In [1]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns

from itertools import combinations
from datetime import datetime

from IPython.display import Markdown

from anomaly_functions import *
from env import *

In [2]:
url = f'mysql+pymysql://{user}:{password}@{host}/curriculum_logs'
query = '''
select *
from logs
join cohorts
on cohorts.id = logs.cohort_id;
'''
df = pd.read_sql(query, url)
df = df.drop(columns=['deleted_at'])

In [3]:
### It appears that cohort_id and id are duplicates

In [4]:
program_dict = {1 : 'PHP Full Stack Web Development',
               2: 'Java Full Stack Web Development',
               3: 'Data Science',
               4: 'Front End Web Development'}
df.replace({'program_id': program_dict}, inplace=True)
df.created_at = pd.to_datetime(df.created_at)
df.updated_at = pd.to_datetime(df.updated_at)
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,PHP Full Stack Web Development
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,PHP Full Stack Web Development
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,PHP Full Stack Web Development
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,PHP Full Stack Web Development
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,Java Full Stack Web Development


In [91]:
def plotProb(df):
    for col in df.columns:
        plt.figure(figsize=(20, 6))
        splot = sns.barplot(x='index', y=col, data=df[col].value_counts(normalize=True).reset_index().sort_values(
            by=col, ascending=False).head(10))
        for p in splot.patches:
            splot.annotate(format(p.get_height() * 100, '.3f') + ' %', 
                       (p.get_x() + p.get_width() / 2., p.get_height()), 
                       ha = 'center', va = 'center', 
                       size=10,
                       xytext = (0, 8), 
                       textcoords = 'offset points')
        plt.title(col.title())
        plt.ylabel('probability')
        plt.show()

In [176]:
def find_answers_per_cohort(df):
    cohort_and_lesson = pd.crosstab(index=df.path,
               columns=df.name)
    cohort_and_lesson['sum'] = cohort_and_lesson.sum(axis=1)
    return cohort_and_lesson.sort_values(by='sum', ascending=False).head(10).index.to_list()

for program in df.program_id.unique():
    print(f'Program {program}')
    print(*find_answers_per_cohort(df[df.program_id == program]), end='\n\n')


Program PHP Full Stack Web Development
/ index.html javascript-i html-css spring java-iii java-ii java-i javascript-ii appendix

Program Java Full Stack Web Development
/ javascript-i toc search/search_index.json java-iii html-css java-ii spring jquery mysql

Program Front End Web Development
content/html-css / content/html-css/gitbook/images/favicon.ico content/html-css/introduction.html

Program Data Science
/ search/search_index.json classification/overview 1-fundamentals/modern-data-scientist.jpg 1-fundamentals/AI-ML-DL-timeline.jpg 1-fundamentals/1.1-intro-to-data-science classification/scale_features_or_not.svg fundamentals/AI-ML-DL-timeline.jpg fundamentals/modern-data-scientist.jpg sql/mysql-overview



In [ ]:
def find_answers_per_cohort(df):
    cohort_and_lesson = pd.crosstab(index=df.path,
               columns=df.name)
    cohort_and_lesson['sum'] = cohort_and_lesson.sum(axis=1)
    return cohort_and_lesson.sort_values(by='sum', ascending=False).head(10).index.to_list()

for program in df.program_id.unique():
    print(f'Program {program}')
    print(*find_answers_per_cohort(df[df.program_id == program]), end='\n\n')

In [6]:
# Avoids any datetime columns
columns = [col for col in df.columns if not np.issubdtype(df[col], np.datetime64)]
# Creates all unique combinations for all columns
combs = set(list(combinations(columns, 2)))

# Anomaly Threshhold
threshhold = .0001

result = f'### Anomaly DataFrames for Threshhold of {threshhold}\n\n'

# List of DataFrames which were lower than the anomaly threshhold
lst_of_anomalies = list()

for comb in combs:
    d1 = comb[0]
    d2 = comb[1]
    new_colname = f'proba_{d1}_given_{d2}'
    tmp_df = pd.DataFrame(df.groupby(d1)[d2].value_counts(normalize=True).rename(new_colname).reset_index())
    # Add to the results Markdown Creation 
    result += f'#### {d1.title()} given {d2.title()}\n\n'
    
    # Add the dataframe to the Markdown to view
    result += tmp_df[tmp_df[new_colname] < threshhold].to_markdown() + '\n\n'
    
    # Add to the lst_of_anomalies to view later if desired
    lst_of_anomalies.append(tmp_df[tmp_df[new_colname] < threshhold])
    

Markdown(result)

### Anomaly DataFrames for Threshhold of 0.0001

#### Date given Cohort_Id

| date   | cohort_id   | proba_date_given_cohort_id   |
|--------|-------------|------------------------------|

#### Path given Id

|       | path                     |   id |   proba_path_given_id |
|------:|:-------------------------|-----:|----------------------:|
|    50 | /                        |    6 |           8.72334e-05 |
|    51 | /                        |    4 |           2.18083e-05 |
|    52 | /                        |    5 |           2.18083e-05 |
|    53 | /                        |    9 |           2.18083e-05 |
|  7288 | html-css                 |    6 |           7.61789e-05 |
|  8011 | java-i                   |    2 |           9.55384e-05 |
|  8012 | java-i                   |   12 |           9.55384e-05 |
|  8260 | java-ii                  |   12 |           8.2122e-05  |
|  8613 | java-iii                 |   12 |           7.59532e-05 |
|  8614 | java-iii                 |  139 |           7.59532e-05 |
|  8991 | javascript-i             |    2 |           5.4936e-05  |
|  8992 | javascript-i             |   12 |           5.4936e-05  |
| 10081 | jquery                   |   19 |           9.05715e-05 |
| 11908 | search/search_index.json |   11 |           5.70321e-05 |
| 11909 | search/search_index.json |   15 |           5.70321e-05 |
| 12479 | spring                   |   55 |           8.41538e-05 |
| 13356 | toc                      |   15 |           5.68473e-05 |

#### Slack given Program_Id

| slack   | program_id   | proba_slack_given_program_id   |
|---------|--------------|--------------------------------|

#### User_Id given Id

| user_id   | id   | proba_user_id_given_id   |
|-----------|------|--------------------------|

#### Id given Program_Id

| id   | program_id   | proba_id_given_program_id   |
|------|--------------|-----------------------------|

#### Date given Start_Date

| date   | start_date   | proba_date_given_start_date   |
|--------|--------------|-------------------------------|

#### Time given Id

| time   | id   | proba_time_given_id   |
|--------|------|-----------------------|

#### Cohort_Id given Id

| cohort_id   | id   | proba_cohort_id_given_id   |
|-------------|------|----------------------------|

#### Time given Path

| time   | path   | proba_time_given_path   |
|--------|--------|-------------------------|

#### Start_Date given Program_Id

| start_date   | program_id   | proba_start_date_given_program_id   |
|--------------|--------------|-------------------------------------|

#### Path given Name

|       | path                     | name       |   proba_path_given_name |
|------:|:-------------------------|:-----------|------------------------:|
|    50 | /                        | Franklin   |             8.72334e-05 |
|    51 | /                        | Apollo     |             2.18083e-05 |
|    52 | /                        | Denali     |             2.18083e-05 |
|    53 | /                        | Everglades |             2.18083e-05 |
|  7288 | html-css                 | Franklin   |             7.61789e-05 |
|  8011 | java-i                   | Badlands   |             9.55384e-05 |
|  8012 | java-i                   | Joshua     |             9.55384e-05 |
|  8260 | java-ii                  | Joshua     |             8.2122e-05  |
|  8613 | java-iii                 | Joshua     |             7.59532e-05 |
|  8614 | java-iii                 | Oberon     |             7.59532e-05 |
|  8991 | javascript-i             | Badlands   |             5.4936e-05  |
|  8992 | javascript-i             | Joshua     |             5.4936e-05  |
| 10081 | jquery                   | Quincy     |             9.05715e-05 |
| 11908 | search/search_index.json | Ike        |             5.70321e-05 |
| 11909 | search/search_index.json | Mammoth    |             5.70321e-05 |
| 12479 | spring                   | Curie      |             8.41538e-05 |
| 13356 | toc                      | Mammoth    |             5.68473e-05 |

#### User_Id given Name

| user_id   | name   | proba_user_id_given_name   |
|-----------|--------|----------------------------|

#### Path given Slack

|       | path                     | slack       |   proba_path_given_slack |
|------:|:-------------------------|:------------|-------------------------:|
|    49 | /                        | #franklin   |              8.72334e-05 |
|    50 | /                        | #apollo     |              2.18083e-05 |
|    51 | /                        | #denali     |              2.18083e-05 |
|    52 | /                        | #everglades |              2.18083e-05 |
|  6946 | html-css                 | #franklin   |              7.61789e-05 |
|  7664 | java-i                   | #badlands   |              9.55384e-05 |
|  7665 | java-i                   | #joshua     |              9.55384e-05 |
|  7911 | java-ii                  | #joshua     |              8.2122e-05  |
|  8263 | java-iii                 | #joshua     |              7.59532e-05 |
|  8264 | java-iii                 | #oberon     |              7.59532e-05 |
|  8640 | javascript-i             | #badlands   |              5.4936e-05  |
|  8641 | javascript-i             | #joshua     |              5.4936e-05  |
|  9726 | jquery                   | #quincy     |              9.05715e-05 |
| 11513 | search/search_index.json | #ike        |              5.70321e-05 |
| 11514 | search/search_index.json | #mammoth    |              5.70321e-05 |
| 12083 | spring                   | #curie      |              8.41538e-05 |
| 12906 | toc                      | #mammoth    |              5.68473e-05 |

#### Date given User_Id

| date   | user_id   | proba_date_given_user_id   |
|--------|-----------|----------------------------|

#### Path given End_Date

|       | path                     | end_date   |   proba_path_given_end_date |
|------:|:-------------------------|:-----------|----------------------------:|
|    48 | /                        | 2015-05-26 |                 8.72334e-05 |
|    49 | /                        | 2015-01-18 |                 2.18083e-05 |
|    50 | /                        | 2015-02-24 |                 2.18083e-05 |
|    51 | /                        | 2015-07-29 |                 2.18083e-05 |
|  7273 | html-css                 | 2015-05-26 |                 7.61789e-05 |
|  7996 | java-i                   | 2014-08-22 |                 9.55384e-05 |
|  7997 | java-i                   | 2016-06-30 |                 9.55384e-05 |
|  8245 | java-ii                  | 2016-06-30 |                 8.2122e-05  |
|  8598 | java-iii                 | 2016-06-30 |                 7.59532e-05 |
|  8599 | java-iii                 | 2021-10-01 |                 7.59532e-05 |
|  8976 | javascript-i             | 2014-08-22 |                 5.4936e-05  |
|  8977 | javascript-i             | 2016-06-30 |                 5.4936e-05  |
| 10066 | jquery                   | 2017-09-22 |                 9.05715e-05 |
| 11890 | search/search_index.json | 2016-05-12 |                 5.70321e-05 |
| 11891 | search/search_index.json | 2017-02-02 |                 5.70321e-05 |
| 12461 | spring                   | 2020-07-07 |                 8.41538e-05 |
| 13338 | toc                      | 2017-02-02 |                 5.68473e-05 |

#### User_Id given Slack

| user_id   | slack   | proba_user_id_given_slack   |
|-----------|---------|-----------------------------|

#### Date given Time

| date   | time   | proba_date_given_time   |
|--------|--------|-------------------------|

#### Time given Name

| time   | name   | proba_time_given_name   |
|--------|--------|-------------------------|

#### Path given Ip

|        | path                     | ip              |   proba_path_given_ip |
|-------:|:-------------------------|:----------------|----------------------:|
|    899 | /                        | 104.181.128.75  |           8.72334e-05 |
|    900 | /                        | 104.5.221.32    |           8.72334e-05 |
|    901 | /                        | 107.129.209.3   |           8.72334e-05 |
|    902 | /                        | 107.77.217.109  |           8.72334e-05 |
|    903 | /                        | 107.77.217.122  |           8.72334e-05 |
|    904 | /                        | 107.77.217.24   |           8.72334e-05 |
|    905 | /                        | 107.77.219.127  |           8.72334e-05 |
|    906 | /                        | 108.222.98.137  |           8.72334e-05 |
|    907 | /                        | 130.45.62.247   |           8.72334e-05 |
|    908 | /                        | 130.45.68.158   |           8.72334e-05 |
|    909 | /                        | 136.50.50.90    |           8.72334e-05 |
|    910 | /                        | 136.50.70.139   |           8.72334e-05 |
|    911 | /                        | 136.50.95.62    |           8.72334e-05 |
|    912 | /                        | 148.66.37.74    |           8.72334e-05 |
|    913 | /                        | 165.231.210.20  |           8.72334e-05 |
|    914 | /                        | 167.24.18.231   |           8.72334e-05 |
|    915 | /                        | 172.127.183.32  |           8.72334e-05 |
|    916 | /                        | 173.173.96.192  |           8.72334e-05 |
|    917 | /                        | 173.173.98.137  |           8.72334e-05 |
|    918 | /                        | 173.175.109.205 |           8.72334e-05 |
|    919 | /                        | 173.175.208.49  |           8.72334e-05 |
|    920 | /                        | 174.246.197.179 |           8.72334e-05 |
|    921 | /                        | 174.255.146.172 |           8.72334e-05 |
|    922 | /                        | 192.107.1.225   |           8.72334e-05 |
|    923 | /                        | 192.138.47.30   |           8.72334e-05 |
|    924 | /                        | 198.190.245.72  |           8.72334e-05 |
|    925 | /                        | 24.155.69.59    |           8.72334e-05 |
|    926 | /                        | 24.160.145.169  |           8.72334e-05 |
|    927 | /                        | 24.160.149.87   |           8.72334e-05 |
|    928 | /                        | 24.28.96.138    |           8.72334e-05 |
|    929 | /                        | 45.131.192.68   |           8.72334e-05 |
|    930 | /                        | 45.21.0.138     |           8.72334e-05 |
|    931 | /                        | 47.183.134.235  |           8.72334e-05 |
|    932 | /                        | 47.218.59.57    |           8.72334e-05 |
|    933 | /                        | 52.144.111.133  |           8.72334e-05 |
|    934 | /                        | 64.134.149.176  |           8.72334e-05 |
|    935 | /                        | 64.134.26.193   |           8.72334e-05 |
|    936 | /                        | 64.250.9.218    |           8.72334e-05 |
|    937 | /                        | 65.36.111.91    |           8.72334e-05 |
|    938 | /                        | 66.196.195.34   |           8.72334e-05 |
|    939 | /                        | 66.25.173.125   |           8.72334e-05 |
|    940 | /                        | 66.69.77.154    |           8.72334e-05 |
|    941 | /                        | 66.69.81.250    |           8.72334e-05 |
|    942 | /                        | 66.90.136.214   |           8.72334e-05 |
|    943 | /                        | 67.10.171.227   |           8.72334e-05 |
|    944 | /                        | 67.11.185.214   |           8.72334e-05 |
|    945 | /                        | 67.11.48.25     |           8.72334e-05 |
|    946 | /                        | 67.11.87.229    |           8.72334e-05 |
|    947 | /                        | 67.78.112.162   |           8.72334e-05 |
|    948 | /                        | 68.206.101.245  |           8.72334e-05 |
|    949 | /                        | 68.206.166.136  |           8.72334e-05 |
|    950 | /                        | 68.77.133.109   |           8.72334e-05 |
|    951 | /                        | 70.114.139.150  |           8.72334e-05 |
|    952 | /                        | 70.114.38.157   |           8.72334e-05 |
|    953 | /                        | 70.120.1.49     |           8.72334e-05 |
|    954 | /                        | 70.120.22.185   |           8.72334e-05 |
|    955 | /                        | 70.120.57.140   |           8.72334e-05 |
|    956 | /                        | 70.120.73.254   |           8.72334e-05 |
|    957 | /                        | 70.121.180.146  |           8.72334e-05 |
|    958 | /                        | 70.121.200.203  |           8.72334e-05 |
|    959 | /                        | 70.122.201.106  |           8.72334e-05 |
|    960 | /                        | 70.123.164.223  |           8.72334e-05 |
|    961 | /                        | 70.123.239.146  |           8.72334e-05 |
|    962 | /                        | 70.123.93.201   |           8.72334e-05 |
|    963 | /                        | 70.125.131.6    |           8.72334e-05 |
|    964 | /                        | 70.130.122.39   |           8.72334e-05 |
|    965 | /                        | 70.133.251.114  |           8.72334e-05 |
|    966 | /                        | 70.233.161.172  |           8.72334e-05 |
|    967 | /                        | 70.94.132.38    |           8.72334e-05 |
|    968 | /                        | 71.221.237.129  |           8.72334e-05 |
|    969 | /                        | 71.40.24.180    |           8.72334e-05 |
|    970 | /                        | 71.41.8.13      |           8.72334e-05 |
|    971 | /                        | 71.42.198.186   |           8.72334e-05 |
|    972 | /                        | 71.78.217.82    |           8.72334e-05 |
|    973 | /                        | 72.177.135.99   |           8.72334e-05 |
|    974 | /                        | 72.181.106.165  |           8.72334e-05 |
|    975 | /                        | 72.181.106.34   |           8.72334e-05 |
|    976 | /                        | 72.181.116.205  |           8.72334e-05 |
|    977 | /                        | 72.181.145.92   |           8.72334e-05 |
|    978 | /                        | 72.181.59.214   |           8.72334e-05 |
|    979 | /                        | 72.190.168.222  |           8.72334e-05 |
|    980 | /                        | 72.191.19.45    |           8.72334e-05 |
|    981 | /                        | 72.191.48.117   |           8.72334e-05 |
|    982 | /                        | 74.81.88.162    |           8.72334e-05 |
|    983 | /                        | 76.115.148.248  |           8.72334e-05 |
|    984 | /                        | 76.170.151.113  |           8.72334e-05 |
|    985 | /                        | 76.185.163.154  |           8.72334e-05 |
|    986 | /                        | 76.185.182.104  |           8.72334e-05 |
|    987 | /                        | 76.185.209.80   |           8.72334e-05 |
|    988 | /                        | 76.185.234.25   |           8.72334e-05 |
|    989 | /                        | 76.187.203.223  |           8.72334e-05 |
|    990 | /                        | 76.25.25.4      |           8.72334e-05 |
|    991 | /                        | 89.187.164.150  |           8.72334e-05 |
|    992 | /                        | 89.187.164.162  |           8.72334e-05 |
|    993 | /                        | 89.187.164.168  |           8.72334e-05 |
|    994 | /                        | 89.187.164.244  |           8.72334e-05 |
|    995 | /                        | 89.187.164.79   |           8.72334e-05 |
|    996 | /                        | 89.187.175.148  |           8.72334e-05 |
|    997 | /                        | 96.8.130.139    |           8.72334e-05 |
|    998 | /                        | 96.8.130.193    |           8.72334e-05 |
|    999 | /                        | 96.8.130.251    |           8.72334e-05 |
|   1000 | /                        | 96.8.130.92     |           8.72334e-05 |
|   1001 | /                        | 96.8.168.147    |           8.72334e-05 |
|   1002 | /                        | 97.77.126.60    |           8.72334e-05 |
|   1003 | /                        | 97.98.85.151    |           8.72334e-05 |
|   1004 | /                        | 99.100.170.238  |           8.72334e-05 |
|   1005 | /                        | 99.132.130.148  |           8.72334e-05 |
|   1006 | /                        | 99.147.236.67   |           8.72334e-05 |
|   1007 | /                        | 99.151.201.25   |           8.72334e-05 |
|   1008 | /                        | 99.158.247.252  |           8.72334e-05 |
|   1009 | /                        | 99.48.228.201   |           8.72334e-05 |
|   1010 | /                        | 99.48.228.206   |           8.72334e-05 |
|   1011 | /                        | 99.62.54.53     |           8.72334e-05 |
|   1012 | /                        | 104.180.86.144  |           6.5425e-05  |
|   1013 | /                        | 104.190.255.78  |           6.5425e-05  |
|   1014 | /                        | 104.191.12.108  |           6.5425e-05  |
|   1015 | /                        | 104.193.225.250 |           6.5425e-05  |
|   1016 | /                        | 104.200.142.196 |           6.5425e-05  |
|   1017 | /                        | 104.219.136.53  |           6.5425e-05  |
|   1018 | /                        | 104.53.202.72   |           6.5425e-05  |
|   1019 | /                        | 104.53.212.101  |           6.5425e-05  |
|   1020 | /                        | 107.140.217.8   |           6.5425e-05  |
|   1021 | /                        | 107.77.169.12   |           6.5425e-05  |
|   1022 | /                        | 107.77.169.8    |           6.5425e-05  |
|   1023 | /                        | 107.77.169.9    |           6.5425e-05  |
|   1024 | /                        | 107.77.189.33   |           6.5425e-05  |
|   1025 | /                        | 107.77.189.9    |           6.5425e-05  |
|   1026 | /                        | 107.77.196.213  |           6.5425e-05  |
|   1027 | /                        | 107.77.218.10   |           6.5425e-05  |
|   1028 | /                        | 107.77.218.190  |           6.5425e-05  |
|   1029 | /                        | 107.77.218.33   |           6.5425e-05  |
|   1030 | /                        | 107.77.218.37   |           6.5425e-05  |
|   1031 | /                        | 107.77.218.48   |           6.5425e-05  |
|   1032 | /                        | 107.77.219.51   |           6.5425e-05  |
|   1033 | /                        | 107.77.219.68   |           6.5425e-05  |
|   1034 | /                        | 107.77.221.127  |           6.5425e-05  |
|   1035 | /                        | 108.174.80.231  |           6.5425e-05  |
|   1036 | /                        | 108.196.159.214 |           6.5425e-05  |
|   1037 | /                        | 108.199.116.156 |           6.5425e-05  |
|   1038 | /                        | 108.199.117.96  |           6.5425e-05  |
|   1039 | /                        | 108.84.76.42    |           6.5425e-05  |
|   1040 | /                        | 12.221.11.38    |           6.5425e-05  |
|   1041 | /                        | 12.52.157.98    |           6.5425e-05  |
|   1042 | /                        | 130.45.55.231   |           6.5425e-05  |
|   1043 | /                        | 136.50.16.179   |           6.5425e-05  |
|   1044 | /                        | 136.50.16.223   |           6.5425e-05  |
|   1045 | /                        | 136.50.36.192   |           6.5425e-05  |
|   1046 | /                        | 136.50.66.159   |           6.5425e-05  |
|   1047 | /                        | 136.50.79.189   |           6.5425e-05  |
|   1048 | /                        | 136.50.97.29    |           6.5425e-05  |
|   1049 | /                        | 137.7.73.123    |           6.5425e-05  |
|   1050 | /                        | 148.66.37.91    |           6.5425e-05  |
|   1051 | /                        | 162.194.247.205 |           6.5425e-05  |
|   1052 | /                        | 162.233.171.61  |           6.5425e-05  |
|   1053 | /                        | 162.253.68.118  |           6.5425e-05  |
|   1054 | /                        | 166.251.106.216 |           6.5425e-05  |
|   1055 | /                        | 170.130.2.251   |           6.5425e-05  |
|   1056 | /                        | 172.110.83.150  |           6.5425e-05  |
|   1057 | /                        | 172.58.67.230   |           6.5425e-05  |
|   1058 | /                        | 172.58.99.111   |           6.5425e-05  |
|   1059 | /                        | 172.85.200.202  |           6.5425e-05  |
|   1060 | /                        | 172.87.151.154  |           6.5425e-05  |
|   1061 | /                        | 172.87.153.157  |           6.5425e-05  |
|   1062 | /                        | 173.173.105.24  |           6.5425e-05  |
|   1063 | /                        | 173.174.164.88  |           6.5425e-05  |
|   1064 | /                        | 173.174.7.203   |           6.5425e-05  |
|   1065 | /                        | 173.30.84.132   |           6.5425e-05  |
|   1066 | /                        | 174.207.5.52    |           6.5425e-05  |
|   1067 | /                        | 174.236.1.254   |           6.5425e-05  |
|   1068 | /                        | 174.255.137.57  |           6.5425e-05  |
|   1069 | /                        | 185.151.12.218  |           6.5425e-05  |
|   1070 | /                        | 199.193.221.114 |           6.5425e-05  |
|   1071 | /                        | 204.60.9.26     |           6.5425e-05  |
|   1072 | /                        | 205.251.148.186 |           6.5425e-05  |
|   1073 | /                        | 205.251.150.146 |           6.5425e-05  |
|   1074 | /                        | 206.193.219.171 |           6.5425e-05  |
|   1075 | /                        | 206.214.78.35   |           6.5425e-05  |
|   1076 | /                        | 208.73.111.171  |           6.5425e-05  |
|   1077 | /                        | 216.177.187.98  |           6.5425e-05  |
|   1078 | /                        | 216.54.250.134  |           6.5425e-05  |
|   1079 | /                        | 23.116.24.226   |           6.5425e-05  |
|   1080 | /                        | 23.123.177.19   |           6.5425e-05  |
|   1081 | /                        | 23.127.174.69   |           6.5425e-05  |
|   1082 | /                        | 24.242.163.11   |           6.5425e-05  |
|   1083 | /                        | 24.243.10.238   |           6.5425e-05  |
|   1084 | /                        | 24.243.31.34    |           6.5425e-05  |
|   1085 | /                        | 24.243.35.7     |           6.5425e-05  |
|   1086 | /                        | 24.243.38.118   |           6.5425e-05  |
|   1087 | /                        | 24.243.7.150    |           6.5425e-05  |
|   1088 | /                        | 24.55.13.38     |           6.5425e-05  |
|   1089 | /                        | 32.141.17.86    |           6.5425e-05  |
|   1090 | /                        | 38.96.210.4     |           6.5425e-05  |
|   1091 | /                        | 45.131.192.231  |           6.5425e-05  |
|   1092 | /                        | 45.17.26.192    |           6.5425e-05  |
|   1093 | /                        | 45.21.39.47     |           6.5425e-05  |
|   1094 | /                        | 45.26.51.187    |           6.5425e-05  |
|   1095 | /                        | 45.30.141.87    |           6.5425e-05  |
|   1096 | /                        | 45.56.151.122   |           6.5425e-05  |
|   1097 | /                        | 45.56.151.182   |           6.5425e-05  |
|   1098 | /                        | 47.221.223.242  |           6.5425e-05  |
|   1099 | /                        | 52.95.4.16      |           6.5425e-05  |
|   1100 | /                        | 64.134.148.108  |           6.5425e-05  |
|   1101 | /                        | 65.36.104.120   |           6.5425e-05  |
|   1102 | /                        | 66.3.38.170     |           6.5425e-05  |
|   1103 | /                        | 66.64.5.233     |           6.5425e-05  |
|   1104 | /                        | 66.90.173.188   |           6.5425e-05  |
|   1105 | /                        | 67.10.153.249   |           6.5425e-05  |
|   1106 | /                        | 67.10.161.67    |           6.5425e-05  |
|   1107 | /                        | 67.10.174.4     |           6.5425e-05  |
|   1108 | /                        | 67.11.111.87    |           6.5425e-05  |
|   1109 | /                        | 67.11.113.73    |           6.5425e-05  |
|   1110 | /                        | 67.11.145.202   |           6.5425e-05  |
|   1111 | /                        | 67.11.185.145   |           6.5425e-05  |
|   1112 | /                        | 67.11.207.37    |           6.5425e-05  |
|   1113 | /                        | 67.11.215.110   |           6.5425e-05  |
|   1114 | /                        | 67.11.233.101   |           6.5425e-05  |
|   1115 | /                        | 67.11.56.174    |           6.5425e-05  |
|   1116 | /                        | 67.164.150.90   |           6.5425e-05  |
|   1117 | /                        | 67.22.173.176   |           6.5425e-05  |
|   1118 | /                        | 67.48.201.230   |           6.5425e-05  |
|   1119 | /                        | 68.203.215.174  |           6.5425e-05  |
|   1120 | /                        | 68.206.198.66   |           6.5425e-05  |
|   1121 | /                        | 68.206.246.215  |           6.5425e-05  |
|   1122 | /                        | 68.206.251.51   |           6.5425e-05  |
|   1123 | /                        | 68.95.172.10    |           6.5425e-05  |
|   1124 | /                        | 69.232.103.179  |           6.5425e-05  |
|   1125 | /                        | 70.113.107.71   |           6.5425e-05  |
|   1126 | /                        | 70.114.29.134   |           6.5425e-05  |
|   1127 | /                        | 70.117.8.118    |           6.5425e-05  |
|   1128 | /                        | 70.118.67.138   |           6.5425e-05  |
|   1129 | /                        | 70.120.27.3     |           6.5425e-05  |
|   1130 | /                        | 70.120.40.174   |           6.5425e-05  |
|   1131 | /                        | 70.121.171.184  |           6.5425e-05  |
|   1132 | /                        | 70.121.181.181  |           6.5425e-05  |
|   1133 | /                        | 70.122.206.28   |           6.5425e-05  |
|   1134 | /                        | 70.123.201.199  |           6.5425e-05  |
|   1135 | /                        | 70.123.239.48   |           6.5425e-05  |
|   1136 | /                        | 70.124.156.202  |           6.5425e-05  |
|   1137 | /                        | 70.125.213.204  |           6.5425e-05  |
|   1138 | /                        | 70.239.184.149  |           6.5425e-05  |
|   1139 | /                        | 70.94.148.57    |           6.5425e-05  |
|   1140 | /                        | 70.94.157.237   |           6.5425e-05  |
|   1141 | /                        | 71.198.180.6    |           6.5425e-05  |
|   1142 | /                        | 71.221.194.87   |           6.5425e-05  |
|   1143 | /                        | 71.221.202.248  |           6.5425e-05  |
|   1144 | /                        | 71.221.207.118  |           6.5425e-05  |
|   1145 | /                        | 71.221.210.29   |           6.5425e-05  |
|   1146 | /                        | 71.221.250.179  |           6.5425e-05  |
|   1147 | /                        | 71.234.171.235  |           6.5425e-05  |
|   1148 | /                        | 71.238.72.160   |           6.5425e-05  |
|   1149 | /                        | 71.42.133.19    |           6.5425e-05  |
|   1150 | /                        | 71.42.196.114   |           6.5425e-05  |
|   1151 | /                        | 71.42.84.202    |           6.5425e-05  |
|   1152 | /                        | 72.177.146.68   |           6.5425e-05  |
|   1153 | /                        | 72.179.108.103  |           6.5425e-05  |
|   1154 | /                        | 72.179.132.10   |           6.5425e-05  |
|   1155 | /                        | 72.179.185.12   |           6.5425e-05  |
|   1156 | /                        | 72.180.149.37   |           6.5425e-05  |
|   1157 | /                        | 72.181.106.116  |           6.5425e-05  |
|   1158 | /                        | 72.181.96.144   |           6.5425e-05  |
|   1159 | /                        | 72.190.143.130  |           6.5425e-05  |
|   1160 | /                        | 72.190.156.176  |           6.5425e-05  |
|   1161 | /                        | 72.191.28.232   |           6.5425e-05  |
|   1162 | /                        | 72.191.34.7     |           6.5425e-05  |
|   1163 | /                        | 72.48.1.140     |           6.5425e-05  |
|   1164 | /                        | 72.48.255.177   |           6.5425e-05  |
|   1165 | /                        | 73.123.183.143  |           6.5425e-05  |
|   1166 | /                        | 73.153.77.234   |           6.5425e-05  |
|   1167 | /                        | 73.233.99.219   |           6.5425e-05  |
|   1168 | /                        | 73.239.66.107   |           6.5425e-05  |
|   1169 | /                        | 74.192.130.90   |           6.5425e-05  |
|   1170 | /                        | 75.58.164.200   |           6.5425e-05  |
|   1171 | /                        | 76.185.209.0    |           6.5425e-05  |
|   1172 | /                        | 76.185.236.152  |           6.5425e-05  |
|   1173 | /                        | 76.185.247.236  |           6.5425e-05  |
|   1174 | /                        | 76.186.156.49   |           6.5425e-05  |
|   1175 | /                        | 76.218.3.43     |           6.5425e-05  |
|   1176 | /                        | 76.218.41.171   |           6.5425e-05  |
|   1177 | /                        | 76.237.205.238  |           6.5425e-05  |
|   1178 | /                        | 76.25.49.194    |           6.5425e-05  |
|   1179 | /                        | 85.106.97.135   |           6.5425e-05  |
|   1180 | /                        | 89.187.164.133  |           6.5425e-05  |
|   1181 | /                        | 89.187.164.134  |           6.5425e-05  |
|   1182 | /                        | 89.187.164.135  |           6.5425e-05  |
|   1183 | /                        | 89.187.164.149  |           6.5425e-05  |
|   1184 | /                        | 89.187.164.163  |           6.5425e-05  |
|   1185 | /                        | 89.187.164.78   |           6.5425e-05  |
|   1186 | /                        | 96.8.128.88     |           6.5425e-05  |
|   1187 | /                        | 96.8.130.143    |           6.5425e-05  |
|   1188 | /                        | 96.8.151.49     |           6.5425e-05  |
|   1189 | /                        | 96.8.177.189    |           6.5425e-05  |
|   1190 | /                        | 97.123.183.37   |           6.5425e-05  |
|   1191 | /                        | 97.77.161.196   |           6.5425e-05  |
|   1192 | /                        | 98.158.243.104  |           6.5425e-05  |
|   1193 | /                        | 98.39.94.41     |           6.5425e-05  |
|   1194 | /                        | 99.132.128.80   |           6.5425e-05  |
|   1195 | /                        | 99.186.6.129    |           6.5425e-05  |
|   1196 | /                        | 99.203.90.58    |           6.5425e-05  |
|   1197 | /                        | 99.27.138.170   |           6.5425e-05  |
|   1198 | /                        | 99.62.54.30     |           6.5425e-05  |
|   1199 | /                        | 99.76.235.191   |           6.5425e-05  |
|   1200 | /                        | 104.1.46.153    |           4.36167e-05 |
|   1201 | /                        | 104.131.47.203  |           4.36167e-05 |
|   1202 | /                        | 104.189.74.143  |           4.36167e-05 |
|   1203 | /                        | 104.190.249.199 |           4.36167e-05 |
|   1204 | /                        | 104.237.80.51   |           4.36167e-05 |
|   1205 | /                        | 104.53.215.7    |           4.36167e-05 |
|   1206 | /                        | 107.128.3.66    |           4.36167e-05 |
|   1207 | /                        | 107.146.141.88  |           4.36167e-05 |
|   1208 | /                        | 107.152.102.138 |           4.36167e-05 |
|   1209 | /                        | 107.77.100.125  |           4.36167e-05 |
|   1210 | /                        | 107.77.100.127  |           4.36167e-05 |
|   1211 | /                        | 107.77.100.47   |           4.36167e-05 |
|   1212 | /                        | 107.77.100.85   |           4.36167e-05 |
|   1213 | /                        | 107.77.105.97   |           4.36167e-05 |
|   1214 | /                        | 107.77.169.10   |           4.36167e-05 |
|   1215 | /                        | 107.77.169.42   |           4.36167e-05 |
|   1216 | /                        | 107.77.189.24   |           4.36167e-05 |
|   1217 | /                        | 107.77.196.195  |           4.36167e-05 |
|   1218 | /                        | 107.77.217.112  |           4.36167e-05 |
|   1219 | /                        | 107.77.217.13   |           4.36167e-05 |
|   1220 | /                        | 107.77.217.156  |           4.36167e-05 |
|   1221 | /                        | 107.77.217.182  |           4.36167e-05 |
|   1222 | /                        | 107.77.217.218  |           4.36167e-05 |
|   1223 | /                        | 107.77.218.18   |           4.36167e-05 |
|   1224 | /                        | 107.77.218.215  |           4.36167e-05 |
|   1225 | /                        | 107.77.218.35   |           4.36167e-05 |
|   1226 | /                        | 107.77.218.81   |           4.36167e-05 |
|   1227 | /                        | 107.77.219.172  |           4.36167e-05 |
|   1228 | /                        | 107.77.219.175  |           4.36167e-05 |
|   1229 | /                        | 107.77.219.190  |           4.36167e-05 |
|   1230 | /                        | 107.77.219.197  |           4.36167e-05 |
|   1231 | /                        | 107.77.219.67   |           4.36167e-05 |
|   1232 | /                        | 107.77.219.96   |           4.36167e-05 |
|   1233 | /                        | 107.77.220.149  |           4.36167e-05 |
|   1234 | /                        | 107.77.220.214  |           4.36167e-05 |
|   1235 | /                        | 107.77.220.6    |           4.36167e-05 |
|   1236 | /                        | 107.77.221.116  |           4.36167e-05 |
|   1237 | /                        | 107.77.221.151  |           4.36167e-05 |
|   1238 | /                        | 107.77.221.157  |           4.36167e-05 |
|   1239 | /                        | 107.77.221.165  |           4.36167e-05 |
|   1240 | /                        | 107.77.221.196  |           4.36167e-05 |
|   1241 | /                        | 107.77.221.96   |           4.36167e-05 |
|   1242 | /                        | 107.77.222.69   |           4.36167e-05 |
|   1243 | /                        | 107.77.66.115   |           4.36167e-05 |
|   1244 | /                        | 108.11.13.210   |           4.36167e-05 |
|   1245 | /                        | 108.160.237.17  |           4.36167e-05 |
|   1246 | /                        | 108.213.189.26  |           4.36167e-05 |
|   1247 | /                        | 12.189.101.90   |           4.36167e-05 |
|   1248 | /                        | 12.247.3.42     |           4.36167e-05 |
|   1249 | /                        | 12.5.63.210     |           4.36167e-05 |
|   1250 | /                        | 12.71.181.245   |           4.36167e-05 |
|   1251 | /                        | 129.115.2.245   |           4.36167e-05 |
|   1252 | /                        | 130.45.46.189   |           4.36167e-05 |
|   1253 | /                        | 130.45.47.106   |           4.36167e-05 |
|   1254 | /                        | 130.45.49.167   |           4.36167e-05 |
|   1255 | /                        | 130.45.51.161   |           4.36167e-05 |
|   1256 | /                        | 130.45.65.31    |           4.36167e-05 |
|   1257 | /                        | 131.44.121.130  |           4.36167e-05 |
|   1258 | /                        | 136.50.104.251  |           4.36167e-05 |
|   1259 | /                        | 136.50.110.0    |           4.36167e-05 |
|   1260 | /                        | 136.50.13.147   |           4.36167e-05 |
|   1261 | /                        | 136.50.72.144   |           4.36167e-05 |
|   1262 | /                        | 136.50.79.102   |           4.36167e-05 |
|   1263 | /                        | 137.242.1.128   |           4.36167e-05 |
|   1264 | /                        | 137.242.1.129   |           4.36167e-05 |
|   1265 | /                        | 148.244.212.19  |           4.36167e-05 |
|   1266 | /                        | 148.59.234.86   |           4.36167e-05 |
|   1267 | /                        | 148.66.37.55    |           4.36167e-05 |
|   1268 | /                        | 148.66.37.82    |           4.36167e-05 |
|   1269 | /                        | 154.3.251.115   |           4.36167e-05 |
|   1270 | /                        | 154.6.28.146    |           4.36167e-05 |
|   1271 | /                        | 162.200.114.251 |           4.36167e-05 |
|   1272 | /                        | 162.233.170.151 |           4.36167e-05 |
|   1273 | /                        | 162.236.252.30  |           4.36167e-05 |
|   1274 | /                        | 162.72.133.33   |           4.36167e-05 |
|   1275 | /                        | 165.91.13.35    |           4.36167e-05 |
|   1276 | /                        | 166.205.97.137  |           4.36167e-05 |
|   1277 | /                        | 166.251.101.230 |           4.36167e-05 |
|   1278 | /                        | 166.251.112.225 |           4.36167e-05 |
|   1279 | /                        | 167.24.104.154  |           4.36167e-05 |
|   1280 | /                        | 172.127.187.185 |           4.36167e-05 |
|   1281 | /                        | 172.5.98.128    |           4.36167e-05 |
|   1282 | /                        | 172.56.14.146   |           4.36167e-05 |
|   1283 | /                        | 172.56.15.15    |           4.36167e-05 |
|   1284 | /                        | 172.56.6.145    |           4.36167e-05 |
|   1285 | /                        | 172.56.7.129    |           4.36167e-05 |
|   1286 | /                        | 172.56.7.146    |           4.36167e-05 |
|   1287 | /                        | 172.58.102.136  |           4.36167e-05 |
|   1288 | /                        | 172.58.102.244  |           4.36167e-05 |
|   1289 | /                        | 172.58.107.194  |           4.36167e-05 |
|   1290 | /                        | 172.58.107.203  |           4.36167e-05 |
|   1291 | /                        | 172.58.108.232  |           4.36167e-05 |
|   1292 | /                        | 172.58.109.206  |           4.36167e-05 |
|   1293 | /                        | 172.58.110.156  |           4.36167e-05 |
|   1294 | /                        | 172.58.110.191  |           4.36167e-05 |
|   1295 | /                        | 172.58.111.159  |           4.36167e-05 |
|   1296 | /                        | 172.58.111.213  |           4.36167e-05 |
|   1297 | /                        | 172.58.67.181   |           4.36167e-05 |
|   1298 | /                        | 172.58.68.171   |           4.36167e-05 |
|   1299 | /                        | 172.58.97.55    |           4.36167e-05 |
|   1300 | /                        | 172.58.99.11    |           4.36167e-05 |
|   1301 | /                        | 172.58.99.191   |           4.36167e-05 |
|   1302 | /                        | 173.123.204.117 |           4.36167e-05 |
|   1303 | /                        | 173.127.40.170  |           4.36167e-05 |
|   1304 | /                        | 173.127.48.40   |           4.36167e-05 |
|   1305 | /                        | 173.173.104.57  |           4.36167e-05 |
|   1306 | /                        | 173.173.121.126 |           4.36167e-05 |
|   1307 | /                        | 173.174.149.192 |           4.36167e-05 |
|   1308 | /                        | 173.174.153.61  |           4.36167e-05 |
|   1309 | /                        | 173.174.178.51  |           4.36167e-05 |
|   1310 | /                        | 173.174.207.225 |           4.36167e-05 |
|   1311 | /                        | 173.174.211.231 |           4.36167e-05 |
|   1312 | /                        | 173.175.112.68  |           4.36167e-05 |
|   1313 | /                        | 173.175.194.1   |           4.36167e-05 |
|   1314 | /                        | 173.232.241.116 |           4.36167e-05 |
|   1315 | /                        | 173.239.232.143 |           4.36167e-05 |
|   1316 | /                        | 173.239.232.164 |           4.36167e-05 |
|   1317 | /                        | 173.239.232.3   |           4.36167e-05 |
|   1318 | /                        | 173.239.232.38  |           4.36167e-05 |
|   1319 | /                        | 173.239.232.62  |           4.36167e-05 |
|   1320 | /                        | 173.239.232.65  |           4.36167e-05 |
|   1321 | /                        | 173.239.232.81  |           4.36167e-05 |
|   1322 | /                        | 173.239.232.82  |           4.36167e-05 |
|   1323 | /                        | 173.239.232.92  |           4.36167e-05 |
|   1324 | /                        | 173.244.217.91  |           4.36167e-05 |
|   1325 | /                        | 174.128.180.71  |           4.36167e-05 |
|   1326 | /                        | 174.128.181.119 |           4.36167e-05 |
|   1327 | /                        | 174.197.19.54   |           4.36167e-05 |
|   1328 | /                        | 174.207.1.54    |           4.36167e-05 |
|   1329 | /                        | 174.207.10.43   |           4.36167e-05 |
|   1330 | /                        | 174.207.17.170  |           4.36167e-05 |
|   1331 | /                        | 174.207.22.170  |           4.36167e-05 |
|   1332 | /                        | 174.207.23.151  |           4.36167e-05 |
|   1333 | /                        | 174.207.4.87    |           4.36167e-05 |
|   1334 | /                        | 174.244.80.215  |           4.36167e-05 |
|   1335 | /                        | 174.51.14.27    |           4.36167e-05 |
|   1336 | /                        | 181.214.227.47  |           4.36167e-05 |
|   1337 | /                        | 184.169.45.4    |           4.36167e-05 |
|   1338 | /                        | 184.203.225.164 |           4.36167e-05 |
|   1339 | /                        | 185.245.87.158  |           4.36167e-05 |
|   1340 | /                        | 187.201.71.67   |           4.36167e-05 |
|   1341 | /                        | 192.198.252.227 |           4.36167e-05 |
|   1342 | /                        | 192.67.54.12    |           4.36167e-05 |
|   1343 | /                        | 195.206.104.108 |           4.36167e-05 |
|   1344 | /                        | 198.202.210.220 |           4.36167e-05 |
|   1345 | /                        | 199.59.33.125   |           4.36167e-05 |
|   1346 | /                        | 205.251.148.98  |           4.36167e-05 |
|   1347 | /                        | 205.251.150.210 |           4.36167e-05 |
|   1348 | /                        | 205.251.150.218 |           4.36167e-05 |
|   1349 | /                        | 205.251.151.42  |           4.36167e-05 |
|   1350 | /                        | 206.214.78.211  |           4.36167e-05 |
|   1351 | /                        | 207.239.160.254 |           4.36167e-05 |
|   1352 | /                        | 208.84.155.68   |           4.36167e-05 |
|   1353 | /                        | 209.194.208.150 |           4.36167e-05 |
|   1354 | /                        | 209.92.162.135  |           4.36167e-05 |
|   1355 | /                        | 218.186.46.251  |           4.36167e-05 |
|   1356 | /                        | 24.155.118.66   |           4.36167e-05 |
|   1357 | /                        | 24.160.133.222  |           4.36167e-05 |
|   1358 | /                        | 24.170.93.9     |           4.36167e-05 |
|   1359 | /                        | 24.242.143.6    |           4.36167e-05 |
|   1360 | /                        | 24.242.93.131   |           4.36167e-05 |
|   1361 | /                        | 24.243.17.137   |           4.36167e-05 |
|   1362 | /                        | 24.243.19.100   |           4.36167e-05 |
|   1363 | /                        | 24.243.20.85    |           4.36167e-05 |
|   1364 | /                        | 24.243.32.134   |           4.36167e-05 |
|   1365 | /                        | 24.243.37.71    |           4.36167e-05 |
|   1366 | /                        | 24.243.58.62    |           4.36167e-05 |
|   1367 | /                        | 24.243.6.109    |           4.36167e-05 |
|   1368 | /                        | 24.243.7.214    |           4.36167e-05 |
|   1369 | /                        | 24.26.244.77    |           4.36167e-05 |
|   1370 | /                        | 24.26.249.169   |           4.36167e-05 |
|   1371 | /                        | 24.27.227.0     |           4.36167e-05 |
|   1372 | /                        | 24.27.227.1     |           4.36167e-05 |
|   1373 | /                        | 24.28.109.234   |           4.36167e-05 |
|   1374 | /                        | 24.28.146.155   |           4.36167e-05 |
|   1375 | /                        | 24.28.152.103   |           4.36167e-05 |
|   1376 | /                        | 24.28.154.138   |           4.36167e-05 |
|   1377 | /                        | 24.54.121.33    |           4.36167e-05 |
|   1378 | /                        | 24.99.149.248   |           4.36167e-05 |
|   1379 | /                        | 31.14.72.17     |           4.36167e-05 |
|   1380 | /                        | 40.128.39.234   |           4.36167e-05 |
|   1381 | /                        | 45.131.192.211  |           4.36167e-05 |
|   1382 | /                        | 45.131.192.220  |           4.36167e-05 |
|   1383 | /                        | 45.131.192.48   |           4.36167e-05 |
|   1384 | /                        | 45.22.96.223    |           4.36167e-05 |
|   1385 | /                        | 45.27.14.73     |           4.36167e-05 |
|   1386 | /                        | 45.49.32.46     |           4.36167e-05 |
|   1387 | /                        | 45.89.173.198   |           4.36167e-05 |
|   1388 | /                        | 47.183.143.86   |           4.36167e-05 |
|   1389 | /                        | 47.187.229.105  |           4.36167e-05 |
|   1390 | /                        | 47.187.237.169  |           4.36167e-05 |
|   1391 | /                        | 47.187.245.12   |           4.36167e-05 |
|   1392 | /                        | 47.189.38.25    |           4.36167e-05 |
|   1393 | /                        | 47.234.188.230  |           4.36167e-05 |
|   1394 | /                        | 50.24.135.88    |           4.36167e-05 |
|   1395 | /                        | 50.27.36.194    |           4.36167e-05 |
|   1396 | /                        | 50.84.78.82     |           4.36167e-05 |
|   1397 | /                        | 52.176.52.196   |           4.36167e-05 |
|   1398 | /                        | 56.0.84.25      |           4.36167e-05 |
|   1399 | /                        | 64.132.229.66   |           4.36167e-05 |
|   1400 | /                        | 64.134.148.200  |           4.36167e-05 |
|   1401 | /                        | 64.134.149.87   |           4.36167e-05 |
|   1402 | /                        | 64.134.159.142  |           4.36167e-05 |
|   1403 | /                        | 65.43.115.179   |           4.36167e-05 |
|   1404 | /                        | 66.102.6.9      |           4.36167e-05 |
|   1405 | /                        | 66.193.245.94   |           4.36167e-05 |
|   1406 | /                        | 66.25.130.242   |           4.36167e-05 |
|   1407 | /                        | 66.25.163.226   |           4.36167e-05 |
|   1408 | /                        | 66.25.165.226   |           4.36167e-05 |
|   1409 | /                        | 66.25.169.158   |           4.36167e-05 |
|   1410 | /                        | 66.25.171.170   |           4.36167e-05 |
|   1411 | /                        | 66.25.175.197   |           4.36167e-05 |
|   1412 | /                        | 66.69.10.7      |           4.36167e-05 |
|   1413 | /                        | 66.69.122.35    |           4.36167e-05 |
|   1414 | /                        | 66.69.124.135   |           4.36167e-05 |
|   1415 | /                        | 66.69.49.204    |           4.36167e-05 |
|   1416 | /                        | 66.69.50.45     |           4.36167e-05 |
|   1417 | /                        | 66.69.6.234     |           4.36167e-05 |
|   1418 | /                        | 66.69.65.3      |           4.36167e-05 |
|   1419 | /                        | 67.10.175.200   |           4.36167e-05 |
|   1420 | /                        | 67.10.183.5     |           4.36167e-05 |
|   1421 | /                        | 67.10.191.167   |           4.36167e-05 |
|   1422 | /                        | 67.11.185.253   |           4.36167e-05 |
|   1423 | /                        | 67.11.208.216   |           4.36167e-05 |
|   1424 | /                        | 67.11.224.135   |           4.36167e-05 |
|   1425 | /                        | 67.11.33.106    |           4.36167e-05 |
|   1426 | /                        | 67.11.39.16     |           4.36167e-05 |
|   1427 | /                        | 67.11.41.202    |           4.36167e-05 |
|   1428 | /                        | 67.11.97.204    |           4.36167e-05 |
|   1429 | /                        | 67.142.235.252  |           4.36167e-05 |
|   1430 | /                        | 67.7.6.9        |           4.36167e-05 |
|   1431 | /                        | 67.79.43.254    |           4.36167e-05 |
|   1432 | /                        | 68.203.222.3    |           4.36167e-05 |
|   1433 | /                        | 68.250.43.43    |           4.36167e-05 |
|   1434 | /                        | 69.166.71.6     |           4.36167e-05 |
|   1435 | /                        | 69.166.89.120   |           4.36167e-05 |
|   1436 | /                        | 69.193.187.30   |           4.36167e-05 |
|   1437 | /                        | 69.236.100.33   |           4.36167e-05 |
|   1438 | /                        | 70.112.212.15   |           4.36167e-05 |
|   1439 | /                        | 70.114.2.244    |           4.36167e-05 |
|   1440 | /                        | 70.114.7.210    |           4.36167e-05 |
|   1441 | /                        | 70.115.158.21   |           4.36167e-05 |
|   1442 | /                        | 70.117.13.151   |           4.36167e-05 |
|   1443 | /                        | 70.117.31.37    |           4.36167e-05 |
|   1444 | /                        | 70.118.34.50    |           4.36167e-05 |
|   1445 | /                        | 70.121.192.194  |           4.36167e-05 |
|   1446 | /                        | 70.121.199.78   |           4.36167e-05 |
|   1447 | /                        | 70.123.193.100  |           4.36167e-05 |
|   1448 | /                        | 70.123.198.193  |           4.36167e-05 |
|   1449 | /                        | 70.123.200.145  |           4.36167e-05 |
|   1450 | /                        | 70.123.219.174  |           4.36167e-05 |
|   1451 | /                        | 70.123.224.242  |           4.36167e-05 |
|   1452 | /                        | 70.123.228.86   |           4.36167e-05 |
|   1453 | /                        | 70.124.183.93   |           4.36167e-05 |
|   1454 | /                        | 70.125.144.209  |           4.36167e-05 |
|   1455 | /                        | 70.130.122.154  |           4.36167e-05 |
|   1456 | /                        | 70.40.28.157    |           4.36167e-05 |
|   1457 | /                        | 70.94.144.103   |           4.36167e-05 |
|   1458 | /                        | 70.94.153.55    |           4.36167e-05 |
|   1459 | /                        | 70.94.164.72    |           4.36167e-05 |
|   1460 | /                        | 71.136.254.125  |           4.36167e-05 |
|   1461 | /                        | 71.137.144.185  |           4.36167e-05 |
|   1462 | /                        | 71.143.84.242   |           4.36167e-05 |
|   1463 | /                        | 71.158.153.146  |           4.36167e-05 |
|   1464 | /                        | 71.40.81.37     |           4.36167e-05 |
|   1465 | /                        | 71.41.191.147   |           4.36167e-05 |
|   1466 | /                        | 71.62.94.245    |           4.36167e-05 |
|   1467 | /                        | 71.78.115.66    |           4.36167e-05 |
|   1468 | /                        | 71.78.62.179    |           4.36167e-05 |
|   1469 | /                        | 72.173.98.235   |           4.36167e-05 |
|   1470 | /                        | 72.177.141.148  |           4.36167e-05 |
|   1471 | /                        | 72.177.185.29   |           4.36167e-05 |
|   1472 | /                        | 72.177.208.53   |           4.36167e-05 |
|   1473 | /                        | 72.178.121.254  |           4.36167e-05 |
|   1474 | /                        | 72.179.168.148  |           4.36167e-05 |
|   1475 | /                        | 72.180.103.93   |           4.36167e-05 |
|   1476 | /                        | 72.180.51.14    |           4.36167e-05 |
|   1477 | /                        | 72.181.105.69   |           4.36167e-05 |
|   1478 | /                        | 72.181.146.118  |           4.36167e-05 |
|   1479 | /                        | 72.183.145.159  |           4.36167e-05 |
|   1480 | /                        | 72.190.151.50   |           4.36167e-05 |
|   1481 | /                        | 72.190.171.65   |           4.36167e-05 |
|   1482 | /                        | 72.190.172.10   |           4.36167e-05 |
|   1483 | /                        | 72.191.11.178   |           4.36167e-05 |
|   1484 | /                        | 72.26.31.59     |           4.36167e-05 |
|   1485 | /                        | 73.219.222.176  |           4.36167e-05 |
|   1486 | /                        | 73.34.177.179   |           4.36167e-05 |
|   1487 | /                        | 73.76.225.232   |           4.36167e-05 |
|   1488 | /                        | 74.50.118.217   |           4.36167e-05 |
|   1489 | /                        | 74.81.88.66     |           4.36167e-05 |
|   1490 | /                        | 75.166.30.247   |           4.36167e-05 |
|   1491 | /                        | 75.41.2.91      |           4.36167e-05 |
|   1492 | /                        | 75.54.32.21     |           4.36167e-05 |
|   1493 | /                        | 75.83.236.223   |           4.36167e-05 |
|   1494 | /                        | 75.87.2.162     |           4.36167e-05 |
|   1495 | /                        | 76.184.52.198   |           4.36167e-05 |
|   1496 | /                        | 76.185.121.198  |           4.36167e-05 |
|   1497 | /                        | 76.185.149.233  |           4.36167e-05 |
|   1498 | /                        | 76.185.173.161  |           4.36167e-05 |
|   1499 | /                        | 76.185.174.229  |           4.36167e-05 |
|   1500 | /                        | 76.185.196.76   |           4.36167e-05 |
|   1501 | /                        | 76.201.6.236    |           4.36167e-05 |
|   1502 | /                        | 76.204.29.215   |           4.36167e-05 |
|   1503 | /                        | 89.187.164.130  |           4.36167e-05 |
|   1504 | /                        | 89.187.164.148  |           4.36167e-05 |
|   1505 | /                        | 89.187.164.167  |           4.36167e-05 |
|   1506 | /                        | 89.187.164.74   |           4.36167e-05 |
|   1507 | /                        | 89.187.164.76   |           4.36167e-05 |
|   1508 | /                        | 89.187.164.77   |           4.36167e-05 |
|   1509 | /                        | 89.187.175.150  |           4.36167e-05 |
|   1510 | /                        | 89.187.175.174  |           4.36167e-05 |
|   1511 | /                        | 89.187.175.23   |           4.36167e-05 |
|   1512 | /                        | 89.187.177.72   |           4.36167e-05 |
|   1513 | /                        | 96.8.130.148    |           4.36167e-05 |
|   1514 | /                        | 96.8.130.21     |           4.36167e-05 |
|   1515 | /                        | 96.8.130.252    |           4.36167e-05 |
|   1516 | /                        | 96.8.131.77     |           4.36167e-05 |
|   1517 | /                        | 96.8.139.196    |           4.36167e-05 |
|   1518 | /                        | 96.8.168.216    |           4.36167e-05 |
|   1519 | /                        | 96.8.178.42     |           4.36167e-05 |
|   1520 | /                        | 96.8.178.61     |           4.36167e-05 |
|   1521 | /                        | 96.8.248.149    |           4.36167e-05 |
|   1522 | /                        | 97.105.137.42   |           4.36167e-05 |
|   1523 | /                        | 97.105.154.139  |           4.36167e-05 |
|   1524 | /                        | 97.105.165.178  |           4.36167e-05 |
|   1525 | /                        | 97.117.115.55   |           4.36167e-05 |
|   1526 | /                        | 97.77.214.243   |           4.36167e-05 |
|   1527 | /                        | 97.98.14.0      |           4.36167e-05 |
|   1528 | /                        | 97.99.239.216   |           4.36167e-05 |
|   1529 | /                        | 98.6.204.179    |           4.36167e-05 |
|   1530 | /                        | 98.6.52.59      |           4.36167e-05 |
|   1531 | /                        | 99.147.232.101  |           4.36167e-05 |
|   1532 | /                        | 99.184.63.30    |           4.36167e-05 |
|   1533 | /                        | 99.203.154.149  |           4.36167e-05 |
|   1534 | /                        | 99.203.155.223  |           4.36167e-05 |
|   1535 | /                        | 99.203.212.98   |           4.36167e-05 |
|   1536 | /                        | 99.203.23.196   |           4.36167e-05 |
|   1537 | /                        | 99.203.26.185   |           4.36167e-05 |
|   1538 | /                        | 99.203.26.250   |           4.36167e-05 |
|   1539 | /                        | 99.203.26.253   |           4.36167e-05 |
|   1540 | /                        | 99.203.26.254   |           4.36167e-05 |
|   1541 | /                        | 99.203.27.106   |           4.36167e-05 |
|   1542 | /                        | 99.203.27.131   |           4.36167e-05 |
|   1543 | /                        | 99.203.27.142   |           4.36167e-05 |
|   1544 | /                        | 99.203.91.15    |           4.36167e-05 |
|   1545 | /                        | 99.41.196.68    |           4.36167e-05 |
|   1546 | /                        | 99.43.106.116   |           4.36167e-05 |
|   1547 | /                        | 99.52.75.127    |           4.36167e-05 |
|   1548 | /                        | 99.6.251.67     |           4.36167e-05 |
|   1549 | /                        | 99.97.210.12    |           4.36167e-05 |
|   1550 | /                        | 99.98.32.97     |           4.36167e-05 |
|   1551 | /                        | 100.35.67.162   |           2.18083e-05 |
|   1552 | /                        | 104.148.30.54   |           2.18083e-05 |
|   1553 | /                        | 104.180.80.54   |           2.18083e-05 |
|   1554 | /                        | 104.182.14.96   |           2.18083e-05 |
|   1555 | /                        | 104.184.230.44  |           2.18083e-05 |
|   1556 | /                        | 104.184.27.54   |           2.18083e-05 |
|   1557 | /                        | 104.187.116.12  |           2.18083e-05 |
|   1558 | /                        | 104.190.247.231 |           2.18083e-05 |
|   1559 | /                        | 104.190.250.60  |           2.18083e-05 |
|   1560 | /                        | 104.191.1.102   |           2.18083e-05 |
|   1561 | /                        | 104.192.171.99  |           2.18083e-05 |
|   1562 | /                        | 104.200.138.33  |           2.18083e-05 |
|   1563 | /                        | 104.218.128.194 |           2.18083e-05 |
|   1564 | /                        | 104.218.78.12   |           2.18083e-05 |
|   1565 | /                        | 104.219.136.105 |           2.18083e-05 |
|   1566 | /                        | 104.223.91.53   |           2.18083e-05 |
|   1567 | /                        | 104.237.80.102  |           2.18083e-05 |
|   1568 | /                        | 104.237.80.109  |           2.18083e-05 |
|   1569 | /                        | 104.237.80.11   |           2.18083e-05 |
|   1570 | /                        | 104.237.80.12   |           2.18083e-05 |
|   1571 | /                        | 104.237.80.140  |           2.18083e-05 |
|   1572 | /                        | 104.237.80.141  |           2.18083e-05 |
|   1573 | /                        | 104.237.80.15   |           2.18083e-05 |
|   1574 | /                        | 104.237.80.151  |           2.18083e-05 |
|   1575 | /                        | 104.237.80.167  |           2.18083e-05 |
|   1576 | /                        | 104.237.80.21   |           2.18083e-05 |
|   1577 | /                        | 104.237.80.53   |           2.18083e-05 |
|   1578 | /                        | 104.237.80.59   |           2.18083e-05 |
|   1579 | /                        | 104.237.80.77   |           2.18083e-05 |
|   1580 | /                        | 104.237.80.81   |           2.18083e-05 |
|   1581 | /                        | 104.237.80.83   |           2.18083e-05 |
|   1582 | /                        | 104.238.45.6    |           2.18083e-05 |
|   1583 | /                        | 104.254.95.84   |           2.18083e-05 |
|   1584 | /                        | 104.37.31.16    |           2.18083e-05 |
|   1585 | /                        | 104.37.31.8     |           2.18083e-05 |
|   1586 | /                        | 104.48.213.50   |           2.18083e-05 |
|   1587 | /                        | 104.5.159.83    |           2.18083e-05 |
|   1588 | /                        | 104.5.221.205   |           2.18083e-05 |
|   1589 | /                        | 104.53.200.122  |           2.18083e-05 |
|   1590 | /                        | 107.117.224.158 |           2.18083e-05 |
|   1591 | /                        | 107.117.224.88  |           2.18083e-05 |
|   1592 | /                        | 107.117.224.90  |           2.18083e-05 |
|   1593 | /                        | 107.117.225.105 |           2.18083e-05 |
|   1594 | /                        | 107.127.0.122   |           2.18083e-05 |
|   1595 | /                        | 107.127.0.123   |           2.18083e-05 |
|   1596 | /                        | 107.127.0.22    |           2.18083e-05 |
|   1597 | /                        | 107.127.0.39    |           2.18083e-05 |
|   1598 | /                        | 107.127.0.89    |           2.18083e-05 |
|   1599 | /                        | 107.127.0.90    |           2.18083e-05 |
|   1600 | /                        | 107.132.199.23  |           2.18083e-05 |
|   1601 | /                        | 107.136.101.76  |           2.18083e-05 |
|   1602 | /                        | 107.152.102.198 |           2.18083e-05 |
|   1603 | /                        | 107.158.15.51   |           2.18083e-05 |
|   1604 | /                        | 107.167.109.95  |           2.18083e-05 |
|   1605 | /                        | 107.191.1.252   |           2.18083e-05 |
|   1606 | /                        | 107.200.109.74  |           2.18083e-05 |
|   1607 | /                        | 107.204.49.22   |           2.18083e-05 |
|   1608 | /                        | 107.207.1.245   |           2.18083e-05 |
|   1609 | /                        | 107.207.7.120   |           2.18083e-05 |
|   1610 | /                        | 107.242.125.32  |           2.18083e-05 |
|   1611 | /                        | 107.242.125.38  |           2.18083e-05 |
|   1612 | /                        | 107.242.125.50  |           2.18083e-05 |
|   1613 | /                        | 107.242.125.8   |           2.18083e-05 |
|   1614 | /                        | 107.77.100.113  |           2.18083e-05 |
|   1615 | /                        | 107.77.100.128  |           2.18083e-05 |
|   1616 | /                        | 107.77.100.129  |           2.18083e-05 |
|   1617 | /                        | 107.77.100.18   |           2.18083e-05 |
|   1618 | /                        | 107.77.100.48   |           2.18083e-05 |
|   1619 | /                        | 107.77.100.79   |           2.18083e-05 |
|   1620 | /                        | 107.77.105.28   |           2.18083e-05 |
|   1621 | /                        | 107.77.105.45   |           2.18083e-05 |
|   1622 | /                        | 107.77.105.75   |           2.18083e-05 |
|   1623 | /                        | 107.77.105.78   |           2.18083e-05 |
|   1624 | /                        | 107.77.105.92   |           2.18083e-05 |
|   1625 | /                        | 107.77.169.1    |           2.18083e-05 |
|   1626 | /                        | 107.77.169.13   |           2.18083e-05 |
|   1627 | /                        | 107.77.169.47   |           2.18083e-05 |
|   1628 | /                        | 107.77.169.55   |           2.18083e-05 |
|   1629 | /                        | 107.77.169.6    |           2.18083e-05 |
|   1630 | /                        | 107.77.169.7    |           2.18083e-05 |
|   1631 | /                        | 107.77.173.2    |           2.18083e-05 |
|   1632 | /                        | 107.77.173.9    |           2.18083e-05 |
|   1633 | /                        | 107.77.189.16   |           2.18083e-05 |
|   1634 | /                        | 107.77.189.17   |           2.18083e-05 |
|   1635 | /                        | 107.77.189.21   |           2.18083e-05 |
|   1636 | /                        | 107.77.189.23   |           2.18083e-05 |
|   1637 | /                        | 107.77.189.27   |           2.18083e-05 |
|   1638 | /                        | 107.77.189.37   |           2.18083e-05 |
|   1639 | /                        | 107.77.189.49   |           2.18083e-05 |
|   1640 | /                        | 107.77.189.50   |           2.18083e-05 |
|   1641 | /                        | 107.77.189.62   |           2.18083e-05 |
|   1642 | /                        | 107.77.189.63   |           2.18083e-05 |
|   1643 | /                        | 107.77.189.64   |           2.18083e-05 |
|   1644 | /                        | 107.77.196.224  |           2.18083e-05 |
|   1645 | /                        | 107.77.196.228  |           2.18083e-05 |
|   1646 | /                        | 107.77.196.38   |           2.18083e-05 |
|   1647 | /                        | 107.77.196.71   |           2.18083e-05 |
|   1648 | /                        | 107.77.197.208  |           2.18083e-05 |
|   1649 | /                        | 107.77.197.78   |           2.18083e-05 |
|   1650 | /                        | 107.77.198.116  |           2.18083e-05 |
|   1651 | /                        | 107.77.198.136  |           2.18083e-05 |
|   1652 | /                        | 107.77.198.169  |           2.18083e-05 |
|   1653 | /                        | 107.77.198.228  |           2.18083e-05 |
|   1654 | /                        | 107.77.200.140  |           2.18083e-05 |
|   1655 | /                        | 107.77.200.166  |           2.18083e-05 |
|   1656 | /                        | 107.77.200.202  |           2.18083e-05 |
|   1657 | /                        | 107.77.200.229  |           2.18083e-05 |
|   1658 | /                        | 107.77.200.83   |           2.18083e-05 |
|   1659 | /                        | 107.77.201.153  |           2.18083e-05 |
|   1660 | /                        | 107.77.201.165  |           2.18083e-05 |
|   1661 | /                        | 107.77.201.226  |           2.18083e-05 |
|   1662 | /                        | 107.77.201.52   |           2.18083e-05 |
|   1663 | /                        | 107.77.208.117  |           2.18083e-05 |
|   1664 | /                        | 107.77.208.209  |           2.18083e-05 |
|   1665 | /                        | 107.77.217.101  |           2.18083e-05 |
|   1666 | /                        | 107.77.217.106  |           2.18083e-05 |
|   1667 | /                        | 107.77.217.115  |           2.18083e-05 |
|   1668 | /                        | 107.77.217.123  |           2.18083e-05 |
|   1669 | /                        | 107.77.217.127  |           2.18083e-05 |
|   1670 | /                        | 107.77.217.151  |           2.18083e-05 |
|   1671 | /                        | 107.77.217.152  |           2.18083e-05 |
|   1672 | /                        | 107.77.217.168  |           2.18083e-05 |
|   1673 | /                        | 107.77.217.195  |           2.18083e-05 |
|   1674 | /                        | 107.77.217.196  |           2.18083e-05 |
|   1675 | /                        | 107.77.217.200  |           2.18083e-05 |
|   1676 | /                        | 107.77.217.229  |           2.18083e-05 |
|   1677 | /                        | 107.77.217.37   |           2.18083e-05 |
|   1678 | /                        | 107.77.217.58   |           2.18083e-05 |
|   1679 | /                        | 107.77.217.65   |           2.18083e-05 |
|   1680 | /                        | 107.77.217.69   |           2.18083e-05 |
|   1681 | /                        | 107.77.218.115  |           2.18083e-05 |
|   1682 | /                        | 107.77.218.122  |           2.18083e-05 |
|   1683 | /                        | 107.77.218.126  |           2.18083e-05 |
|   1684 | /                        | 107.77.218.136  |           2.18083e-05 |
|   1685 | /                        | 107.77.218.139  |           2.18083e-05 |
|   1686 | /                        | 107.77.218.153  |           2.18083e-05 |
|   1687 | /                        | 107.77.218.154  |           2.18083e-05 |
|   1688 | /                        | 107.77.218.167  |           2.18083e-05 |
|   1689 | /                        | 107.77.218.168  |           2.18083e-05 |
|   1690 | /                        | 107.77.218.195  |           2.18083e-05 |
|   1691 | /                        | 107.77.218.199  |           2.18083e-05 |
|   1692 | /                        | 107.77.218.203  |           2.18083e-05 |
|   1693 | /                        | 107.77.218.226  |           2.18083e-05 |
|   1694 | /                        | 107.77.218.230  |           2.18083e-05 |
|   1695 | /                        | 107.77.218.55   |           2.18083e-05 |
|   1696 | /                        | 107.77.218.58   |           2.18083e-05 |
|   1697 | /                        | 107.77.218.65   |           2.18083e-05 |
|   1698 | /                        | 107.77.218.8    |           2.18083e-05 |
|   1699 | /                        | 107.77.218.83   |           2.18083e-05 |
|   1700 | /                        | 107.77.218.91   |           2.18083e-05 |
|   1701 | /                        | 107.77.219.100  |           2.18083e-05 |
|   1702 | /                        | 107.77.219.108  |           2.18083e-05 |
|   1703 | /                        | 107.77.219.114  |           2.18083e-05 |
|   1704 | /                        | 107.77.219.13   |           2.18083e-05 |
|   1705 | /                        | 107.77.219.187  |           2.18083e-05 |
|   1706 | /                        | 107.77.219.189  |           2.18083e-05 |
|   1707 | /                        | 107.77.219.19   |           2.18083e-05 |
|   1708 | /                        | 107.77.219.194  |           2.18083e-05 |
|   1709 | /                        | 107.77.219.202  |           2.18083e-05 |
|   1710 | /                        | 107.77.219.217  |           2.18083e-05 |
|   1711 | /                        | 107.77.219.219  |           2.18083e-05 |
|   1712 | /                        | 107.77.219.22   |           2.18083e-05 |
|   1713 | /                        | 107.77.219.39   |           2.18083e-05 |
|   1714 | /                        | 107.77.219.48   |           2.18083e-05 |
|   1715 | /                        | 107.77.219.66   |           2.18083e-05 |
|   1716 | /                        | 107.77.219.83   |           2.18083e-05 |
|   1717 | /                        | 107.77.220.106  |           2.18083e-05 |
|   1718 | /                        | 107.77.220.107  |           2.18083e-05 |
|   1719 | /                        | 107.77.220.108  |           2.18083e-05 |
|   1720 | /                        | 107.77.220.12   |           2.18083e-05 |
|   1721 | /                        | 107.77.220.123  |           2.18083e-05 |
|   1722 | /                        | 107.77.220.125  |           2.18083e-05 |
|   1723 | /                        | 107.77.220.154  |           2.18083e-05 |
|   1724 | /                        | 107.77.220.176  |           2.18083e-05 |
|   1725 | /                        | 107.77.220.190  |           2.18083e-05 |
|   1726 | /                        | 107.77.220.191  |           2.18083e-05 |
|   1727 | /                        | 107.77.220.195  |           2.18083e-05 |
|   1728 | /                        | 107.77.220.202  |           2.18083e-05 |
|   1729 | /                        | 107.77.220.21   |           2.18083e-05 |
|   1730 | /                        | 107.77.220.215  |           2.18083e-05 |
|   1731 | /                        | 107.77.220.231  |           2.18083e-05 |
|   1732 | /                        | 107.77.220.26   |           2.18083e-05 |
|   1733 | /                        | 107.77.220.42   |           2.18083e-05 |
|   1734 | /                        | 107.77.220.47   |           2.18083e-05 |
|   1735 | /                        | 107.77.220.51   |           2.18083e-05 |
|   1736 | /                        | 107.77.220.64   |           2.18083e-05 |
|   1737 | /                        | 107.77.220.81   |           2.18083e-05 |
|   1738 | /                        | 107.77.220.86   |           2.18083e-05 |
|   1739 | /                        | 107.77.220.90   |           2.18083e-05 |
|   1740 | /                        | 107.77.220.95   |           2.18083e-05 |
|   1741 | /                        | 107.77.221.10   |           2.18083e-05 |
|   1742 | /                        | 107.77.221.101  |           2.18083e-05 |
|   1743 | /                        | 107.77.221.107  |           2.18083e-05 |
|   1744 | /                        | 107.77.221.13   |           2.18083e-05 |
|   1745 | /                        | 107.77.221.141  |           2.18083e-05 |
|   1746 | /                        | 107.77.221.152  |           2.18083e-05 |
|   1747 | /                        | 107.77.221.153  |           2.18083e-05 |
|   1748 | /                        | 107.77.221.173  |           2.18083e-05 |
|   1749 | /                        | 107.77.221.19   |           2.18083e-05 |
|   1750 | /                        | 107.77.221.194  |           2.18083e-05 |
|   1751 | /                        | 107.77.221.200  |           2.18083e-05 |
|   1752 | /                        | 107.77.221.202  |           2.18083e-05 |
|   1753 | /                        | 107.77.221.203  |           2.18083e-05 |
|   1754 | /                        | 107.77.221.210  |           2.18083e-05 |
|   1755 | /                        | 107.77.221.22   |           2.18083e-05 |
|   1756 | /                        | 107.77.221.39   |           2.18083e-05 |
|   1757 | /                        | 107.77.221.40   |           2.18083e-05 |
|   1758 | /                        | 107.77.221.49   |           2.18083e-05 |
|   1759 | /                        | 107.77.221.57   |           2.18083e-05 |
|   1760 | /                        | 107.77.221.7    |           2.18083e-05 |
|   1761 | /                        | 107.77.221.82   |           2.18083e-05 |
|   1762 | /                        | 107.77.221.91   |           2.18083e-05 |
|   1763 | /                        | 107.77.222.108  |           2.18083e-05 |
|   1764 | /                        | 107.77.222.115  |           2.18083e-05 |
|   1765 | /                        | 107.77.222.125  |           2.18083e-05 |
|   1766 | /                        | 107.77.222.127  |           2.18083e-05 |
|   1767 | /                        | 107.77.222.128  |           2.18083e-05 |
|   1768 | /                        | 107.77.222.13   |           2.18083e-05 |
|   1769 | /                        | 107.77.222.131  |           2.18083e-05 |
|   1770 | /                        | 107.77.222.132  |           2.18083e-05 |
|   1771 | /                        | 107.77.222.138  |           2.18083e-05 |
|   1772 | /                        | 107.77.222.141  |           2.18083e-05 |
|   1773 | /                        | 107.77.222.142  |           2.18083e-05 |
|   1774 | /                        | 107.77.222.153  |           2.18083e-05 |
|   1775 | /                        | 107.77.222.159  |           2.18083e-05 |
|   1776 | /                        | 107.77.222.165  |           2.18083e-05 |
|   1777 | /                        | 107.77.222.168  |           2.18083e-05 |
|   1778 | /                        | 107.77.222.17   |           2.18083e-05 |
|   1779 | /                        | 107.77.222.175  |           2.18083e-05 |
|   1780 | /                        | 107.77.222.18   |           2.18083e-05 |
|   1781 | /                        | 107.77.222.187  |           2.18083e-05 |
|   1782 | /                        | 107.77.222.195  |           2.18083e-05 |
|   1783 | /                        | 107.77.222.211  |           2.18083e-05 |
|   1784 | /                        | 107.77.222.216  |           2.18083e-05 |
|   1785 | /                        | 107.77.222.219  |           2.18083e-05 |
|   1786 | /                        | 107.77.222.31   |           2.18083e-05 |
|   1787 | /                        | 107.77.222.37   |           2.18083e-05 |
|   1788 | /                        | 107.77.222.41   |           2.18083e-05 |
|   1789 | /                        | 107.77.222.63   |           2.18083e-05 |
|   1790 | /                        | 107.77.222.8    |           2.18083e-05 |
|   1791 | /                        | 107.77.234.226  |           2.18083e-05 |
|   1792 | /                        | 107.77.236.53   |           2.18083e-05 |
|   1793 | /                        | 107.77.245.7    |           2.18083e-05 |
|   1794 | /                        | 107.77.64.130   |           2.18083e-05 |
|   1795 | /                        | 107.77.64.21    |           2.18083e-05 |
|   1796 | /                        | 107.77.64.27    |           2.18083e-05 |
|   1797 | /                        | 107.77.64.50    |           2.18083e-05 |
|   1798 | /                        | 107.77.64.52    |           2.18083e-05 |
|   1799 | /                        | 107.77.66.121   |           2.18083e-05 |
|   1800 | /                        | 107.77.66.26    |           2.18083e-05 |
|   1801 | /                        | 107.77.66.54    |           2.18083e-05 |
|   1802 | /                        | 107.77.66.78    |           2.18083e-05 |
|   1803 | /                        | 107.77.66.81    |           2.18083e-05 |
|   1804 | /                        | 107.77.72.112   |           2.18083e-05 |
|   1805 | /                        | 107.77.72.127   |           2.18083e-05 |
|   1806 | /                        | 107.77.72.64    |           2.18083e-05 |
|   1807 | /                        | 107.77.72.90    |           2.18083e-05 |
|   1808 | /                        | 107.77.72.97    |           2.18083e-05 |
|   1809 | /                        | 107.77.80.107   |           2.18083e-05 |
|   1810 | /                        | 107.77.80.125   |           2.18083e-05 |
|   1811 | /                        | 107.77.80.128   |           2.18083e-05 |
|   1812 | /                        | 107.77.80.20    |           2.18083e-05 |
|   1813 | /                        | 107.77.80.29    |           2.18083e-05 |
|   1814 | /                        | 107.77.80.82    |           2.18083e-05 |
|   1815 | /                        | 107.77.80.83    |           2.18083e-05 |
|   1816 | /                        | 107.77.80.85    |           2.18083e-05 |
|   1817 | /                        | 107.77.80.91    |           2.18083e-05 |
|   1818 | /                        | 107.77.94.128   |           2.18083e-05 |
|   1819 | /                        | 107.77.94.80    |           2.18083e-05 |
|   1820 | /                        | 108.119.105.160 |           2.18083e-05 |
|   1821 | /                        | 108.178.71.190  |           2.18083e-05 |
|   1822 | /                        | 108.178.77.202  |           2.18083e-05 |
|   1823 | /                        | 108.18.249.150  |           2.18083e-05 |
|   1824 | /                        | 108.199.117.106 |           2.18083e-05 |
|   1825 | /                        | 108.200.115.87  |           2.18083e-05 |
|   1826 | /                        | 108.205.6.177   |           2.18083e-05 |
|   1827 | /                        | 108.213.191.10  |           2.18083e-05 |
|   1828 | /                        | 108.218.193.28  |           2.18083e-05 |
|   1829 | /                        | 108.247.106.74  |           2.18083e-05 |
|   1830 | /                        | 108.249.226.232 |           2.18083e-05 |
|   1831 | /                        | 108.59.0.37     |           2.18083e-05 |
|   1832 | /                        | 108.7.195.50    |           2.18083e-05 |
|   1833 | /                        | 108.77.11.169   |           2.18083e-05 |
|   1834 | /                        | 108.93.117.34   |           2.18083e-05 |
|   1835 | /                        | 108.93.12.131   |           2.18083e-05 |
|   1836 | /                        | 110.78.138.179  |           2.18083e-05 |
|   1837 | /                        | 110.78.138.254  |           2.18083e-05 |
|   1838 | /                        | 12.131.11.26    |           2.18083e-05 |
|   1839 | /                        | 12.151.185.131  |           2.18083e-05 |
|   1840 | /                        | 12.154.218.253  |           2.18083e-05 |
|   1841 | /                        | 12.16.232.100   |           2.18083e-05 |
|   1842 | /                        | 12.162.134.81   |           2.18083e-05 |
|   1843 | /                        | 12.175.215.244  |           2.18083e-05 |
|   1844 | /                        | 12.197.199.226  |           2.18083e-05 |
|   1845 | /                        | 12.252.48.110   |           2.18083e-05 |
|   1846 | /                        | 12.27.152.137   |           2.18083e-05 |
|   1847 | /                        | 12.31.71.58     |           2.18083e-05 |
|   1848 | /                        | 12.33.49.18     |           2.18083e-05 |
|   1849 | /                        | 12.68.243.204   |           2.18083e-05 |
|   1850 | /                        | 12.71.182.66    |           2.18083e-05 |
|   1851 | /                        | 12.87.39.218    |           2.18083e-05 |
|   1852 | /                        | 12.97.187.12    |           2.18083e-05 |
|   1853 | /                        | 12.97.187.13    |           2.18083e-05 |
|   1854 | /                        | 12.97.187.15    |           2.18083e-05 |
|   1855 | /                        | 129.111.179.104 |           2.18083e-05 |
|   1856 | /                        | 129.111.179.55  |           2.18083e-05 |
|   1857 | /                        | 129.112.109.40  |           2.18083e-05 |
|   1858 | /                        | 129.112.109.42  |           2.18083e-05 |
|   1859 | /                        | 129.115.195.26  |           2.18083e-05 |
|   1860 | /                        | 129.115.195.45  |           2.18083e-05 |
|   1861 | /                        | 13.58.162.201   |           2.18083e-05 |
|   1862 | /                        | 130.45.25.120   |           2.18083e-05 |
|   1863 | /                        | 130.45.33.133   |           2.18083e-05 |
|   1864 | /                        | 130.45.52.50    |           2.18083e-05 |
|   1865 | /                        | 131.194.172.188 |           2.18083e-05 |
|   1866 | /                        | 131.194.172.221 |           2.18083e-05 |
|   1867 | /                        | 131.194.172.242 |           2.18083e-05 |
|   1868 | /                        | 131.194.172.30  |           2.18083e-05 |
|   1869 | /                        | 131.194.172.36  |           2.18083e-05 |
|   1870 | /                        | 131.194.172.51  |           2.18083e-05 |
|   1871 | /                        | 131.194.172.75  |           2.18083e-05 |
|   1872 | /                        | 131.194.173.118 |           2.18083e-05 |
|   1873 | /                        | 131.194.173.20  |           2.18083e-05 |
|   1874 | /                        | 131.194.173.46  |           2.18083e-05 |
|   1875 | /                        | 131.194.184.123 |           2.18083e-05 |
|   1876 | /                        | 131.194.184.74  |           2.18083e-05 |
|   1877 | /                        | 131.194.98.179  |           2.18083e-05 |
|   1878 | /                        | 134.250.1.82    |           2.18083e-05 |
|   1879 | /                        | 136.34.163.5    |           2.18083e-05 |
|   1880 | /                        | 136.49.100.118  |           2.18083e-05 |
|   1881 | /                        | 136.49.16.164   |           2.18083e-05 |
|   1882 | /                        | 136.49.177.117  |           2.18083e-05 |
|   1883 | /                        | 136.49.24.186   |           2.18083e-05 |
|   1884 | /                        | 136.50.0.214    |           2.18083e-05 |
|   1885 | /                        | 136.50.22.155   |           2.18083e-05 |
|   1886 | /                        | 136.50.54.38    |           2.18083e-05 |
|   1887 | /                        | 136.50.61.52    |           2.18083e-05 |
|   1888 | /                        | 136.50.75.159   |           2.18083e-05 |
|   1889 | /                        | 136.50.83.85    |           2.18083e-05 |
|   1890 | /                        | 136.50.89.106   |           2.18083e-05 |
|   1891 | /                        | 136.50.95.200   |           2.18083e-05 |
|   1892 | /                        | 136.62.156.201  |           2.18083e-05 |
|   1893 | /                        | 136.62.157.140  |           2.18083e-05 |
|   1894 | /                        | 136.62.18.97    |           2.18083e-05 |
|   1895 | /                        | 137.254.7.165   |           2.18083e-05 |
|   1896 | /                        | 137.254.7.169   |           2.18083e-05 |
|   1897 | /                        | 137.254.7.172   |           2.18083e-05 |
|   1898 | /                        | 137.254.7.173   |           2.18083e-05 |
|   1899 | /                        | 137.254.7.174   |           2.18083e-05 |
|   1900 | /                        | 137.254.7.176   |           2.18083e-05 |
|   1901 | /                        | 137.254.7.178   |           2.18083e-05 |
|   1902 | /                        | 137.254.7.184   |           2.18083e-05 |
|   1903 | /                        | 144.178.1.46    |           2.18083e-05 |
|   1904 | /                        | 144.178.1.54    |           2.18083e-05 |
|   1905 | /                        | 148.59.156.46   |           2.18083e-05 |
|   1906 | /                        | 148.66.38.18    |           2.18083e-05 |
|   1907 | /                        | 148.66.38.220   |           2.18083e-05 |
|   1908 | /                        | 148.87.23.18    |           2.18083e-05 |
|   1909 | /                        | 154.29.131.27   |           2.18083e-05 |
|   1910 | /                        | 154.3.250.108   |           2.18083e-05 |
|   1911 | /                        | 154.3.250.165   |           2.18083e-05 |
|   1912 | /                        | 154.3.250.64    |           2.18083e-05 |
|   1913 | /                        | 155.94.241.194  |           2.18083e-05 |
|   1914 | /                        | 155.95.87.116   |           2.18083e-05 |
|   1915 | /                        | 155.95.98.43    |           2.18083e-05 |
|   1916 | /                        | 155.95.98.61    |           2.18083e-05 |
|   1917 | /                        | 156.146.38.132  |           2.18083e-05 |
|   1918 | /                        | 156.146.43.65   |           2.18083e-05 |
|   1919 | /                        | 159.118.138.70  |           2.18083e-05 |
|   1920 | /                        | 159.192.243.186 |           2.18083e-05 |
|   1921 | /                        | 162.195.148.122 |           2.18083e-05 |
|   1922 | /                        | 162.197.40.243  |           2.18083e-05 |
|   1923 | /                        | 162.197.43.200  |           2.18083e-05 |
|   1924 | /                        | 162.200.113.53  |           2.18083e-05 |
|   1925 | /                        | 162.218.230.219 |           2.18083e-05 |
|   1926 | /                        | 162.222.246.181 |           2.18083e-05 |
|   1927 | /                        | 162.239.219.93  |           2.18083e-05 |
|   1928 | /                        | 162.245.202.3   |           2.18083e-05 |
|   1929 | /                        | 162.248.14.168  |           2.18083e-05 |
|   1930 | /                        | 162.250.18.82   |           2.18083e-05 |
|   1931 | /                        | 162.72.0.173    |           2.18083e-05 |
|   1932 | /                        | 162.72.2.198    |           2.18083e-05 |
|   1933 | /                        | 162.89.0.47     |           2.18083e-05 |
|   1934 | /                        | 165.225.48.84   |           2.18083e-05 |
|   1935 | /                        | 165.225.48.85   |           2.18083e-05 |
|   1936 | /                        | 165.225.48.86   |           2.18083e-05 |
|   1937 | /                        | 165.225.48.90   |           2.18083e-05 |
|   1938 | /                        | 165.225.48.93   |           2.18083e-05 |
|   1939 | /                        | 165.91.12.128   |           2.18083e-05 |
|   1940 | /                        | 166.137.118.110 |           2.18083e-05 |
|   1941 | /                        | 166.137.118.47  |           2.18083e-05 |
|   1942 | /                        | 166.137.118.83  |           2.18083e-05 |
|   1943 | /                        | 166.137.118.89  |           2.18083e-05 |
|   1944 | /                        | 166.137.125.28  |           2.18083e-05 |
|   1945 | /                        | 166.137.125.75  |           2.18083e-05 |
|   1946 | /                        | 166.137.126.119 |           2.18083e-05 |
|   1947 | /                        | 166.137.126.34  |           2.18083e-05 |
|   1948 | /                        | 166.137.136.50  |           2.18083e-05 |
|   1949 | /                        | 166.137.139.115 |           2.18083e-05 |
|   1950 | /                        | 166.137.139.56  |           2.18083e-05 |
|   1951 | /                        | 166.137.143.123 |           2.18083e-05 |
|   1952 | /                        | 166.141.90.7    |           2.18083e-05 |
|   1953 | /                        | 166.205.97.126  |           2.18083e-05 |
|   1954 | /                        | 166.247.169.116 |           2.18083e-05 |
|   1955 | /                        | 166.251.101.177 |           2.18083e-05 |
|   1956 | /                        | 166.251.101.2   |           2.18083e-05 |
|   1957 | /                        | 166.251.101.210 |           2.18083e-05 |
|   1958 | /                        | 166.251.101.233 |           2.18083e-05 |
|   1959 | /                        | 166.251.101.68  |           2.18083e-05 |
|   1960 | /                        | 166.251.106.160 |           2.18083e-05 |
|   1961 | /                        | 166.251.106.189 |           2.18083e-05 |
|   1962 | /                        | 166.251.106.194 |           2.18083e-05 |
|   1963 | /                        | 166.251.106.84  |           2.18083e-05 |
|   1964 | /                        | 166.251.109.196 |           2.18083e-05 |
|   1965 | /                        | 166.251.109.235 |           2.18083e-05 |
|   1966 | /                        | 166.251.109.241 |           2.18083e-05 |
|   1967 | /                        | 166.251.109.48  |           2.18083e-05 |
|   1968 | /                        | 166.251.109.68  |           2.18083e-05 |
|   1969 | /                        | 167.71.126.76   |           2.18083e-05 |
|   1970 | /                        | 167.99.96.214   |           2.18083e-05 |
|   1971 | /                        | 169.57.90.139   |           2.18083e-05 |
|   1972 | /                        | 170.130.1.67    |           2.18083e-05 |
|   1973 | /                        | 170.225.9.141   |           2.18083e-05 |
|   1974 | /                        | 172.127.182.0   |           2.18083e-05 |
|   1975 | /                        | 172.127.185.99  |           2.18083e-05 |
|   1976 | /                        | 172.223.38.19   |           2.18083e-05 |
|   1977 | /                        | 172.5.103.185   |           2.18083e-05 |
|   1978 | /                        | 172.56.14.200   |           2.18083e-05 |
|   1979 | /                        | 172.56.14.78    |           2.18083e-05 |
|   1980 | /                        | 172.56.15.135   |           2.18083e-05 |
|   1981 | /                        | 172.56.15.203   |           2.18083e-05 |
|   1982 | /                        | 172.56.15.46    |           2.18083e-05 |
|   1983 | /                        | 172.56.15.50    |           2.18083e-05 |
|   1984 | /                        | 172.56.15.51    |           2.18083e-05 |
|   1985 | /                        | 172.56.15.73    |           2.18083e-05 |
|   1986 | /                        | 172.56.6.112    |           2.18083e-05 |
|   1987 | /                        | 172.56.6.129    |           2.18083e-05 |
|   1988 | /                        | 172.56.6.130    |           2.18083e-05 |
|   1989 | /                        | 172.56.6.150    |           2.18083e-05 |
|   1990 | /                        | 172.56.6.151    |           2.18083e-05 |
|   1991 | /                        | 172.56.6.20     |           2.18083e-05 |
|   1992 | /                        | 172.56.6.247    |           2.18083e-05 |
|   1993 | /                        | 172.56.6.62     |           2.18083e-05 |
|   1994 | /                        | 172.56.6.79     |           2.18083e-05 |
|   1995 | /                        | 172.56.7.144    |           2.18083e-05 |
|   1996 | /                        | 172.56.7.16     |           2.18083e-05 |
|   1997 | /                        | 172.56.7.180    |           2.18083e-05 |
|   1998 | /                        | 172.56.7.21     |           2.18083e-05 |
|   1999 | /                        | 172.56.7.212    |           2.18083e-05 |
|   2000 | /                        | 172.56.7.218    |           2.18083e-05 |
|   2001 | /                        | 172.56.7.23     |           2.18083e-05 |
|   2002 | /                        | 172.56.7.232    |           2.18083e-05 |
|   2003 | /                        | 172.56.7.238    |           2.18083e-05 |
|   2004 | /                        | 172.56.7.241    |           2.18083e-05 |
|   2005 | /                        | 172.56.7.25     |           2.18083e-05 |
|   2006 | /                        | 172.56.7.46     |           2.18083e-05 |
|   2007 | /                        | 172.56.7.48     |           2.18083e-05 |
|   2008 | /                        | 172.56.7.56     |           2.18083e-05 |
|   2009 | /                        | 172.56.7.58     |           2.18083e-05 |
|   2010 | /                        | 172.58.100.107  |           2.18083e-05 |
|   2011 | /                        | 172.58.100.127  |           2.18083e-05 |
|   2012 | /                        | 172.58.100.199  |           2.18083e-05 |
|   2013 | /                        | 172.58.101.148  |           2.18083e-05 |
|   2014 | /                        | 172.58.101.208  |           2.18083e-05 |
|   2015 | /                        | 172.58.101.215  |           2.18083e-05 |
|   2016 | /                        | 172.58.101.45   |           2.18083e-05 |
|   2017 | /                        | 172.58.102.138  |           2.18083e-05 |
|   2018 | /                        | 172.58.102.140  |           2.18083e-05 |
|   2019 | /                        | 172.58.102.153  |           2.18083e-05 |
|   2020 | /                        | 172.58.102.155  |           2.18083e-05 |
|   2021 | /                        | 172.58.102.159  |           2.18083e-05 |
|   2022 | /                        | 172.58.102.174  |           2.18083e-05 |
|   2023 | /                        | 172.58.102.176  |           2.18083e-05 |
|   2024 | /                        | 172.58.102.188  |           2.18083e-05 |
|   2025 | /                        | 172.58.102.201  |           2.18083e-05 |
|   2026 | /                        | 172.58.102.207  |           2.18083e-05 |
|   2027 | /                        | 172.58.102.220  |           2.18083e-05 |
|   2028 | /                        | 172.58.102.222  |           2.18083e-05 |
|   2029 | /                        | 172.58.102.226  |           2.18083e-05 |
|   2030 | /                        | 172.58.102.228  |           2.18083e-05 |
|   2031 | /                        | 172.58.102.230  |           2.18083e-05 |
|   2032 | /                        | 172.58.102.231  |           2.18083e-05 |
|   2033 | /                        | 172.58.102.235  |           2.18083e-05 |
|   2034 | /                        | 172.58.102.240  |           2.18083e-05 |
|   2035 | /                        | 172.58.102.250  |           2.18083e-05 |
|   2036 | /                        | 172.58.103.143  |           2.18083e-05 |
|   2037 | /                        | 172.58.103.188  |           2.18083e-05 |
|   2038 | /                        | 172.58.103.55   |           2.18083e-05 |
|   2039 | /                        | 172.58.105.205  |           2.18083e-05 |
|   2040 | /                        | 172.58.106.143  |           2.18083e-05 |
|   2041 | /                        | 172.58.106.58   |           2.18083e-05 |
|   2042 | /                        | 172.58.107.104  |           2.18083e-05 |
|   2043 | /                        | 172.58.107.117  |           2.18083e-05 |
|   2044 | /                        | 172.58.107.129  |           2.18083e-05 |
|   2045 | /                        | 172.58.107.177  |           2.18083e-05 |
|   2046 | /                        | 172.58.107.178  |           2.18083e-05 |
|   2047 | /                        | 172.58.107.190  |           2.18083e-05 |
|   2048 | /                        | 172.58.107.204  |           2.18083e-05 |
|   2049 | /                        | 172.58.107.207  |           2.18083e-05 |
|   2050 | /                        | 172.58.107.212  |           2.18083e-05 |
|   2051 | /                        | 172.58.107.223  |           2.18083e-05 |
|   2052 | /                        | 172.58.107.225  |           2.18083e-05 |
|   2053 | /                        | 172.58.107.226  |           2.18083e-05 |
|   2054 | /                        | 172.58.107.229  |           2.18083e-05 |
|   2055 | /                        | 172.58.107.23   |           2.18083e-05 |
|   2056 | /                        | 172.58.107.230  |           2.18083e-05 |
|   2057 | /                        | 172.58.107.234  |           2.18083e-05 |
|   2058 | /                        | 172.58.107.237  |           2.18083e-05 |
|   2059 | /                        | 172.58.107.239  |           2.18083e-05 |
|   2060 | /                        | 172.58.107.34   |           2.18083e-05 |
|   2061 | /                        | 172.58.107.51   |           2.18083e-05 |
|   2062 | /                        | 172.58.107.56   |           2.18083e-05 |
|   2063 | /                        | 172.58.107.61   |           2.18083e-05 |
|   2064 | /                        | 172.58.107.62   |           2.18083e-05 |
|   2065 | /                        | 172.58.107.74   |           2.18083e-05 |
|   2066 | /                        | 172.58.107.84   |           2.18083e-05 |
|   2067 | /                        | 172.58.107.85   |           2.18083e-05 |
|   2068 | /                        | 172.58.108.134  |           2.18083e-05 |
|   2069 | /                        | 172.58.108.142  |           2.18083e-05 |
|   2070 | /                        | 172.58.108.147  |           2.18083e-05 |
|   2071 | /                        | 172.58.108.153  |           2.18083e-05 |
|   2072 | /                        | 172.58.108.157  |           2.18083e-05 |
|   2073 | /                        | 172.58.108.202  |           2.18083e-05 |
|   2074 | /                        | 172.58.108.214  |           2.18083e-05 |
|   2075 | /                        | 172.58.108.216  |           2.18083e-05 |
|   2076 | /                        | 172.58.108.5    |           2.18083e-05 |
|   2077 | /                        | 172.58.108.72   |           2.18083e-05 |
|   2078 | /                        | 172.58.108.94   |           2.18083e-05 |
|   2079 | /                        | 172.58.109.108  |           2.18083e-05 |
|   2080 | /                        | 172.58.109.109  |           2.18083e-05 |
|   2081 | /                        | 172.58.109.121  |           2.18083e-05 |
|   2082 | /                        | 172.58.109.124  |           2.18083e-05 |
|   2083 | /                        | 172.58.109.13   |           2.18083e-05 |
|   2084 | /                        | 172.58.109.133  |           2.18083e-05 |
|   2085 | /                        | 172.58.109.140  |           2.18083e-05 |
|   2086 | /                        | 172.58.109.155  |           2.18083e-05 |
|   2087 | /                        | 172.58.109.173  |           2.18083e-05 |
|   2088 | /                        | 172.58.109.175  |           2.18083e-05 |
|   2089 | /                        | 172.58.109.183  |           2.18083e-05 |
|   2090 | /                        | 172.58.109.222  |           2.18083e-05 |
|   2091 | /                        | 172.58.109.223  |           2.18083e-05 |
|   2092 | /                        | 172.58.109.225  |           2.18083e-05 |
|   2093 | /                        | 172.58.109.232  |           2.18083e-05 |
|   2094 | /                        | 172.58.109.241  |           2.18083e-05 |
|   2095 | /                        | 172.58.109.245  |           2.18083e-05 |
|   2096 | /                        | 172.58.109.251  |           2.18083e-05 |
|   2097 | /                        | 172.58.109.30   |           2.18083e-05 |
|   2098 | /                        | 172.58.109.33   |           2.18083e-05 |
|   2099 | /                        | 172.58.109.44   |           2.18083e-05 |
|   2100 | /                        | 172.58.109.54   |           2.18083e-05 |
|   2101 | /                        | 172.58.109.64   |           2.18083e-05 |
|   2102 | /                        | 172.58.109.69   |           2.18083e-05 |
|   2103 | /                        | 172.58.109.83   |           2.18083e-05 |
|   2104 | /                        | 172.58.110.140  |           2.18083e-05 |
|   2105 | /                        | 172.58.110.159  |           2.18083e-05 |
|   2106 | /                        | 172.58.110.164  |           2.18083e-05 |
|   2107 | /                        | 172.58.110.170  |           2.18083e-05 |
|   2108 | /                        | 172.58.110.184  |           2.18083e-05 |
|   2109 | /                        | 172.58.110.189  |           2.18083e-05 |
|   2110 | /                        | 172.58.110.195  |           2.18083e-05 |
|   2111 | /                        | 172.58.110.206  |           2.18083e-05 |
|   2112 | /                        | 172.58.110.231  |           2.18083e-05 |
|   2113 | /                        | 172.58.110.236  |           2.18083e-05 |
|   2114 | /                        | 172.58.110.240  |           2.18083e-05 |
|   2115 | /                        | 172.58.111.140  |           2.18083e-05 |
|   2116 | /                        | 172.58.111.141  |           2.18083e-05 |
|   2117 | /                        | 172.58.111.153  |           2.18083e-05 |
|   2118 | /                        | 172.58.111.172  |           2.18083e-05 |
|   2119 | /                        | 172.58.111.175  |           2.18083e-05 |
|   2120 | /                        | 172.58.111.180  |           2.18083e-05 |
|   2121 | /                        | 172.58.111.192  |           2.18083e-05 |
|   2122 | /                        | 172.58.111.202  |           2.18083e-05 |
|   2123 | /                        | 172.58.111.203  |           2.18083e-05 |
|   2124 | /                        | 172.58.111.204  |           2.18083e-05 |
|   2125 | /                        | 172.58.111.245  |           2.18083e-05 |
|   2126 | /                        | 172.58.111.247  |           2.18083e-05 |
|   2127 | /                        | 172.58.111.255  |           2.18083e-05 |
|   2128 | /                        | 172.58.111.26   |           2.18083e-05 |
|   2129 | /                        | 172.58.111.5    |           2.18083e-05 |
|   2130 | /                        | 172.58.14.166   |           2.18083e-05 |
|   2131 | /                        | 172.58.30.200   |           2.18083e-05 |
|   2132 | /                        | 172.58.67.105   |           2.18083e-05 |
|   2133 | /                        | 172.58.67.120   |           2.18083e-05 |
|   2134 | /                        | 172.58.67.127   |           2.18083e-05 |
|   2135 | /                        | 172.58.67.200   |           2.18083e-05 |
|   2136 | /                        | 172.58.67.201   |           2.18083e-05 |
|   2137 | /                        | 172.58.67.207   |           2.18083e-05 |
|   2138 | /                        | 172.58.67.210   |           2.18083e-05 |
|   2139 | /                        | 172.58.67.239   |           2.18083e-05 |
|   2140 | /                        | 172.58.67.35    |           2.18083e-05 |
|   2141 | /                        | 172.58.67.38    |           2.18083e-05 |
|   2142 | /                        | 172.58.67.44    |           2.18083e-05 |
|   2143 | /                        | 172.58.67.65    |           2.18083e-05 |
|   2144 | /                        | 172.58.67.72    |           2.18083e-05 |
|   2145 | /                        | 172.58.67.77    |           2.18083e-05 |
|   2146 | /                        | 172.58.68.125   |           2.18083e-05 |
|   2147 | /                        | 172.58.68.245   |           2.18083e-05 |
|   2148 | /                        | 172.58.68.25    |           2.18083e-05 |
|   2149 | /                        | 172.58.70.161   |           2.18083e-05 |
|   2150 | /                        | 172.58.70.163   |           2.18083e-05 |
|   2151 | /                        | 172.58.70.191   |           2.18083e-05 |
|   2152 | /                        | 172.58.70.194   |           2.18083e-05 |
|   2153 | /                        | 172.58.70.201   |           2.18083e-05 |
|   2154 | /                        | 172.58.71.157   |           2.18083e-05 |
|   2155 | /                        | 172.58.73.208   |           2.18083e-05 |
|   2156 | /                        | 172.58.76.154   |           2.18083e-05 |
|   2157 | /                        | 172.58.76.54    |           2.18083e-05 |
|   2158 | /                        | 172.58.76.86    |           2.18083e-05 |
|   2159 | /                        | 172.58.96.108   |           2.18083e-05 |
|   2160 | /                        | 172.58.96.115   |           2.18083e-05 |
|   2161 | /                        | 172.58.96.163   |           2.18083e-05 |
|   2162 | /                        | 172.58.96.172   |           2.18083e-05 |
|   2163 | /                        | 172.58.96.200   |           2.18083e-05 |
|   2164 | /                        | 172.58.96.230   |           2.18083e-05 |
|   2165 | /                        | 172.58.97.107   |           2.18083e-05 |
|   2166 | /                        | 172.58.97.120   |           2.18083e-05 |
|   2167 | /                        | 172.58.97.152   |           2.18083e-05 |
|   2168 | /                        | 172.58.97.192   |           2.18083e-05 |
|   2169 | /                        | 172.58.97.72    |           2.18083e-05 |
|   2170 | /                        | 172.58.99.105   |           2.18083e-05 |
|   2171 | /                        | 172.58.99.108   |           2.18083e-05 |
|   2172 | /                        | 172.58.99.109   |           2.18083e-05 |
|   2173 | /                        | 172.58.99.121   |           2.18083e-05 |
|   2174 | /                        | 172.58.99.124   |           2.18083e-05 |
|   2175 | /                        | 172.58.99.125   |           2.18083e-05 |
|   2176 | /                        | 172.58.99.127   |           2.18083e-05 |
|   2177 | /                        | 172.58.99.132   |           2.18083e-05 |
|   2178 | /                        | 172.58.99.136   |           2.18083e-05 |
|   2179 | /                        | 172.58.99.139   |           2.18083e-05 |
|   2180 | /                        | 172.58.99.14    |           2.18083e-05 |
|   2181 | /                        | 172.58.99.155   |           2.18083e-05 |
|   2182 | /                        | 172.58.99.165   |           2.18083e-05 |
|   2183 | /                        | 172.58.99.175   |           2.18083e-05 |
|   2184 | /                        | 172.58.99.178   |           2.18083e-05 |
|   2185 | /                        | 172.58.99.179   |           2.18083e-05 |
|   2186 | /                        | 172.58.99.186   |           2.18083e-05 |
|   2187 | /                        | 172.58.99.192   |           2.18083e-05 |
|   2188 | /                        | 172.58.99.194   |           2.18083e-05 |
|   2189 | /                        | 172.58.99.212   |           2.18083e-05 |
|   2190 | /                        | 172.58.99.234   |           2.18083e-05 |
|   2191 | /                        | 172.58.99.25    |           2.18083e-05 |
|   2192 | /                        | 172.58.99.251   |           2.18083e-05 |
|   2193 | /                        | 172.58.99.51    |           2.18083e-05 |
|   2194 | /                        | 172.58.99.53    |           2.18083e-05 |
|   2195 | /                        | 172.58.99.57    |           2.18083e-05 |
|   2196 | /                        | 172.58.99.59    |           2.18083e-05 |
|   2197 | /                        | 172.58.99.67    |           2.18083e-05 |
|   2198 | /                        | 172.58.99.99    |           2.18083e-05 |
|   2199 | /                        | 172.8.173.98    |           2.18083e-05 |
|   2200 | /                        | 172.8.175.156   |           2.18083e-05 |
|   2201 | /                        | 172.83.43.134   |           2.18083e-05 |
|   2202 | /                        | 172.87.148.22   |           2.18083e-05 |
|   2203 | /                        | 172.87.181.101  |           2.18083e-05 |
|   2204 | /                        | 172.98.66.16    |           2.18083e-05 |
|   2205 | /                        | 173.123.149.65  |           2.18083e-05 |
|   2206 | /                        | 173.123.157.175 |           2.18083e-05 |
|   2207 | /                        | 173.123.161.160 |           2.18083e-05 |
|   2208 | /                        | 173.123.186.102 |           2.18083e-05 |
|   2209 | /                        | 173.123.192.224 |           2.18083e-05 |
|   2210 | /                        | 173.123.212.81  |           2.18083e-05 |
|   2211 | /                        | 173.127.10.4    |           2.18083e-05 |
|   2212 | /                        | 173.127.11.100  |           2.18083e-05 |
|   2213 | /                        | 173.127.123.160 |           2.18083e-05 |
|   2214 | /                        | 173.127.36.190  |           2.18083e-05 |
|   2215 | /                        | 173.127.44.65   |           2.18083e-05 |
|   2216 | /                        | 173.127.47.45   |           2.18083e-05 |
|   2217 | /                        | 173.127.49.144  |           2.18083e-05 |
|   2218 | /                        | 173.127.55.135  |           2.18083e-05 |
|   2219 | /                        | 173.127.59.159  |           2.18083e-05 |
|   2220 | /                        | 173.127.8.49    |           2.18083e-05 |
|   2221 | /                        | 173.127.80.254  |           2.18083e-05 |
|   2222 | /                        | 173.127.89.155  |           2.18083e-05 |
|   2223 | /                        | 173.127.93.78   |           2.18083e-05 |
|   2224 | /                        | 173.134.114.250 |           2.18083e-05 |
|   2225 | /                        | 173.149.129.239 |           2.18083e-05 |
|   2226 | /                        | 173.149.142.35  |           2.18083e-05 |
|   2227 | /                        | 173.149.157.75  |           2.18083e-05 |
|   2228 | /                        | 173.149.158.44  |           2.18083e-05 |
|   2229 | /                        | 173.149.192.116 |           2.18083e-05 |
|   2230 | /                        | 173.149.192.150 |           2.18083e-05 |
|   2231 | /                        | 173.149.209.79  |           2.18083e-05 |
|   2232 | /                        | 173.172.166.90  |           2.18083e-05 |
|   2233 | /                        | 173.173.100.45  |           2.18083e-05 |
|   2234 | /                        | 173.173.101.109 |           2.18083e-05 |
|   2235 | /                        | 173.173.102.198 |           2.18083e-05 |
|   2236 | /                        | 173.173.104.202 |           2.18083e-05 |
|   2237 | /                        | 173.173.107.110 |           2.18083e-05 |
|   2238 | /                        | 173.173.111.160 |           2.18083e-05 |
|   2239 | /                        | 173.173.113.96  |           2.18083e-05 |
|   2240 | /                        | 173.173.116.10  |           2.18083e-05 |
|   2241 | /                        | 173.173.118.83  |           2.18083e-05 |
|   2242 | /                        | 173.173.119.75  |           2.18083e-05 |
|   2243 | /                        | 173.173.121.51  |           2.18083e-05 |
|   2244 | /                        | 173.173.173.208 |           2.18083e-05 |
|   2245 | /                        | 173.174.164.47  |           2.18083e-05 |
|   2246 | /                        | 173.174.172.52  |           2.18083e-05 |
|   2247 | /                        | 173.174.184.26  |           2.18083e-05 |
|   2248 | /                        | 173.174.186.93  |           2.18083e-05 |
|   2249 | /                        | 173.174.192.127 |           2.18083e-05 |
|   2250 | /                        | 173.174.196.212 |           2.18083e-05 |
|   2251 | /                        | 173.174.201.87  |           2.18083e-05 |
|   2252 | /                        | 173.174.212.111 |           2.18083e-05 |
|   2253 | /                        | 173.174.217.236 |           2.18083e-05 |
|   2254 | /                        | 173.174.218.183 |           2.18083e-05 |
|   2255 | /                        | 173.174.223.192 |           2.18083e-05 |
|   2256 | /                        | 173.174.240.181 |           2.18083e-05 |
|   2257 | /                        | 173.175.104.33  |           2.18083e-05 |
|   2258 | /                        | 173.175.109.235 |           2.18083e-05 |
|   2259 | /                        | 173.175.118.119 |           2.18083e-05 |
|   2260 | /                        | 173.175.172.89  |           2.18083e-05 |
|   2261 | /                        | 173.175.82.164  |           2.18083e-05 |
|   2262 | /                        | 173.227.210.1   |           2.18083e-05 |
|   2263 | /                        | 173.232.243.3   |           2.18083e-05 |
|   2264 | /                        | 173.239.198.230 |           2.18083e-05 |
|   2265 | /                        | 173.239.218.8   |           2.18083e-05 |
|   2266 | /                        | 173.239.232.103 |           2.18083e-05 |
|   2267 | /                        | 173.239.232.104 |           2.18083e-05 |
|   2268 | /                        | 173.239.232.11  |           2.18083e-05 |
|   2269 | /                        | 173.239.232.112 |           2.18083e-05 |
|   2270 | /                        | 173.239.232.113 |           2.18083e-05 |
|   2271 | /                        | 173.239.232.119 |           2.18083e-05 |
|   2272 | /                        | 173.239.232.12  |           2.18083e-05 |
|   2273 | /                        | 173.239.232.121 |           2.18083e-05 |
|   2274 | /                        | 173.239.232.125 |           2.18083e-05 |
|   2275 | /                        | 173.239.232.13  |           2.18083e-05 |
|   2276 | /                        | 173.239.232.134 |           2.18083e-05 |
|   2277 | /                        | 173.239.232.141 |           2.18083e-05 |
|   2278 | /                        | 173.239.232.142 |           2.18083e-05 |
|   2279 | /                        | 173.239.232.146 |           2.18083e-05 |
|   2280 | /                        | 173.239.232.149 |           2.18083e-05 |
|   2281 | /                        | 173.239.232.153 |           2.18083e-05 |
|   2282 | /                        | 173.239.232.177 |           2.18083e-05 |
|   2283 | /                        | 173.239.232.20  |           2.18083e-05 |
|   2284 | /                        | 173.239.232.22  |           2.18083e-05 |
|   2285 | /                        | 173.239.232.24  |           2.18083e-05 |
|   2286 | /                        | 173.239.232.26  |           2.18083e-05 |
|   2287 | /                        | 173.239.232.37  |           2.18083e-05 |
|   2288 | /                        | 173.239.232.4   |           2.18083e-05 |
|   2289 | /                        | 173.239.232.41  |           2.18083e-05 |
|   2290 | /                        | 173.239.232.42  |           2.18083e-05 |
|   2291 | /                        | 173.239.232.45  |           2.18083e-05 |
|   2292 | /                        | 173.239.232.46  |           2.18083e-05 |
|   2293 | /                        | 173.239.232.48  |           2.18083e-05 |
|   2294 | /                        | 173.239.232.49  |           2.18083e-05 |
|   2295 | /                        | 173.239.232.5   |           2.18083e-05 |
|   2296 | /                        | 173.239.232.52  |           2.18083e-05 |
|   2297 | /                        | 173.239.232.64  |           2.18083e-05 |
|   2298 | /                        | 173.239.232.69  |           2.18083e-05 |
|   2299 | /                        | 173.239.232.79  |           2.18083e-05 |
|   2300 | /                        | 173.239.232.88  |           2.18083e-05 |
|   2301 | /                        | 173.239.232.90  |           2.18083e-05 |
|   2302 | /                        | 173.239.232.95  |           2.18083e-05 |
|   2303 | /                        | 173.239.232.97  |           2.18083e-05 |
|   2304 | /                        | 173.255.139.216 |           2.18083e-05 |
|   2305 | /                        | 174.128.177.247 |           2.18083e-05 |
|   2306 | /                        | 174.128.180.102 |           2.18083e-05 |
|   2307 | /                        | 174.128.180.115 |           2.18083e-05 |
|   2308 | /                        | 174.128.180.152 |           2.18083e-05 |
|   2309 | /                        | 174.128.180.19  |           2.18083e-05 |
|   2310 | /                        | 174.128.180.20  |           2.18083e-05 |
|   2311 | /                        | 174.128.180.21  |           2.18083e-05 |
|   2312 | /                        | 174.128.180.212 |           2.18083e-05 |
|   2313 | /                        | 174.128.180.67  |           2.18083e-05 |
|   2314 | /                        | 174.128.181.105 |           2.18083e-05 |
|   2315 | /                        | 174.128.181.152 |           2.18083e-05 |
|   2316 | /                        | 174.128.181.166 |           2.18083e-05 |
|   2317 | /                        | 174.128.181.232 |           2.18083e-05 |
|   2318 | /                        | 174.128.181.39  |           2.18083e-05 |
|   2319 | /                        | 174.128.181.69  |           2.18083e-05 |
|   2320 | /                        | 174.128.181.89  |           2.18083e-05 |
|   2321 | /                        | 174.193.4.63    |           2.18083e-05 |
|   2322 | /                        | 174.194.1.50    |           2.18083e-05 |
|   2323 | /                        | 174.196.196.21  |           2.18083e-05 |
|   2324 | /                        | 174.197.0.7     |           2.18083e-05 |
|   2325 | /                        | 174.197.1.205   |           2.18083e-05 |
|   2326 | /                        | 174.197.1.248   |           2.18083e-05 |
|   2327 | /                        | 174.197.1.83    |           2.18083e-05 |
|   2328 | /                        | 174.197.10.65   |           2.18083e-05 |
|   2329 | /                        | 174.197.2.198   |           2.18083e-05 |
|   2330 | /                        | 174.197.4.51    |           2.18083e-05 |
|   2331 | /                        | 174.197.6.132   |           2.18083e-05 |
|   2332 | /                        | 174.197.6.43    |           2.18083e-05 |
|   2333 | /                        | 174.197.7.127   |           2.18083e-05 |
|   2334 | /                        | 174.197.7.219   |           2.18083e-05 |
|   2335 | /                        | 174.197.8.92    |           2.18083e-05 |
|   2336 | /                        | 174.206.18.197  |           2.18083e-05 |
|   2337 | /                        | 174.206.2.148   |           2.18083e-05 |
|   2338 | /                        | 174.206.26.70   |           2.18083e-05 |
|   2339 | /                        | 174.207.0.103   |           2.18083e-05 |
|   2340 | /                        | 174.207.0.183   |           2.18083e-05 |
|   2341 | /                        | 174.207.0.84    |           2.18083e-05 |
|   2342 | /                        | 174.207.1.185   |           2.18083e-05 |
|   2343 | /                        | 174.207.11.138  |           2.18083e-05 |
|   2344 | /                        | 174.207.11.139  |           2.18083e-05 |
|   2345 | /                        | 174.207.12.122  |           2.18083e-05 |
|   2346 | /                        | 174.207.12.44   |           2.18083e-05 |
|   2347 | /                        | 174.207.13.82   |           2.18083e-05 |
|   2348 | /                        | 174.207.14.251  |           2.18083e-05 |
|   2349 | /                        | 174.207.14.75   |           2.18083e-05 |
|   2350 | /                        | 174.207.15.123  |           2.18083e-05 |
|   2351 | /                        | 174.207.15.137  |           2.18083e-05 |
|   2352 | /                        | 174.207.15.176  |           2.18083e-05 |
|   2353 | /                        | 174.207.15.199  |           2.18083e-05 |
|   2354 | /                        | 174.207.15.228  |           2.18083e-05 |
|   2355 | /                        | 174.207.16.65   |           2.18083e-05 |
|   2356 | /                        | 174.207.17.141  |           2.18083e-05 |
|   2357 | /                        | 174.207.17.181  |           2.18083e-05 |
|   2358 | /                        | 174.207.17.75   |           2.18083e-05 |
|   2359 | /                        | 174.207.17.95   |           2.18083e-05 |
|   2360 | /                        | 174.207.18.107  |           2.18083e-05 |
|   2361 | /                        | 174.207.18.173  |           2.18083e-05 |
|   2362 | /                        | 174.207.19.222  |           2.18083e-05 |
|   2363 | /                        | 174.207.2.122   |           2.18083e-05 |
|   2364 | /                        | 174.207.2.208   |           2.18083e-05 |
|   2365 | /                        | 174.207.2.32    |           2.18083e-05 |
|   2366 | /                        | 174.207.21.3    |           2.18083e-05 |
|   2367 | /                        | 174.207.22.2    |           2.18083e-05 |
|   2368 | /                        | 174.207.23.110  |           2.18083e-05 |
|   2369 | /                        | 174.207.23.167  |           2.18083e-05 |
|   2370 | /                        | 174.207.23.38   |           2.18083e-05 |
|   2371 | /                        | 174.207.23.41   |           2.18083e-05 |
|   2372 | /                        | 174.207.24.231  |           2.18083e-05 |
|   2373 | /                        | 174.207.24.245  |           2.18083e-05 |
|   2374 | /                        | 174.207.24.42   |           2.18083e-05 |
|   2375 | /                        | 174.207.25.219  |           2.18083e-05 |
|   2376 | /                        | 174.207.25.229  |           2.18083e-05 |
|   2377 | /                        | 174.207.26.100  |           2.18083e-05 |
|   2378 | /                        | 174.207.26.73   |           2.18083e-05 |
|   2379 | /                        | 174.207.3.185   |           2.18083e-05 |
|   2380 | /                        | 174.207.3.247   |           2.18083e-05 |
|   2381 | /                        | 174.207.3.54    |           2.18083e-05 |
|   2382 | /                        | 174.207.34.2    |           2.18083e-05 |
|   2383 | /                        | 174.207.4.226   |           2.18083e-05 |
|   2384 | /                        | 174.207.4.5     |           2.18083e-05 |
|   2385 | /                        | 174.207.5.143   |           2.18083e-05 |
|   2386 | /                        | 174.207.5.154   |           2.18083e-05 |
|   2387 | /                        | 174.207.5.245   |           2.18083e-05 |
|   2388 | /                        | 174.207.6.110   |           2.18083e-05 |
|   2389 | /                        | 174.207.8.123   |           2.18083e-05 |
|   2390 | /                        | 174.207.8.153   |           2.18083e-05 |
|   2391 | /                        | 174.236.11.194  |           2.18083e-05 |
|   2392 | /                        | 174.236.18.237  |           2.18083e-05 |
|   2393 | /                        | 174.236.3.129   |           2.18083e-05 |
|   2394 | /                        | 174.236.6.113   |           2.18083e-05 |
|   2395 | /                        | 174.240.134.5   |           2.18083e-05 |
|   2396 | /                        | 174.242.79.208  |           2.18083e-05 |
|   2397 | /                        | 174.244.67.71   |           2.18083e-05 |
|   2398 | /                        | 174.244.81.146  |           2.18083e-05 |
|   2399 | /                        | 174.244.81.200  |           2.18083e-05 |
|   2400 | /                        | 174.244.81.249  |           2.18083e-05 |
|   2401 | /                        | 174.244.81.91   |           2.18083e-05 |
|   2402 | /                        | 174.246.129.39  |           2.18083e-05 |
|   2403 | /                        | 174.246.133.136 |           2.18083e-05 |
|   2404 | /                        | 174.246.161.224 |           2.18083e-05 |
|   2405 | /                        | 174.246.192.202 |           2.18083e-05 |
|   2406 | /                        | 174.246.192.204 |           2.18083e-05 |
|   2407 | /                        | 174.246.193.161 |           2.18083e-05 |
|   2408 | /                        | 174.246.193.205 |           2.18083e-05 |
|   2409 | /                        | 174.246.193.51  |           2.18083e-05 |
|   2410 | /                        | 174.246.194.63  |           2.18083e-05 |
|   2411 | /                        | 174.246.196.144 |           2.18083e-05 |
|   2412 | /                        | 174.246.196.212 |           2.18083e-05 |
|   2413 | /                        | 174.246.196.23  |           2.18083e-05 |
|   2414 | /                        | 174.246.197.129 |           2.18083e-05 |
|   2415 | /                        | 174.246.198.28  |           2.18083e-05 |
|   2416 | /                        | 174.246.201.223 |           2.18083e-05 |
|   2417 | /                        | 174.246.202.47  |           2.18083e-05 |
|   2418 | /                        | 174.249.45.199  |           2.18083e-05 |
|   2419 | /                        | 174.25.169.61   |           2.18083e-05 |
|   2420 | /                        | 174.255.131.179 |           2.18083e-05 |
|   2421 | /                        | 174.255.131.241 |           2.18083e-05 |
|   2422 | /                        | 174.255.132.117 |           2.18083e-05 |
|   2423 | /                        | 174.255.132.169 |           2.18083e-05 |
|   2424 | /                        | 174.255.134.225 |           2.18083e-05 |
|   2425 | /                        | 174.255.134.83  |           2.18083e-05 |
|   2426 | /                        | 174.255.136.80  |           2.18083e-05 |
|   2427 | /                        | 174.255.137.106 |           2.18083e-05 |
|   2428 | /                        | 174.255.145.205 |           2.18083e-05 |
|   2429 | /                        | 174.255.146.154 |           2.18083e-05 |
|   2430 | /                        | 174.255.148.28  |           2.18083e-05 |
|   2431 | /                        | 174.255.154.236 |           2.18083e-05 |
|   2432 | /                        | 174.255.155.223 |           2.18083e-05 |
|   2433 | /                        | 174.51.184.241  |           2.18083e-05 |
|   2434 | /                        | 177.247.132.187 |           2.18083e-05 |
|   2435 | /                        | 18.237.174.64   |           2.18083e-05 |
|   2436 | /                        | 184.203.105.40  |           2.18083e-05 |
|   2437 | /                        | 184.203.172.25  |           2.18083e-05 |
|   2438 | /                        | 184.203.184.92  |           2.18083e-05 |
|   2439 | /                        | 184.203.19.207  |           2.18083e-05 |
|   2440 | /                        | 184.203.221.94  |           2.18083e-05 |
|   2441 | /                        | 184.203.236.132 |           2.18083e-05 |
|   2442 | /                        | 184.203.5.0     |           2.18083e-05 |
|   2443 | /                        | 184.203.52.137  |           2.18083e-05 |
|   2444 | /                        | 184.226.102.210 |           2.18083e-05 |
|   2445 | /                        | 184.226.116.113 |           2.18083e-05 |
|   2446 | /                        | 184.226.56.72   |           2.18083e-05 |
|   2447 | /                        | 184.226.71.155  |           2.18083e-05 |
|   2448 | /                        | 184.226.95.31   |           2.18083e-05 |
|   2449 | /                        | 184.75.223.44   |           2.18083e-05 |
|   2450 | /                        | 184.80.232.34   |           2.18083e-05 |
|   2451 | /                        | 185.153.179.81  |           2.18083e-05 |
|   2452 | /                        | 185.216.33.82   |           2.18083e-05 |
|   2453 | /                        | 185.245.86.170  |           2.18083e-05 |
|   2454 | /                        | 185.245.87.238  |           2.18083e-05 |
|   2455 | /                        | 185.247.70.173  |           2.18083e-05 |
|   2456 | /                        | 185.247.70.196  |           2.18083e-05 |
|   2457 | /                        | 185.247.70.228  |           2.18083e-05 |
|   2458 | /                        | 187.201.77.208  |           2.18083e-05 |
|   2459 | /                        | 187.220.23.224  |           2.18083e-05 |
|   2460 | /                        | 187.237.231.72  |           2.18083e-05 |
|   2461 | /                        | 187.237.25.33   |           2.18083e-05 |
|   2462 | /                        | 189.140.41.75   |           2.18083e-05 |
|   2463 | /                        | 189.140.49.80   |           2.18083e-05 |
|   2464 | /                        | 189.166.144.213 |           2.18083e-05 |
|   2465 | /                        | 189.166.150.196 |           2.18083e-05 |
|   2466 | /                        | 191.96.67.108   |           2.18083e-05 |
|   2467 | /                        | 191.96.67.5     |           2.18083e-05 |
|   2468 | /                        | 192.145.119.105 |           2.18083e-05 |
|   2469 | /                        | 192.171.117.234 |           2.18083e-05 |
|   2470 | /                        | 192.24.18.31    |           2.18083e-05 |
|   2471 | /                        | 192.241.205.66  |           2.18083e-05 |
|   2472 | /                        | 192.241.233.20  |           2.18083e-05 |
|   2473 | /                        | 192.241.235.228 |           2.18083e-05 |
|   2474 | /                        | 193.148.18.75   |           2.18083e-05 |
|   2475 | /                        | 193.37.252.52   |           2.18083e-05 |
|   2476 | /                        | 193.37.252.82   |           2.18083e-05 |
|   2477 | /                        | 193.56.117.42   |           2.18083e-05 |
|   2478 | /                        | 195.206.104.106 |           2.18083e-05 |
|   2479 | /                        | 195.206.104.109 |           2.18083e-05 |
|   2480 | /                        | 196.247.56.62   |           2.18083e-05 |
|   2481 | /                        | 196.52.2.25     |           2.18083e-05 |
|   2482 | /                        | 196.52.39.2     |           2.18083e-05 |
|   2483 | /                        | 196.52.39.23    |           2.18083e-05 |
|   2484 | /                        | 196.52.39.29    |           2.18083e-05 |
|   2485 | /                        | 198.205.24.208  |           2.18083e-05 |
|   2486 | /                        | 198.98.87.16    |           2.18083e-05 |
|   2487 | /                        | 199.10.64.84    |           2.18083e-05 |
|   2488 | /                        | 199.116.115.137 |           2.18083e-05 |
|   2489 | /                        | 199.116.118.132 |           2.18083e-05 |
|   2490 | /                        | 200.68.128.26   |           2.18083e-05 |
|   2491 | /                        | 201.174.135.51  |           2.18083e-05 |
|   2492 | /                        | 205.251.148.138 |           2.18083e-05 |
|   2493 | /                        | 205.251.148.146 |           2.18083e-05 |
|   2494 | /                        | 205.251.148.178 |           2.18083e-05 |
|   2495 | /                        | 205.251.148.26  |           2.18083e-05 |
|   2496 | /                        | 205.251.148.34  |           2.18083e-05 |
|   2497 | /                        | 205.251.148.66  |           2.18083e-05 |
|   2498 | /                        | 205.251.150.162 |           2.18083e-05 |
|   2499 | /                        | 205.251.150.194 |           2.18083e-05 |
|   2500 | /                        | 206.109.5.6     |           2.18083e-05 |
|   2501 | /                        | 206.217.136.93  |           2.18083e-05 |
|   2502 | /                        | 206.81.232.6    |           2.18083e-05 |
|   2503 | /                        | 207.119.3.149   |           2.18083e-05 |
|   2504 | /                        | 207.188.6.23    |           2.18083e-05 |
|   2505 | /                        | 207.189.160.146 |           2.18083e-05 |
|   2506 | /                        | 207.189.160.150 |           2.18083e-05 |
|   2507 | /                        | 207.68.209.17   |           2.18083e-05 |
|   2508 | /                        | 208.180.188.245 |           2.18083e-05 |
|   2509 | /                        | 208.184.162.149 |           2.18083e-05 |
|   2510 | /                        | 208.54.83.145   |           2.18083e-05 |
|   2511 | /                        | 208.54.83.163   |           2.18083e-05 |
|   2512 | /                        | 208.54.83.194   |           2.18083e-05 |
|   2513 | /                        | 208.54.83.234   |           2.18083e-05 |
|   2514 | /                        | 208.54.86.183   |           2.18083e-05 |
|   2515 | /                        | 208.54.86.247   |           2.18083e-05 |
|   2516 | /                        | 209.107.189.151 |           2.18083e-05 |
|   2517 | /                        | 209.107.189.26  |           2.18083e-05 |
|   2518 | /                        | 209.107.196.134 |           2.18083e-05 |
|   2519 | /                        | 209.112.234.226 |           2.18083e-05 |
|   2520 | /                        | 209.12.105.54   |           2.18083e-05 |
|   2521 | /                        | 209.135.132.136 |           2.18083e-05 |
|   2522 | /                        | 209.181.112.90  |           2.18083e-05 |
|   2523 | /                        | 209.184.113.32  |           2.18083e-05 |
|   2524 | /                        | 209.184.113.43  |           2.18083e-05 |
|   2525 | /                        | 209.205.120.222 |           2.18083e-05 |
|   2526 | /                        | 209.205.221.162 |           2.18083e-05 |
|   2527 | /                        | 209.36.87.18    |           2.18083e-05 |
|   2528 | /                        | 209.40.156.14   |           2.18083e-05 |
|   2529 | /                        | 209.58.142.160  |           2.18083e-05 |
|   2530 | /                        | 209.58.147.231  |           2.18083e-05 |
|   2531 | /                        | 209.58.154.12   |           2.18083e-05 |
|   2532 | /                        | 212.102.40.134  |           2.18083e-05 |
|   2533 | /                        | 212.102.40.17   |           2.18083e-05 |
|   2534 | /                        | 212.102.44.99   |           2.18083e-05 |
|   2535 | /                        | 212.102.45.115  |           2.18083e-05 |
|   2536 | /                        | 212.102.45.83   |           2.18083e-05 |
|   2537 | /                        | 216.1.153.162   |           2.18083e-05 |
|   2538 | /                        | 216.162.109.195 |           2.18083e-05 |
|   2539 | /                        | 216.3.131.238   |           2.18083e-05 |
|   2540 | /                        | 216.82.129.193  |           2.18083e-05 |
|   2541 | /                        | 217.138.206.86  |           2.18083e-05 |
|   2542 | /                        | 217.167.133.44  |           2.18083e-05 |
|   2543 | /                        | 23.105.149.2    |           2.18083e-05 |
|   2544 | /                        | 23.117.144.200  |           2.18083e-05 |
|   2545 | /                        | 23.122.42.151   |           2.18083e-05 |
|   2546 | /                        | 23.127.172.66   |           2.18083e-05 |
|   2547 | /                        | 23.24.208.203   |           2.18083e-05 |
|   2548 | /                        | 24.130.120.24   |           2.18083e-05 |
|   2549 | /                        | 24.153.137.59   |           2.18083e-05 |
|   2550 | /                        | 24.153.246.211  |           2.18083e-05 |
|   2551 | /                        | 24.155.149.210  |           2.18083e-05 |
|   2552 | /                        | 24.160.141.134  |           2.18083e-05 |
|   2553 | /                        | 24.160.158.182  |           2.18083e-05 |
|   2554 | /                        | 24.160.4.6      |           2.18083e-05 |
|   2555 | /                        | 24.167.9.61     |           2.18083e-05 |
|   2556 | /                        | 24.170.84.159   |           2.18083e-05 |
|   2557 | /                        | 24.214.254.251  |           2.18083e-05 |
|   2558 | /                        | 24.227.244.110  |           2.18083e-05 |
|   2559 | /                        | 24.227.245.186  |           2.18083e-05 |
|   2560 | /                        | 24.242.183.202  |           2.18083e-05 |
|   2561 | /                        | 24.243.14.190   |           2.18083e-05 |
|   2562 | /                        | 24.243.20.39    |           2.18083e-05 |
|   2563 | /                        | 24.243.34.38    |           2.18083e-05 |
|   2564 | /                        | 24.243.40.41    |           2.18083e-05 |
|   2565 | /                        | 24.243.50.45    |           2.18083e-05 |
|   2566 | /                        | 24.243.71.31    |           2.18083e-05 |
|   2567 | /                        | 24.26.227.191   |           2.18083e-05 |
|   2568 | /                        | 24.26.238.218   |           2.18083e-05 |
|   2569 | /                        | 24.26.241.169   |           2.18083e-05 |
|   2570 | /                        | 24.26.248.56    |           2.18083e-05 |
|   2571 | /                        | 24.27.227.129   |           2.18083e-05 |
|   2572 | /                        | 24.27.56.43     |           2.18083e-05 |
|   2573 | /                        | 24.28.101.172   |           2.18083e-05 |
|   2574 | /                        | 24.28.101.217   |           2.18083e-05 |
|   2575 | /                        | 24.28.146.220   |           2.18083e-05 |
|   2576 | /                        | 24.28.41.114    |           2.18083e-05 |
|   2577 | /                        | 24.28.41.37     |           2.18083e-05 |
|   2578 | /                        | 24.54.124.11    |           2.18083e-05 |
|   2579 | /                        | 24.55.39.25     |           2.18083e-05 |
|   2580 | /                        | 24.9.177.142    |           2.18083e-05 |
|   2581 | /                        | 24.90.204.177   |           2.18083e-05 |
|   2582 | /                        | 3.101.140.94    |           2.18083e-05 |
|   2583 | /                        | 3.128.201.114   |           2.18083e-05 |
|   2584 | /                        | 3.236.43.206    |           2.18083e-05 |
|   2585 | /                        | 31.14.72.25     |           2.18083e-05 |
|   2586 | /                        | 34.203.225.141  |           2.18083e-05 |
|   2587 | /                        | 37.60.189.18    |           2.18083e-05 |
|   2588 | /                        | 38.140.212.203  |           2.18083e-05 |
|   2589 | /                        | 38.70.11.18     |           2.18083e-05 |
|   2590 | /                        | 38.95.108.230   |           2.18083e-05 |
|   2591 | /                        | 4.15.231.83     |           2.18083e-05 |
|   2592 | /                        | 4.34.182.238    |           2.18083e-05 |
|   2593 | /                        | 40.128.78.90    |           2.18083e-05 |
|   2594 | /                        | 40.132.252.74   |           2.18083e-05 |
|   2595 | /                        | 45.131.192.135  |           2.18083e-05 |
|   2596 | /                        | 45.131.192.151  |           2.18083e-05 |
|   2597 | /                        | 45.131.192.232  |           2.18083e-05 |
|   2598 | /                        | 45.131.192.63   |           2.18083e-05 |
|   2599 | /                        | 45.132.115.22   |           2.18083e-05 |
|   2600 | /                        | 45.21.2.131     |           2.18083e-05 |
|   2601 | /                        | 45.248.77.171   |           2.18083e-05 |
|   2602 | /                        | 45.30.140.223   |           2.18083e-05 |
|   2603 | /                        | 45.41.134.89    |           2.18083e-05 |
|   2604 | /                        | 45.56.151.110   |           2.18083e-05 |
|   2605 | /                        | 45.56.151.222   |           2.18083e-05 |
|   2606 | /                        | 45.56.151.44    |           2.18083e-05 |
|   2607 | /                        | 45.56.151.8     |           2.18083e-05 |
|   2608 | /                        | 45.56.151.93    |           2.18083e-05 |
|   2609 | /                        | 45.83.89.58     |           2.18083e-05 |
|   2610 | /                        | 46.218.181.194  |           2.18083e-05 |
|   2611 | /                        | 47.183.194.27   |           2.18083e-05 |
|   2612 | /                        | 47.184.42.157   |           2.18083e-05 |
|   2613 | /                        | 47.187.240.128  |           2.18083e-05 |
|   2614 | /                        | 47.189.243.89   |           2.18083e-05 |
|   2615 | /                        | 47.214.184.25   |           2.18083e-05 |
|   2616 | /                        | 47.234.152.61   |           2.18083e-05 |
|   2617 | /                        | 47.234.244.208  |           2.18083e-05 |
|   2618 | /                        | 47.25.103.90    |           2.18083e-05 |
|   2619 | /                        | 5.181.234.114   |           2.18083e-05 |
|   2620 | /                        | 5.181.234.38    |           2.18083e-05 |
|   2621 | /                        | 50.202.129.130  |           2.18083e-05 |
|   2622 | /                        | 50.206.173.154  |           2.18083e-05 |
|   2623 | /                        | 50.207.115.213  |           2.18083e-05 |
|   2624 | /                        | 50.207.193.220  |           2.18083e-05 |
|   2625 | /                        | 50.237.25.227   |           2.18083e-05 |
|   2626 | /                        | 50.250.214.161  |           2.18083e-05 |
|   2627 | /                        | 50.30.92.29     |           2.18083e-05 |
|   2628 | /                        | 50.84.153.254   |           2.18083e-05 |
|   2629 | /                        | 50.84.194.42    |           2.18083e-05 |
|   2630 | /                        | 50.84.244.130   |           2.18083e-05 |
|   2631 | /                        | 50.84.26.206    |           2.18083e-05 |
|   2632 | /                        | 50.84.77.251    |           2.18083e-05 |
|   2633 | /                        | 51.9.81.63      |           2.18083e-05 |
|   2634 | /                        | 52.45.96.252    |           2.18083e-05 |
|   2635 | /                        | 54.218.132.133  |           2.18083e-05 |
|   2636 | /                        | 56.0.143.25     |           2.18083e-05 |
|   2637 | /                        | 56.0.84.26      |           2.18083e-05 |
|   2638 | /                        | 63.240.174.244  |           2.18083e-05 |
|   2639 | /                        | 64.124.33.178   |           2.18083e-05 |
|   2640 | /                        | 64.125.109.186  |           2.18083e-05 |
|   2641 | /                        | 64.134.149.33   |           2.18083e-05 |
|   2642 | /                        | 64.134.159.125  |           2.18083e-05 |
|   2643 | /                        | 64.134.31.44    |           2.18083e-05 |
|   2644 | /                        | 64.233.172.171  |           2.18083e-05 |
|   2645 | /                        | 64.233.172.173  |           2.18083e-05 |
|   2646 | /                        | 64.250.7.235    |           2.18083e-05 |
|   2647 | /                        | 64.9.239.13     |           2.18083e-05 |
|   2648 | /                        | 64.9.239.21     |           2.18083e-05 |
|   2649 | /                        | 64.9.239.30     |           2.18083e-05 |
|   2650 | /                        | 64.92.13.226    |           2.18083e-05 |
|   2651 | /                        | 64.92.22.156    |           2.18083e-05 |
|   2652 | /                        | 64.92.28.203    |           2.18083e-05 |
|   2653 | /                        | 65.112.228.222  |           2.18083e-05 |
|   2654 | /                        | 65.121.50.194   |           2.18083e-05 |
|   2655 | /                        | 65.204.10.233   |           2.18083e-05 |
|   2656 | /                        | 65.36.106.75    |           2.18083e-05 |
|   2657 | /                        | 66.102.6.171    |           2.18083e-05 |
|   2658 | /                        | 66.111.118.45   |           2.18083e-05 |
|   2659 | /                        | 66.139.85.115   |           2.18083e-05 |
|   2660 | /                        | 66.194.160.106  |           2.18083e-05 |
|   2661 | /                        | 66.225.111.109  |           2.18083e-05 |
|   2662 | /                        | 66.25.13.167    |           2.18083e-05 |
|   2663 | /                        | 66.25.154.61    |           2.18083e-05 |
|   2664 | /                        | 66.25.168.77    |           2.18083e-05 |
|   2665 | /                        | 66.25.172.244   |           2.18083e-05 |
|   2666 | /                        | 66.25.67.202    |           2.18083e-05 |
|   2667 | /                        | 66.25.7.76      |           2.18083e-05 |
|   2668 | /                        | 66.25.7.9       |           2.18083e-05 |
|   2669 | /                        | 66.25.80.16     |           2.18083e-05 |
|   2670 | /                        | 66.25.88.97     |           2.18083e-05 |
|   2671 | /                        | 66.3.38.202     |           2.18083e-05 |
|   2672 | /                        | 66.31.40.49     |           2.18083e-05 |
|   2673 | /                        | 66.42.139.162   |           2.18083e-05 |
|   2674 | /                        | 66.68.28.182    |           2.18083e-05 |
|   2675 | /                        | 66.68.53.12     |           2.18083e-05 |
|   2676 | /                        | 66.69.111.115   |           2.18083e-05 |
|   2677 | /                        | 66.69.119.209   |           2.18083e-05 |
|   2678 | /                        | 66.69.157.64    |           2.18083e-05 |
|   2679 | /                        | 66.69.33.137    |           2.18083e-05 |
|   2680 | /                        | 66.69.37.36     |           2.18083e-05 |
|   2681 | /                        | 66.69.65.32     |           2.18083e-05 |
|   2682 | /                        | 66.69.79.50     |           2.18083e-05 |
|   2683 | /                        | 66.90.156.73    |           2.18083e-05 |
|   2684 | /                        | 67.10.129.157   |           2.18083e-05 |
|   2685 | /                        | 67.10.154.68    |           2.18083e-05 |
|   2686 | /                        | 67.10.164.203   |           2.18083e-05 |
|   2687 | /                        | 67.10.165.185   |           2.18083e-05 |
|   2688 | /                        | 67.10.185.18    |           2.18083e-05 |
|   2689 | /                        | 67.10.186.142   |           2.18083e-05 |
|   2690 | /                        | 67.10.213.134   |           2.18083e-05 |
|   2691 | /                        | 67.11.100.255   |           2.18083e-05 |
|   2692 | /                        | 67.11.137.215   |           2.18083e-05 |
|   2693 | /                        | 67.11.16.255    |           2.18083e-05 |
|   2694 | /                        | 67.11.167.95    |           2.18083e-05 |
|   2695 | /                        | 67.11.199.195   |           2.18083e-05 |
|   2696 | /                        | 67.11.236.189   |           2.18083e-05 |
|   2697 | /                        | 67.11.35.239    |           2.18083e-05 |
|   2698 | /                        | 67.11.35.80     |           2.18083e-05 |
|   2699 | /                        | 67.11.52.225    |           2.18083e-05 |
|   2700 | /                        | 67.11.84.66     |           2.18083e-05 |
|   2701 | /                        | 67.11.86.130    |           2.18083e-05 |
|   2702 | /                        | 67.198.103.97   |           2.18083e-05 |
|   2703 | /                        | 67.198.116.192  |           2.18083e-05 |
|   2704 | /                        | 67.214.78.138   |           2.18083e-05 |
|   2705 | /                        | 67.44.176.135   |           2.18083e-05 |
|   2706 | /                        | 67.48.50.218    |           2.18083e-05 |
|   2707 | /                        | 67.48.59.123    |           2.18083e-05 |
|   2708 | /                        | 67.54.151.131   |           2.18083e-05 |
|   2709 | /                        | 67.7.3.203      |           2.18083e-05 |
|   2710 | /                        | 67.7.7.139      |           2.18083e-05 |
|   2711 | /                        | 67.78.73.114    |           2.18083e-05 |
|   2712 | /                        | 67.79.15.43     |           2.18083e-05 |
|   2713 | /                        | 67.79.20.162    |           2.18083e-05 |
|   2714 | /                        | 67.79.36.55     |           2.18083e-05 |
|   2715 | /                        | 67.79.40.22     |           2.18083e-05 |
|   2716 | /                        | 67.79.62.147    |           2.18083e-05 |
|   2717 | /                        | 68.103.108.6    |           2.18083e-05 |
|   2718 | /                        | 68.134.170.110  |           2.18083e-05 |
|   2719 | /                        | 68.201.208.69   |           2.18083e-05 |
|   2720 | /                        | 68.201.240.17   |           2.18083e-05 |
|   2721 | /                        | 68.201.249.18   |           2.18083e-05 |
|   2722 | /                        | 68.201.78.226   |           2.18083e-05 |
|   2723 | /                        | 68.203.198.221  |           2.18083e-05 |
|   2724 | /                        | 68.206.161.84   |           2.18083e-05 |
|   2725 | /                        | 68.206.249.115  |           2.18083e-05 |
|   2726 | /                        | 68.227.97.247   |           2.18083e-05 |
|   2727 | /                        | 68.241.59.80    |           2.18083e-05 |
|   2728 | /                        | 68.47.165.83    |           2.18083e-05 |
|   2729 | /                        | 68.73.197.187   |           2.18083e-05 |
|   2730 | /                        | 69.12.94.123    |           2.18083e-05 |
|   2731 | /                        | 69.130.123.181  |           2.18083e-05 |
|   2732 | /                        | 69.148.0.213    |           2.18083e-05 |
|   2733 | /                        | 69.149.54.202   |           2.18083e-05 |
|   2734 | /                        | 69.154.52.90    |           2.18083e-05 |
|   2735 | /                        | 69.160.252.231  |           2.18083e-05 |
|   2736 | /                        | 69.166.80.80    |           2.18083e-05 |
|   2737 | /                        | 69.172.183.120  |           2.18083e-05 |
|   2738 | /                        | 69.232.101.106  |           2.18083e-05 |
|   2739 | /                        | 69.59.85.121    |           2.18083e-05 |
|   2740 | /                        | 69.9.44.150     |           2.18083e-05 |
|   2741 | /                        | 69.9.44.152     |           2.18083e-05 |
|   2742 | /                        | 69.9.44.166     |           2.18083e-05 |
|   2743 | /                        | 69.9.44.167     |           2.18083e-05 |
|   2744 | /                        | 69.9.44.186     |           2.18083e-05 |
|   2745 | /                        | 69.9.44.217     |           2.18083e-05 |
|   2746 | /                        | 70.112.123.248  |           2.18083e-05 |
|   2747 | /                        | 70.113.20.26    |           2.18083e-05 |
|   2748 | /                        | 70.113.50.67    |           2.18083e-05 |
|   2749 | /                        | 70.113.52.10    |           2.18083e-05 |
|   2750 | /                        | 70.114.239.51   |           2.18083e-05 |
|   2751 | /                        | 70.114.28.44    |           2.18083e-05 |
|   2752 | /                        | 70.114.3.78     |           2.18083e-05 |
|   2753 | /                        | 70.117.19.13    |           2.18083e-05 |
|   2754 | /                        | 70.117.29.41    |           2.18083e-05 |
|   2755 | /                        | 70.117.71.59    |           2.18083e-05 |
|   2756 | /                        | 70.117.9.9      |           2.18083e-05 |
|   2757 | /                        | 70.118.48.163   |           2.18083e-05 |
|   2758 | /                        | 70.118.9.107    |           2.18083e-05 |
|   2759 | /                        | 70.119.130.42   |           2.18083e-05 |
|   2760 | /                        | 70.120.15.165   |           2.18083e-05 |
|   2761 | /                        | 70.120.28.245   |           2.18083e-05 |
|   2762 | /                        | 70.120.28.81    |           2.18083e-05 |
|   2763 | /                        | 70.120.3.91     |           2.18083e-05 |
|   2764 | /                        | 70.120.39.192   |           2.18083e-05 |
|   2765 | /                        | 70.120.43.13    |           2.18083e-05 |
|   2766 | /                        | 70.120.49.220   |           2.18083e-05 |
|   2767 | /                        | 70.120.75.219   |           2.18083e-05 |
|   2768 | /                        | 70.120.93.125   |           2.18083e-05 |
|   2769 | /                        | 70.121.136.2    |           2.18083e-05 |
|   2770 | /                        | 70.121.160.151  |           2.18083e-05 |
|   2771 | /                        | 70.121.171.212  |           2.18083e-05 |
|   2772 | /                        | 70.121.182.222  |           2.18083e-05 |
|   2773 | /                        | 70.121.184.72   |           2.18083e-05 |
|   2774 | /                        | 70.121.187.122  |           2.18083e-05 |
|   2775 | /                        | 70.121.188.16   |           2.18083e-05 |
|   2776 | /                        | 70.121.191.85   |           2.18083e-05 |
|   2777 | /                        | 70.121.196.204  |           2.18083e-05 |
|   2778 | /                        | 70.121.216.135  |           2.18083e-05 |
|   2779 | /                        | 70.121.228.51   |           2.18083e-05 |
|   2780 | /                        | 70.122.206.138  |           2.18083e-05 |
|   2781 | /                        | 70.122.230.225  |           2.18083e-05 |
|   2782 | /                        | 70.122.37.185   |           2.18083e-05 |
|   2783 | /                        | 70.122.42.101   |           2.18083e-05 |
|   2784 | /                        | 70.123.110.205  |           2.18083e-05 |
|   2785 | /                        | 70.123.161.21   |           2.18083e-05 |
|   2786 | /                        | 70.123.195.48   |           2.18083e-05 |
|   2787 | /                        | 70.123.205.224  |           2.18083e-05 |
|   2788 | /                        | 70.123.233.35   |           2.18083e-05 |
|   2789 | /                        | 70.123.252.47   |           2.18083e-05 |
|   2790 | /                        | 70.124.99.231   |           2.18083e-05 |
|   2791 | /                        | 70.125.138.191  |           2.18083e-05 |
|   2792 | /                        | 70.125.150.41   |           2.18083e-05 |
|   2793 | /                        | 70.125.151.73   |           2.18083e-05 |
|   2794 | /                        | 70.125.199.213  |           2.18083e-05 |
|   2795 | /                        | 70.125.211.106  |           2.18083e-05 |
|   2796 | /                        | 70.125.229.227  |           2.18083e-05 |
|   2797 | /                        | 70.125.99.193   |           2.18083e-05 |
|   2798 | /                        | 70.238.240.119  |           2.18083e-05 |
|   2799 | /                        | 70.243.68.60    |           2.18083e-05 |
|   2800 | /                        | 70.40.21.86     |           2.18083e-05 |
|   2801 | /                        | 70.40.28.106    |           2.18083e-05 |
|   2802 | /                        | 70.40.29.112    |           2.18083e-05 |
|   2803 | /                        | 70.94.129.79    |           2.18083e-05 |
|   2804 | /                        | 70.94.160.147   |           2.18083e-05 |
|   2805 | /                        | 70.94.167.166   |           2.18083e-05 |
|   2806 | /                        | 70.94.183.166   |           2.18083e-05 |
|   2807 | /                        | 71.132.254.162  |           2.18083e-05 |
|   2808 | /                        | 71.136.250.27   |           2.18083e-05 |
|   2809 | /                        | 71.136.252.180  |           2.18083e-05 |
|   2810 | /                        | 71.136.255.148  |           2.18083e-05 |
|   2811 | /                        | 71.146.33.118   |           2.18083e-05 |
|   2812 | /                        | 71.183.39.187   |           2.18083e-05 |
|   2813 | /                        | 71.221.223.195  |           2.18083e-05 |
|   2814 | /                        | 71.221.235.149  |           2.18083e-05 |
|   2815 | /                        | 71.235.91.18    |           2.18083e-05 |
|   2816 | /                        | 71.40.1.172     |           2.18083e-05 |
|   2817 | /                        | 71.40.12.162    |           2.18083e-05 |
|   2818 | /                        | 71.41.156.141   |           2.18083e-05 |
|   2819 | /                        | 71.41.242.68    |           2.18083e-05 |
|   2820 | /                        | 71.41.242.76    |           2.18083e-05 |
|   2821 | /                        | 71.41.243.19    |           2.18083e-05 |
|   2822 | /                        | 71.42.105.146   |           2.18083e-05 |
|   2823 | /                        | 71.42.105.182   |           2.18083e-05 |
|   2824 | /                        | 71.42.218.131   |           2.18083e-05 |
|   2825 | /                        | 71.42.245.250   |           2.18083e-05 |
|   2826 | /                        | 71.77.210.52    |           2.18083e-05 |
|   2827 | /                        | 71.78.167.131   |           2.18083e-05 |
|   2828 | /                        | 71.78.185.26    |           2.18083e-05 |
|   2829 | /                        | 71.78.189.122   |           2.18083e-05 |
|   2830 | /                        | 71.78.236.131   |           2.18083e-05 |
|   2831 | /                        | 71.78.37.150    |           2.18083e-05 |
|   2832 | /                        | 71.78.46.154    |           2.18083e-05 |
|   2833 | /                        | 71.81.176.46    |           2.18083e-05 |
|   2834 | /                        | 72.128.139.130  |           2.18083e-05 |
|   2835 | /                        | 72.128.140.202  |           2.18083e-05 |
|   2836 | /                        | 72.177.116.71   |           2.18083e-05 |
|   2837 | /                        | 72.177.173.161  |           2.18083e-05 |
|   2838 | /                        | 72.177.174.171  |           2.18083e-05 |
|   2839 | /                        | 72.177.209.244  |           2.18083e-05 |
|   2840 | /                        | 72.177.229.125  |           2.18083e-05 |
|   2841 | /                        | 72.177.230.241  |           2.18083e-05 |
|   2842 | /                        | 72.177.233.3    |           2.18083e-05 |
|   2843 | /                        | 72.177.237.78   |           2.18083e-05 |
|   2844 | /                        | 72.177.245.198  |           2.18083e-05 |
|   2845 | /                        | 72.178.188.133  |           2.18083e-05 |
|   2846 | /                        | 72.178.76.70    |           2.18083e-05 |
|   2847 | /                        | 72.179.161.188  |           2.18083e-05 |
|   2848 | /                        | 72.179.183.68   |           2.18083e-05 |
|   2849 | /                        | 72.179.184.241  |           2.18083e-05 |
|   2850 | /                        | 72.180.171.13   |           2.18083e-05 |
|   2851 | /                        | 72.180.46.191   |           2.18083e-05 |
|   2852 | /                        | 72.180.53.75    |           2.18083e-05 |
|   2853 | /                        | 72.181.100.92   |           2.18083e-05 |
|   2854 | /                        | 72.181.101.87   |           2.18083e-05 |
|   2855 | /                        | 72.181.103.160  |           2.18083e-05 |
|   2856 | /                        | 72.181.112.227  |           2.18083e-05 |
|   2857 | /                        | 72.181.113.170  |           2.18083e-05 |
|   2858 | /                        | 72.181.117.243  |           2.18083e-05 |
|   2859 | /                        | 72.181.120.120  |           2.18083e-05 |
|   2860 | /                        | 72.181.120.63   |           2.18083e-05 |
|   2861 | /                        | 72.181.121.127  |           2.18083e-05 |
|   2862 | /                        | 72.181.121.84   |           2.18083e-05 |
|   2863 | /                        | 72.181.126.124  |           2.18083e-05 |
|   2864 | /                        | 72.181.37.22    |           2.18083e-05 |
|   2865 | /                        | 72.181.97.207   |           2.18083e-05 |
|   2866 | /                        | 72.181.97.35    |           2.18083e-05 |
|   2867 | /                        | 72.181.98.126   |           2.18083e-05 |
|   2868 | /                        | 72.181.99.110   |           2.18083e-05 |
|   2869 | /                        | 72.190.128.167  |           2.18083e-05 |
|   2870 | /                        | 72.190.130.87   |           2.18083e-05 |
|   2871 | /                        | 72.190.165.220  |           2.18083e-05 |
|   2872 | /                        | 72.190.178.21   |           2.18083e-05 |
|   2873 | /                        | 72.190.180.197  |           2.18083e-05 |
|   2874 | /                        | 72.190.185.235  |           2.18083e-05 |
|   2875 | /                        | 72.190.221.42   |           2.18083e-05 |
|   2876 | /                        | 72.190.232.108  |           2.18083e-05 |
|   2877 | /                        | 72.190.237.180  |           2.18083e-05 |
|   2878 | /                        | 72.190.36.102   |           2.18083e-05 |
|   2879 | /                        | 72.191.140.150  |           2.18083e-05 |
|   2880 | /                        | 72.191.141.122  |           2.18083e-05 |
|   2881 | /                        | 72.191.22.4     |           2.18083e-05 |
|   2882 | /                        | 72.191.26.144   |           2.18083e-05 |
|   2883 | /                        | 72.191.29.194   |           2.18083e-05 |
|   2884 | /                        | 72.191.30.213   |           2.18083e-05 |
|   2885 | /                        | 72.191.32.140   |           2.18083e-05 |
|   2886 | /                        | 72.191.4.136    |           2.18083e-05 |
|   2887 | /                        | 72.205.179.123  |           2.18083e-05 |
|   2888 | /                        | 72.205.214.27   |           2.18083e-05 |
|   2889 | /                        | 72.206.103.198  |           2.18083e-05 |
|   2890 | /                        | 72.21.196.64    |           2.18083e-05 |
|   2891 | /                        | 72.45.179.227   |           2.18083e-05 |
|   2892 | /                        | 72.48.1.25      |           2.18083e-05 |
|   2893 | /                        | 72.48.11.18     |           2.18083e-05 |
|   2894 | /                        | 72.48.254.209   |           2.18083e-05 |
|   2895 | /                        | 72.48.60.84     |           2.18083e-05 |
|   2896 | /                        | 73.115.118.19   |           2.18083e-05 |
|   2897 | /                        | 73.115.127.168  |           2.18083e-05 |
|   2898 | /                        | 73.119.63.36    |           2.18083e-05 |
|   2899 | /                        | 73.14.59.241    |           2.18083e-05 |
|   2900 | /                        | 73.153.134.112  |           2.18083e-05 |
|   2901 | /                        | 73.158.71.104   |           2.18083e-05 |
|   2902 | /                        | 73.166.32.184   |           2.18083e-05 |
|   2903 | /                        | 73.240.127.111  |           2.18083e-05 |
|   2904 | /                        | 73.31.215.224   |           2.18083e-05 |
|   2905 | /                        | 73.76.119.133   |           2.18083e-05 |
|   2906 | /                        | 73.85.100.144   |           2.18083e-05 |
|   2907 | /                        | 74.197.69.72    |           2.18083e-05 |
|   2908 | /                        | 74.75.232.45    |           2.18083e-05 |
|   2909 | /                        | 74.81.88.114    |           2.18083e-05 |
|   2910 | /                        | 74.81.88.122    |           2.18083e-05 |
|   2911 | /                        | 74.81.88.42     |           2.18083e-05 |
|   2912 | /                        | 74.81.88.74     |           2.18083e-05 |
|   2913 | /                        | 75.1.128.18     |           2.18083e-05 |
|   2914 | /                        | 75.1.145.2      |           2.18083e-05 |
|   2915 | /                        | 75.1.166.50     |           2.18083e-05 |
|   2916 | /                        | 75.1.201.239    |           2.18083e-05 |
|   2917 | /                        | 75.1.32.205     |           2.18083e-05 |
|   2918 | /                        | 75.1.44.127     |           2.18083e-05 |
|   2919 | /                        | 75.15.242.245   |           2.18083e-05 |
|   2920 | /                        | 75.28.18.125    |           2.18083e-05 |
|   2921 | /                        | 75.41.19.43     |           2.18083e-05 |
|   2922 | /                        | 75.54.32.17     |           2.18083e-05 |
|   2923 | /                        | 75.54.32.20     |           2.18083e-05 |
|   2924 | /                        | 75.54.32.79     |           2.18083e-05 |
|   2925 | /                        | 75.54.32.9      |           2.18083e-05 |
|   2926 | /                        | 75.54.33.109    |           2.18083e-05 |
|   2927 | /                        | 75.54.33.161    |           2.18083e-05 |
|   2928 | /                        | 75.54.33.183    |           2.18083e-05 |
|   2929 | /                        | 75.54.33.188    |           2.18083e-05 |
|   2930 | /                        | 75.54.33.190    |           2.18083e-05 |
|   2931 | /                        | 75.57.25.192    |           2.18083e-05 |
|   2932 | /                        | 75.81.88.191    |           2.18083e-05 |
|   2933 | /                        | 75.87.28.58     |           2.18083e-05 |
|   2934 | /                        | 76.182.166.199  |           2.18083e-05 |
|   2935 | /                        | 76.183.251.188  |           2.18083e-05 |
|   2936 | /                        | 76.185.136.75   |           2.18083e-05 |
|   2937 | /                        | 76.185.158.93   |           2.18083e-05 |
|   2938 | /                        | 76.185.168.145  |           2.18083e-05 |
|   2939 | /                        | 76.185.17.248   |           2.18083e-05 |
|   2940 | /                        | 76.185.175.160  |           2.18083e-05 |
|   2941 | /                        | 76.185.176.98   |           2.18083e-05 |
|   2942 | /                        | 76.185.206.252  |           2.18083e-05 |
|   2943 | /                        | 76.185.216.146  |           2.18083e-05 |
|   2944 | /                        | 76.185.227.82   |           2.18083e-05 |
|   2945 | /                        | 76.185.252.88   |           2.18083e-05 |
|   2946 | /                        | 76.186.167.31   |           2.18083e-05 |
|   2947 | /                        | 76.186.2.234    |           2.18083e-05 |
|   2948 | /                        | 76.186.27.252   |           2.18083e-05 |
|   2949 | /                        | 76.187.101.165  |           2.18083e-05 |
|   2950 | /                        | 76.193.64.119   |           2.18083e-05 |
|   2951 | /                        | 76.193.65.187   |           2.18083e-05 |
|   2952 | /                        | 76.214.115.41   |           2.18083e-05 |
|   2953 | /                        | 76.228.200.216  |           2.18083e-05 |
|   2954 | /                        | 76.237.206.217  |           2.18083e-05 |
|   2955 | /                        | 76.250.148.104  |           2.18083e-05 |
|   2956 | /                        | 76.85.101.80    |           2.18083e-05 |
|   2957 | /                        | 77.111.247.10   |           2.18083e-05 |
|   2958 | /                        | 8.31.0.128      |           2.18083e-05 |
|   2959 | /                        | 80.137.93.59    |           2.18083e-05 |
|   2960 | /                        | 84.14.10.91     |           2.18083e-05 |
|   2961 | /                        | 84.17.45.159    |           2.18083e-05 |
|   2962 | /                        | 85.106.100.99   |           2.18083e-05 |
|   2963 | /                        | 85.106.99.108   |           2.18083e-05 |
|   2964 | /                        | 87.158.135.133  |           2.18083e-05 |
|   2965 | /                        | 88.241.49.202   |           2.18083e-05 |
|   2966 | /                        | 89.187.164.129  |           2.18083e-05 |
|   2967 | /                        | 89.187.164.131  |           2.18083e-05 |
|   2968 | /                        | 89.187.164.145  |           2.18083e-05 |
|   2969 | /                        | 89.187.164.146  |           2.18083e-05 |
|   2970 | /                        | 89.187.164.147  |           2.18083e-05 |
|   2971 | /                        | 89.187.164.151  |           2.18083e-05 |
|   2972 | /                        | 89.187.164.161  |           2.18083e-05 |
|   2973 | /                        | 89.187.164.164  |           2.18083e-05 |
|   2974 | /                        | 89.187.164.165  |           2.18083e-05 |
|   2975 | /                        | 89.187.164.166  |           2.18083e-05 |
|   2976 | /                        | 89.187.164.75   |           2.18083e-05 |
|   2977 | /                        | 89.187.175.105  |           2.18083e-05 |
|   2978 | /                        | 89.187.175.149  |           2.18083e-05 |
|   2979 | /                        | 89.187.175.48   |           2.18083e-05 |
|   2980 | /                        | 89.81.166.143   |           2.18083e-05 |
|   2981 | /                        | 91.132.138.164  |           2.18083e-05 |
|   2982 | /                        | 92.38.148.10    |           2.18083e-05 |
|   2983 | /                        | 92.38.148.65    |           2.18083e-05 |
|   2984 | /                        | 96.35.248.138   |           2.18083e-05 |
|   2985 | /                        | 96.40.234.116   |           2.18083e-05 |
|   2986 | /                        | 96.44.144.194   |           2.18083e-05 |
|   2987 | /                        | 96.46.160.20    |           2.18083e-05 |
|   2988 | /                        | 96.46.160.21    |           2.18083e-05 |
|   2989 | /                        | 96.64.121.249   |           2.18083e-05 |
|   2990 | /                        | 96.64.210.98    |           2.18083e-05 |
|   2991 | /                        | 96.77.154.186   |           2.18083e-05 |
|   2992 | /                        | 96.8.128.193    |           2.18083e-05 |
|   2993 | /                        | 96.8.128.205    |           2.18083e-05 |
|   2994 | /                        | 96.8.128.98     |           2.18083e-05 |
|   2995 | /                        | 96.8.130.11     |           2.18083e-05 |
|   2996 | /                        | 96.8.130.167    |           2.18083e-05 |
|   2997 | /                        | 96.8.130.194    |           2.18083e-05 |
|   2998 | /                        | 96.8.130.197    |           2.18083e-05 |
|   2999 | /                        | 96.8.130.209    |           2.18083e-05 |
|   3000 | /                        | 96.8.160.28     |           2.18083e-05 |
|   3001 | /                        | 96.8.177.123    |           2.18083e-05 |
|   3002 | /                        | 96.8.178.119    |           2.18083e-05 |
|   3003 | /                        | 96.8.178.150    |           2.18083e-05 |
|   3004 | /                        | 96.8.178.180    |           2.18083e-05 |
|   3005 | /                        | 96.8.178.202    |           2.18083e-05 |
|   3006 | /                        | 96.8.178.97     |           2.18083e-05 |
|   3007 | /                        | 96.8.193.160    |           2.18083e-05 |
|   3008 | /                        | 96.8.248.115    |           2.18083e-05 |
|   3009 | /                        | 96.8.248.136    |           2.18083e-05 |
|   3010 | /                        | 96.89.11.37     |           2.18083e-05 |
|   3011 | /                        | 97.100.13.134   |           2.18083e-05 |
|   3012 | /                        | 97.105.101.131  |           2.18083e-05 |
|   3013 | /                        | 97.105.102.61   |           2.18083e-05 |
|   3014 | /                        | 97.105.164.91   |           2.18083e-05 |
|   3015 | /                        | 97.105.40.214   |           2.18083e-05 |
|   3016 | /                        | 97.105.54.251   |           2.18083e-05 |
|   3017 | /                        | 97.105.62.162   |           2.18083e-05 |
|   3018 | /                        | 97.105.69.130   |           2.18083e-05 |
|   3019 | /                        | 97.105.81.147   |           2.18083e-05 |
|   3020 | /                        | 97.118.255.115  |           2.18083e-05 |
|   3021 | /                        | 97.123.26.103   |           2.18083e-05 |
|   3022 | /                        | 97.77.146.130   |           2.18083e-05 |
|   3023 | /                        | 97.77.16.42     |           2.18083e-05 |
|   3024 | /                        | 97.77.203.60    |           2.18083e-05 |
|   3025 | /                        | 97.77.206.98    |           2.18083e-05 |
|   3026 | /                        | 97.77.227.131   |           2.18083e-05 |
|   3027 | /                        | 97.77.27.166    |           2.18083e-05 |
|   3028 | /                        | 97.79.128.78    |           2.18083e-05 |
|   3029 | /                        | 97.79.140.198   |           2.18083e-05 |
|   3030 | /                        | 97.98.105.129   |           2.18083e-05 |
|   3031 | /                        | 97.98.108.135   |           2.18083e-05 |
|   3032 | /                        | 97.98.3.168     |           2.18083e-05 |
|   3033 | /                        | 97.99.68.133    |           2.18083e-05 |
|   3034 | /                        | 98.158.241.75   |           2.18083e-05 |
|   3035 | /                        | 98.158.244.8    |           2.18083e-05 |
|   3036 | /                        | 98.6.1.177      |           2.18083e-05 |
|   3037 | /                        | 98.6.147.203    |           2.18083e-05 |
|   3038 | /                        | 98.6.203.30     |           2.18083e-05 |
|   3039 | /                        | 98.6.90.2       |           2.18083e-05 |
|   3040 | /                        | 99.147.239.234  |           2.18083e-05 |
|   3041 | /                        | 99.158.248.118  |           2.18083e-05 |
|   3042 | /                        | 99.158.251.110  |           2.18083e-05 |
|   3043 | /                        | 99.162.240.204  |           2.18083e-05 |
|   3044 | /                        | 99.203.143.2    |           2.18083e-05 |
|   3045 | /                        | 99.203.154.100  |           2.18083e-05 |
|   3046 | /                        | 99.203.154.104  |           2.18083e-05 |
|   3047 | /                        | 99.203.154.130  |           2.18083e-05 |
|   3048 | /                        | 99.203.154.173  |           2.18083e-05 |
|   3049 | /                        | 99.203.154.182  |           2.18083e-05 |
|   3050 | /                        | 99.203.154.231  |           2.18083e-05 |
|   3051 | /                        | 99.203.154.244  |           2.18083e-05 |
|   3052 | /                        | 99.203.154.250  |           2.18083e-05 |
|   3053 | /                        | 99.203.154.58   |           2.18083e-05 |
|   3054 | /                        | 99.203.154.85   |           2.18083e-05 |
|   3055 | /                        | 99.203.154.89   |           2.18083e-05 |
|   3056 | /                        | 99.203.154.95   |           2.18083e-05 |
|   3057 | /                        | 99.203.154.98   |           2.18083e-05 |
|   3058 | /                        | 99.203.155.120  |           2.18083e-05 |
|   3059 | /                        | 99.203.155.126  |           2.18083e-05 |
|   3060 | /                        | 99.203.155.134  |           2.18083e-05 |
|   3061 | /                        | 99.203.155.145  |           2.18083e-05 |
|   3062 | /                        | 99.203.155.147  |           2.18083e-05 |
|   3063 | /                        | 99.203.155.177  |           2.18083e-05 |
|   3064 | /                        | 99.203.155.182  |           2.18083e-05 |
|   3065 | /                        | 99.203.155.78   |           2.18083e-05 |
|   3066 | /                        | 99.203.155.98   |           2.18083e-05 |
|   3067 | /                        | 99.203.212.119  |           2.18083e-05 |
|   3068 | /                        | 99.203.212.122  |           2.18083e-05 |
|   3069 | /                        | 99.203.212.173  |           2.18083e-05 |
|   3070 | /                        | 99.203.212.200  |           2.18083e-05 |
|   3071 | /                        | 99.203.212.205  |           2.18083e-05 |
|   3072 | /                        | 99.203.212.207  |           2.18083e-05 |
|   3073 | /                        | 99.203.212.61   |           2.18083e-05 |
|   3074 | /                        | 99.203.213.209  |           2.18083e-05 |
|   3075 | /                        | 99.203.213.32   |           2.18083e-05 |
|   3076 | /                        | 99.203.213.59   |           2.18083e-05 |
|   3077 | /                        | 99.203.22.172   |           2.18083e-05 |
|   3078 | /                        | 99.203.22.39    |           2.18083e-05 |
|   3079 | /                        | 99.203.23.16    |           2.18083e-05 |
|   3080 | /                        | 99.203.23.170   |           2.18083e-05 |
|   3081 | /                        | 99.203.26.103   |           2.18083e-05 |
|   3082 | /                        | 99.203.26.115   |           2.18083e-05 |
|   3083 | /                        | 99.203.26.125   |           2.18083e-05 |
|   3084 | /                        | 99.203.26.138   |           2.18083e-05 |
|   3085 | /                        | 99.203.26.146   |           2.18083e-05 |
|   3086 | /                        | 99.203.26.158   |           2.18083e-05 |
|   3087 | /                        | 99.203.26.190   |           2.18083e-05 |
|   3088 | /                        | 99.203.26.199   |           2.18083e-05 |
|   3089 | /                        | 99.203.26.200   |           2.18083e-05 |
|   3090 | /                        | 99.203.26.208   |           2.18083e-05 |
|   3091 | /                        | 99.203.26.214   |           2.18083e-05 |
|   3092 | /                        | 99.203.26.216   |           2.18083e-05 |
|   3093 | /                        | 99.203.26.237   |           2.18083e-05 |
|   3094 | /                        | 99.203.26.41    |           2.18083e-05 |
|   3095 | /                        | 99.203.26.48    |           2.18083e-05 |
|   3096 | /                        | 99.203.26.5     |           2.18083e-05 |
|   3097 | /                        | 99.203.26.55    |           2.18083e-05 |
|   3098 | /                        | 99.203.26.80    |           2.18083e-05 |
|   3099 | /                        | 99.203.26.86    |           2.18083e-05 |
|   3100 | /                        | 99.203.26.9     |           2.18083e-05 |
|   3101 | /                        | 99.203.26.93    |           2.18083e-05 |
|   3102 | /                        | 99.203.27.1     |           2.18083e-05 |
|   3103 | /                        | 99.203.27.10    |           2.18083e-05 |
|   3104 | /                        | 99.203.27.103   |           2.18083e-05 |
|   3105 | /                        | 99.203.27.127   |           2.18083e-05 |
|   3106 | /                        | 99.203.27.141   |           2.18083e-05 |
|   3107 | /                        | 99.203.27.150   |           2.18083e-05 |
|   3108 | /                        | 99.203.27.176   |           2.18083e-05 |
|   3109 | /                        | 99.203.27.180   |           2.18083e-05 |
|   3110 | /                        | 99.203.27.185   |           2.18083e-05 |
|   3111 | /                        | 99.203.27.199   |           2.18083e-05 |
|   3112 | /                        | 99.203.27.213   |           2.18083e-05 |
|   3113 | /                        | 99.203.27.218   |           2.18083e-05 |
|   3114 | /                        | 99.203.27.231   |           2.18083e-05 |
|   3115 | /                        | 99.203.27.241   |           2.18083e-05 |
|   3116 | /                        | 99.203.27.254   |           2.18083e-05 |
|   3117 | /                        | 99.203.27.36    |           2.18083e-05 |
|   3118 | /                        | 99.203.27.45    |           2.18083e-05 |
|   3119 | /                        | 99.203.27.83    |           2.18083e-05 |
|   3120 | /                        | 99.203.27.90    |           2.18083e-05 |
|   3121 | /                        | 99.203.27.94    |           2.18083e-05 |
|   3122 | /                        | 99.203.31.78    |           2.18083e-05 |
|   3123 | /                        | 99.203.90.115   |           2.18083e-05 |
|   3124 | /                        | 99.203.90.140   |           2.18083e-05 |
|   3125 | /                        | 99.203.90.176   |           2.18083e-05 |
|   3126 | /                        | 99.203.90.208   |           2.18083e-05 |
|   3127 | /                        | 99.203.90.71    |           2.18083e-05 |
|   3128 | /                        | 99.203.90.84    |           2.18083e-05 |
|   3129 | /                        | 99.203.91.132   |           2.18083e-05 |
|   3130 | /                        | 99.48.230.105   |           2.18083e-05 |
|   3131 | /                        | 99.52.78.67     |           2.18083e-05 |
|   3132 | /                        | 99.57.20.163    |           2.18083e-05 |
|   3133 | /                        | 99.61.71.158    |           2.18083e-05 |
|   3134 | /                        | 99.62.48.254    |           2.18083e-05 |
|   3135 | /                        | 99.66.104.96    |           2.18083e-05 |
|   3136 | /                        | 99.97.208.154   |           2.18083e-05 |
|  36313 | html-css                 | 104.190.250.60  |           7.61789e-05 |
|  36314 | html-css                 | 104.191.12.108  |           7.61789e-05 |
|  36315 | html-css                 | 104.192.171.99  |           7.61789e-05 |
|  36316 | html-css                 | 104.218.78.12   |           7.61789e-05 |
|  36317 | html-css                 | 104.219.136.53  |           7.61789e-05 |
|  36318 | html-css                 | 104.237.80.15   |           7.61789e-05 |
|  36319 | html-css                 | 104.237.80.150  |           7.61789e-05 |
|  36320 | html-css                 | 104.237.80.167  |           7.61789e-05 |
|  36321 | html-css                 | 104.237.80.77   |           7.61789e-05 |
|  36322 | html-css                 | 104.238.45.20   |           7.61789e-05 |
|  36323 | html-css                 | 104.48.213.50   |           7.61789e-05 |
|  36324 | html-css                 | 104.5.159.83    |           7.61789e-05 |
|  36325 | html-css                 | 104.53.200.122  |           7.61789e-05 |
|  36326 | html-css                 | 104.58.135.163  |           7.61789e-05 |
|  36327 | html-css                 | 107.117.225.105 |           7.61789e-05 |
|  36328 | html-css                 | 107.129.209.3   |           7.61789e-05 |
|  36329 | html-css                 | 107.132.199.23  |           7.61789e-05 |
|  36330 | html-css                 | 107.138.213.232 |           7.61789e-05 |
|  36331 | html-css                 | 107.146.141.88  |           7.61789e-05 |
|  36332 | html-css                 | 107.204.52.180  |           7.61789e-05 |
|  36333 | html-css                 | 107.207.7.120   |           7.61789e-05 |
|  36334 | html-css                 | 107.242.125.32  |           7.61789e-05 |
|  36335 | html-css                 | 107.77.100.18   |           7.61789e-05 |
|  36336 | html-css                 | 107.77.105.36   |           7.61789e-05 |
|  36337 | html-css                 | 107.77.169.54   |           7.61789e-05 |
|  36338 | html-css                 | 107.77.169.7    |           7.61789e-05 |
|  36339 | html-css                 | 107.77.197.208  |           7.61789e-05 |
|  36340 | html-css                 | 107.77.200.166  |           7.61789e-05 |
|  36341 | html-css                 | 107.77.217.151  |           7.61789e-05 |
|  36342 | html-css                 | 107.77.217.156  |           7.61789e-05 |
|  36343 | html-css                 | 107.77.218.37   |           7.61789e-05 |
|  36344 | html-css                 | 107.77.218.64   |           7.61789e-05 |
|  36345 | html-css                 | 107.77.219.108  |           7.61789e-05 |
|  36346 | html-css                 | 107.77.219.141  |           7.61789e-05 |
|  36347 | html-css                 | 107.77.219.187  |           7.61789e-05 |
|  36348 | html-css                 | 107.77.219.67   |           7.61789e-05 |
|  36349 | html-css                 | 107.77.220.130  |           7.61789e-05 |
|  36350 | html-css                 | 107.77.220.202  |           7.61789e-05 |
|  36351 | html-css                 | 107.77.221.10   |           7.61789e-05 |
|  36352 | html-css                 | 107.77.221.101  |           7.61789e-05 |
|  36353 | html-css                 | 107.77.221.116  |           7.61789e-05 |
|  36354 | html-css                 | 107.77.221.150  |           7.61789e-05 |
|  36355 | html-css                 | 107.77.221.91   |           7.61789e-05 |
|  36356 | html-css                 | 107.77.222.127  |           7.61789e-05 |
|  36357 | html-css                 | 107.77.222.141  |           7.61789e-05 |
|  36358 | html-css                 | 107.77.222.142  |           7.61789e-05 |
|  36359 | html-css                 | 107.77.222.159  |           7.61789e-05 |
|  36360 | html-css                 | 107.77.222.165  |           7.61789e-05 |
|  36361 | html-css                 | 107.77.234.226  |           7.61789e-05 |
|  36362 | html-css                 | 107.77.72.127   |           7.61789e-05 |
|  36363 | html-css                 | 107.77.80.29    |           7.61789e-05 |
|  36364 | html-css                 | 107.77.80.32    |           7.61789e-05 |
|  36365 | html-css                 | 107.77.80.85    |           7.61789e-05 |
|  36366 | html-css                 | 108.178.64.170  |           7.61789e-05 |
|  36367 | html-css                 | 108.207.130.89  |           7.61789e-05 |
|  36368 | html-css                 | 108.222.98.137  |           7.61789e-05 |
|  36369 | html-css                 | 12.189.101.90   |           7.61789e-05 |
|  36370 | html-css                 | 12.247.3.42     |           7.61789e-05 |
|  36371 | html-css                 | 12.33.49.18     |           7.61789e-05 |
|  36372 | html-css                 | 12.52.157.98    |           7.61789e-05 |
|  36373 | html-css                 | 12.71.181.245   |           7.61789e-05 |
|  36374 | html-css                 | 12.91.190.130   |           7.61789e-05 |
|  36375 | html-css                 | 129.112.109.40  |           7.61789e-05 |
|  36376 | html-css                 | 130.45.33.209   |           7.61789e-05 |
|  36377 | html-css                 | 130.45.35.109   |           7.61789e-05 |
|  36378 | html-css                 | 130.45.43.174   |           7.61789e-05 |
|  36379 | html-css                 | 130.45.45.231   |           7.61789e-05 |
|  36380 | html-css                 | 130.45.53.218   |           7.61789e-05 |
|  36381 | html-css                 | 131.194.172.217 |           7.61789e-05 |
|  36382 | html-css                 | 131.194.172.221 |           7.61789e-05 |
|  36383 | html-css                 | 131.194.172.36  |           7.61789e-05 |
|  36384 | html-css                 | 131.194.172.51  |           7.61789e-05 |
|  36385 | html-css                 | 136.50.104.158  |           7.61789e-05 |
|  36386 | html-css                 | 136.50.13.147   |           7.61789e-05 |
|  36387 | html-css                 | 136.50.24.253   |           7.61789e-05 |
|  36388 | html-css                 | 136.50.28.52    |           7.61789e-05 |
|  36389 | html-css                 | 136.50.29.193   |           7.61789e-05 |
|  36390 | html-css                 | 136.50.54.38    |           7.61789e-05 |
|  36391 | html-css                 | 137.242.1.129   |           7.61789e-05 |
|  36392 | html-css                 | 148.59.156.46   |           7.61789e-05 |
|  36393 | html-css                 | 154.29.131.27   |           7.61789e-05 |
|  36394 | html-css                 | 154.3.250.165   |           7.61789e-05 |
|  36395 | html-css                 | 154.3.251.130   |           7.61789e-05 |
|  36396 | html-css                 | 159.192.242.43  |           7.61789e-05 |
|  36397 | html-css                 | 162.197.40.243  |           7.61789e-05 |
|  36398 | html-css                 | 162.218.230.219 |           7.61789e-05 |
|  36399 | html-css                 | 162.233.171.61  |           7.61789e-05 |
|  36400 | html-css                 | 162.248.14.168  |           7.61789e-05 |
|  36401 | html-css                 | 162.72.0.173    |           7.61789e-05 |
|  36402 | html-css                 | 162.72.133.33   |           7.61789e-05 |
|  36403 | html-css                 | 165.231.210.20  |           7.61789e-05 |
|  36404 | html-css                 | 166.137.118.47  |           7.61789e-05 |
|  36405 | html-css                 | 166.137.118.83  |           7.61789e-05 |
|  36406 | html-css                 | 166.137.143.123 |           7.61789e-05 |
|  36407 | html-css                 | 166.216.159.156 |           7.61789e-05 |
|  36408 | html-css                 | 172.110.83.150  |           7.61789e-05 |
|  36409 | html-css                 | 172.124.174.241 |           7.61789e-05 |
|  36410 | html-css                 | 172.127.182.225 |           7.61789e-05 |
|  36411 | html-css                 | 172.3.220.202   |           7.61789e-05 |
|  36412 | html-css                 | 172.56.6.71     |           7.61789e-05 |
|  36413 | html-css                 | 172.56.7.119    |           7.61789e-05 |
|  36414 | html-css                 | 172.56.7.28     |           7.61789e-05 |
|  36415 | html-css                 | 172.56.7.43     |           7.61789e-05 |
|  36416 | html-css                 | 172.58.101.191  |           7.61789e-05 |
|  36417 | html-css                 | 172.58.110.145  |           7.61789e-05 |
|  36418 | html-css                 | 172.58.110.189  |           7.61789e-05 |
|  36419 | html-css                 | 172.58.67.201   |           7.61789e-05 |
|  36420 | html-css                 | 172.58.67.44    |           7.61789e-05 |
|  36421 | html-css                 | 172.58.67.77    |           7.61789e-05 |
|  36422 | html-css                 | 172.58.97.127   |           7.61789e-05 |
|  36423 | html-css                 | 172.58.99.155   |           7.61789e-05 |
|  36424 | html-css                 | 172.58.99.43    |           7.61789e-05 |
|  36425 | html-css                 | 172.85.200.202  |           7.61789e-05 |
|  36426 | html-css                 | 172.87.148.22   |           7.61789e-05 |
|  36427 | html-css                 | 172.87.181.101  |           7.61789e-05 |
|  36428 | html-css                 | 172.9.3.42      |           7.61789e-05 |
|  36429 | html-css                 | 173.123.161.160 |           7.61789e-05 |
|  36430 | html-css                 | 173.123.163.49  |           7.61789e-05 |
|  36431 | html-css                 | 173.127.114.36  |           7.61789e-05 |
|  36432 | html-css                 | 173.127.13.65   |           7.61789e-05 |
|  36433 | html-css                 | 173.127.40.170  |           7.61789e-05 |
|  36434 | html-css                 | 173.127.49.144  |           7.61789e-05 |
|  36435 | html-css                 | 173.172.166.90  |           7.61789e-05 |
|  36436 | html-css                 | 173.172.202.149 |           7.61789e-05 |
|  36437 | html-css                 | 173.173.104.14  |           7.61789e-05 |
|  36438 | html-css                 | 173.173.104.202 |           7.61789e-05 |
|  36439 | html-css                 | 173.173.111.218 |           7.61789e-05 |
|  36440 | html-css                 | 173.173.98.244  |           7.61789e-05 |
|  36441 | html-css                 | 173.174.153.61  |           7.61789e-05 |
|  36442 | html-css                 | 173.174.159.45  |           7.61789e-05 |
|  36443 | html-css                 | 173.174.169.114 |           7.61789e-05 |
|  36444 | html-css                 | 173.174.218.223 |           7.61789e-05 |
|  36445 | html-css                 | 173.174.233.67  |           7.61789e-05 |
|  36446 | html-css                 | 173.175.112.130 |           7.61789e-05 |
|  36447 | html-css                 | 173.175.118.119 |           7.61789e-05 |
|  36448 | html-css                 | 173.175.82.164  |           7.61789e-05 |
|  36449 | html-css                 | 173.232.241.228 |           7.61789e-05 |
|  36450 | html-css                 | 173.239.232.103 |           7.61789e-05 |
|  36451 | html-css                 | 173.239.232.106 |           7.61789e-05 |
|  36452 | html-css                 | 173.239.232.119 |           7.61789e-05 |
|  36453 | html-css                 | 173.239.232.134 |           7.61789e-05 |
|  36454 | html-css                 | 173.239.232.167 |           7.61789e-05 |
|  36455 | html-css                 | 173.239.232.174 |           7.61789e-05 |
|  36456 | html-css                 | 173.239.232.39  |           7.61789e-05 |
|  36457 | html-css                 | 173.239.232.41  |           7.61789e-05 |
|  36458 | html-css                 | 173.239.232.43  |           7.61789e-05 |
|  36459 | html-css                 | 173.239.232.46  |           7.61789e-05 |
|  36460 | html-css                 | 173.239.232.47  |           7.61789e-05 |
|  36461 | html-css                 | 173.239.232.49  |           7.61789e-05 |
|  36462 | html-css                 | 173.239.232.64  |           7.61789e-05 |
|  36463 | html-css                 | 173.239.232.89  |           7.61789e-05 |
|  36464 | html-css                 | 173.239.232.92  |           7.61789e-05 |
|  36465 | html-css                 | 173.239.232.95  |           7.61789e-05 |
|  36466 | html-css                 | 174.110.151.124 |           7.61789e-05 |
|  36467 | html-css                 | 174.128.180.102 |           7.61789e-05 |
|  36468 | html-css                 | 174.128.180.115 |           7.61789e-05 |
|  36469 | html-css                 | 174.128.180.19  |           7.61789e-05 |
|  36470 | html-css                 | 174.128.180.9   |           7.61789e-05 |
|  36471 | html-css                 | 174.128.181.69  |           7.61789e-05 |
|  36472 | html-css                 | 174.128.181.89  |           7.61789e-05 |
|  36473 | html-css                 | 174.207.17.250  |           7.61789e-05 |
|  36474 | html-css                 | 174.207.18.107  |           7.61789e-05 |
|  36475 | html-css                 | 174.207.23.110  |           7.61789e-05 |
|  36476 | html-css                 | 174.207.4.5     |           7.61789e-05 |
|  36477 | html-css                 | 174.246.132.173 |           7.61789e-05 |
|  36478 | html-css                 | 174.67.9.84     |           7.61789e-05 |
|  36479 | html-css                 | 184.203.6.32    |           7.61789e-05 |
|  36480 | html-css                 | 185.245.86.170  |           7.61789e-05 |
|  36481 | html-css                 | 189.156.27.111  |           7.61789e-05 |
|  36482 | html-css                 | 192.107.1.225   |           7.61789e-05 |
|  36483 | html-css                 | 192.241.233.20  |           7.61789e-05 |
|  36484 | html-css                 | 196.52.39.2     |           7.61789e-05 |
|  36485 | html-css                 | 196.52.39.32    |           7.61789e-05 |
|  36486 | html-css                 | 198.147.24.90   |           7.61789e-05 |
|  36487 | html-css                 | 198.202.210.220 |           7.61789e-05 |
|  36488 | html-css                 | 198.98.87.16    |           7.61789e-05 |
|  36489 | html-css                 | 199.116.118.221 |           7.61789e-05 |
|  36490 | html-css                 | 199.193.220.210 |           7.61789e-05 |
|  36491 | html-css                 | 199.193.221.114 |           7.61789e-05 |
|  36492 | html-css                 | 199.46.105.229  |           7.61789e-05 |
|  36493 | html-css                 | 206.109.23.139  |           7.61789e-05 |
|  36494 | html-css                 | 206.81.232.6    |           7.61789e-05 |
|  36495 | html-css                 | 207.119.3.149   |           7.61789e-05 |
|  36496 | html-css                 | 207.119.48.226  |           7.61789e-05 |
|  36497 | html-css                 | 207.119.54.40   |           7.61789e-05 |
|  36498 | html-css                 | 207.119.54.88   |           7.61789e-05 |
|  36499 | html-css                 | 207.119.59.156  |           7.61789e-05 |
|  36500 | html-css                 | 207.189.160.150 |           7.61789e-05 |
|  36501 | html-css                 | 208.73.111.171  |           7.61789e-05 |
|  36502 | html-css                 | 209.12.105.54   |           7.61789e-05 |
|  36503 | html-css                 | 209.64.120.2    |           7.61789e-05 |
|  36504 | html-css                 | 209.92.162.135  |           7.61789e-05 |
|  36505 | html-css                 | 212.102.44.99   |           7.61789e-05 |
|  36506 | html-css                 | 23.117.144.200  |           7.61789e-05 |
|  36507 | html-css                 | 23.122.42.151   |           7.61789e-05 |
|  36508 | html-css                 | 24.155.140.118  |           7.61789e-05 |
|  36509 | html-css                 | 24.160.145.169  |           7.61789e-05 |
|  36510 | html-css                 | 24.160.158.182  |           7.61789e-05 |
|  36511 | html-css                 | 24.162.162.164  |           7.61789e-05 |
|  36512 | html-css                 | 24.242.143.6    |           7.61789e-05 |
|  36513 | html-css                 | 24.243.16.96    |           7.61789e-05 |
|  36514 | html-css                 | 24.243.18.90    |           7.61789e-05 |
|  36515 | html-css                 | 24.243.20.159   |           7.61789e-05 |
|  36516 | html-css                 | 24.243.20.85    |           7.61789e-05 |
|  36517 | html-css                 | 24.243.22.68    |           7.61789e-05 |
|  36518 | html-css                 | 24.243.35.7     |           7.61789e-05 |
|  36519 | html-css                 | 24.26.227.191   |           7.61789e-05 |
|  36520 | html-css                 | 24.26.232.177   |           7.61789e-05 |
|  36521 | html-css                 | 24.26.235.139   |           7.61789e-05 |
|  36522 | html-css                 | 24.26.238.218   |           7.61789e-05 |
|  36523 | html-css                 | 24.26.251.39    |           7.61789e-05 |
|  36524 | html-css                 | 24.27.227.0     |           7.61789e-05 |
|  36525 | html-css                 | 24.28.148.245   |           7.61789e-05 |
|  36526 | html-css                 | 24.55.39.25     |           7.61789e-05 |
|  36527 | html-css                 | 24.99.149.248   |           7.61789e-05 |
|  36528 | html-css                 | 31.13.189.90    |           7.61789e-05 |
|  36529 | html-css                 | 4.34.182.238    |           7.61789e-05 |
|  36530 | html-css                 | 40.128.78.90    |           7.61789e-05 |
|  36531 | html-css                 | 45.131.192.127  |           7.61789e-05 |
|  36532 | html-css                 | 45.131.192.180  |           7.61789e-05 |
|  36533 | html-css                 | 45.131.192.231  |           7.61789e-05 |
|  36534 | html-css                 | 45.16.112.177   |           7.61789e-05 |
|  36535 | html-css                 | 45.56.151.222   |           7.61789e-05 |
|  36536 | html-css                 | 46.218.181.194  |           7.61789e-05 |
|  36537 | html-css                 | 47.183.128.23   |           7.61789e-05 |
|  36538 | html-css                 | 47.183.138.169  |           7.61789e-05 |
|  36539 | html-css                 | 47.183.143.149  |           7.61789e-05 |
|  36540 | html-css                 | 47.187.245.12   |           7.61789e-05 |
|  36541 | html-css                 | 47.187.247.193  |           7.61789e-05 |
|  36542 | html-css                 | 47.187.247.99   |           7.61789e-05 |
|  36543 | html-css                 | 47.234.136.72   |           7.61789e-05 |
|  36544 | html-css                 | 47.234.144.124  |           7.61789e-05 |
|  36545 | html-css                 | 50.202.129.130  |           7.61789e-05 |
|  36546 | html-css                 | 50.84.103.219   |           7.61789e-05 |
|  36547 | html-css                 | 50.84.26.206    |           7.61789e-05 |
|  36548 | html-css                 | 64.134.148.200  |           7.61789e-05 |
|  36549 | html-css                 | 64.134.149.176  |           7.61789e-05 |
|  36550 | html-css                 | 64.134.159.142  |           7.61789e-05 |
|  36551 | html-css                 | 65.131.228.148  |           7.61789e-05 |
|  36552 | html-css                 | 65.36.104.120   |           7.61789e-05 |
|  36553 | html-css                 | 65.36.111.91    |           7.61789e-05 |
|  36554 | html-css                 | 66.193.245.94   |           7.61789e-05 |
|  36555 | html-css                 | 66.25.145.212   |           7.61789e-05 |
|  36556 | html-css                 | 66.25.154.61    |           7.61789e-05 |
|  36557 | html-css                 | 66.25.163.2     |           7.61789e-05 |
|  36558 | html-css                 | 66.25.167.65    |           7.61789e-05 |
|  36559 | html-css                 | 66.25.169.158   |           7.61789e-05 |
|  36560 | html-css                 | 66.25.172.244   |           7.61789e-05 |
|  36561 | html-css                 | 66.25.175.197   |           7.61789e-05 |
|  36562 | html-css                 | 66.25.67.202    |           7.61789e-05 |
|  36563 | html-css                 | 66.25.88.97     |           7.61789e-05 |
|  36564 | html-css                 | 66.68.149.199   |           7.61789e-05 |
|  36565 | html-css                 | 66.69.122.35    |           7.61789e-05 |
|  36566 | html-css                 | 66.69.24.31     |           7.61789e-05 |
|  36567 | html-css                 | 66.69.37.238    |           7.61789e-05 |
|  36568 | html-css                 | 66.69.77.154    |           7.61789e-05 |
|  36569 | html-css                 | 66.90.180.216   |           7.61789e-05 |
|  36570 | html-css                 | 67.10.169.200   |           7.61789e-05 |
|  36571 | html-css                 | 67.10.213.134   |           7.61789e-05 |
|  36572 | html-css                 | 67.11.100.255   |           7.61789e-05 |
|  36573 | html-css                 | 67.11.104.47    |           7.61789e-05 |
|  36574 | html-css                 | 67.11.111.87    |           7.61789e-05 |
|  36575 | html-css                 | 67.11.137.215   |           7.61789e-05 |
|  36576 | html-css                 | 67.11.167.95    |           7.61789e-05 |
|  36577 | html-css                 | 67.11.215.110   |           7.61789e-05 |
|  36578 | html-css                 | 67.11.35.80     |           7.61789e-05 |
|  36579 | html-css                 | 67.11.48.25     |           7.61789e-05 |
|  36580 | html-css                 | 67.11.82.179    |           7.61789e-05 |
|  36581 | html-css                 | 67.48.40.83     |           7.61789e-05 |
|  36582 | html-css                 | 67.48.59.123    |           7.61789e-05 |
|  36583 | html-css                 | 67.78.112.162   |           7.61789e-05 |
|  36584 | html-css                 | 67.79.43.254    |           7.61789e-05 |
|  36585 | html-css                 | 68.206.161.84   |           7.61789e-05 |
|  36586 | html-css                 | 68.206.249.115  |           7.61789e-05 |
|  36587 | html-css                 | 68.206.251.51   |           7.61789e-05 |
|  36588 | html-css                 | 68.73.197.187   |           7.61789e-05 |
|  36589 | html-css                 | 69.153.19.127   |           7.61789e-05 |
|  36590 | html-css                 | 69.166.80.80    |           7.61789e-05 |
|  36591 | html-css                 | 69.166.89.120   |           7.61789e-05 |
|  36592 | html-css                 | 69.193.187.30   |           7.61789e-05 |
|  36593 | html-css                 | 69.232.101.106  |           7.61789e-05 |
|  36594 | html-css                 | 69.239.143.192  |           7.61789e-05 |
|  36595 | html-css                 | 70.112.103.21   |           7.61789e-05 |
|  36596 | html-css                 | 70.114.239.51   |           7.61789e-05 |
|  36597 | html-css                 | 70.114.36.54    |           7.61789e-05 |
|  36598 | html-css                 | 70.114.38.157   |           7.61789e-05 |
|  36599 | html-css                 | 70.115.158.21   |           7.61789e-05 |
|  36600 | html-css                 | 70.117.3.179    |           7.61789e-05 |
|  36601 | html-css                 | 70.117.71.59    |           7.61789e-05 |
|  36602 | html-css                 | 70.118.48.163   |           7.61789e-05 |
|  36603 | html-css                 | 70.118.9.107    |           7.61789e-05 |
|  36604 | html-css                 | 70.120.16.42    |           7.61789e-05 |
|  36605 | html-css                 | 70.120.17.11    |           7.61789e-05 |
|  36606 | html-css                 | 70.120.230.205  |           7.61789e-05 |
|  36607 | html-css                 | 70.120.24.167   |           7.61789e-05 |
|  36608 | html-css                 | 70.120.28.81    |           7.61789e-05 |
|  36609 | html-css                 | 70.120.3.242    |           7.61789e-05 |
|  36610 | html-css                 | 70.120.49.220   |           7.61789e-05 |
|  36611 | html-css                 | 70.120.52.142   |           7.61789e-05 |
|  36612 | html-css                 | 70.121.136.246  |           7.61789e-05 |
|  36613 | html-css                 | 70.121.180.146  |           7.61789e-05 |
|  36614 | html-css                 | 70.121.182.222  |           7.61789e-05 |
|  36615 | html-css                 | 70.121.205.219  |           7.61789e-05 |
|  36616 | html-css                 | 70.121.214.153  |           7.61789e-05 |
|  36617 | html-css                 | 70.123.162.206  |           7.61789e-05 |
|  36618 | html-css                 | 70.123.169.229  |           7.61789e-05 |
|  36619 | html-css                 | 70.123.193.100  |           7.61789e-05 |
|  36620 | html-css                 | 70.123.195.48   |           7.61789e-05 |
|  36621 | html-css                 | 70.123.200.22   |           7.61789e-05 |
|  36622 | html-css                 | 70.123.201.199  |           7.61789e-05 |
|  36623 | html-css                 | 70.123.205.224  |           7.61789e-05 |
|  36624 | html-css                 | 70.123.215.49   |           7.61789e-05 |
|  36625 | html-css                 | 70.123.215.59   |           7.61789e-05 |
|  36626 | html-css                 | 70.123.55.55    |           7.61789e-05 |
|  36627 | html-css                 | 70.123.93.201   |           7.61789e-05 |
|  36628 | html-css                 | 70.125.131.6    |           7.61789e-05 |
|  36629 | html-css                 | 70.125.99.193   |           7.61789e-05 |
|  36630 | html-css                 | 70.130.122.39   |           7.61789e-05 |
|  36631 | html-css                 | 70.130.124.250  |           7.61789e-05 |
|  36632 | html-css                 | 70.94.153.55    |           7.61789e-05 |
|  36633 | html-css                 | 70.94.164.72    |           7.61789e-05 |
|  36634 | html-css                 | 71.132.254.162  |           7.61789e-05 |
|  36635 | html-css                 | 71.136.254.125  |           7.61789e-05 |
|  36636 | html-css                 | 71.150.218.181  |           7.61789e-05 |
|  36637 | html-css                 | 71.158.153.146  |           7.61789e-05 |
|  36638 | html-css                 | 71.158.155.118  |           7.61789e-05 |
|  36639 | html-css                 | 71.221.202.248  |           7.61789e-05 |
|  36640 | html-css                 | 71.40.12.162    |           7.61789e-05 |
|  36641 | html-css                 | 71.41.134.238   |           7.61789e-05 |
|  36642 | html-css                 | 71.41.156.141   |           7.61789e-05 |
|  36643 | html-css                 | 71.41.242.76    |           7.61789e-05 |
|  36644 | html-css                 | 71.42.105.146   |           7.61789e-05 |
|  36645 | html-css                 | 71.42.105.171   |           7.61789e-05 |
|  36646 | html-css                 | 71.42.105.172   |           7.61789e-05 |
|  36647 | html-css                 | 71.42.105.182   |           7.61789e-05 |
|  36648 | html-css                 | 71.42.140.139   |           7.61789e-05 |
|  36649 | html-css                 | 71.42.218.131   |           7.61789e-05 |
|  36650 | html-css                 | 71.78.115.66    |           7.61789e-05 |
|  36651 | html-css                 | 71.78.167.131   |           7.61789e-05 |
|  36652 | html-css                 | 71.78.236.131   |           7.61789e-05 |
|  36653 | html-css                 | 72.12.112.229   |           7.61789e-05 |
|  36654 | html-css                 | 72.177.208.53   |           7.61789e-05 |
|  36655 | html-css                 | 72.177.209.77   |           7.61789e-05 |
|  36656 | html-css                 | 72.178.117.75   |           7.61789e-05 |
|  36657 | html-css                 | 72.179.136.146  |           7.61789e-05 |
|  36658 | html-css                 | 72.179.183.68   |           7.61789e-05 |
|  36659 | html-css                 | 72.180.51.14    |           7.61789e-05 |
|  36660 | html-css                 | 72.181.110.80   |           7.61789e-05 |
|  36661 | html-css                 | 72.181.117.187  |           7.61789e-05 |
|  36662 | html-css                 | 72.181.117.226  |           7.61789e-05 |
|  36663 | html-css                 | 72.181.117.243  |           7.61789e-05 |
|  36664 | html-css                 | 72.181.125.198  |           7.61789e-05 |
|  36665 | html-css                 | 72.181.149.13   |           7.61789e-05 |
|  36666 | html-css                 | 72.181.97.136   |           7.61789e-05 |
|  36667 | html-css                 | 72.181.99.44    |           7.61789e-05 |
|  36668 | html-css                 | 72.190.171.65   |           7.61789e-05 |
|  36669 | html-css                 | 72.190.232.108  |           7.61789e-05 |
|  36670 | html-css                 | 72.190.237.130  |           7.61789e-05 |
|  36671 | html-css                 | 72.191.140.150  |           7.61789e-05 |
|  36672 | html-css                 | 72.191.22.4     |           7.61789e-05 |
|  36673 | html-css                 | 72.191.28.232   |           7.61789e-05 |
|  36674 | html-css                 | 72.191.50.176   |           7.61789e-05 |
|  36675 | html-css                 | 72.201.107.178  |           7.61789e-05 |
|  36676 | html-css                 | 72.205.214.27   |           7.61789e-05 |
|  36677 | html-css                 | 72.45.179.227   |           7.61789e-05 |
|  36678 | html-css                 | 72.48.8.218     |           7.61789e-05 |
|  36679 | html-css                 | 73.14.59.241    |           7.61789e-05 |
|  36680 | html-css                 | 73.153.77.234   |           7.61789e-05 |
|  36681 | html-css                 | 73.166.32.184   |           7.61789e-05 |
|  36682 | html-css                 | 73.219.222.176  |           7.61789e-05 |
|  36683 | html-css                 | 73.77.171.11    |           7.61789e-05 |
|  36684 | html-css                 | 73.85.100.144   |           7.61789e-05 |
|  36685 | html-css                 | 74.105.240.236  |           7.61789e-05 |
|  36686 | html-css                 | 74.197.69.72    |           7.61789e-05 |
|  36687 | html-css                 | 74.75.232.45    |           7.61789e-05 |
|  36688 | html-css                 | 75.1.32.205     |           7.61789e-05 |
|  36689 | html-css                 | 75.166.30.247   |           7.61789e-05 |
|  36690 | html-css                 | 75.31.192.132   |           7.61789e-05 |
|  36691 | html-css                 | 75.33.163.123   |           7.61789e-05 |
|  36692 | html-css                 | 75.35.140.23    |           7.61789e-05 |
|  36693 | html-css                 | 75.54.32.19     |           7.61789e-05 |
|  36694 | html-css                 | 75.54.32.20     |           7.61789e-05 |
|  36695 | html-css                 | 75.54.33.190    |           7.61789e-05 |
|  36696 | html-css                 | 76.184.212.53   |           7.61789e-05 |
|  36697 | html-css                 | 76.185.128.151  |           7.61789e-05 |
|  36698 | html-css                 | 76.185.158.93   |           7.61789e-05 |
|  36699 | html-css                 | 76.185.234.111  |           7.61789e-05 |
|  36700 | html-css                 | 76.185.236.152  |           7.61789e-05 |
|  36701 | html-css                 | 76.186.26.141   |           7.61789e-05 |
|  36702 | html-css                 | 76.186.54.220   |           7.61789e-05 |
|  36703 | html-css                 | 76.187.101.165  |           7.61789e-05 |
|  36704 | html-css                 | 76.192.103.34   |           7.61789e-05 |
|  36705 | html-css                 | 8.31.0.128      |           7.61789e-05 |
|  36706 | html-css                 | 84.14.10.91     |           7.61789e-05 |
|  36707 | html-css                 | 87.158.135.133  |           7.61789e-05 |
|  36708 | html-css                 | 88.241.49.202   |           7.61789e-05 |
|  36709 | html-css                 | 89.159.14.216   |           7.61789e-05 |
|  36710 | html-css                 | 89.187.164.129  |           7.61789e-05 |
|  36711 | html-css                 | 89.187.164.132  |           7.61789e-05 |
|  36712 | html-css                 | 89.187.164.135  |           7.61789e-05 |
|  36713 | html-css                 | 89.187.164.146  |           7.61789e-05 |
|  36714 | html-css                 | 89.187.164.147  |           7.61789e-05 |
|  36715 | html-css                 | 89.187.164.150  |           7.61789e-05 |
|  36716 | html-css                 | 89.187.164.151  |           7.61789e-05 |
|  36717 | html-css                 | 89.187.164.152  |           7.61789e-05 |
|  36718 | html-css                 | 89.187.164.161  |           7.61789e-05 |
|  36719 | html-css                 | 89.187.164.164  |           7.61789e-05 |
|  36720 | html-css                 | 89.187.164.167  |           7.61789e-05 |
|  36721 | html-css                 | 89.187.164.77   |           7.61789e-05 |
|  36722 | html-css                 | 96.46.160.14    |           7.61789e-05 |
|  36723 | html-css                 | 96.8.128.88     |           7.61789e-05 |
|  36724 | html-css                 | 96.8.130.0      |           7.61789e-05 |
|  36725 | html-css                 | 96.8.130.12     |           7.61789e-05 |
|  36726 | html-css                 | 96.8.130.21     |           7.61789e-05 |
|  36727 | html-css                 | 96.8.131.77     |           7.61789e-05 |
|  36728 | html-css                 | 96.8.160.28     |           7.61789e-05 |
|  36729 | html-css                 | 96.8.177.189    |           7.61789e-05 |
|  36730 | html-css                 | 96.8.178.180    |           7.61789e-05 |
|  36731 | html-css                 | 96.8.178.42     |           7.61789e-05 |
|  36732 | html-css                 | 96.8.178.61     |           7.61789e-05 |
|  36733 | html-css                 | 96.8.248.115    |           7.61789e-05 |
|  36734 | html-css                 | 96.8.248.15     |           7.61789e-05 |
|  36735 | html-css                 | 97.105.154.139  |           7.61789e-05 |
|  36736 | html-css                 | 97.105.165.178  |           7.61789e-05 |
|  36737 | html-css                 | 97.105.90.179   |           7.61789e-05 |
|  36738 | html-css                 | 97.123.183.37   |           7.61789e-05 |
|  36739 | html-css                 | 97.123.26.103   |           7.61789e-05 |
|  36740 | html-css                 | 97.77.126.60    |           7.61789e-05 |
|  36741 | html-css                 | 97.98.108.135   |           7.61789e-05 |
|  36742 | html-css                 | 98.6.75.74      |           7.61789e-05 |
|  36743 | html-css                 | 98.6.94.51      |           7.61789e-05 |
|  36744 | html-css                 | 99.132.130.214  |           7.61789e-05 |
|  36745 | html-css                 | 99.137.88.24    |           7.61789e-05 |
|  36746 | html-css                 | 99.147.239.234  |           7.61789e-05 |
|  36747 | html-css                 | 99.158.247.252  |           7.61789e-05 |
|  36748 | html-css                 | 99.184.61.243   |           7.61789e-05 |
|  36749 | html-css                 | 99.202.73.143   |           7.61789e-05 |
|  36750 | html-css                 | 99.203.212.200  |           7.61789e-05 |
|  36751 | html-css                 | 99.203.26.146   |           7.61789e-05 |
|  36752 | html-css                 | 99.203.26.48    |           7.61789e-05 |
|  36753 | html-css                 | 99.203.26.80    |           7.61789e-05 |
|  36754 | html-css                 | 99.203.27.240   |           7.61789e-05 |
|  36755 | html-css                 | 99.203.90.147   |           7.61789e-05 |
|  36756 | html-css                 | 99.203.90.211   |           7.61789e-05 |
|  36757 | html-css                 | 99.27.138.170   |           7.61789e-05 |
|  36758 | html-css                 | 99.43.106.116   |           7.61789e-05 |
|  36759 | html-css                 | 99.43.138.137   |           7.61789e-05 |
|  36760 | html-css                 | 99.52.75.127    |           7.61789e-05 |
|  36761 | html-css                 | 99.52.78.67     |           7.61789e-05 |
|  36762 | html-css                 | 99.6.251.67     |           7.61789e-05 |
|  36763 | html-css                 | 99.62.48.254    |           7.61789e-05 |
|  46082 | java-i                   | 104.131.47.203  |           9.55384e-05 |
|  46083 | java-i                   | 104.191.12.108  |           9.55384e-05 |
|  46084 | java-i                   | 104.200.142.196 |           9.55384e-05 |
|  46085 | java-i                   | 104.219.136.105 |           9.55384e-05 |
|  46086 | java-i                   | 104.237.80.81   |           9.55384e-05 |
|  46087 | java-i                   | 104.37.31.16    |           9.55384e-05 |
|  46088 | java-i                   | 104.37.31.8     |           9.55384e-05 |
|  46089 | java-i                   | 104.5.159.147   |           9.55384e-05 |
|  46090 | java-i                   | 104.53.202.72   |           9.55384e-05 |
|  46091 | java-i                   | 104.53.206.121  |           9.55384e-05 |
|  46092 | java-i                   | 107.128.5.96    |           9.55384e-05 |
|  46093 | java-i                   | 107.129.209.3   |           9.55384e-05 |
|  46094 | java-i                   | 107.146.141.88  |           9.55384e-05 |
|  46095 | java-i                   | 107.242.125.38  |           9.55384e-05 |
|  46096 | java-i                   | 107.77.100.108  |           9.55384e-05 |
|  46097 | java-i                   | 107.77.169.7    |           9.55384e-05 |
|  46098 | java-i                   | 107.77.189.40   |           9.55384e-05 |
|  46099 | java-i                   | 107.77.196.224  |           9.55384e-05 |
|  46100 | java-i                   | 107.77.196.71   |           9.55384e-05 |
|  46101 | java-i                   | 107.77.201.226  |           9.55384e-05 |
|  46102 | java-i                   | 107.77.217.123  |           9.55384e-05 |
|  46103 | java-i                   | 107.77.217.54   |           9.55384e-05 |
|  46104 | java-i                   | 107.77.218.122  |           9.55384e-05 |
|  46105 | java-i                   | 107.77.218.22   |           9.55384e-05 |
|  46106 | java-i                   | 107.77.219.141  |           9.55384e-05 |
|  46107 | java-i                   | 107.77.219.171  |           9.55384e-05 |
|  46108 | java-i                   | 107.77.219.20   |           9.55384e-05 |
|  46109 | java-i                   | 107.77.219.49   |           9.55384e-05 |
|  46110 | java-i                   | 107.77.219.68   |           9.55384e-05 |
|  46111 | java-i                   | 107.77.219.93   |           9.55384e-05 |
|  46112 | java-i                   | 107.77.220.203  |           9.55384e-05 |
|  46113 | java-i                   | 107.77.220.86   |           9.55384e-05 |
|  46114 | java-i                   | 107.77.221.108  |           9.55384e-05 |
|  46115 | java-i                   | 107.77.221.203  |           9.55384e-05 |
|  46116 | java-i                   | 107.77.222.127  |           9.55384e-05 |
|  46117 | java-i                   | 107.77.66.78    |           9.55384e-05 |
|  46118 | java-i                   | 108.199.117.96  |           9.55384e-05 |
|  46119 | java-i                   | 108.200.112.192 |           9.55384e-05 |
|  46120 | java-i                   | 108.87.103.243  |           9.55384e-05 |
|  46121 | java-i                   | 108.93.12.131   |           9.55384e-05 |
|  46122 | java-i                   | 110.78.138.179  |           9.55384e-05 |
|  46123 | java-i                   | 12.154.218.253  |           9.55384e-05 |
|  46124 | java-i                   | 12.16.232.100   |           9.55384e-05 |
|  46125 | java-i                   | 12.247.3.42     |           9.55384e-05 |
|  46126 | java-i                   | 12.31.71.58     |           9.55384e-05 |
|  46127 | java-i                   | 12.34.237.242   |           9.55384e-05 |
|  46128 | java-i                   | 129.115.195.22  |           9.55384e-05 |
|  46129 | java-i                   | 129.115.2.251   |           9.55384e-05 |
|  46130 | java-i                   | 130.45.33.209   |           9.55384e-05 |
|  46131 | java-i                   | 130.45.35.109   |           9.55384e-05 |
|  46132 | java-i                   | 130.45.53.218   |           9.55384e-05 |
|  46133 | java-i                   | 131.194.172.128 |           9.55384e-05 |
|  46134 | java-i                   | 131.194.173.118 |           9.55384e-05 |
|  46135 | java-i                   | 131.44.121.130  |           9.55384e-05 |
|  46136 | java-i                   | 136.50.110.0    |           9.55384e-05 |
|  46137 | java-i                   | 136.50.20.17    |           9.55384e-05 |
|  46138 | java-i                   | 136.50.22.155   |           9.55384e-05 |
|  46139 | java-i                   | 136.50.29.193   |           9.55384e-05 |
|  46140 | java-i                   | 136.50.50.187   |           9.55384e-05 |
|  46141 | java-i                   | 136.50.54.38    |           9.55384e-05 |
|  46142 | java-i                   | 137.242.1.128   |           9.55384e-05 |
|  46143 | java-i                   | 137.242.1.129   |           9.55384e-05 |
|  46144 | java-i                   | 148.59.156.46   |           9.55384e-05 |
|  46145 | java-i                   | 148.66.37.74    |           9.55384e-05 |
|  46146 | java-i                   | 148.66.37.91    |           9.55384e-05 |
|  46147 | java-i                   | 148.66.38.18    |           9.55384e-05 |
|  46148 | java-i                   | 162.229.6.124   |           9.55384e-05 |
|  46149 | java-i                   | 162.233.170.149 |           9.55384e-05 |
|  46150 | java-i                   | 162.245.202.3   |           9.55384e-05 |
|  46151 | java-i                   | 162.253.68.118  |           9.55384e-05 |
|  46152 | java-i                   | 162.72.133.33   |           9.55384e-05 |
|  46153 | java-i                   | 165.231.210.20  |           9.55384e-05 |
|  46154 | java-i                   | 166.247.169.116 |           9.55384e-05 |
|  46155 | java-i                   | 166.251.101.230 |           9.55384e-05 |
|  46156 | java-i                   | 166.251.101.233 |           9.55384e-05 |
|  46157 | java-i                   | 172.108.140.130 |           9.55384e-05 |
|  46158 | java-i                   | 172.3.220.202   |           9.55384e-05 |
|  46159 | java-i                   | 172.3.28.125    |           9.55384e-05 |
|  46160 | java-i                   | 172.56.6.54     |           9.55384e-05 |
|  46161 | java-i                   | 172.58.102.174  |           9.55384e-05 |
|  46162 | java-i                   | 172.58.107.99   |           9.55384e-05 |
|  46163 | java-i                   | 172.58.70.175   |           9.55384e-05 |
|  46164 | java-i                   | 172.58.97.32    |           9.55384e-05 |
|  46165 | java-i                   | 172.58.99.188   |           9.55384e-05 |
|  46166 | java-i                   | 173.126.107.20  |           9.55384e-05 |
|  46167 | java-i                   | 173.127.123.160 |           9.55384e-05 |
|  46168 | java-i                   | 173.127.23.253  |           9.55384e-05 |
|  46169 | java-i                   | 173.127.6.41    |           9.55384e-05 |
|  46170 | java-i                   | 173.127.80.231  |           9.55384e-05 |
|  46171 | java-i                   | 173.127.89.155  |           9.55384e-05 |
|  46172 | java-i                   | 173.134.114.250 |           9.55384e-05 |
|  46173 | java-i                   | 173.173.101.207 |           9.55384e-05 |
|  46174 | java-i                   | 173.173.102.136 |           9.55384e-05 |
|  46175 | java-i                   | 173.173.107.110 |           9.55384e-05 |
|  46176 | java-i                   | 173.173.121.126 |           9.55384e-05 |
|  46177 | java-i                   | 173.173.127.101 |           9.55384e-05 |
|  46178 | java-i                   | 173.173.47.94   |           9.55384e-05 |
|  46179 | java-i                   | 173.173.98.244  |           9.55384e-05 |
|  46180 | java-i                   | 173.174.153.61  |           9.55384e-05 |
|  46181 | java-i                   | 173.174.164.88  |           9.55384e-05 |
|  46182 | java-i                   | 173.174.212.111 |           9.55384e-05 |
|  46183 | java-i                   | 173.174.233.67  |           9.55384e-05 |
|  46184 | java-i                   | 173.175.109.205 |           9.55384e-05 |
|  46185 | java-i                   | 173.175.109.235 |           9.55384e-05 |
|  46186 | java-i                   | 173.239.232.107 |           9.55384e-05 |
|  46187 | java-i                   | 173.239.232.12  |           9.55384e-05 |
|  46188 | java-i                   | 173.239.232.123 |           9.55384e-05 |
|  46189 | java-i                   | 173.239.232.134 |           9.55384e-05 |
|  46190 | java-i                   | 173.239.232.143 |           9.55384e-05 |
|  46191 | java-i                   | 173.239.232.166 |           9.55384e-05 |
|  46192 | java-i                   | 173.239.232.20  |           9.55384e-05 |
|  46193 | java-i                   | 173.239.232.26  |           9.55384e-05 |
|  46194 | java-i                   | 173.239.232.56  |           9.55384e-05 |
|  46195 | java-i                   | 173.239.232.57  |           9.55384e-05 |
|  46196 | java-i                   | 173.239.232.64  |           9.55384e-05 |
|  46197 | java-i                   | 173.239.232.70  |           9.55384e-05 |
|  46198 | java-i                   | 173.239.232.73  |           9.55384e-05 |
|  46199 | java-i                   | 173.239.232.82  |           9.55384e-05 |
|  46200 | java-i                   | 173.239.232.91  |           9.55384e-05 |
|  46201 | java-i                   | 173.239.232.92  |           9.55384e-05 |
|  46202 | java-i                   | 173.239.232.99  |           9.55384e-05 |
|  46203 | java-i                   | 173.244.217.91  |           9.55384e-05 |
|  46204 | java-i                   | 173.255.139.216 |           9.55384e-05 |
|  46205 | java-i                   | 174.207.1.185   |           9.55384e-05 |
|  46206 | java-i                   | 174.207.17.114  |           9.55384e-05 |
|  46207 | java-i                   | 174.207.3.54    |           9.55384e-05 |
|  46208 | java-i                   | 174.207.5.31    |           9.55384e-05 |
|  46209 | java-i                   | 174.207.7.167   |           9.55384e-05 |
|  46210 | java-i                   | 174.235.7.82    |           9.55384e-05 |
|  46211 | java-i                   | 174.236.18.237  |           9.55384e-05 |
|  46212 | java-i                   | 174.236.3.162   |           9.55384e-05 |
|  46213 | java-i                   | 174.246.199.81  |           9.55384e-05 |
|  46214 | java-i                   | 174.249.45.199  |           9.55384e-05 |
|  46215 | java-i                   | 174.249.47.236  |           9.55384e-05 |
|  46216 | java-i                   | 184.203.110.171 |           9.55384e-05 |
|  46217 | java-i                   | 184.226.100.247 |           9.55384e-05 |
|  46218 | java-i                   | 184.226.102.146 |           9.55384e-05 |
|  46219 | java-i                   | 184.226.56.72   |           9.55384e-05 |
|  46220 | java-i                   | 184.226.93.17   |           9.55384e-05 |
|  46221 | java-i                   | 189.140.49.80   |           9.55384e-05 |
|  46222 | java-i                   | 193.37.252.52   |           9.55384e-05 |
|  46223 | java-i                   | 193.56.117.75   |           9.55384e-05 |
|  46224 | java-i                   | 195.206.104.108 |           9.55384e-05 |
|  46225 | java-i                   | 196.52.39.29    |           9.55384e-05 |
|  46226 | java-i                   | 196.52.39.32    |           9.55384e-05 |
|  46227 | java-i                   | 198.147.23.196  |           9.55384e-05 |
|  46228 | java-i                   | 198.147.24.90   |           9.55384e-05 |
|  46229 | java-i                   | 198.190.245.10  |           9.55384e-05 |
|  46230 | java-i                   | 198.202.210.220 |           9.55384e-05 |
|  46231 | java-i                   | 198.98.87.16    |           9.55384e-05 |
|  46232 | java-i                   | 199.116.115.137 |           9.55384e-05 |
|  46233 | java-i                   | 204.44.112.76   |           9.55384e-05 |
|  46234 | java-i                   | 205.251.148.122 |           9.55384e-05 |
|  46235 | java-i                   | 205.251.148.146 |           9.55384e-05 |
|  46236 | java-i                   | 205.251.148.154 |           9.55384e-05 |
|  46237 | java-i                   | 205.251.148.98  |           9.55384e-05 |
|  46238 | java-i                   | 206.217.136.93  |           9.55384e-05 |
|  46239 | java-i                   | 207.119.52.22   |           9.55384e-05 |
|  46240 | java-i                   | 207.119.54.40   |           9.55384e-05 |
|  46241 | java-i                   | 207.119.59.216  |           9.55384e-05 |
|  46242 | java-i                   | 207.119.61.70   |           9.55384e-05 |
|  46243 | java-i                   | 209.12.105.53   |           9.55384e-05 |
|  46244 | java-i                   | 209.205.221.178 |           9.55384e-05 |
|  46245 | java-i                   | 212.102.40.134  |           9.55384e-05 |
|  46246 | java-i                   | 23.116.24.226   |           9.55384e-05 |
|  46247 | java-i                   | 23.117.144.200  |           9.55384e-05 |
|  46248 | java-i                   | 23.227.134.242  |           9.55384e-05 |
|  46249 | java-i                   | 24.153.137.59   |           9.55384e-05 |
|  46250 | java-i                   | 24.155.140.118  |           9.55384e-05 |
|  46251 | java-i                   | 24.155.149.210  |           9.55384e-05 |
|  46252 | java-i                   | 24.160.139.79   |           9.55384e-05 |
|  46253 | java-i                   | 24.160.140.11   |           9.55384e-05 |
|  46254 | java-i                   | 24.160.149.87   |           9.55384e-05 |
|  46255 | java-i                   | 24.160.9.56     |           9.55384e-05 |
|  46256 | java-i                   | 24.175.231.111  |           9.55384e-05 |
|  46257 | java-i                   | 24.243.10.53    |           9.55384e-05 |
|  46258 | java-i                   | 24.243.19.100   |           9.55384e-05 |
|  46259 | java-i                   | 24.243.35.7     |           9.55384e-05 |
|  46260 | java-i                   | 24.26.235.139   |           9.55384e-05 |
|  46261 | java-i                   | 24.26.239.74    |           9.55384e-05 |
|  46262 | java-i                   | 24.26.242.103   |           9.55384e-05 |
|  46263 | java-i                   | 24.26.255.20    |           9.55384e-05 |
|  46264 | java-i                   | 24.28.96.138    |           9.55384e-05 |
|  46265 | java-i                   | 38.96.210.4     |           9.55384e-05 |
|  46266 | java-i                   | 40.128.39.234   |           9.55384e-05 |
|  46267 | java-i                   | 45.131.192.222  |           9.55384e-05 |
|  46268 | java-i                   | 45.17.26.192    |           9.55384e-05 |
|  46269 | java-i                   | 45.21.2.123     |           9.55384e-05 |
|  46270 | java-i                   | 45.21.39.47     |           9.55384e-05 |
|  46271 | java-i                   | 45.31.212.143   |           9.55384e-05 |
|  46272 | java-i                   | 45.31.212.88    |           9.55384e-05 |
|  46273 | java-i                   | 45.56.151.110   |           9.55384e-05 |
|  46274 | java-i                   | 45.56.151.122   |           9.55384e-05 |
|  46275 | java-i                   | 45.56.151.50    |           9.55384e-05 |
|  46276 | java-i                   | 45.56.151.8     |           9.55384e-05 |
|  46277 | java-i                   | 45.56.151.93    |           9.55384e-05 |
|  46278 | java-i                   | 47.218.59.57    |           9.55384e-05 |
|  46279 | java-i                   | 47.221.223.242  |           9.55384e-05 |
|  46280 | java-i                   | 47.234.244.208  |           9.55384e-05 |
|  46281 | java-i                   | 50.226.114.86   |           9.55384e-05 |
|  46282 | java-i                   | 50.84.152.78    |           9.55384e-05 |
|  46283 | java-i                   | 50.84.218.3     |           9.55384e-05 |
|  46284 | java-i                   | 64.132.229.66   |           9.55384e-05 |
|  46285 | java-i                   | 64.250.9.218    |           9.55384e-05 |
|  46286 | java-i                   | 65.131.228.148  |           9.55384e-05 |
|  46287 | java-i                   | 65.154.35.138   |           9.55384e-05 |
|  46288 | java-i                   | 65.220.10.3     |           9.55384e-05 |
|  46289 | java-i                   | 65.43.113.131   |           9.55384e-05 |
|  46290 | java-i                   | 66.196.195.34   |           9.55384e-05 |
|  46291 | java-i                   | 66.25.130.242   |           9.55384e-05 |
|  46292 | java-i                   | 66.25.171.170   |           9.55384e-05 |
|  46293 | java-i                   | 66.25.175.197   |           9.55384e-05 |
|  46294 | java-i                   | 66.25.246.159   |           9.55384e-05 |
|  46295 | java-i                   | 66.69.48.5      |           9.55384e-05 |
|  46296 | java-i                   | 66.69.58.238    |           9.55384e-05 |
|  46297 | java-i                   | 66.69.6.234     |           9.55384e-05 |
|  46298 | java-i                   | 66.69.60.109    |           9.55384e-05 |
|  46299 | java-i                   | 66.69.79.50     |           9.55384e-05 |
|  46300 | java-i                   | 66.90.136.214   |           9.55384e-05 |
|  46301 | java-i                   | 66.90.156.125   |           9.55384e-05 |
|  46302 | java-i                   | 66.90.180.216   |           9.55384e-05 |
|  46303 | java-i                   | 67.10.153.249   |           9.55384e-05 |
|  46304 | java-i                   | 67.10.181.177   |           9.55384e-05 |
|  46305 | java-i                   | 67.11.100.82    |           9.55384e-05 |
|  46306 | java-i                   | 67.11.169.7     |           9.55384e-05 |
|  46307 | java-i                   | 67.11.229.0     |           9.55384e-05 |
|  46308 | java-i                   | 67.11.35.239    |           9.55384e-05 |
|  46309 | java-i                   | 67.11.35.80     |           9.55384e-05 |
|  46310 | java-i                   | 67.11.39.16     |           9.55384e-05 |
|  46311 | java-i                   | 67.11.84.66     |           9.55384e-05 |
|  46312 | java-i                   | 67.48.42.163    |           9.55384e-05 |
|  46313 | java-i                   | 67.79.15.43     |           9.55384e-05 |
|  46314 | java-i                   | 67.79.43.254    |           9.55384e-05 |
|  46315 | java-i                   | 67.79.62.147    |           9.55384e-05 |
|  46316 | java-i                   | 68.203.203.127  |           9.55384e-05 |
|  46317 | java-i                   | 68.206.160.201  |           9.55384e-05 |
|  46318 | java-i                   | 68.206.251.51   |           9.55384e-05 |
|  46319 | java-i                   | 68.206.254.17   |           9.55384e-05 |
|  46320 | java-i                   | 68.206.254.198  |           9.55384e-05 |
|  46321 | java-i                   | 68.47.165.83    |           9.55384e-05 |
|  46322 | java-i                   | 69.153.19.127   |           9.55384e-05 |
|  46323 | java-i                   | 69.232.103.179  |           9.55384e-05 |
|  46324 | java-i                   | 70.114.36.54    |           9.55384e-05 |
|  46325 | java-i                   | 70.115.114.110  |           9.55384e-05 |
|  46326 | java-i                   | 70.117.31.37    |           9.55384e-05 |
|  46327 | java-i                   | 70.117.9.9      |           9.55384e-05 |
|  46328 | java-i                   | 70.118.168.239  |           9.55384e-05 |
|  46329 | java-i                   | 70.118.48.163   |           9.55384e-05 |
|  46330 | java-i                   | 70.120.213.195  |           9.55384e-05 |
|  46331 | java-i                   | 70.120.5.227    |           9.55384e-05 |
|  46332 | java-i                   | 70.120.52.142   |           9.55384e-05 |
|  46333 | java-i                   | 70.120.6.177    |           9.55384e-05 |
|  46334 | java-i                   | 70.121.182.158  |           9.55384e-05 |
|  46335 | java-i                   | 70.121.185.146  |           9.55384e-05 |
|  46336 | java-i                   | 70.121.187.122  |           9.55384e-05 |
|  46337 | java-i                   | 70.121.214.153  |           9.55384e-05 |
|  46338 | java-i                   | 70.121.221.55   |           9.55384e-05 |
|  46339 | java-i                   | 70.122.201.106  |           9.55384e-05 |
|  46340 | java-i                   | 70.122.206.138  |           9.55384e-05 |
|  46341 | java-i                   | 70.122.206.28   |           9.55384e-05 |
|  46342 | java-i                   | 70.123.135.83   |           9.55384e-05 |
|  46343 | java-i                   | 70.123.215.59   |           9.55384e-05 |
|  46344 | java-i                   | 70.123.233.35   |           9.55384e-05 |
|  46345 | java-i                   | 70.128.146.8    |           9.55384e-05 |
|  46346 | java-i                   | 70.130.122.154  |           9.55384e-05 |
|  46347 | java-i                   | 70.133.251.114  |           9.55384e-05 |
|  46348 | java-i                   | 70.240.199.22   |           9.55384e-05 |
|  46349 | java-i                   | 70.94.144.103   |           9.55384e-05 |
|  46350 | java-i                   | 70.94.167.166   |           9.55384e-05 |
|  46351 | java-i                   | 71.12.226.214   |           9.55384e-05 |
|  46352 | java-i                   | 71.136.251.103  |           9.55384e-05 |
|  46353 | java-i                   | 71.137.144.185  |           9.55384e-05 |
|  46354 | java-i                   | 71.137.146.71   |           9.55384e-05 |
|  46355 | java-i                   | 71.221.202.248  |           9.55384e-05 |
|  46356 | java-i                   | 71.221.210.29   |           9.55384e-05 |
|  46357 | java-i                   | 71.40.24.180    |           9.55384e-05 |
|  46358 | java-i                   | 71.41.134.238   |           9.55384e-05 |
|  46359 | java-i                   | 71.41.243.19    |           9.55384e-05 |
|  46360 | java-i                   | 71.42.199.34    |           9.55384e-05 |
|  46361 | java-i                   | 71.78.167.131   |           9.55384e-05 |
|  46362 | java-i                   | 71.78.236.131   |           9.55384e-05 |
|  46363 | java-i                   | 72.12.112.229   |           9.55384e-05 |
|  46364 | java-i                   | 72.177.165.210  |           9.55384e-05 |
|  46365 | java-i                   | 72.177.173.215  |           9.55384e-05 |
|  46366 | java-i                   | 72.177.209.77   |           9.55384e-05 |
|  46367 | java-i                   | 72.177.237.78   |           9.55384e-05 |
|  46368 | java-i                   | 72.178.117.75   |           9.55384e-05 |
|  46369 | java-i                   | 72.178.188.133  |           9.55384e-05 |
|  46370 | java-i                   | 72.179.108.103  |           9.55384e-05 |
|  46371 | java-i                   | 72.179.132.10   |           9.55384e-05 |
|  46372 | java-i                   | 72.179.175.39   |           9.55384e-05 |
|  46373 | java-i                   | 72.179.184.241  |           9.55384e-05 |
|  46374 | java-i                   | 72.180.46.191   |           9.55384e-05 |
|  46375 | java-i                   | 72.180.51.14    |           9.55384e-05 |
|  46376 | java-i                   | 72.181.104.49   |           9.55384e-05 |
|  46377 | java-i                   | 72.181.106.165  |           9.55384e-05 |
|  46378 | java-i                   | 72.181.106.34   |           9.55384e-05 |
|  46379 | java-i                   | 72.181.107.144  |           9.55384e-05 |
|  46380 | java-i                   | 72.181.117.187  |           9.55384e-05 |
|  46381 | java-i                   | 72.181.117.226  |           9.55384e-05 |
|  46382 | java-i                   | 72.181.120.73   |           9.55384e-05 |
|  46383 | java-i                   | 72.181.99.132   |           9.55384e-05 |
|  46384 | java-i                   | 72.182.64.120   |           9.55384e-05 |
|  46385 | java-i                   | 72.190.139.120  |           9.55384e-05 |
|  46386 | java-i                   | 72.190.170.159  |           9.55384e-05 |
|  46387 | java-i                   | 72.190.36.102   |           9.55384e-05 |
|  46388 | java-i                   | 72.191.11.178   |           9.55384e-05 |
|  46389 | java-i                   | 72.191.29.194   |           9.55384e-05 |
|  46390 | java-i                   | 72.201.107.178  |           9.55384e-05 |
|  46391 | java-i                   | 72.21.196.64    |           9.55384e-05 |
|  46392 | java-i                   | 72.45.179.227   |           9.55384e-05 |
|  46393 | java-i                   | 72.48.60.84     |           9.55384e-05 |
|  46394 | java-i                   | 72.48.8.218     |           9.55384e-05 |
|  46395 | java-i                   | 73.153.77.234   |           9.55384e-05 |
|  46396 | java-i                   | 73.166.32.184   |           9.55384e-05 |
|  46397 | java-i                   | 73.166.86.204   |           9.55384e-05 |
|  46398 | java-i                   | 73.34.63.18     |           9.55384e-05 |
|  46399 | java-i                   | 73.39.175.210   |           9.55384e-05 |
|  46400 | java-i                   | 73.77.171.11    |           9.55384e-05 |
|  46401 | java-i                   | 74.105.240.236  |           9.55384e-05 |
|  46402 | java-i                   | 74.50.118.217   |           9.55384e-05 |
|  46403 | java-i                   | 75.166.30.247   |           9.55384e-05 |
|  46404 | java-i                   | 75.173.105.106  |           9.55384e-05 |
|  46405 | java-i                   | 75.41.18.87     |           9.55384e-05 |
|  46406 | java-i                   | 75.54.32.17     |           9.55384e-05 |
|  46407 | java-i                   | 75.54.32.45     |           9.55384e-05 |
|  46408 | java-i                   | 75.54.33.183    |           9.55384e-05 |
|  46409 | java-i                   | 75.87.2.162     |           9.55384e-05 |
|  46410 | java-i                   | 76.115.148.248  |           9.55384e-05 |
|  46411 | java-i                   | 76.185.121.198  |           9.55384e-05 |
|  46412 | java-i                   | 76.185.135.244  |           9.55384e-05 |
|  46413 | java-i                   | 76.185.164.25   |           9.55384e-05 |
|  46414 | java-i                   | 76.185.172.182  |           9.55384e-05 |
|  46415 | java-i                   | 76.185.173.254  |           9.55384e-05 |
|  46416 | java-i                   | 76.185.193.115  |           9.55384e-05 |
|  46417 | java-i                   | 76.185.209.80   |           9.55384e-05 |
|  46418 | java-i                   | 76.185.216.146  |           9.55384e-05 |
|  46419 | java-i                   | 76.185.227.82   |           9.55384e-05 |
|  46420 | java-i                   | 76.185.234.111  |           9.55384e-05 |
|  46421 | java-i                   | 76.185.236.152  |           9.55384e-05 |
|  46422 | java-i                   | 76.198.200.203  |           9.55384e-05 |
|  46423 | java-i                   | 76.211.179.43   |           9.55384e-05 |
|  46424 | java-i                   | 76.214.156.42   |           9.55384e-05 |
|  46425 | java-i                   | 76.237.205.238  |           9.55384e-05 |
|  46426 | java-i                   | 76.250.226.254  |           9.55384e-05 |
|  46427 | java-i                   | 79.220.63.16    |           9.55384e-05 |
|  46428 | java-i                   | 85.106.99.108   |           9.55384e-05 |
|  46429 | java-i                   | 88.241.49.202   |           9.55384e-05 |
|  46430 | java-i                   | 89.187.164.131  |           9.55384e-05 |
|  46431 | java-i                   | 89.187.164.134  |           9.55384e-05 |
|  46432 | java-i                   | 89.187.164.136  |           9.55384e-05 |
|  46433 | java-i                   | 89.187.164.145  |           9.55384e-05 |
|  46434 | java-i                   | 89.187.164.146  |           9.55384e-05 |
|  46435 | java-i                   | 89.187.164.168  |           9.55384e-05 |
|  46436 | java-i                   | 89.187.164.76   |           9.55384e-05 |
|  46437 | java-i                   | 89.187.175.33   |           9.55384e-05 |
|  46438 | java-i                   | 92.38.148.10    |           9.55384e-05 |
|  46439 | java-i                   | 95.181.238.13   |           9.55384e-05 |
|  46440 | java-i                   | 96.8.130.0      |           9.55384e-05 |
|  46441 | java-i                   | 96.8.130.11     |           9.55384e-05 |
|  46442 | java-i                   | 96.8.172.51     |           9.55384e-05 |
|  46443 | java-i                   | 96.8.177.36     |           9.55384e-05 |
|  46444 | java-i                   | 96.8.193.113    |           9.55384e-05 |
|  46445 | java-i                   | 96.8.248.232    |           9.55384e-05 |
|  46446 | java-i                   | 97.105.101.131  |           9.55384e-05 |
|  46447 | java-i                   | 97.105.154.139  |           9.55384e-05 |
|  46448 | java-i                   | 97.105.40.214   |           9.55384e-05 |
|  46449 | java-i                   | 97.77.146.130   |           9.55384e-05 |
|  46450 | java-i                   | 97.77.161.196   |           9.55384e-05 |
|  46451 | java-i                   | 97.77.203.60    |           9.55384e-05 |
|  46452 | java-i                   | 97.77.27.166    |           9.55384e-05 |
|  46453 | java-i                   | 97.98.85.151    |           9.55384e-05 |
|  46454 | java-i                   | 97.99.239.216   |           9.55384e-05 |
|  46455 | java-i                   | 98.156.147.200  |           9.55384e-05 |
|  46456 | java-i                   | 98.6.122.83     |           9.55384e-05 |
|  46457 | java-i                   | 98.6.94.51      |           9.55384e-05 |
|  46458 | java-i                   | 99.132.130.148  |           9.55384e-05 |
|  46459 | java-i                   | 99.151.195.187  |           9.55384e-05 |
|  46460 | java-i                   | 99.151.203.164  |           9.55384e-05 |
|  46461 | java-i                   | 99.152.157.19   |           9.55384e-05 |
|  46462 | java-i                   | 99.158.247.252  |           9.55384e-05 |
|  46463 | java-i                   | 99.162.243.169  |           9.55384e-05 |
|  46464 | java-i                   | 99.203.154.20   |           9.55384e-05 |
|  46465 | java-i                   | 99.203.26.214   |           9.55384e-05 |
|  46466 | java-i                   | 99.203.27.36    |           9.55384e-05 |
|  46467 | java-i                   | 99.33.48.253    |           9.55384e-05 |
|  46468 | java-i                   | 99.48.230.93    |           9.55384e-05 |
|  46469 | java-i                   | 99.66.104.96    |           9.55384e-05 |
|  46470 | java-i                   | 99.97.208.154   |           9.55384e-05 |
|  46471 | java-i                   | 99.98.36.168    |           9.55384e-05 |
|  50615 | java-ii                  | 104.131.47.203  |           8.2122e-05  |
|  50616 | java-ii                  | 104.189.72.26   |           8.2122e-05  |
|  50617 | java-ii                  | 104.191.11.147  |           8.2122e-05  |
|  50618 | java-ii                  | 104.219.136.53  |           8.2122e-05  |
|  50619 | java-ii                  | 104.53.200.122  |           8.2122e-05  |
|  50620 | java-ii                  | 104.53.202.72   |           8.2122e-05  |
|  50621 | java-ii                  | 107.204.50.40   |           8.2122e-05  |
|  50622 | java-ii                  | 107.77.189.15   |           8.2122e-05  |
|  50623 | java-ii                  | 107.77.196.224  |           8.2122e-05  |
|  50624 | java-ii                  | 107.77.217.195  |           8.2122e-05  |
|  50625 | java-ii                  | 107.77.217.6    |           8.2122e-05  |
|  50626 | java-ii                  | 107.77.217.70   |           8.2122e-05  |
|  50627 | java-ii                  | 107.77.217.96   |           8.2122e-05  |
|  50628 | java-ii                  | 107.77.218.122  |           8.2122e-05  |
|  50629 | java-ii                  | 107.77.219.141  |           8.2122e-05  |
|  50630 | java-ii                  | 107.77.219.64   |           8.2122e-05  |
|  50631 | java-ii                  | 107.77.219.67   |           8.2122e-05  |
|  50632 | java-ii                  | 107.77.219.68   |           8.2122e-05  |
|  50633 | java-ii                  | 107.77.220.125  |           8.2122e-05  |
|  50634 | java-ii                  | 107.77.220.183  |           8.2122e-05  |
|  50635 | java-ii                  | 107.77.220.203  |           8.2122e-05  |
|  50636 | java-ii                  | 107.77.220.47   |           8.2122e-05  |
|  50637 | java-ii                  | 107.77.220.56   |           8.2122e-05  |
|  50638 | java-ii                  | 107.77.221.101  |           8.2122e-05  |
|  50639 | java-ii                  | 107.77.221.108  |           8.2122e-05  |
|  50640 | java-ii                  | 107.77.221.199  |           8.2122e-05  |
|  50641 | java-ii                  | 107.77.221.96   |           8.2122e-05  |
|  50642 | java-ii                  | 107.77.66.78    |           8.2122e-05  |
|  50643 | java-ii                  | 108.192.181.20  |           8.2122e-05  |
|  50644 | java-ii                  | 108.196.159.214 |           8.2122e-05  |
|  50645 | java-ii                  | 108.200.112.192 |           8.2122e-05  |
|  50646 | java-ii                  | 108.87.103.243  |           8.2122e-05  |
|  50647 | java-ii                  | 110.78.138.179  |           8.2122e-05  |
|  50648 | java-ii                  | 12.16.232.100   |           8.2122e-05  |
|  50649 | java-ii                  | 12.247.3.42     |           8.2122e-05  |
|  50650 | java-ii                  | 12.247.3.50     |           8.2122e-05  |
|  50651 | java-ii                  | 129.112.109.42  |           8.2122e-05  |
|  50652 | java-ii                  | 130.45.35.109   |           8.2122e-05  |
|  50653 | java-ii                  | 130.45.50.55    |           8.2122e-05  |
|  50654 | java-ii                  | 130.45.62.247   |           8.2122e-05  |
|  50655 | java-ii                  | 131.194.172.75  |           8.2122e-05  |
|  50656 | java-ii                  | 131.44.121.130  |           8.2122e-05  |
|  50657 | java-ii                  | 136.50.110.0    |           8.2122e-05  |
|  50658 | java-ii                  | 136.50.13.79    |           8.2122e-05  |
|  50659 | java-ii                  | 136.50.22.155   |           8.2122e-05  |
|  50660 | java-ii                  | 136.50.49.145   |           8.2122e-05  |
|  50661 | java-ii                  | 136.50.61.231   |           8.2122e-05  |
|  50662 | java-ii                  | 136.50.63.72    |           8.2122e-05  |
|  50663 | java-ii                  | 136.50.68.20    |           8.2122e-05  |
|  50664 | java-ii                  | 136.50.97.29    |           8.2122e-05  |
|  50665 | java-ii                  | 137.242.1.128   |           8.2122e-05  |
|  50666 | java-ii                  | 148.59.156.46   |           8.2122e-05  |
|  50667 | java-ii                  | 148.66.37.74    |           8.2122e-05  |
|  50668 | java-ii                  | 148.66.37.82    |           8.2122e-05  |
|  50669 | java-ii                  | 162.198.96.75   |           8.2122e-05  |
|  50670 | java-ii                  | 162.229.6.124   |           8.2122e-05  |
|  50671 | java-ii                  | 162.233.170.149 |           8.2122e-05  |
|  50672 | java-ii                  | 162.245.202.3   |           8.2122e-05  |
|  50673 | java-ii                  | 162.250.18.154  |           8.2122e-05  |
|  50674 | java-ii                  | 165.225.48.86   |           8.2122e-05  |
|  50675 | java-ii                  | 166.205.190.126 |           8.2122e-05  |
|  50676 | java-ii                  | 167.24.24.150   |           8.2122e-05  |
|  50677 | java-ii                  | 172.242.120.198 |           8.2122e-05  |
|  50678 | java-ii                  | 172.3.29.85     |           8.2122e-05  |
|  50679 | java-ii                  | 172.56.15.15    |           8.2122e-05  |
|  50680 | java-ii                  | 172.56.6.41     |           8.2122e-05  |
|  50681 | java-ii                  | 172.56.7.90     |           8.2122e-05  |
|  50682 | java-ii                  | 172.58.102.188  |           8.2122e-05  |
|  50683 | java-ii                  | 172.58.107.124  |           8.2122e-05  |
|  50684 | java-ii                  | 172.58.107.237  |           8.2122e-05  |
|  50685 | java-ii                  | 172.58.107.99   |           8.2122e-05  |
|  50686 | java-ii                  | 172.58.109.241  |           8.2122e-05  |
|  50687 | java-ii                  | 172.58.67.225   |           8.2122e-05  |
|  50688 | java-ii                  | 172.58.67.65    |           8.2122e-05  |
|  50689 | java-ii                  | 172.58.97.32    |           8.2122e-05  |
|  50690 | java-ii                  | 172.8.173.98    |           8.2122e-05  |
|  50691 | java-ii                  | 173.172.166.90  |           8.2122e-05  |
|  50692 | java-ii                  | 173.173.100.45  |           8.2122e-05  |
|  50693 | java-ii                  | 173.173.101.207 |           8.2122e-05  |
|  50694 | java-ii                  | 173.173.107.110 |           8.2122e-05  |
|  50695 | java-ii                  | 173.173.111.160 |           8.2122e-05  |
|  50696 | java-ii                  | 173.173.127.101 |           8.2122e-05  |
|  50697 | java-ii                  | 173.173.47.94   |           8.2122e-05  |
|  50698 | java-ii                  | 173.173.96.192  |           8.2122e-05  |
|  50699 | java-ii                  | 173.173.98.137  |           8.2122e-05  |
|  50700 | java-ii                  | 173.174.159.45  |           8.2122e-05  |
|  50701 | java-ii                  | 173.174.164.88  |           8.2122e-05  |
|  50702 | java-ii                  | 173.174.220.17  |           8.2122e-05  |
|  50703 | java-ii                  | 173.174.233.67  |           8.2122e-05  |
|  50704 | java-ii                  | 173.175.109.205 |           8.2122e-05  |
|  50705 | java-ii                  | 173.175.109.235 |           8.2122e-05  |
|  50706 | java-ii                  | 173.239.232.120 |           8.2122e-05  |
|  50707 | java-ii                  | 173.239.232.122 |           8.2122e-05  |
|  50708 | java-ii                  | 173.239.232.123 |           8.2122e-05  |
|  50709 | java-ii                  | 173.239.232.124 |           8.2122e-05  |
|  50710 | java-ii                  | 173.239.232.13  |           8.2122e-05  |
|  50711 | java-ii                  | 173.239.232.131 |           8.2122e-05  |
|  50712 | java-ii                  | 173.239.232.134 |           8.2122e-05  |
|  50713 | java-ii                  | 173.239.232.137 |           8.2122e-05  |
|  50714 | java-ii                  | 173.239.232.140 |           8.2122e-05  |
|  50715 | java-ii                  | 173.239.232.20  |           8.2122e-05  |
|  50716 | java-ii                  | 173.239.232.22  |           8.2122e-05  |
|  50717 | java-ii                  | 173.239.232.3   |           8.2122e-05  |
|  50718 | java-ii                  | 173.239.232.33  |           8.2122e-05  |
|  50719 | java-ii                  | 173.239.232.42  |           8.2122e-05  |
|  50720 | java-ii                  | 173.239.232.48  |           8.2122e-05  |
|  50721 | java-ii                  | 173.239.232.62  |           8.2122e-05  |
|  50722 | java-ii                  | 173.239.232.64  |           8.2122e-05  |
|  50723 | java-ii                  | 173.239.232.73  |           8.2122e-05  |
|  50724 | java-ii                  | 173.239.232.74  |           8.2122e-05  |
|  50725 | java-ii                  | 173.239.232.88  |           8.2122e-05  |
|  50726 | java-ii                  | 173.239.232.91  |           8.2122e-05  |
|  50727 | java-ii                  | 173.239.232.99  |           8.2122e-05  |
|  50728 | java-ii                  | 174.128.180.212 |           8.2122e-05  |
|  50729 | java-ii                  | 174.197.12.70   |           8.2122e-05  |
|  50730 | java-ii                  | 174.207.17.114  |           8.2122e-05  |
|  50731 | java-ii                  | 174.207.26.100  |           8.2122e-05  |
|  50732 | java-ii                  | 174.207.3.54    |           8.2122e-05  |
|  50733 | java-ii                  | 174.235.7.82    |           8.2122e-05  |
|  50734 | java-ii                  | 174.236.8.119   |           8.2122e-05  |
|  50735 | java-ii                  | 174.246.194.63  |           8.2122e-05  |
|  50736 | java-ii                  | 174.249.47.236  |           8.2122e-05  |
|  50737 | java-ii                  | 174.51.14.27    |           8.2122e-05  |
|  50738 | java-ii                  | 184.203.19.207  |           8.2122e-05  |
|  50739 | java-ii                  | 184.226.20.138  |           8.2122e-05  |
|  50740 | java-ii                  | 184.226.93.17   |           8.2122e-05  |
|  50741 | java-ii                  | 185.245.87.238  |           8.2122e-05  |
|  50742 | java-ii                  | 189.146.56.56   |           8.2122e-05  |
|  50743 | java-ii                  | 192.138.47.30   |           8.2122e-05  |
|  50744 | java-ii                  | 193.148.18.75   |           8.2122e-05  |
|  50745 | java-ii                  | 193.37.252.42   |           8.2122e-05  |
|  50746 | java-ii                  | 193.37.252.48   |           8.2122e-05  |
|  50747 | java-ii                  | 195.206.104.108 |           8.2122e-05  |
|  50748 | java-ii                  | 196.52.39.32    |           8.2122e-05  |
|  50749 | java-ii                  | 198.202.210.220 |           8.2122e-05  |
|  50750 | java-ii                  | 199.193.220.210 |           8.2122e-05  |
|  50751 | java-ii                  | 204.44.112.76   |           8.2122e-05  |
|  50752 | java-ii                  | 205.251.148.122 |           8.2122e-05  |
|  50753 | java-ii                  | 205.251.148.146 |           8.2122e-05  |
|  50754 | java-ii                  | 205.251.150.146 |           8.2122e-05  |
|  50755 | java-ii                  | 207.119.61.70   |           8.2122e-05  |
|  50756 | java-ii                  | 209.12.105.53   |           8.2122e-05  |
|  50757 | java-ii                  | 209.182.242.18  |           8.2122e-05  |
|  50758 | java-ii                  | 209.205.221.178 |           8.2122e-05  |
|  50759 | java-ii                  | 212.103.49.182  |           8.2122e-05  |
|  50760 | java-ii                  | 216.177.187.98  |           8.2122e-05  |
|  50761 | java-ii                  | 23.116.24.226   |           8.2122e-05  |
|  50762 | java-ii                  | 24.153.137.59   |           8.2122e-05  |
|  50763 | java-ii                  | 24.155.118.66   |           8.2122e-05  |
|  50764 | java-ii                  | 24.155.140.118  |           8.2122e-05  |
|  50765 | java-ii                  | 24.160.135.99   |           8.2122e-05  |
|  50766 | java-ii                  | 24.160.149.87   |           8.2122e-05  |
|  50767 | java-ii                  | 24.242.143.6    |           8.2122e-05  |
|  50768 | java-ii                  | 24.242.150.231  |           8.2122e-05  |
|  50769 | java-ii                  | 24.243.10.53    |           8.2122e-05  |
|  50770 | java-ii                  | 24.243.19.100   |           8.2122e-05  |
|  50771 | java-ii                  | 24.243.35.7     |           8.2122e-05  |
|  50772 | java-ii                  | 24.243.50.45    |           8.2122e-05  |
|  50773 | java-ii                  | 24.243.55.34    |           8.2122e-05  |
|  50774 | java-ii                  | 24.243.59.138   |           8.2122e-05  |
|  50775 | java-ii                  | 24.243.59.161   |           8.2122e-05  |
|  50776 | java-ii                  | 24.243.7.150    |           8.2122e-05  |
|  50777 | java-ii                  | 24.243.74.160   |           8.2122e-05  |
|  50778 | java-ii                  | 24.26.241.169   |           8.2122e-05  |
|  50779 | java-ii                  | 24.27.227.1     |           8.2122e-05  |
|  50780 | java-ii                  | 24.28.101.217   |           8.2122e-05  |
|  50781 | java-ii                  | 24.28.109.234   |           8.2122e-05  |
|  50782 | java-ii                  | 24.99.149.248   |           8.2122e-05  |
|  50783 | java-ii                  | 45.131.192.222  |           8.2122e-05  |
|  50784 | java-ii                  | 45.21.32.233    |           8.2122e-05  |
|  50785 | java-ii                  | 45.31.212.88    |           8.2122e-05  |
|  50786 | java-ii                  | 45.56.151.50    |           8.2122e-05  |
|  50787 | java-ii                  | 45.56.151.8     |           8.2122e-05  |
|  50788 | java-ii                  | 47.187.228.190  |           8.2122e-05  |
|  50789 | java-ii                  | 47.234.152.236  |           8.2122e-05  |
|  50790 | java-ii                  | 50.84.218.3     |           8.2122e-05  |
|  50791 | java-ii                  | 64.134.149.176  |           8.2122e-05  |
|  50792 | java-ii                  | 64.134.159.125  |           8.2122e-05  |
|  50793 | java-ii                  | 64.148.247.227  |           8.2122e-05  |
|  50794 | java-ii                  | 65.131.228.148  |           8.2122e-05  |
|  50795 | java-ii                  | 65.220.10.3     |           8.2122e-05  |
|  50796 | java-ii                  | 65.43.113.131   |           8.2122e-05  |
|  50797 | java-ii                  | 66.136.56.254   |           8.2122e-05  |
|  50798 | java-ii                  | 66.196.195.34   |           8.2122e-05  |
|  50799 | java-ii                  | 66.25.162.204   |           8.2122e-05  |
|  50800 | java-ii                  | 66.25.163.226   |           8.2122e-05  |
|  50801 | java-ii                  | 66.25.168.113   |           8.2122e-05  |
|  50802 | java-ii                  | 66.69.122.35    |           8.2122e-05  |
|  50803 | java-ii                  | 66.69.57.38     |           8.2122e-05  |
|  50804 | java-ii                  | 66.69.79.50     |           8.2122e-05  |
|  50805 | java-ii                  | 66.90.156.125   |           8.2122e-05  |
|  50806 | java-ii                  | 66.90.180.216   |           8.2122e-05  |
|  50807 | java-ii                  | 67.10.128.187   |           8.2122e-05  |
|  50808 | java-ii                  | 67.10.185.18    |           8.2122e-05  |
|  50809 | java-ii                  | 67.11.120.84    |           8.2122e-05  |
|  50810 | java-ii                  | 67.11.131.55    |           8.2122e-05  |
|  50811 | java-ii                  | 67.11.169.7     |           8.2122e-05  |
|  50812 | java-ii                  | 67.11.84.66     |           8.2122e-05  |
|  50813 | java-ii                  | 67.11.97.204    |           8.2122e-05  |
|  50814 | java-ii                  | 67.48.201.230   |           8.2122e-05  |
|  50815 | java-ii                  | 67.79.15.43     |           8.2122e-05  |
|  50816 | java-ii                  | 68.206.160.201  |           8.2122e-05  |
|  50817 | java-ii                  | 68.206.254.198  |           8.2122e-05  |
|  50818 | java-ii                  | 68.252.68.142   |           8.2122e-05  |
|  50819 | java-ii                  | 68.47.165.83    |           8.2122e-05  |
|  50820 | java-ii                  | 69.153.19.127   |           8.2122e-05  |
|  50821 | java-ii                  | 69.154.52.98    |           8.2122e-05  |
|  50822 | java-ii                  | 69.172.183.120  |           8.2122e-05  |
|  50823 | java-ii                  | 70.114.139.150  |           8.2122e-05  |
|  50824 | java-ii                  | 70.114.36.54    |           8.2122e-05  |
|  50825 | java-ii                  | 70.114.38.157   |           8.2122e-05  |
|  50826 | java-ii                  | 70.115.114.110  |           8.2122e-05  |
|  50827 | java-ii                  | 70.117.3.179    |           8.2122e-05  |
|  50828 | java-ii                  | 70.117.31.37    |           8.2122e-05  |
|  50829 | java-ii                  | 70.117.5.148    |           8.2122e-05  |
|  50830 | java-ii                  | 70.118.168.239  |           8.2122e-05  |
|  50831 | java-ii                  | 70.118.67.138   |           8.2122e-05  |
|  50832 | java-ii                  | 70.118.9.107    |           8.2122e-05  |
|  50833 | java-ii                  | 70.120.230.205  |           8.2122e-05  |
|  50834 | java-ii                  | 70.120.3.242    |           8.2122e-05  |
|  50835 | java-ii                  | 70.120.6.177    |           8.2122e-05  |
|  50836 | java-ii                  | 70.121.136.2    |           8.2122e-05  |
|  50837 | java-ii                  | 70.121.185.146  |           8.2122e-05  |
|  50838 | java-ii                  | 70.121.187.122  |           8.2122e-05  |
|  50839 | java-ii                  | 70.121.188.16   |           8.2122e-05  |
|  50840 | java-ii                  | 70.121.192.194  |           8.2122e-05  |
|  50841 | java-ii                  | 70.121.194.226  |           8.2122e-05  |
|  50842 | java-ii                  | 70.121.214.153  |           8.2122e-05  |
|  50843 | java-ii                  | 70.122.206.28   |           8.2122e-05  |
|  50844 | java-ii                  | 70.122.250.222  |           8.2122e-05  |
|  50845 | java-ii                  | 70.123.135.83   |           8.2122e-05  |
|  50846 | java-ii                  | 70.123.164.223  |           8.2122e-05  |
|  50847 | java-ii                  | 70.123.169.229  |           8.2122e-05  |
|  50848 | java-ii                  | 70.123.239.48   |           8.2122e-05  |
|  50849 | java-ii                  | 70.125.148.96   |           8.2122e-05  |
|  50850 | java-ii                  | 70.125.178.113  |           8.2122e-05  |
|  50851 | java-ii                  | 70.130.122.154  |           8.2122e-05  |
|  50852 | java-ii                  | 70.130.122.39   |           8.2122e-05  |
|  50853 | java-ii                  | 70.130.124.250  |           8.2122e-05  |
|  50854 | java-ii                  | 70.94.165.250   |           8.2122e-05  |
|  50855 | java-ii                  | 71.158.155.118  |           8.2122e-05  |
|  50856 | java-ii                  | 71.221.198.41   |           8.2122e-05  |
|  50857 | java-ii                  | 71.40.81.37     |           8.2122e-05  |
|  50858 | java-ii                  | 72.12.112.229   |           8.2122e-05  |
|  50859 | java-ii                  | 72.173.98.235   |           8.2122e-05  |
|  50860 | java-ii                  | 72.177.165.210  |           8.2122e-05  |
|  50861 | java-ii                  | 72.177.199.59   |           8.2122e-05  |
|  50862 | java-ii                  | 72.177.209.244  |           8.2122e-05  |
|  50863 | java-ii                  | 72.177.210.199  |           8.2122e-05  |
|  50864 | java-ii                  | 72.178.117.75   |           8.2122e-05  |
|  50865 | java-ii                  | 72.178.188.133  |           8.2122e-05  |
|  50866 | java-ii                  | 72.179.185.12   |           8.2122e-05  |
|  50867 | java-ii                  | 72.181.104.49   |           8.2122e-05  |
|  50868 | java-ii                  | 72.181.106.34   |           8.2122e-05  |
|  50869 | java-ii                  | 72.181.119.126  |           8.2122e-05  |
|  50870 | java-ii                  | 72.181.98.94    |           8.2122e-05  |
|  50871 | java-ii                  | 72.181.99.44    |           8.2122e-05  |
|  50872 | java-ii                  | 72.190.170.159  |           8.2122e-05  |
|  50873 | java-ii                  | 72.190.172.10   |           8.2122e-05  |
|  50874 | java-ii                  | 72.191.26.144   |           8.2122e-05  |
|  50875 | java-ii                  | 72.191.29.194   |           8.2122e-05  |
|  50876 | java-ii                  | 72.191.58.78    |           8.2122e-05  |
|  50877 | java-ii                  | 72.191.59.31    |           8.2122e-05  |
|  50878 | java-ii                  | 72.21.196.64    |           8.2122e-05  |
|  50879 | java-ii                  | 72.45.179.227   |           8.2122e-05  |
|  50880 | java-ii                  | 72.48.255.177   |           8.2122e-05  |
|  50881 | java-ii                  | 72.48.8.218     |           8.2122e-05  |
|  50882 | java-ii                  | 73.143.162.24   |           8.2122e-05  |
|  50883 | java-ii                  | 73.233.99.219   |           8.2122e-05  |
|  50884 | java-ii                  | 73.31.215.224   |           8.2122e-05  |
|  50885 | java-ii                  | 73.39.175.210   |           8.2122e-05  |
|  50886 | java-ii                  | 75.15.242.25    |           8.2122e-05  |
|  50887 | java-ii                  | 75.173.105.106  |           8.2122e-05  |
|  50888 | java-ii                  | 75.28.18.14     |           8.2122e-05  |
|  50889 | java-ii                  | 75.41.18.87     |           8.2122e-05  |
|  50890 | java-ii                  | 75.54.33.182    |           8.2122e-05  |
|  50891 | java-ii                  | 76.182.226.251  |           8.2122e-05  |
|  50892 | java-ii                  | 76.185.121.198  |           8.2122e-05  |
|  50893 | java-ii                  | 76.185.135.35   |           8.2122e-05  |
|  50894 | java-ii                  | 76.185.142.244  |           8.2122e-05  |
|  50895 | java-ii                  | 76.185.145.231  |           8.2122e-05  |
|  50896 | java-ii                  | 76.185.168.145  |           8.2122e-05  |
|  50897 | java-ii                  | 76.185.172.182  |           8.2122e-05  |
|  50898 | java-ii                  | 76.185.173.254  |           8.2122e-05  |
|  50899 | java-ii                  | 76.185.182.104  |           8.2122e-05  |
|  50900 | java-ii                  | 76.185.193.115  |           8.2122e-05  |
|  50901 | java-ii                  | 76.185.227.82   |           8.2122e-05  |
|  50902 | java-ii                  | 76.186.189.183  |           8.2122e-05  |
|  50903 | java-ii                  | 76.186.54.46    |           8.2122e-05  |
|  50904 | java-ii                  | 76.187.7.114    |           8.2122e-05  |
|  50905 | java-ii                  | 76.198.200.203  |           8.2122e-05  |
|  50906 | java-ii                  | 76.198.205.9    |           8.2122e-05  |
|  50907 | java-ii                  | 79.220.63.16    |           8.2122e-05  |
|  50908 | java-ii                  | 85.106.100.99   |           8.2122e-05  |
|  50909 | java-ii                  | 87.158.135.133  |           8.2122e-05  |
|  50910 | java-ii                  | 88.241.49.202   |           8.2122e-05  |
|  50911 | java-ii                  | 89.187.164.135  |           8.2122e-05  |
|  50912 | java-ii                  | 89.187.164.146  |           8.2122e-05  |
|  50913 | java-ii                  | 89.187.164.148  |           8.2122e-05  |
|  50914 | java-ii                  | 89.187.164.164  |           8.2122e-05  |
|  50915 | java-ii                  | 89.187.164.166  |           8.2122e-05  |
|  50916 | java-ii                  | 89.187.164.167  |           8.2122e-05  |
|  50917 | java-ii                  | 89.187.164.77   |           8.2122e-05  |
|  50918 | java-ii                  | 89.187.164.78   |           8.2122e-05  |
|  50919 | java-ii                  | 89.187.177.72   |           8.2122e-05  |
|  50920 | java-ii                  | 92.38.148.10    |           8.2122e-05  |
|  50921 | java-ii                  | 95.181.238.24   |           8.2122e-05  |
|  50922 | java-ii                  | 95.181.238.28   |           8.2122e-05  |
|  50923 | java-ii                  | 95.181.238.36   |           8.2122e-05  |
|  50924 | java-ii                  | 95.181.238.6    |           8.2122e-05  |
|  50925 | java-ii                  | 95.181.238.9    |           8.2122e-05  |
|  50926 | java-ii                  | 96.8.130.0      |           8.2122e-05  |
|  50927 | java-ii                  | 96.8.130.11     |           8.2122e-05  |
|  50928 | java-ii                  | 96.8.130.92     |           8.2122e-05  |
|  50929 | java-ii                  | 96.8.177.189    |           8.2122e-05  |
|  50930 | java-ii                  | 96.8.177.36     |           8.2122e-05  |
|  50931 | java-ii                  | 96.8.178.42     |           8.2122e-05  |
|  50932 | java-ii                  | 96.8.193.113    |           8.2122e-05  |
|  50933 | java-ii                  | 96.8.193.160    |           8.2122e-05  |
|  50934 | java-ii                  | 96.8.193.73     |           8.2122e-05  |
|  50935 | java-ii                  | 96.8.198.155    |           8.2122e-05  |
|  50936 | java-ii                  | 96.8.248.20     |           8.2122e-05  |
|  50937 | java-ii                  | 97.105.165.178  |           8.2122e-05  |
|  50938 | java-ii                  | 97.105.53.195   |           8.2122e-05  |
|  50939 | java-ii                  | 97.105.81.147   |           8.2122e-05  |
|  50940 | java-ii                  | 97.77.161.196   |           8.2122e-05  |
|  50941 | java-ii                  | 97.79.128.78    |           8.2122e-05  |
|  50942 | java-ii                  | 98.39.12.204    |           8.2122e-05  |
|  50943 | java-ii                  | 98.6.122.83     |           8.2122e-05  |
|  50944 | java-ii                  | 98.6.203.30     |           8.2122e-05  |
|  50945 | java-ii                  | 98.6.75.74      |           8.2122e-05  |
|  50946 | java-ii                  | 98.6.94.51      |           8.2122e-05  |
|  50947 | java-ii                  | 99.137.88.24    |           8.2122e-05  |
|  50948 | java-ii                  | 99.147.236.67   |           8.2122e-05  |
|  50949 | java-ii                  | 99.147.238.134  |           8.2122e-05  |
|  50950 | java-ii                  | 99.151.195.187  |           8.2122e-05  |
|  50951 | java-ii                  | 99.151.203.164  |           8.2122e-05  |
|  50952 | java-ii                  | 99.152.157.19   |           8.2122e-05  |
|  50953 | java-ii                  | 99.158.247.252  |           8.2122e-05  |
|  50954 | java-ii                  | 99.158.248.118  |           8.2122e-05  |
|  50955 | java-ii                  | 99.162.240.204  |           8.2122e-05  |
|  50956 | java-ii                  | 99.162.243.169  |           8.2122e-05  |
|  50957 | java-ii                  | 99.184.61.243   |           8.2122e-05  |
|  50958 | java-ii                  | 99.184.63.30    |           8.2122e-05  |
|  50959 | java-ii                  | 99.203.154.20   |           8.2122e-05  |
|  50960 | java-ii                  | 99.203.26.18    |           8.2122e-05  |
|  50961 | java-ii                  | 99.203.27.120   |           8.2122e-05  |
|  50962 | java-ii                  | 99.203.27.13    |           8.2122e-05  |
|  50963 | java-ii                  | 99.203.27.34    |           8.2122e-05  |
|  50964 | java-ii                  | 99.203.27.66    |           8.2122e-05  |
|  50965 | java-ii                  | 99.27.138.170   |           8.2122e-05  |
|  50966 | java-ii                  | 99.30.242.101   |           8.2122e-05  |
|  50967 | java-ii                  | 99.43.137.218   |           8.2122e-05  |
|  50968 | java-ii                  | 99.48.230.93    |           8.2122e-05  |
|  50969 | java-ii                  | 99.98.36.168    |           8.2122e-05  |
|  56148 | java-iii                 | 104.180.86.144  |           7.59532e-05 |
|  56149 | java-iii                 | 104.182.15.214  |           7.59532e-05 |
|  56150 | java-iii                 | 104.191.11.147  |           7.59532e-05 |
|  56151 | java-iii                 | 107.200.109.74  |           7.59532e-05 |
|  56152 | java-iii                 | 107.77.100.88   |           7.59532e-05 |
|  56153 | java-iii                 | 107.77.169.10   |           7.59532e-05 |
|  56154 | java-iii                 | 107.77.196.106  |           7.59532e-05 |
|  56155 | java-iii                 | 107.77.198.116  |           7.59532e-05 |
|  56156 | java-iii                 | 107.77.198.150  |           7.59532e-05 |
|  56157 | java-iii                 | 107.77.201.4    |           7.59532e-05 |
|  56158 | java-iii                 | 107.77.217.123  |           7.59532e-05 |
|  56159 | java-iii                 | 107.77.217.233  |           7.59532e-05 |
|  56160 | java-iii                 | 107.77.218.122  |           7.59532e-05 |
|  56161 | java-iii                 | 107.77.219.94   |           7.59532e-05 |
|  56162 | java-iii                 | 107.77.221.153  |           7.59532e-05 |
|  56163 | java-iii                 | 107.77.221.191  |           7.59532e-05 |
|  56164 | java-iii                 | 107.77.221.209  |           7.59532e-05 |
|  56165 | java-iii                 | 107.77.221.33   |           7.59532e-05 |
|  56166 | java-iii                 | 107.77.222.188  |           7.59532e-05 |
|  56167 | java-iii                 | 107.77.222.190  |           7.59532e-05 |
|  56168 | java-iii                 | 107.77.66.78    |           7.59532e-05 |
|  56169 | java-iii                 | 107.77.66.96    |           7.59532e-05 |
|  56170 | java-iii                 | 107.77.80.125   |           7.59532e-05 |
|  56171 | java-iii                 | 108.11.13.210   |           7.59532e-05 |
|  56172 | java-iii                 | 108.192.181.20  |           7.59532e-05 |
|  56173 | java-iii                 | 108.196.159.214 |           7.59532e-05 |
|  56174 | java-iii                 | 108.199.115.75  |           7.59532e-05 |
|  56175 | java-iii                 | 108.218.193.28  |           7.59532e-05 |
|  56176 | java-iii                 | 108.59.0.37     |           7.59532e-05 |
|  56177 | java-iii                 | 108.65.244.91   |           7.59532e-05 |
|  56178 | java-iii                 | 108.66.243.159  |           7.59532e-05 |
|  56179 | java-iii                 | 108.93.10.214   |           7.59532e-05 |
|  56180 | java-iii                 | 12.16.232.100   |           7.59532e-05 |
|  56181 | java-iii                 | 12.247.3.42     |           7.59532e-05 |
|  56182 | java-iii                 | 12.28.69.2      |           7.59532e-05 |
|  56183 | java-iii                 | 12.34.237.242   |           7.59532e-05 |
|  56184 | java-iii                 | 12.49.174.18    |           7.59532e-05 |
|  56185 | java-iii                 | 129.111.179.55  |           7.59532e-05 |
|  56186 | java-iii                 | 129.112.109.42  |           7.59532e-05 |
|  56187 | java-iii                 | 129.115.195.28  |           7.59532e-05 |
|  56188 | java-iii                 | 130.45.33.209   |           7.59532e-05 |
|  56189 | java-iii                 | 130.45.35.109   |           7.59532e-05 |
|  56190 | java-iii                 | 130.45.53.218   |           7.59532e-05 |
|  56191 | java-iii                 | 130.45.62.247   |           7.59532e-05 |
|  56192 | java-iii                 | 131.194.172.199 |           7.59532e-05 |
|  56193 | java-iii                 | 136.49.100.118  |           7.59532e-05 |
|  56194 | java-iii                 | 136.50.110.0    |           7.59532e-05 |
|  56195 | java-iii                 | 136.50.13.79    |           7.59532e-05 |
|  56196 | java-iii                 | 136.50.53.26    |           7.59532e-05 |
|  56197 | java-iii                 | 136.50.54.38    |           7.59532e-05 |
|  56198 | java-iii                 | 136.50.64.148   |           7.59532e-05 |
|  56199 | java-iii                 | 136.50.92.234   |           7.59532e-05 |
|  56200 | java-iii                 | 148.59.156.46   |           7.59532e-05 |
|  56201 | java-iii                 | 154.3.250.108   |           7.59532e-05 |
|  56202 | java-iii                 | 159.192.242.43  |           7.59532e-05 |
|  56203 | java-iii                 | 162.195.148.122 |           7.59532e-05 |
|  56204 | java-iii                 | 162.236.252.30  |           7.59532e-05 |
|  56205 | java-iii                 | 162.250.18.154  |           7.59532e-05 |
|  56206 | java-iii                 | 162.253.68.115  |           7.59532e-05 |
|  56207 | java-iii                 | 166.247.169.116 |           7.59532e-05 |
|  56208 | java-iii                 | 166.251.101.2   |           7.59532e-05 |
|  56209 | java-iii                 | 169.57.90.139   |           7.59532e-05 |
|  56210 | java-iii                 | 172.110.80.227  |           7.59532e-05 |
|  56211 | java-iii                 | 172.242.120.198 |           7.59532e-05 |
|  56212 | java-iii                 | 172.3.220.202   |           7.59532e-05 |
|  56213 | java-iii                 | 172.56.6.211    |           7.59532e-05 |
|  56214 | java-iii                 | 172.56.7.244    |           7.59532e-05 |
|  56215 | java-iii                 | 172.56.7.25     |           7.59532e-05 |
|  56216 | java-iii                 | 172.58.100.199  |           7.59532e-05 |
|  56217 | java-iii                 | 172.58.101.191  |           7.59532e-05 |
|  56218 | java-iii                 | 172.58.101.208  |           7.59532e-05 |
|  56219 | java-iii                 | 172.58.103.253  |           7.59532e-05 |
|  56220 | java-iii                 | 172.58.107.204  |           7.59532e-05 |
|  56221 | java-iii                 | 172.58.107.66   |           7.59532e-05 |
|  56222 | java-iii                 | 172.58.110.151  |           7.59532e-05 |
|  56223 | java-iii                 | 172.58.110.193  |           7.59532e-05 |
|  56224 | java-iii                 | 172.58.110.217  |           7.59532e-05 |
|  56225 | java-iii                 | 172.58.97.32    |           7.59532e-05 |
|  56226 | java-iii                 | 172.58.99.230   |           7.59532e-05 |
|  56227 | java-iii                 | 172.58.99.241   |           7.59532e-05 |
|  56228 | java-iii                 | 172.58.99.43    |           7.59532e-05 |
|  56229 | java-iii                 | 172.87.148.22   |           7.59532e-05 |
|  56230 | java-iii                 | 173.172.166.90  |           7.59532e-05 |
|  56231 | java-iii                 | 173.173.101.207 |           7.59532e-05 |
|  56232 | java-iii                 | 173.173.104.57  |           7.59532e-05 |
|  56233 | java-iii                 | 173.173.107.110 |           7.59532e-05 |
|  56234 | java-iii                 | 173.173.111.160 |           7.59532e-05 |
|  56235 | java-iii                 | 173.173.121.126 |           7.59532e-05 |
|  56236 | java-iii                 | 173.173.96.192  |           7.59532e-05 |
|  56237 | java-iii                 | 173.173.96.82   |           7.59532e-05 |
|  56238 | java-iii                 | 173.174.149.192 |           7.59532e-05 |
|  56239 | java-iii                 | 173.174.196.212 |           7.59532e-05 |
|  56240 | java-iii                 | 173.174.198.2   |           7.59532e-05 |
|  56241 | java-iii                 | 173.174.218.183 |           7.59532e-05 |
|  56242 | java-iii                 | 173.174.240.181 |           7.59532e-05 |
|  56243 | java-iii                 | 173.175.109.205 |           7.59532e-05 |
|  56244 | java-iii                 | 173.175.109.235 |           7.59532e-05 |
|  56245 | java-iii                 | 173.175.19.30   |           7.59532e-05 |
|  56246 | java-iii                 | 173.239.232.131 |           7.59532e-05 |
|  56247 | java-iii                 | 173.239.232.170 |           7.59532e-05 |
|  56248 | java-iii                 | 173.239.232.20  |           7.59532e-05 |
|  56249 | java-iii                 | 173.239.232.27  |           7.59532e-05 |
|  56250 | java-iii                 | 173.239.232.42  |           7.59532e-05 |
|  56251 | java-iii                 | 173.239.232.60  |           7.59532e-05 |
|  56252 | java-iii                 | 173.239.232.64  |           7.59532e-05 |
|  56253 | java-iii                 | 173.239.232.84  |           7.59532e-05 |
|  56254 | java-iii                 | 173.239.232.93  |           7.59532e-05 |
|  56255 | java-iii                 | 173.239.232.99  |           7.59532e-05 |
|  56256 | java-iii                 | 173.30.84.132   |           7.59532e-05 |
|  56257 | java-iii                 | 174.207.1.54    |           7.59532e-05 |
|  56258 | java-iii                 | 174.207.14.66   |           7.59532e-05 |
|  56259 | java-iii                 | 174.207.17.75   |           7.59532e-05 |
|  56260 | java-iii                 | 174.207.2.137   |           7.59532e-05 |
|  56261 | java-iii                 | 174.236.1.254   |           7.59532e-05 |
|  56262 | java-iii                 | 174.244.48.221  |           7.59532e-05 |
|  56263 | java-iii                 | 174.246.199.81  |           7.59532e-05 |
|  56264 | java-iii                 | 174.255.137.57  |           7.59532e-05 |
|  56265 | java-iii                 | 174.255.145.205 |           7.59532e-05 |
|  56266 | java-iii                 | 174.255.146.154 |           7.59532e-05 |
|  56267 | java-iii                 | 176.113.73.30   |           7.59532e-05 |
|  56268 | java-iii                 | 181.214.227.47  |           7.59532e-05 |
|  56269 | java-iii                 | 184.203.172.25  |           7.59532e-05 |
|  56270 | java-iii                 | 184.226.100.247 |           7.59532e-05 |
|  56271 | java-iii                 | 184.226.30.104  |           7.59532e-05 |
|  56272 | java-iii                 | 185.216.33.82   |           7.59532e-05 |
|  56273 | java-iii                 | 189.146.56.56   |           7.59532e-05 |
|  56274 | java-iii                 | 192.241.205.66  |           7.59532e-05 |
|  56275 | java-iii                 | 193.37.252.42   |           7.59532e-05 |
|  56276 | java-iii                 | 193.37.252.48   |           7.59532e-05 |
|  56277 | java-iii                 | 195.206.104.107 |           7.59532e-05 |
|  56278 | java-iii                 | 196.52.39.32    |           7.59532e-05 |
|  56279 | java-iii                 | 198.202.210.220 |           7.59532e-05 |
|  56280 | java-iii                 | 204.156.181.62  |           7.59532e-05 |
|  56281 | java-iii                 | 204.44.112.76   |           7.59532e-05 |
|  56282 | java-iii                 | 205.251.148.146 |           7.59532e-05 |
|  56283 | java-iii                 | 205.251.148.26  |           7.59532e-05 |
|  56284 | java-iii                 | 205.251.148.34  |           7.59532e-05 |
|  56285 | java-iii                 | 205.251.148.98  |           7.59532e-05 |
|  56286 | java-iii                 | 205.251.150.146 |           7.59532e-05 |
|  56287 | java-iii                 | 205.251.150.162 |           7.59532e-05 |
|  56288 | java-iii                 | 205.251.150.218 |           7.59532e-05 |
|  56289 | java-iii                 | 205.251.151.42  |           7.59532e-05 |
|  56290 | java-iii                 | 206.255.81.238  |           7.59532e-05 |
|  56291 | java-iii                 | 207.119.59.216  |           7.59532e-05 |
|  56292 | java-iii                 | 207.119.63.85   |           7.59532e-05 |
|  56293 | java-iii                 | 208.73.111.171  |           7.59532e-05 |
|  56294 | java-iii                 | 209.12.105.53   |           7.59532e-05 |
|  56295 | java-iii                 | 209.182.242.18  |           7.59532e-05 |
|  56296 | java-iii                 | 209.64.120.2    |           7.59532e-05 |
|  56297 | java-iii                 | 216.177.187.98  |           7.59532e-05 |
|  56298 | java-iii                 | 24.153.217.26   |           7.59532e-05 |
|  56299 | java-iii                 | 24.242.143.6    |           7.59532e-05 |
|  56300 | java-iii                 | 24.242.180.178  |           7.59532e-05 |
|  56301 | java-iii                 | 24.243.16.96    |           7.59532e-05 |
|  56302 | java-iii                 | 24.243.19.100   |           7.59532e-05 |
|  56303 | java-iii                 | 24.243.31.34    |           7.59532e-05 |
|  56304 | java-iii                 | 24.243.31.59    |           7.59532e-05 |
|  56305 | java-iii                 | 24.243.32.134   |           7.59532e-05 |
|  56306 | java-iii                 | 24.26.241.169   |           7.59532e-05 |
|  56307 | java-iii                 | 24.28.101.217   |           7.59532e-05 |
|  56308 | java-iii                 | 24.28.149.24    |           7.59532e-05 |
|  56309 | java-iii                 | 24.55.13.38     |           7.59532e-05 |
|  56310 | java-iii                 | 24.99.149.248   |           7.59532e-05 |
|  56311 | java-iii                 | 31.14.72.25     |           7.59532e-05 |
|  56312 | java-iii                 | 38.132.103.150  |           7.59532e-05 |
|  56313 | java-iii                 | 45.131.192.107  |           7.59532e-05 |
|  56314 | java-iii                 | 45.131.192.16   |           7.59532e-05 |
|  56315 | java-iii                 | 45.131.192.166  |           7.59532e-05 |
|  56316 | java-iii                 | 45.131.192.220  |           7.59532e-05 |
|  56317 | java-iii                 | 45.16.112.177   |           7.59532e-05 |
|  56318 | java-iii                 | 45.21.32.233    |           7.59532e-05 |
|  56319 | java-iii                 | 45.31.212.88    |           7.59532e-05 |
|  56320 | java-iii                 | 45.31.213.223   |           7.59532e-05 |
|  56321 | java-iii                 | 45.41.134.89    |           7.59532e-05 |
|  56322 | java-iii                 | 45.56.151.122   |           7.59532e-05 |
|  56323 | java-iii                 | 45.56.151.177   |           7.59532e-05 |
|  56324 | java-iii                 | 45.56.151.8     |           7.59532e-05 |
|  56325 | java-iii                 | 45.83.89.58     |           7.59532e-05 |
|  56326 | java-iii                 | 47.187.228.190  |           7.59532e-05 |
|  56327 | java-iii                 | 47.188.180.229  |           7.59532e-05 |
|  56328 | java-iii                 | 47.221.223.242  |           7.59532e-05 |
|  56329 | java-iii                 | 5.181.234.38    |           7.59532e-05 |
|  56330 | java-iii                 | 50.84.103.219   |           7.59532e-05 |
|  56331 | java-iii                 | 56.0.84.25      |           7.59532e-05 |
|  56332 | java-iii                 | 63.246.163.161  |           7.59532e-05 |
|  56333 | java-iii                 | 64.134.155.129  |           7.59532e-05 |
|  56334 | java-iii                 | 64.44.80.196    |           7.59532e-05 |
|  56335 | java-iii                 | 64.44.81.84     |           7.59532e-05 |
|  56336 | java-iii                 | 65.112.228.222  |           7.59532e-05 |
|  56337 | java-iii                 | 65.131.228.148  |           7.59532e-05 |
|  56338 | java-iii                 | 66.25.162.204   |           7.59532e-05 |
|  56339 | java-iii                 | 66.25.168.113   |           7.59532e-05 |
|  56340 | java-iii                 | 66.69.122.35    |           7.59532e-05 |
|  56341 | java-iii                 | 66.69.48.5      |           7.59532e-05 |
|  56342 | java-iii                 | 66.69.58.238    |           7.59532e-05 |
|  56343 | java-iii                 | 66.69.66.113    |           7.59532e-05 |
|  56344 | java-iii                 | 66.90.180.216   |           7.59532e-05 |
|  56345 | java-iii                 | 67.10.128.187   |           7.59532e-05 |
|  56346 | java-iii                 | 67.10.164.55    |           7.59532e-05 |
|  56347 | java-iii                 | 67.10.184.254   |           7.59532e-05 |
|  56348 | java-iii                 | 67.10.191.167   |           7.59532e-05 |
|  56349 | java-iii                 | 67.11.111.87    |           7.59532e-05 |
|  56350 | java-iii                 | 67.11.245.224   |           7.59532e-05 |
|  56351 | java-iii                 | 67.198.103.97   |           7.59532e-05 |
|  56352 | java-iii                 | 67.48.201.230   |           7.59532e-05 |
|  56353 | java-iii                 | 67.48.40.83     |           7.59532e-05 |
|  56354 | java-iii                 | 67.7.7.139      |           7.59532e-05 |
|  56355 | java-iii                 | 67.80.48.111    |           7.59532e-05 |
|  56356 | java-iii                 | 68.201.204.191  |           7.59532e-05 |
|  56357 | java-iii                 | 68.206.160.201  |           7.59532e-05 |
|  56358 | java-iii                 | 68.206.161.84   |           7.59532e-05 |
|  56359 | java-iii                 | 68.206.198.66   |           7.59532e-05 |
|  56360 | java-iii                 | 69.132.169.243  |           7.59532e-05 |
|  56361 | java-iii                 | 69.154.52.98    |           7.59532e-05 |
|  56362 | java-iii                 | 69.166.89.120   |           7.59532e-05 |
|  56363 | java-iii                 | 69.212.157.148  |           7.59532e-05 |
|  56364 | java-iii                 | 69.221.149.11   |           7.59532e-05 |
|  56365 | java-iii                 | 69.236.100.33   |           7.59532e-05 |
|  56366 | java-iii                 | 70.114.36.54    |           7.59532e-05 |
|  56367 | java-iii                 | 70.115.114.110  |           7.59532e-05 |
|  56368 | java-iii                 | 70.117.5.148    |           7.59532e-05 |
|  56369 | java-iii                 | 70.118.67.138   |           7.59532e-05 |
|  56370 | java-iii                 | 70.120.24.167   |           7.59532e-05 |
|  56371 | java-iii                 | 70.121.136.2    |           7.59532e-05 |
|  56372 | java-iii                 | 70.121.187.122  |           7.59532e-05 |
|  56373 | java-iii                 | 70.121.191.85   |           7.59532e-05 |
|  56374 | java-iii                 | 70.121.200.203  |           7.59532e-05 |
|  56375 | java-iii                 | 70.121.205.5    |           7.59532e-05 |
|  56376 | java-iii                 | 70.121.214.153  |           7.59532e-05 |
|  56377 | java-iii                 | 70.122.201.106  |           7.59532e-05 |
|  56378 | java-iii                 | 70.122.206.28   |           7.59532e-05 |
|  56379 | java-iii                 | 70.122.37.65    |           7.59532e-05 |
|  56380 | java-iii                 | 70.123.135.83   |           7.59532e-05 |
|  56381 | java-iii                 | 70.123.162.206  |           7.59532e-05 |
|  56382 | java-iii                 | 70.123.169.229  |           7.59532e-05 |
|  56383 | java-iii                 | 70.123.193.100  |           7.59532e-05 |
|  56384 | java-iii                 | 70.123.201.199  |           7.59532e-05 |
|  56385 | java-iii                 | 70.123.228.62   |           7.59532e-05 |
|  56386 | java-iii                 | 70.123.239.48   |           7.59532e-05 |
|  56387 | java-iii                 | 70.125.138.191  |           7.59532e-05 |
|  56388 | java-iii                 | 70.130.122.154  |           7.59532e-05 |
|  56389 | java-iii                 | 70.130.124.250  |           7.59532e-05 |
|  56390 | java-iii                 | 70.94.165.250   |           7.59532e-05 |
|  56391 | java-iii                 | 71.132.254.162  |           7.59532e-05 |
|  56392 | java-iii                 | 71.136.255.165  |           7.59532e-05 |
|  56393 | java-iii                 | 71.78.167.131   |           7.59532e-05 |
|  56394 | java-iii                 | 72.177.141.148  |           7.59532e-05 |
|  56395 | java-iii                 | 72.177.199.59   |           7.59532e-05 |
|  56396 | java-iii                 | 72.177.209.77   |           7.59532e-05 |
|  56397 | java-iii                 | 72.177.210.199  |           7.59532e-05 |
|  56398 | java-iii                 | 72.177.241.50   |           7.59532e-05 |
|  56399 | java-iii                 | 72.178.117.75   |           7.59532e-05 |
|  56400 | java-iii                 | 72.179.108.103  |           7.59532e-05 |
|  56401 | java-iii                 | 72.179.184.241  |           7.59532e-05 |
|  56402 | java-iii                 | 72.181.117.187  |           7.59532e-05 |
|  56403 | java-iii                 | 72.181.119.126  |           7.59532e-05 |
|  56404 | java-iii                 | 72.181.96.144   |           7.59532e-05 |
|  56405 | java-iii                 | 72.181.98.94    |           7.59532e-05 |
|  56406 | java-iii                 | 72.182.51.48    |           7.59532e-05 |
|  56407 | java-iii                 | 72.190.131.54   |           7.59532e-05 |
|  56408 | java-iii                 | 72.190.170.159  |           7.59532e-05 |
|  56409 | java-iii                 | 72.190.171.65   |           7.59532e-05 |
|  56410 | java-iii                 | 72.190.173.219  |           7.59532e-05 |
|  56411 | java-iii                 | 72.191.11.178   |           7.59532e-05 |
|  56412 | java-iii                 | 72.191.22.4     |           7.59532e-05 |
|  56413 | java-iii                 | 72.191.59.31    |           7.59532e-05 |
|  56414 | java-iii                 | 72.48.8.218     |           7.59532e-05 |
|  56415 | java-iii                 | 73.153.134.112  |           7.59532e-05 |
|  56416 | java-iii                 | 73.166.86.204   |           7.59532e-05 |
|  56417 | java-iii                 | 73.34.63.18     |           7.59532e-05 |
|  56418 | java-iii                 | 74.81.88.130    |           7.59532e-05 |
|  56419 | java-iii                 | 75.15.242.25    |           7.59532e-05 |
|  56420 | java-iii                 | 75.41.2.91      |           7.59532e-05 |
|  56421 | java-iii                 | 75.83.236.223   |           7.59532e-05 |
|  56422 | java-iii                 | 75.87.2.162     |           7.59532e-05 |
|  56423 | java-iii                 | 76.185.193.115  |           7.59532e-05 |
|  56424 | java-iii                 | 76.185.227.82   |           7.59532e-05 |
|  56425 | java-iii                 | 76.185.236.152  |           7.59532e-05 |
|  56426 | java-iii                 | 76.186.167.31   |           7.59532e-05 |
|  56427 | java-iii                 | 76.186.189.183  |           7.59532e-05 |
|  56428 | java-iii                 | 76.186.2.234    |           7.59532e-05 |
|  56429 | java-iii                 | 76.187.7.114    |           7.59532e-05 |
|  56430 | java-iii                 | 76.250.156.19   |           7.59532e-05 |
|  56431 | java-iii                 | 77.136.40.123   |           7.59532e-05 |
|  56432 | java-iii                 | 80.137.93.59    |           7.59532e-05 |
|  56433 | java-iii                 | 84.17.45.159    |           7.59532e-05 |
|  56434 | java-iii                 | 85.106.103.181  |           7.59532e-05 |
|  56435 | java-iii                 | 85.106.99.195   |           7.59532e-05 |
|  56436 | java-iii                 | 87.158.135.133  |           7.59532e-05 |
|  56437 | java-iii                 | 89.159.14.216   |           7.59532e-05 |
|  56438 | java-iii                 | 89.187.164.129  |           7.59532e-05 |
|  56439 | java-iii                 | 89.187.164.131  |           7.59532e-05 |
|  56440 | java-iii                 | 89.187.164.134  |           7.59532e-05 |
|  56441 | java-iii                 | 89.187.164.146  |           7.59532e-05 |
|  56442 | java-iii                 | 89.187.164.147  |           7.59532e-05 |
|  56443 | java-iii                 | 89.187.164.148  |           7.59532e-05 |
|  56444 | java-iii                 | 89.187.164.151  |           7.59532e-05 |
|  56445 | java-iii                 | 89.187.164.162  |           7.59532e-05 |
|  56446 | java-iii                 | 89.187.164.163  |           7.59532e-05 |
|  56447 | java-iii                 | 89.187.164.164  |           7.59532e-05 |
|  56448 | java-iii                 | 89.187.164.165  |           7.59532e-05 |
|  56449 | java-iii                 | 92.38.148.56    |           7.59532e-05 |
|  56450 | java-iii                 | 95.181.238.19   |           7.59532e-05 |
|  56451 | java-iii                 | 96.40.234.116   |           7.59532e-05 |
|  56452 | java-iii                 | 96.46.160.20    |           7.59532e-05 |
|  56453 | java-iii                 | 96.8.130.11     |           7.59532e-05 |
|  56454 | java-iii                 | 96.8.130.92     |           7.59532e-05 |
|  56455 | java-iii                 | 96.8.131.77     |           7.59532e-05 |
|  56456 | java-iii                 | 96.8.177.189    |           7.59532e-05 |
|  56457 | java-iii                 | 96.8.178.32     |           7.59532e-05 |
|  56458 | java-iii                 | 96.8.193.113    |           7.59532e-05 |
|  56459 | java-iii                 | 97.105.101.131  |           7.59532e-05 |
|  56460 | java-iii                 | 97.105.165.178  |           7.59532e-05 |
|  56461 | java-iii                 | 97.105.81.147   |           7.59532e-05 |
|  56462 | java-iii                 | 97.77.161.196   |           7.59532e-05 |
|  56463 | java-iii                 | 97.77.232.50    |           7.59532e-05 |
|  56464 | java-iii                 | 97.98.85.151    |           7.59532e-05 |
|  56465 | java-iii                 | 98.6.95.27      |           7.59532e-05 |
|  56466 | java-iii                 | 99.147.239.234  |           7.59532e-05 |
|  56467 | java-iii                 | 99.151.203.164  |           7.59532e-05 |
|  56468 | java-iii                 | 99.158.247.126  |           7.59532e-05 |
|  56469 | java-iii                 | 99.203.154.190  |           7.59532e-05 |
|  56470 | java-iii                 | 99.203.212.121  |           7.59532e-05 |
|  56471 | java-iii                 | 99.203.213.116  |           7.59532e-05 |
|  56472 | java-iii                 | 99.203.26.184   |           7.59532e-05 |
|  56473 | java-iii                 | 99.203.26.56    |           7.59532e-05 |
|  56474 | java-iii                 | 99.203.27.131   |           7.59532e-05 |
|  56475 | java-iii                 | 99.203.90.115   |           7.59532e-05 |
|  56476 | java-iii                 | 99.98.36.168    |           7.59532e-05 |
|  61517 | javascript-i             | 104.129.18.241  |           5.4936e-05  |
|  61518 | javascript-i             | 104.14.116.184  |           5.4936e-05  |
|  61519 | javascript-i             | 104.143.92.81   |           5.4936e-05  |
|  61520 | javascript-i             | 104.180.80.54   |           5.4936e-05  |
|  61521 | javascript-i             | 104.218.78.12   |           5.4936e-05  |
|  61522 | javascript-i             | 104.219.136.100 |           5.4936e-05  |
|  61523 | javascript-i             | 104.219.136.53  |           5.4936e-05  |
|  61524 | javascript-i             | 104.237.80.11   |           5.4936e-05  |
|  61525 | javascript-i             | 104.237.80.151  |           5.4936e-05  |
|  61526 | javascript-i             | 104.237.80.157  |           5.4936e-05  |
|  61527 | javascript-i             | 104.237.80.25   |           5.4936e-05  |
|  61528 | javascript-i             | 104.237.80.48   |           5.4936e-05  |
|  61529 | javascript-i             | 104.237.80.51   |           5.4936e-05  |
|  61530 | javascript-i             | 104.237.80.77   |           5.4936e-05  |
|  61531 | javascript-i             | 104.237.80.81   |           5.4936e-05  |
|  61532 | javascript-i             | 104.237.80.87   |           5.4936e-05  |
|  61533 | javascript-i             | 104.5.159.147   |           5.4936e-05  |
|  61534 | javascript-i             | 104.5.221.32    |           5.4936e-05  |
|  61535 | javascript-i             | 104.53.206.121  |           5.4936e-05  |
|  61536 | javascript-i             | 104.53.215.21   |           5.4936e-05  |
|  61537 | javascript-i             | 107.152.102.198 |           5.4936e-05  |
|  61538 | javascript-i             | 107.200.109.74  |           5.4936e-05  |
|  61539 | javascript-i             | 107.207.1.245   |           5.4936e-05  |
|  61540 | javascript-i             | 107.77.100.129  |           5.4936e-05  |
|  61541 | javascript-i             | 107.77.100.48   |           5.4936e-05  |
|  61542 | javascript-i             | 107.77.169.54   |           5.4936e-05  |
|  61543 | javascript-i             | 107.77.169.7    |           5.4936e-05  |
|  61544 | javascript-i             | 107.77.189.32   |           5.4936e-05  |
|  61545 | javascript-i             | 107.77.196.54   |           5.4936e-05  |
|  61546 | javascript-i             | 107.77.196.71   |           5.4936e-05  |
|  61547 | javascript-i             | 107.77.198.116  |           5.4936e-05  |
|  61548 | javascript-i             | 107.77.200.151  |           5.4936e-05  |
|  61549 | javascript-i             | 107.77.200.226  |           5.4936e-05  |
|  61550 | javascript-i             | 107.77.217.114  |           5.4936e-05  |
|  61551 | javascript-i             | 107.77.217.122  |           5.4936e-05  |
|  61552 | javascript-i             | 107.77.217.151  |           5.4936e-05  |
|  61553 | javascript-i             | 107.77.218.122  |           5.4936e-05  |
|  61554 | javascript-i             | 107.77.218.215  |           5.4936e-05  |
|  61555 | javascript-i             | 107.77.218.40   |           5.4936e-05  |
|  61556 | javascript-i             | 107.77.218.42   |           5.4936e-05  |
|  61557 | javascript-i             | 107.77.218.48   |           5.4936e-05  |
|  61558 | javascript-i             | 107.77.219.171  |           5.4936e-05  |
|  61559 | javascript-i             | 107.77.219.175  |           5.4936e-05  |
|  61560 | javascript-i             | 107.77.220.107  |           5.4936e-05  |
|  61561 | javascript-i             | 107.77.220.202  |           5.4936e-05  |
|  61562 | javascript-i             | 107.77.220.56   |           5.4936e-05  |
|  61563 | javascript-i             | 107.77.220.6    |           5.4936e-05  |
|  61564 | javascript-i             | 107.77.221.10   |           5.4936e-05  |
|  61565 | javascript-i             | 107.77.221.101  |           5.4936e-05  |
|  61566 | javascript-i             | 107.77.221.108  |           5.4936e-05  |
|  61567 | javascript-i             | 107.77.221.116  |           5.4936e-05  |
|  61568 | javascript-i             | 107.77.221.152  |           5.4936e-05  |
|  61569 | javascript-i             | 107.77.221.39   |           5.4936e-05  |
|  61570 | javascript-i             | 107.77.222.116  |           5.4936e-05  |
|  61571 | javascript-i             | 107.77.80.29    |           5.4936e-05  |
|  61572 | javascript-i             | 107.77.94.80    |           5.4936e-05  |
|  61573 | javascript-i             | 108.178.77.202  |           5.4936e-05  |
|  61574 | javascript-i             | 108.199.117.106 |           5.4936e-05  |
|  61575 | javascript-i             | 108.218.193.28  |           5.4936e-05  |
|  61576 | javascript-i             | 108.222.98.137  |           5.4936e-05  |
|  61577 | javascript-i             | 110.78.138.179  |           5.4936e-05  |
|  61578 | javascript-i             | 12.16.232.100   |           5.4936e-05  |
|  61579 | javascript-i             | 12.162.135.250  |           5.4936e-05  |
|  61580 | javascript-i             | 12.19.204.162   |           5.4936e-05  |
|  61581 | javascript-i             | 12.247.3.42     |           5.4936e-05  |
|  61582 | javascript-i             | 12.31.71.58     |           5.4936e-05  |
|  61583 | javascript-i             | 12.71.181.245   |           5.4936e-05  |
|  61584 | javascript-i             | 12.71.182.66    |           5.4936e-05  |
|  61585 | javascript-i             | 129.112.109.42  |           5.4936e-05  |
|  61586 | javascript-i             | 130.45.35.109   |           5.4936e-05  |
|  61587 | javascript-i             | 130.45.43.174   |           5.4936e-05  |
|  61588 | javascript-i             | 130.45.53.218   |           5.4936e-05  |
|  61589 | javascript-i             | 130.45.68.158   |           5.4936e-05  |
|  61590 | javascript-i             | 131.194.172.221 |           5.4936e-05  |
|  61591 | javascript-i             | 131.194.172.51  |           5.4936e-05  |
|  61592 | javascript-i             | 131.194.173.46  |           5.4936e-05  |
|  61593 | javascript-i             | 136.49.90.35    |           5.4936e-05  |
|  61594 | javascript-i             | 136.50.104.158  |           5.4936e-05  |
|  61595 | javascript-i             | 136.50.24.253   |           5.4936e-05  |
|  61596 | javascript-i             | 136.50.59.188   |           5.4936e-05  |
|  61597 | javascript-i             | 136.50.95.200   |           5.4936e-05  |
|  61598 | javascript-i             | 148.59.156.46   |           5.4936e-05  |
|  61599 | javascript-i             | 148.66.37.55    |           5.4936e-05  |
|  61600 | javascript-i             | 148.66.38.220   |           5.4936e-05  |
|  61601 | javascript-i             | 154.3.251.115   |           5.4936e-05  |
|  61602 | javascript-i             | 159.192.242.43  |           5.4936e-05  |
|  61603 | javascript-i             | 162.194.247.205 |           5.4936e-05  |
|  61604 | javascript-i             | 162.195.148.122 |           5.4936e-05  |
|  61605 | javascript-i             | 162.236.252.30  |           5.4936e-05  |
|  61606 | javascript-i             | 162.72.133.33   |           5.4936e-05  |
|  61607 | javascript-i             | 166.128.126.112 |           5.4936e-05  |
|  61608 | javascript-i             | 166.137.118.110 |           5.4936e-05  |
|  61609 | javascript-i             | 166.137.126.34  |           5.4936e-05  |
|  61610 | javascript-i             | 166.137.143.91  |           5.4936e-05  |
|  61611 | javascript-i             | 166.251.106.160 |           5.4936e-05  |
|  61612 | javascript-i             | 167.71.126.76   |           5.4936e-05  |
|  61613 | javascript-i             | 167.99.96.214   |           5.4936e-05  |
|  61614 | javascript-i             | 172.110.80.227  |           5.4936e-05  |
|  61615 | javascript-i             | 172.124.174.241 |           5.4936e-05  |
|  61616 | javascript-i             | 172.127.182.225 |           5.4936e-05  |
|  61617 | javascript-i             | 172.3.29.85     |           5.4936e-05  |
|  61618 | javascript-i             | 172.56.6.146    |           5.4936e-05  |
|  61619 | javascript-i             | 172.56.7.188    |           5.4936e-05  |
|  61620 | javascript-i             | 172.56.7.43     |           5.4936e-05  |
|  61621 | javascript-i             | 172.56.7.58     |           5.4936e-05  |
|  61622 | javascript-i             | 172.58.102.136  |           5.4936e-05  |
|  61623 | javascript-i             | 172.58.102.198  |           5.4936e-05  |
|  61624 | javascript-i             | 172.58.107.187  |           5.4936e-05  |
|  61625 | javascript-i             | 172.58.107.221  |           5.4936e-05  |
|  61626 | javascript-i             | 172.58.107.51   |           5.4936e-05  |
|  61627 | javascript-i             | 172.58.107.99   |           5.4936e-05  |
|  61628 | javascript-i             | 172.58.110.145  |           5.4936e-05  |
|  61629 | javascript-i             | 172.58.110.220  |           5.4936e-05  |
|  61630 | javascript-i             | 172.58.14.166   |           5.4936e-05  |
|  61631 | javascript-i             | 172.58.67.120   |           5.4936e-05  |
|  61632 | javascript-i             | 172.58.67.201   |           5.4936e-05  |
|  61633 | javascript-i             | 172.58.67.252   |           5.4936e-05  |
|  61634 | javascript-i             | 172.58.67.77    |           5.4936e-05  |
|  61635 | javascript-i             | 172.58.73.208   |           5.4936e-05  |
|  61636 | javascript-i             | 172.58.96.164   |           5.4936e-05  |
|  61637 | javascript-i             | 172.58.96.200   |           5.4936e-05  |
|  61638 | javascript-i             | 172.58.96.230   |           5.4936e-05  |
|  61639 | javascript-i             | 172.58.96.244   |           5.4936e-05  |
|  61640 | javascript-i             | 172.58.97.32    |           5.4936e-05  |
|  61641 | javascript-i             | 172.58.97.72    |           5.4936e-05  |
|  61642 | javascript-i             | 172.58.99.11    |           5.4936e-05  |
|  61643 | javascript-i             | 172.58.99.175   |           5.4936e-05  |
|  61644 | javascript-i             | 172.58.99.207   |           5.4936e-05  |
|  61645 | javascript-i             | 172.58.99.241   |           5.4936e-05  |
|  61646 | javascript-i             | 172.58.99.70    |           5.4936e-05  |
|  61647 | javascript-i             | 172.8.173.98    |           5.4936e-05  |
|  61648 | javascript-i             | 172.8.175.156   |           5.4936e-05  |
|  61649 | javascript-i             | 173.127.36.190  |           5.4936e-05  |
|  61650 | javascript-i             | 173.127.40.170  |           5.4936e-05  |
|  61651 | javascript-i             | 173.149.159.126 |           5.4936e-05  |
|  61652 | javascript-i             | 173.149.177.187 |           5.4936e-05  |
|  61653 | javascript-i             | 173.149.180.43  |           5.4936e-05  |
|  61654 | javascript-i             | 173.149.202.225 |           5.4936e-05  |
|  61655 | javascript-i             | 173.149.241.156 |           5.4936e-05  |
|  61656 | javascript-i             | 173.173.101.207 |           5.4936e-05  |
|  61657 | javascript-i             | 173.173.109.125 |           5.4936e-05  |
|  61658 | javascript-i             | 173.173.96.192  |           5.4936e-05  |
|  61659 | javascript-i             | 173.173.96.202  |           5.4936e-05  |
|  61660 | javascript-i             | 173.174.184.26  |           5.4936e-05  |
|  61661 | javascript-i             | 173.174.191.137 |           5.4936e-05  |
|  61662 | javascript-i             | 173.174.240.181 |           5.4936e-05  |
|  61663 | javascript-i             | 173.175.109.205 |           5.4936e-05  |
|  61664 | javascript-i             | 173.175.117.44  |           5.4936e-05  |
|  61665 | javascript-i             | 173.239.232.106 |           5.4936e-05  |
|  61666 | javascript-i             | 173.239.232.11  |           5.4936e-05  |
|  61667 | javascript-i             | 173.239.232.120 |           5.4936e-05  |
|  61668 | javascript-i             | 173.239.232.135 |           5.4936e-05  |
|  61669 | javascript-i             | 173.239.232.141 |           5.4936e-05  |
|  61670 | javascript-i             | 173.239.232.146 |           5.4936e-05  |
|  61671 | javascript-i             | 173.239.232.149 |           5.4936e-05  |
|  61672 | javascript-i             | 173.239.232.164 |           5.4936e-05  |
|  61673 | javascript-i             | 173.239.232.174 |           5.4936e-05  |
|  61674 | javascript-i             | 173.239.232.21  |           5.4936e-05  |
|  61675 | javascript-i             | 173.239.232.33  |           5.4936e-05  |
|  61676 | javascript-i             | 173.239.232.45  |           5.4936e-05  |
|  61677 | javascript-i             | 173.239.232.48  |           5.4936e-05  |
|  61678 | javascript-i             | 173.239.232.50  |           5.4936e-05  |
|  61679 | javascript-i             | 173.239.232.60  |           5.4936e-05  |
|  61680 | javascript-i             | 173.239.232.77  |           5.4936e-05  |
|  61681 | javascript-i             | 173.239.232.97  |           5.4936e-05  |
|  61682 | javascript-i             | 173.239.232.98  |           5.4936e-05  |
|  61683 | javascript-i             | 173.30.84.132   |           5.4936e-05  |
|  61684 | javascript-i             | 174.128.180.24  |           5.4936e-05  |
|  61685 | javascript-i             | 174.128.181.152 |           5.4936e-05  |
|  61686 | javascript-i             | 174.128.181.163 |           5.4936e-05  |
|  61687 | javascript-i             | 174.128.181.232 |           5.4936e-05  |
|  61688 | javascript-i             | 174.128.181.39  |           5.4936e-05  |
|  61689 | javascript-i             | 174.128.181.71  |           5.4936e-05  |
|  61690 | javascript-i             | 174.207.0.141   |           5.4936e-05  |
|  61691 | javascript-i             | 174.207.14.229  |           5.4936e-05  |
|  61692 | javascript-i             | 174.207.17.95   |           5.4936e-05  |
|  61693 | javascript-i             | 174.207.2.137   |           5.4936e-05  |
|  61694 | javascript-i             | 174.207.25.229  |           5.4936e-05  |
|  61695 | javascript-i             | 174.207.5.100   |           5.4936e-05  |
|  61696 | javascript-i             | 174.235.7.82    |           5.4936e-05  |
|  61697 | javascript-i             | 174.246.193.161 |           5.4936e-05  |
|  61698 | javascript-i             | 174.246.194.0   |           5.4936e-05  |
|  61699 | javascript-i             | 174.246.194.48  |           5.4936e-05  |
|  61700 | javascript-i             | 174.246.199.81  |           5.4936e-05  |
|  61701 | javascript-i             | 174.255.134.83  |           5.4936e-05  |
|  61702 | javascript-i             | 174.255.137.57  |           5.4936e-05  |
|  61703 | javascript-i             | 184.203.153.83  |           5.4936e-05  |
|  61704 | javascript-i             | 184.203.236.132 |           5.4936e-05  |
|  61705 | javascript-i             | 184.203.6.32    |           5.4936e-05  |
|  61706 | javascript-i             | 184.226.105.72  |           5.4936e-05  |
|  61707 | javascript-i             | 184.226.56.72   |           5.4936e-05  |
|  61708 | javascript-i             | 184.226.95.31   |           5.4936e-05  |
|  61709 | javascript-i             | 189.156.27.111  |           5.4936e-05  |
|  61710 | javascript-i             | 191.96.67.5     |           5.4936e-05  |
|  61711 | javascript-i             | 192.171.117.214 |           5.4936e-05  |
|  61712 | javascript-i             | 192.241.233.20  |           5.4936e-05  |
|  61713 | javascript-i             | 192.241.235.228 |           5.4936e-05  |
|  61714 | javascript-i             | 193.37.252.42   |           5.4936e-05  |
|  61715 | javascript-i             | 193.56.117.100  |           5.4936e-05  |
|  61716 | javascript-i             | 193.56.117.16   |           5.4936e-05  |
|  61717 | javascript-i             | 195.206.104.107 |           5.4936e-05  |
|  61718 | javascript-i             | 196.52.2.25     |           5.4936e-05  |
|  61719 | javascript-i             | 196.52.39.2     |           5.4936e-05  |
|  61720 | javascript-i             | 196.52.39.32    |           5.4936e-05  |
|  61721 | javascript-i             | 196.52.60.56    |           5.4936e-05  |
|  61722 | javascript-i             | 198.147.23.211  |           5.4936e-05  |
|  61723 | javascript-i             | 198.55.125.205  |           5.4936e-05  |
|  61724 | javascript-i             | 199.193.220.106 |           5.4936e-05  |
|  61725 | javascript-i             | 199.193.221.114 |           5.4936e-05  |
|  61726 | javascript-i             | 204.44.112.76   |           5.4936e-05  |
|  61727 | javascript-i             | 207.189.160.150 |           5.4936e-05  |
|  61728 | javascript-i             | 208.84.155.36   |           5.4936e-05  |
|  61729 | javascript-i             | 209.12.105.54   |           5.4936e-05  |
|  61730 | javascript-i             | 209.58.154.12   |           5.4936e-05  |
|  61731 | javascript-i             | 209.64.120.2    |           5.4936e-05  |
|  61732 | javascript-i             | 209.64.85.248   |           5.4936e-05  |
|  61733 | javascript-i             | 212.102.44.84   |           5.4936e-05  |
|  61734 | javascript-i             | 216.162.109.195 |           5.4936e-05  |
|  61735 | javascript-i             | 216.54.250.134  |           5.4936e-05  |
|  61736 | javascript-i             | 23.105.149.2    |           5.4936e-05  |
|  61737 | javascript-i             | 23.227.143.114  |           5.4936e-05  |
|  61738 | javascript-i             | 24.155.140.118  |           5.4936e-05  |
|  61739 | javascript-i             | 24.160.9.56     |           5.4936e-05  |
|  61740 | javascript-i             | 24.214.254.251  |           5.4936e-05  |
|  61741 | javascript-i             | 24.242.143.6    |           5.4936e-05  |
|  61742 | javascript-i             | 24.243.10.238   |           5.4936e-05  |
|  61743 | javascript-i             | 24.243.14.190   |           5.4936e-05  |
|  61744 | javascript-i             | 24.243.3.85     |           5.4936e-05  |
|  61745 | javascript-i             | 24.243.31.34    |           5.4936e-05  |
|  61746 | javascript-i             | 24.243.32.134   |           5.4936e-05  |
|  61747 | javascript-i             | 24.243.6.109    |           5.4936e-05  |
|  61748 | javascript-i             | 24.26.227.191   |           5.4936e-05  |
|  61749 | javascript-i             | 24.26.232.177   |           5.4936e-05  |
|  61750 | javascript-i             | 24.26.244.77    |           5.4936e-05  |
|  61751 | javascript-i             | 24.28.146.220   |           5.4936e-05  |
|  61752 | javascript-i             | 24.28.149.24    |           5.4936e-05  |
|  61753 | javascript-i             | 24.28.152.103   |           5.4936e-05  |
|  61754 | javascript-i             | 24.54.124.11    |           5.4936e-05  |
|  61755 | javascript-i             | 24.55.39.25     |           5.4936e-05  |
|  61756 | javascript-i             | 24.73.246.228   |           5.4936e-05  |
|  61757 | javascript-i             | 38.142.208.162  |           5.4936e-05  |
|  61758 | javascript-i             | 38.96.210.4     |           5.4936e-05  |
|  61759 | javascript-i             | 45.131.192.220  |           5.4936e-05  |
|  61760 | javascript-i             | 45.131.192.222  |           5.4936e-05  |
|  61761 | javascript-i             | 45.30.141.87    |           5.4936e-05  |
|  61762 | javascript-i             | 45.31.213.223   |           5.4936e-05  |
|  61763 | javascript-i             | 45.56.151.222   |           5.4936e-05  |
|  61764 | javascript-i             | 45.56.151.99    |           5.4936e-05  |
|  61765 | javascript-i             | 47.183.143.149  |           5.4936e-05  |
|  61766 | javascript-i             | 47.187.238.163  |           5.4936e-05  |
|  61767 | javascript-i             | 47.187.247.99   |           5.4936e-05  |
|  61768 | javascript-i             | 47.218.59.57    |           5.4936e-05  |
|  61769 | javascript-i             | 47.234.152.236  |           5.4936e-05  |
|  61770 | javascript-i             | 5.181.234.114   |           5.4936e-05  |
|  61771 | javascript-i             | 5.181.234.118   |           5.4936e-05  |
|  61772 | javascript-i             | 50.226.114.86   |           5.4936e-05  |
|  61773 | javascript-i             | 50.84.229.194   |           5.4936e-05  |
|  61774 | javascript-i             | 64.124.33.178   |           5.4936e-05  |
|  61775 | javascript-i             | 64.134.149.176  |           5.4936e-05  |
|  61776 | javascript-i             | 64.134.26.193   |           5.4936e-05  |
|  61777 | javascript-i             | 64.92.13.226    |           5.4936e-05  |
|  61778 | javascript-i             | 64.92.15.138    |           5.4936e-05  |
|  61779 | javascript-i             | 64.92.28.203    |           5.4936e-05  |
|  61780 | javascript-i             | 65.220.10.3     |           5.4936e-05  |
|  61781 | javascript-i             | 65.36.104.120   |           5.4936e-05  |
|  61782 | javascript-i             | 66.162.83.130   |           5.4936e-05  |
|  61783 | javascript-i             | 66.196.195.34   |           5.4936e-05  |
|  61784 | javascript-i             | 66.25.149.223   |           5.4936e-05  |
|  61785 | javascript-i             | 66.25.154.61    |           5.4936e-05  |
|  61786 | javascript-i             | 66.25.67.202    |           5.4936e-05  |
|  61787 | javascript-i             | 66.25.80.16     |           5.4936e-05  |
|  61788 | javascript-i             | 66.25.81.21     |           5.4936e-05  |
|  61789 | javascript-i             | 66.64.5.233     |           5.4936e-05  |
|  61790 | javascript-i             | 66.69.24.31     |           5.4936e-05  |
|  61791 | javascript-i             | 66.69.36.47     |           5.4936e-05  |
|  61792 | javascript-i             | 66.69.37.238    |           5.4936e-05  |
|  61793 | javascript-i             | 66.69.65.3      |           5.4936e-05  |
|  61794 | javascript-i             | 67.10.129.157   |           5.4936e-05  |
|  61795 | javascript-i             | 67.10.191.167   |           5.4936e-05  |
|  61796 | javascript-i             | 67.11.100.255   |           5.4936e-05  |
|  61797 | javascript-i             | 67.11.16.255    |           5.4936e-05  |
|  61798 | javascript-i             | 67.11.167.40    |           5.4936e-05  |
|  61799 | javascript-i             | 67.11.167.95    |           5.4936e-05  |
|  61800 | javascript-i             | 67.11.172.66    |           5.4936e-05  |
|  61801 | javascript-i             | 67.11.204.68    |           5.4936e-05  |
|  61802 | javascript-i             | 67.11.33.106    |           5.4936e-05  |
|  61803 | javascript-i             | 67.11.48.25     |           5.4936e-05  |
|  61804 | javascript-i             | 67.11.62.128    |           5.4936e-05  |
|  61805 | javascript-i             | 67.164.150.90   |           5.4936e-05  |
|  61806 | javascript-i             | 67.44.176.135   |           5.4936e-05  |
|  61807 | javascript-i             | 67.48.40.83     |           5.4936e-05  |
|  61808 | javascript-i             | 67.7.11.231     |           5.4936e-05  |
|  61809 | javascript-i             | 67.7.3.203      |           5.4936e-05  |
|  61810 | javascript-i             | 68.206.246.215  |           5.4936e-05  |
|  61811 | javascript-i             | 68.95.172.10    |           5.4936e-05  |
|  61812 | javascript-i             | 69.153.19.127   |           5.4936e-05  |
|  61813 | javascript-i             | 69.154.52.90    |           5.4936e-05  |
|  61814 | javascript-i             | 69.193.187.30   |           5.4936e-05  |
|  61815 | javascript-i             | 69.212.157.148  |           5.4936e-05  |
|  61816 | javascript-i             | 69.232.49.24    |           5.4936e-05  |
|  61817 | javascript-i             | 70.112.179.142  |           5.4936e-05  |
|  61818 | javascript-i             | 70.113.90.221   |           5.4936e-05  |
|  61819 | javascript-i             | 70.114.29.134   |           5.4936e-05  |
|  61820 | javascript-i             | 70.114.38.157   |           5.4936e-05  |
|  61821 | javascript-i             | 70.116.81.203   |           5.4936e-05  |
|  61822 | javascript-i             | 70.117.71.59    |           5.4936e-05  |
|  61823 | javascript-i             | 70.118.67.138   |           5.4936e-05  |
|  61824 | javascript-i             | 70.120.213.195  |           5.4936e-05  |
|  61825 | javascript-i             | 70.120.3.242    |           5.4936e-05  |
|  61826 | javascript-i             | 70.120.5.227    |           5.4936e-05  |
|  61827 | javascript-i             | 70.121.136.246  |           5.4936e-05  |
|  61828 | javascript-i             | 70.121.181.181  |           5.4936e-05  |
|  61829 | javascript-i             | 70.121.184.72   |           5.4936e-05  |
|  61830 | javascript-i             | 70.123.160.242  |           5.4936e-05  |
|  61831 | javascript-i             | 70.123.164.223  |           5.4936e-05  |
|  61832 | javascript-i             | 70.123.169.229  |           5.4936e-05  |
|  61833 | javascript-i             | 70.123.194.234  |           5.4936e-05  |
|  61834 | javascript-i             | 70.123.215.59   |           5.4936e-05  |
|  61835 | javascript-i             | 70.123.224.242  |           5.4936e-05  |
|  61836 | javascript-i             | 70.123.239.48   |           5.4936e-05  |
|  61837 | javascript-i             | 70.124.99.231   |           5.4936e-05  |
|  61838 | javascript-i             | 70.125.148.96   |           5.4936e-05  |
|  61839 | javascript-i             | 70.133.251.114  |           5.4936e-05  |
|  61840 | javascript-i             | 71.132.254.162  |           5.4936e-05  |
|  61841 | javascript-i             | 71.136.255.148  |           5.4936e-05  |
|  61842 | javascript-i             | 71.234.171.235  |           5.4936e-05  |
|  61843 | javascript-i             | 71.40.1.172     |           5.4936e-05  |
|  61844 | javascript-i             | 71.42.105.146   |           5.4936e-05  |
|  61845 | javascript-i             | 71.42.105.172   |           5.4936e-05  |
|  61846 | javascript-i             | 71.42.133.19    |           5.4936e-05  |
|  61847 | javascript-i             | 71.42.140.139   |           5.4936e-05  |
|  61848 | javascript-i             | 71.62.94.245    |           5.4936e-05  |
|  61849 | javascript-i             | 71.78.167.131   |           5.4936e-05  |
|  61850 | javascript-i             | 71.81.176.46    |           5.4936e-05  |
|  61851 | javascript-i             | 72.173.98.235   |           5.4936e-05  |
|  61852 | javascript-i             | 72.177.116.71   |           5.4936e-05  |
|  61853 | javascript-i             | 72.177.146.68   |           5.4936e-05  |
|  61854 | javascript-i             | 72.177.233.3    |           5.4936e-05  |
|  61855 | javascript-i             | 72.180.149.37   |           5.4936e-05  |
|  61856 | javascript-i             | 72.181.104.30   |           5.4936e-05  |
|  61857 | javascript-i             | 72.181.106.12   |           5.4936e-05  |
|  61858 | javascript-i             | 72.181.145.92   |           5.4936e-05  |
|  61859 | javascript-i             | 72.181.59.214   |           5.4936e-05  |
|  61860 | javascript-i             | 72.181.97.136   |           5.4936e-05  |
|  61861 | javascript-i             | 72.182.214.180  |           5.4936e-05  |
|  61862 | javascript-i             | 72.183.145.159  |           5.4936e-05  |
|  61863 | javascript-i             | 72.190.151.50   |           5.4936e-05  |
|  61864 | javascript-i             | 72.190.168.217  |           5.4936e-05  |
|  61865 | javascript-i             | 72.190.170.159  |           5.4936e-05  |
|  61866 | javascript-i             | 72.190.232.108  |           5.4936e-05  |
|  61867 | javascript-i             | 72.190.251.104  |           5.4936e-05  |
|  61868 | javascript-i             | 72.191.11.178   |           5.4936e-05  |
|  61869 | javascript-i             | 72.191.29.130   |           5.4936e-05  |
|  61870 | javascript-i             | 72.191.32.140   |           5.4936e-05  |
|  61871 | javascript-i             | 73.14.59.241    |           5.4936e-05  |
|  61872 | javascript-i             | 73.153.134.112  |           5.4936e-05  |
|  61873 | javascript-i             | 73.153.77.234   |           5.4936e-05  |
|  61874 | javascript-i             | 73.219.222.176  |           5.4936e-05  |
|  61875 | javascript-i             | 73.77.171.11    |           5.4936e-05  |
|  61876 | javascript-i             | 73.85.100.144   |           5.4936e-05  |
|  61877 | javascript-i             | 73.85.64.161    |           5.4936e-05  |
|  61878 | javascript-i             | 74.105.240.236  |           5.4936e-05  |
|  61879 | javascript-i             | 74.50.118.217   |           5.4936e-05  |
|  61880 | javascript-i             | 75.166.30.247   |           5.4936e-05  |
|  61881 | javascript-i             | 75.21.227.97    |           5.4936e-05  |
|  61882 | javascript-i             | 75.35.140.23    |           5.4936e-05  |
|  61883 | javascript-i             | 75.54.32.20     |           5.4936e-05  |
|  61884 | javascript-i             | 75.54.32.23     |           5.4936e-05  |
|  61885 | javascript-i             | 75.54.32.79     |           5.4936e-05  |
|  61886 | javascript-i             | 75.54.32.9      |           5.4936e-05  |
|  61887 | javascript-i             | 75.57.25.192    |           5.4936e-05  |
|  61888 | javascript-i             | 75.58.164.200   |           5.4936e-05  |
|  61889 | javascript-i             | 75.87.28.58     |           5.4936e-05  |
|  61890 | javascript-i             | 76.184.212.53   |           5.4936e-05  |
|  61891 | javascript-i             | 76.185.128.151  |           5.4936e-05  |
|  61892 | javascript-i             | 76.185.142.244  |           5.4936e-05  |
|  61893 | javascript-i             | 76.185.149.233  |           5.4936e-05  |
|  61894 | javascript-i             | 76.185.197.205  |           5.4936e-05  |
|  61895 | javascript-i             | 76.185.234.25   |           5.4936e-05  |
|  61896 | javascript-i             | 76.185.236.152  |           5.4936e-05  |
|  61897 | javascript-i             | 76.186.54.46    |           5.4936e-05  |
|  61898 | javascript-i             | 76.218.41.171   |           5.4936e-05  |
|  61899 | javascript-i             | 76.85.110.35    |           5.4936e-05  |
|  61900 | javascript-i             | 79.220.63.16    |           5.4936e-05  |
|  61901 | javascript-i             | 89.187.164.131  |           5.4936e-05  |
|  61902 | javascript-i             | 89.187.164.132  |           5.4936e-05  |
|  61903 | javascript-i             | 89.187.164.146  |           5.4936e-05  |
|  61904 | javascript-i             | 89.187.164.147  |           5.4936e-05  |
|  61905 | javascript-i             | 89.187.164.161  |           5.4936e-05  |
|  61906 | javascript-i             | 89.187.164.163  |           5.4936e-05  |
|  61907 | javascript-i             | 89.187.164.164  |           5.4936e-05  |
|  61908 | javascript-i             | 89.187.164.166  |           5.4936e-05  |
|  61909 | javascript-i             | 89.187.164.167  |           5.4936e-05  |
|  61910 | javascript-i             | 89.187.164.75   |           5.4936e-05  |
|  61911 | javascript-i             | 89.187.164.79   |           5.4936e-05  |
|  61912 | javascript-i             | 89.187.175.174  |           5.4936e-05  |
|  61913 | javascript-i             | 92.38.148.10    |           5.4936e-05  |
|  61914 | javascript-i             | 96.46.160.20    |           5.4936e-05  |
|  61915 | javascript-i             | 96.8.128.88     |           5.4936e-05  |
|  61916 | javascript-i             | 96.8.128.94     |           5.4936e-05  |
|  61917 | javascript-i             | 96.8.130.11     |           5.4936e-05  |
|  61918 | javascript-i             | 96.8.248.149    |           5.4936e-05  |
|  61919 | javascript-i             | 96.8.248.15     |           5.4936e-05  |
|  61920 | javascript-i             | 97.105.19.60    |           5.4936e-05  |
|  61921 | javascript-i             | 97.105.40.214   |           5.4936e-05  |
|  61922 | javascript-i             | 97.105.54.251   |           5.4936e-05  |
|  61923 | javascript-i             | 97.105.69.130   |           5.4936e-05  |
|  61924 | javascript-i             | 97.105.81.147   |           5.4936e-05  |
|  61925 | javascript-i             | 97.117.115.55   |           5.4936e-05  |
|  61926 | javascript-i             | 97.118.255.115  |           5.4936e-05  |
|  61927 | javascript-i             | 97.77.126.60    |           5.4936e-05  |
|  61928 | javascript-i             | 97.77.238.3     |           5.4936e-05  |
|  61929 | javascript-i             | 97.79.147.122   |           5.4936e-05  |
|  61930 | javascript-i             | 98.158.244.8    |           5.4936e-05  |
|  61931 | javascript-i             | 98.6.203.30     |           5.4936e-05  |
|  61932 | javascript-i             | 99.113.230.177  |           5.4936e-05  |
|  61933 | javascript-i             | 99.126.116.28   |           5.4936e-05  |
|  61934 | javascript-i             | 99.132.129.169  |           5.4936e-05  |
|  61935 | javascript-i             | 99.132.130.148  |           5.4936e-05  |
|  61936 | javascript-i             | 99.140.253.42   |           5.4936e-05  |
|  61937 | javascript-i             | 99.147.238.134  |           5.4936e-05  |
|  61938 | javascript-i             | 99.147.239.234  |           5.4936e-05  |
|  61939 | javascript-i             | 99.151.195.187  |           5.4936e-05  |
|  61940 | javascript-i             | 99.184.63.30    |           5.4936e-05  |
|  61941 | javascript-i             | 99.203.154.231  |           5.4936e-05  |
|  61942 | javascript-i             | 99.203.154.58   |           5.4936e-05  |
|  61943 | javascript-i             | 99.203.23.16    |           5.4936e-05  |
|  61944 | javascript-i             | 99.203.90.84    |           5.4936e-05  |
|  61945 | javascript-i             | 99.27.139.172   |           5.4936e-05  |
|  61946 | javascript-i             | 99.48.230.77    |           5.4936e-05  |
|  61947 | javascript-i             | 99.5.223.96     |           5.4936e-05  |
|  61948 | javascript-i             | 99.52.75.127    |           5.4936e-05  |
|  61949 | javascript-i             | 99.52.78.67     |           5.4936e-05  |
|  61950 | javascript-i             | 99.6.251.67     |           5.4936e-05  |
|  61951 | javascript-i             | 99.62.54.53     |           5.4936e-05  |
|  61952 | javascript-i             | 99.76.234.122   |           5.4936e-05  |
|  61953 | javascript-i             | 99.76.235.191   |           5.4936e-05  |
|  74774 | javascript-ii            | 104.180.86.144  |           9.7144e-05  |
|  74775 | javascript-ii            | 104.182.14.96   |           9.7144e-05  |
|  74776 | javascript-ii            | 104.190.249.199 |           9.7144e-05  |
|  74777 | javascript-ii            | 104.219.136.100 |           9.7144e-05  |
|  74778 | javascript-ii            | 104.237.80.135  |           9.7144e-05  |
|  74779 | javascript-ii            | 104.237.80.25   |           9.7144e-05  |
|  74780 | javascript-ii            | 104.237.80.81   |           9.7144e-05  |
|  74781 | javascript-ii            | 104.237.80.87   |           9.7144e-05  |
|  74782 | javascript-ii            | 104.5.159.147   |           9.7144e-05  |
|  74783 | javascript-ii            | 104.5.221.32    |           9.7144e-05  |
|  74784 | javascript-ii            | 104.53.206.121  |           9.7144e-05  |
|  74785 | javascript-ii            | 104.53.215.21   |           9.7144e-05  |
|  74786 | javascript-ii            | 104.53.215.7    |           9.7144e-05  |
|  74787 | javascript-ii            | 107.129.209.3   |           9.7144e-05  |
|  74788 | javascript-ii            | 107.146.141.88  |           9.7144e-05  |
|  74789 | javascript-ii            | 107.200.109.74  |           9.7144e-05  |
|  74790 | javascript-ii            | 107.77.100.108  |           9.7144e-05  |
|  74791 | javascript-ii            | 107.77.105.117  |           9.7144e-05  |
|  74792 | javascript-ii            | 107.77.105.28   |           9.7144e-05  |
|  74793 | javascript-ii            | 107.77.105.75   |           9.7144e-05  |
|  74794 | javascript-ii            | 107.77.169.12   |           9.7144e-05  |
|  74795 | javascript-ii            | 107.77.169.8    |           9.7144e-05  |
|  74796 | javascript-ii            | 107.77.201.165  |           9.7144e-05  |
|  74797 | javascript-ii            | 107.77.217.115  |           9.7144e-05  |
|  74798 | javascript-ii            | 107.77.217.231  |           9.7144e-05  |
|  74799 | javascript-ii            | 107.77.218.122  |           9.7144e-05  |
|  74800 | javascript-ii            | 107.77.218.184  |           9.7144e-05  |
|  74801 | javascript-ii            | 107.77.218.22   |           9.7144e-05  |
|  74802 | javascript-ii            | 107.77.218.91   |           9.7144e-05  |
|  74803 | javascript-ii            | 107.77.219.141  |           9.7144e-05  |
|  74804 | javascript-ii            | 107.77.219.171  |           9.7144e-05  |
|  74805 | javascript-ii            | 107.77.219.49   |           9.7144e-05  |
|  74806 | javascript-ii            | 107.77.219.67   |           9.7144e-05  |
|  74807 | javascript-ii            | 107.77.220.191  |           9.7144e-05  |
|  74808 | javascript-ii            | 107.77.221.209  |           9.7144e-05  |
|  74809 | javascript-ii            | 107.77.221.70   |           9.7144e-05  |
|  74810 | javascript-ii            | 108.174.85.238  |           9.7144e-05  |
|  74811 | javascript-ii            | 108.18.249.150  |           9.7144e-05  |
|  74812 | javascript-ii            | 108.196.159.214 |           9.7144e-05  |
|  74813 | javascript-ii            | 108.199.115.75  |           9.7144e-05  |
|  74814 | javascript-ii            | 108.199.117.96  |           9.7144e-05  |
|  74815 | javascript-ii            | 108.213.189.26  |           9.7144e-05  |
|  74816 | javascript-ii            | 108.249.226.232 |           9.7144e-05  |
|  74817 | javascript-ii            | 108.93.10.214   |           9.7144e-05  |
|  74818 | javascript-ii            | 12.16.232.100   |           9.7144e-05  |
|  74819 | javascript-ii            | 12.162.135.250  |           9.7144e-05  |
|  74820 | javascript-ii            | 12.19.204.162   |           9.7144e-05  |
|  74821 | javascript-ii            | 12.247.3.42     |           9.7144e-05  |
|  74822 | javascript-ii            | 12.5.63.210     |           9.7144e-05  |
|  74823 | javascript-ii            | 129.112.109.42  |           9.7144e-05  |
|  74824 | javascript-ii            | 129.115.195.52  |           9.7144e-05  |
|  74825 | javascript-ii            | 129.115.2.245   |           9.7144e-05  |
|  74826 | javascript-ii            | 130.45.33.209   |           9.7144e-05  |
|  74827 | javascript-ii            | 131.194.173.119 |           9.7144e-05  |
|  74828 | javascript-ii            | 131.44.121.129  |           9.7144e-05  |
|  74829 | javascript-ii            | 136.49.100.118  |           9.7144e-05  |
|  74830 | javascript-ii            | 136.50.54.38    |           9.7144e-05  |
|  74831 | javascript-ii            | 136.50.92.234   |           9.7144e-05  |
|  74832 | javascript-ii            | 137.254.7.173   |           9.7144e-05  |
|  74833 | javascript-ii            | 148.66.37.91    |           9.7144e-05  |
|  74834 | javascript-ii            | 154.6.28.146    |           9.7144e-05  |
|  74835 | javascript-ii            | 159.192.242.43  |           9.7144e-05  |
|  74836 | javascript-ii            | 159.192.243.186 |           9.7144e-05  |
|  74837 | javascript-ii            | 162.200.119.143 |           9.7144e-05  |
|  74838 | javascript-ii            | 162.233.170.151 |           9.7144e-05  |
|  74839 | javascript-ii            | 162.236.252.30  |           9.7144e-05  |
|  74840 | javascript-ii            | 162.71.236.115  |           9.7144e-05  |
|  74841 | javascript-ii            | 165.225.48.86   |           9.7144e-05  |
|  74842 | javascript-ii            | 166.137.139.115 |           9.7144e-05  |
|  74843 | javascript-ii            | 166.137.139.56  |           9.7144e-05  |
|  74844 | javascript-ii            | 166.251.106.84  |           9.7144e-05  |
|  74845 | javascript-ii            | 166.251.109.223 |           9.7144e-05  |
|  74846 | javascript-ii            | 166.251.112.88  |           9.7144e-05  |
|  74847 | javascript-ii            | 172.108.140.130 |           9.7144e-05  |
|  74848 | javascript-ii            | 172.127.187.185 |           9.7144e-05  |
|  74849 | javascript-ii            | 172.242.120.198 |           9.7144e-05  |
|  74850 | javascript-ii            | 172.3.29.85     |           9.7144e-05  |
|  74851 | javascript-ii            | 172.56.6.32     |           9.7144e-05  |
|  74852 | javascript-ii            | 172.56.7.113    |           9.7144e-05  |
|  74853 | javascript-ii            | 172.56.7.119    |           9.7144e-05  |
|  74854 | javascript-ii            | 172.56.7.23     |           9.7144e-05  |
|  74855 | javascript-ii            | 172.56.7.58     |           9.7144e-05  |
|  74856 | javascript-ii            | 172.58.107.104  |           9.7144e-05  |
|  74857 | javascript-ii            | 172.58.107.124  |           9.7144e-05  |
|  74858 | javascript-ii            | 172.58.107.187  |           9.7144e-05  |
|  74859 | javascript-ii            | 172.58.109.59   |           9.7144e-05  |
|  74860 | javascript-ii            | 172.58.97.240   |           9.7144e-05  |
|  74861 | javascript-ii            | 172.58.97.32    |           9.7144e-05  |
|  74862 | javascript-ii            | 172.58.99.165   |           9.7144e-05  |
|  74863 | javascript-ii            | 172.58.99.188   |           9.7144e-05  |
|  74864 | javascript-ii            | 172.58.99.69    |           9.7144e-05  |
|  74865 | javascript-ii            | 172.58.99.71    |           9.7144e-05  |
|  74866 | javascript-ii            | 172.87.148.22   |           9.7144e-05  |
|  74867 | javascript-ii            | 172.87.181.101  |           9.7144e-05  |
|  74868 | javascript-ii            | 173.123.129.83  |           9.7144e-05  |
|  74869 | javascript-ii            | 173.149.129.239 |           9.7144e-05  |
|  74870 | javascript-ii            | 173.149.159.126 |           9.7144e-05  |
|  74871 | javascript-ii            | 173.149.177.187 |           9.7144e-05  |
|  74872 | javascript-ii            | 173.149.202.225 |           9.7144e-05  |
|  74873 | javascript-ii            | 173.149.249.154 |           9.7144e-05  |
|  74874 | javascript-ii            | 173.173.101.207 |           9.7144e-05  |
|  74875 | javascript-ii            | 173.173.102.182 |           9.7144e-05  |
|  74876 | javascript-ii            | 173.173.109.125 |           9.7144e-05  |
|  74877 | javascript-ii            | 173.173.127.101 |           9.7144e-05  |
|  74878 | javascript-ii            | 173.173.47.94   |           9.7144e-05  |
|  74879 | javascript-ii            | 173.173.96.192  |           9.7144e-05  |
|  74880 | javascript-ii            | 173.173.96.82   |           9.7144e-05  |
|  74881 | javascript-ii            | 173.174.153.61  |           9.7144e-05  |
|  74882 | javascript-ii            | 173.174.155.172 |           9.7144e-05  |
|  74883 | javascript-ii            | 173.174.164.88  |           9.7144e-05  |
|  74884 | javascript-ii            | 173.174.190.61  |           9.7144e-05  |
|  74885 | javascript-ii            | 173.174.208.178 |           9.7144e-05  |
|  74886 | javascript-ii            | 173.174.220.17  |           9.7144e-05  |
|  74887 | javascript-ii            | 173.174.243.231 |           9.7144e-05  |
|  74888 | javascript-ii            | 173.175.109.205 |           9.7144e-05  |
|  74889 | javascript-ii            | 173.175.122.123 |           9.7144e-05  |
|  74890 | javascript-ii            | 173.239.232.114 |           9.7144e-05  |
|  74891 | javascript-ii            | 173.239.232.120 |           9.7144e-05  |
|  74892 | javascript-ii            | 173.239.232.137 |           9.7144e-05  |
|  74893 | javascript-ii            | 173.239.232.176 |           9.7144e-05  |
|  74894 | javascript-ii            | 173.239.232.21  |           9.7144e-05  |
|  74895 | javascript-ii            | 173.239.232.33  |           9.7144e-05  |
|  74896 | javascript-ii            | 173.239.232.64  |           9.7144e-05  |
|  74897 | javascript-ii            | 173.239.232.67  |           9.7144e-05  |
|  74898 | javascript-ii            | 173.239.232.72  |           9.7144e-05  |
|  74899 | javascript-ii            | 173.239.232.82  |           9.7144e-05  |
|  74900 | javascript-ii            | 174.128.181.163 |           9.7144e-05  |
|  74901 | javascript-ii            | 174.128.181.232 |           9.7144e-05  |
|  74902 | javascript-ii            | 174.128.181.39  |           9.7144e-05  |
|  74903 | javascript-ii            | 174.207.0.141   |           9.7144e-05  |
|  74904 | javascript-ii            | 174.207.14.75   |           9.7144e-05  |
|  74905 | javascript-ii            | 174.207.15.91   |           9.7144e-05  |
|  74906 | javascript-ii            | 174.207.24.42   |           9.7144e-05  |
|  74907 | javascript-ii            | 174.207.25.219  |           9.7144e-05  |
|  74908 | javascript-ii            | 174.246.193.51  |           9.7144e-05  |
|  74909 | javascript-ii            | 174.246.196.4   |           9.7144e-05  |
|  74910 | javascript-ii            | 174.255.137.57  |           9.7144e-05  |
|  74911 | javascript-ii            | 174.255.145.205 |           9.7144e-05  |
|  74912 | javascript-ii            | 184.169.45.4    |           9.7144e-05  |
|  74913 | javascript-ii            | 184.62.220.53   |           9.7144e-05  |
|  74914 | javascript-ii            | 185.151.12.242  |           9.7144e-05  |
|  74915 | javascript-ii            | 185.247.70.196  |           9.7144e-05  |
|  74916 | javascript-ii            | 189.140.49.80   |           9.7144e-05  |
|  74917 | javascript-ii            | 191.96.67.226   |           9.7144e-05  |
|  74918 | javascript-ii            | 191.96.67.34    |           9.7144e-05  |
|  74919 | javascript-ii            | 192.107.1.225   |           9.7144e-05  |
|  74920 | javascript-ii            | 192.171.117.214 |           9.7144e-05  |
|  74921 | javascript-ii            | 193.56.117.75   |           9.7144e-05  |
|  74922 | javascript-ii            | 195.206.104.107 |           9.7144e-05  |
|  74923 | javascript-ii            | 196.52.39.32    |           9.7144e-05  |
|  74924 | javascript-ii            | 198.147.23.196  |           9.7144e-05  |
|  74925 | javascript-ii            | 199.59.33.126   |           9.7144e-05  |
|  74926 | javascript-ii            | 204.44.112.76   |           9.7144e-05  |
|  74927 | javascript-ii            | 205.251.148.154 |           9.7144e-05  |
|  74928 | javascript-ii            | 205.251.148.162 |           9.7144e-05  |
|  74929 | javascript-ii            | 205.251.148.82  |           9.7144e-05  |
|  74930 | javascript-ii            | 207.119.56.213  |           9.7144e-05  |
|  74931 | javascript-ii            | 207.189.160.150 |           9.7144e-05  |
|  74932 | javascript-ii            | 208.54.86.183   |           9.7144e-05  |
|  74933 | javascript-ii            | 209.205.120.222 |           9.7144e-05  |
|  74934 | javascript-ii            | 209.40.156.14   |           9.7144e-05  |
|  74935 | javascript-ii            | 209.58.147.231  |           9.7144e-05  |
|  74936 | javascript-ii            | 209.65.254.25   |           9.7144e-05  |
|  74937 | javascript-ii            | 23.116.24.226   |           9.7144e-05  |
|  74938 | javascript-ii            | 23.117.144.200  |           9.7144e-05  |
|  74939 | javascript-ii            | 24.153.246.211  |           9.7144e-05  |
|  74940 | javascript-ii            | 24.155.140.118  |           9.7144e-05  |
|  74941 | javascript-ii            | 24.155.24.134   |           9.7144e-05  |
|  74942 | javascript-ii            | 24.160.140.11   |           9.7144e-05  |
|  74943 | javascript-ii            | 24.175.74.13    |           9.7144e-05  |
|  74944 | javascript-ii            | 24.242.143.6    |           9.7144e-05  |
|  74945 | javascript-ii            | 24.243.10.238   |           9.7144e-05  |
|  74946 | javascript-ii            | 24.243.31.34    |           9.7144e-05  |
|  74947 | javascript-ii            | 24.243.59.95    |           9.7144e-05  |
|  74948 | javascript-ii            | 24.26.235.139   |           9.7144e-05  |
|  74949 | javascript-ii            | 24.27.227.1     |           9.7144e-05  |
|  74950 | javascript-ii            | 24.73.246.228   |           9.7144e-05  |
|  74951 | javascript-ii            | 24.99.149.248   |           9.7144e-05  |
|  74952 | javascript-ii            | 4.34.44.52      |           9.7144e-05  |
|  74953 | javascript-ii            | 45.131.192.16   |           9.7144e-05  |
|  74954 | javascript-ii            | 45.131.192.166  |           9.7144e-05  |
|  74955 | javascript-ii            | 45.131.192.220  |           9.7144e-05  |
|  74956 | javascript-ii            | 45.131.192.222  |           9.7144e-05  |
|  74957 | javascript-ii            | 45.21.0.138     |           9.7144e-05  |
|  74958 | javascript-ii            | 45.22.97.134    |           9.7144e-05  |
|  74959 | javascript-ii            | 45.31.212.88    |           9.7144e-05  |
|  74960 | javascript-ii            | 45.56.151.117   |           9.7144e-05  |
|  74961 | javascript-ii            | 45.56.151.44    |           9.7144e-05  |
|  74962 | javascript-ii            | 45.56.151.8     |           9.7144e-05  |
|  74963 | javascript-ii            | 45.56.151.93    |           9.7144e-05  |
|  74964 | javascript-ii            | 45.56.151.99    |           9.7144e-05  |
|  74965 | javascript-ii            | 47.183.142.121  |           9.7144e-05  |
|  74966 | javascript-ii            | 47.185.105.67   |           9.7144e-05  |
|  74967 | javascript-ii            | 47.187.238.163  |           9.7144e-05  |
|  74968 | javascript-ii            | 47.189.38.25    |           9.7144e-05  |
|  74969 | javascript-ii            | 47.218.59.57    |           9.7144e-05  |
|  74970 | javascript-ii            | 47.234.152.61   |           9.7144e-05  |
|  74971 | javascript-ii            | 5.181.234.114   |           9.7144e-05  |
|  74972 | javascript-ii            | 64.148.247.227  |           9.7144e-05  |
|  74973 | javascript-ii            | 64.191.50.235   |           9.7144e-05  |
|  74974 | javascript-ii            | 64.250.7.235    |           9.7144e-05  |
|  74975 | javascript-ii            | 64.44.81.84     |           9.7144e-05  |
|  74976 | javascript-ii            | 64.92.13.226    |           9.7144e-05  |
|  74977 | javascript-ii            | 65.220.10.3     |           9.7144e-05  |
|  74978 | javascript-ii            | 66.162.83.130   |           9.7144e-05  |
|  74979 | javascript-ii            | 66.225.111.109  |           9.7144e-05  |
|  74980 | javascript-ii            | 66.25.162.204   |           9.7144e-05  |
|  74981 | javascript-ii            | 66.25.164.171   |           9.7144e-05  |
|  74982 | javascript-ii            | 66.25.168.171   |           9.7144e-05  |
|  74983 | javascript-ii            | 66.25.171.31    |           9.7144e-05  |
|  74984 | javascript-ii            | 66.25.172.244   |           9.7144e-05  |
|  74985 | javascript-ii            | 66.25.7.9       |           9.7144e-05  |
|  74986 | javascript-ii            | 66.69.36.47     |           9.7144e-05  |
|  74987 | javascript-ii            | 66.69.42.51     |           9.7144e-05  |
|  74988 | javascript-ii            | 66.69.48.5      |           9.7144e-05  |
|  74989 | javascript-ii            | 66.69.58.238    |           9.7144e-05  |
|  74990 | javascript-ii            | 66.90.180.216   |           9.7144e-05  |
|  74991 | javascript-ii            | 67.10.128.179   |           9.7144e-05  |
|  74992 | javascript-ii            | 67.10.153.249   |           9.7144e-05  |
|  74993 | javascript-ii            | 67.10.169.200   |           9.7144e-05  |
|  74994 | javascript-ii            | 67.10.213.134   |           9.7144e-05  |
|  74995 | javascript-ii            | 67.11.100.255   |           9.7144e-05  |
|  74996 | javascript-ii            | 67.11.100.82    |           9.7144e-05  |
|  74997 | javascript-ii            | 67.11.145.202   |           9.7144e-05  |
|  74998 | javascript-ii            | 67.11.172.66    |           9.7144e-05  |
|  74999 | javascript-ii            | 67.11.207.37    |           9.7144e-05  |
|  75000 | javascript-ii            | 67.11.215.110   |           9.7144e-05  |
|  75001 | javascript-ii            | 67.11.223.207   |           9.7144e-05  |
|  75002 | javascript-ii            | 67.11.28.235    |           9.7144e-05  |
|  75003 | javascript-ii            | 67.11.48.25     |           9.7144e-05  |
|  75004 | javascript-ii            | 67.11.87.229    |           9.7144e-05  |
|  75005 | javascript-ii            | 67.48.40.83     |           9.7144e-05  |
|  75006 | javascript-ii            | 67.48.50.218    |           9.7144e-05  |
|  75007 | javascript-ii            | 67.48.59.123    |           9.7144e-05  |
|  75008 | javascript-ii            | 67.79.222.75    |           9.7144e-05  |
|  75009 | javascript-ii            | 67.80.48.111    |           9.7144e-05  |
|  75010 | javascript-ii            | 68.206.161.84   |           9.7144e-05  |
|  75011 | javascript-ii            | 68.249.0.44     |           9.7144e-05  |
|  75012 | javascript-ii            | 69.232.100.94   |           9.7144e-05  |
|  75013 | javascript-ii            | 69.232.103.179  |           9.7144e-05  |
|  75014 | javascript-ii            | 69.236.100.33   |           9.7144e-05  |
|  75015 | javascript-ii            | 70.112.164.11   |           9.7144e-05  |
|  75016 | javascript-ii            | 70.113.107.71   |           9.7144e-05  |
|  75017 | javascript-ii            | 70.113.50.67    |           9.7144e-05  |
|  75018 | javascript-ii            | 70.114.239.51   |           9.7144e-05  |
|  75019 | javascript-ii            | 70.115.114.110  |           9.7144e-05  |
|  75020 | javascript-ii            | 70.116.81.203   |           9.7144e-05  |
|  75021 | javascript-ii            | 70.117.5.148    |           9.7144e-05  |
|  75022 | javascript-ii            | 70.120.1.49     |           9.7144e-05  |
|  75023 | javascript-ii            | 70.120.3.91     |           9.7144e-05  |
|  75024 | javascript-ii            | 70.120.31.121   |           9.7144e-05  |
|  75025 | javascript-ii            | 70.120.49.220   |           9.7144e-05  |
|  75026 | javascript-ii            | 70.120.83.9     |           9.7144e-05  |
|  75027 | javascript-ii            | 70.120.93.125   |           9.7144e-05  |
|  75028 | javascript-ii            | 70.121.136.246  |           9.7144e-05  |
|  75029 | javascript-ii            | 70.121.171.184  |           9.7144e-05  |
|  75030 | javascript-ii            | 70.121.182.222  |           9.7144e-05  |
|  75031 | javascript-ii            | 70.121.187.122  |           9.7144e-05  |
|  75032 | javascript-ii            | 70.122.206.138  |           9.7144e-05  |
|  75033 | javascript-ii            | 70.123.135.83   |           9.7144e-05  |
|  75034 | javascript-ii            | 70.123.162.206  |           9.7144e-05  |
|  75035 | javascript-ii            | 70.123.200.22   |           9.7144e-05  |
|  75036 | javascript-ii            | 70.123.201.199  |           9.7144e-05  |
|  75037 | javascript-ii            | 70.123.224.242  |           9.7144e-05  |
|  75038 | javascript-ii            | 70.123.239.48   |           9.7144e-05  |
|  75039 | javascript-ii            | 70.123.247.196  |           9.7144e-05  |
|  75040 | javascript-ii            | 70.125.138.191  |           9.7144e-05  |
|  75041 | javascript-ii            | 70.128.146.8    |           9.7144e-05  |
|  75042 | javascript-ii            | 71.132.254.162  |           9.7144e-05  |
|  75043 | javascript-ii            | 71.136.251.103  |           9.7144e-05  |
|  75044 | javascript-ii            | 71.136.252.51   |           9.7144e-05  |
|  75045 | javascript-ii            | 71.136.254.125  |           9.7144e-05  |
|  75046 | javascript-ii            | 71.146.116.173  |           9.7144e-05  |
|  75047 | javascript-ii            | 71.158.155.118  |           9.7144e-05  |
|  75048 | javascript-ii            | 71.221.194.87   |           9.7144e-05  |
|  75049 | javascript-ii            | 71.221.198.41   |           9.7144e-05  |
|  75050 | javascript-ii            | 71.221.202.248  |           9.7144e-05  |
|  75051 | javascript-ii            | 71.221.237.129  |           9.7144e-05  |
|  75052 | javascript-ii            | 71.221.250.179  |           9.7144e-05  |
|  75053 | javascript-ii            | 71.40.24.180    |           9.7144e-05  |
|  75054 | javascript-ii            | 71.40.81.37     |           9.7144e-05  |
|  75055 | javascript-ii            | 71.77.210.52    |           9.7144e-05  |
|  75056 | javascript-ii            | 71.78.167.131   |           9.7144e-05  |
|  75057 | javascript-ii            | 72.177.147.207  |           9.7144e-05  |
|  75058 | javascript-ii            | 72.177.148.181  |           9.7144e-05  |
|  75059 | javascript-ii            | 72.177.226.58   |           9.7144e-05  |
|  75060 | javascript-ii            | 72.177.232.192  |           9.7144e-05  |
|  75061 | javascript-ii            | 72.177.241.50   |           9.7144e-05  |
|  75062 | javascript-ii            | 72.179.136.146  |           9.7144e-05  |
|  75063 | javascript-ii            | 72.179.173.100  |           9.7144e-05  |
|  75064 | javascript-ii            | 72.179.174.165  |           9.7144e-05  |
|  75065 | javascript-ii            | 72.179.175.39   |           9.7144e-05  |
|  75066 | javascript-ii            | 72.181.104.30   |           9.7144e-05  |
|  75067 | javascript-ii            | 72.181.104.49   |           9.7144e-05  |
|  75068 | javascript-ii            | 72.181.106.12   |           9.7144e-05  |
|  75069 | javascript-ii            | 72.181.113.238  |           9.7144e-05  |
|  75070 | javascript-ii            | 72.181.119.126  |           9.7144e-05  |
|  75071 | javascript-ii            | 72.181.59.214   |           9.7144e-05  |
|  75072 | javascript-ii            | 72.181.96.144   |           9.7144e-05  |
|  75073 | javascript-ii            | 72.182.64.120   |           9.7144e-05  |
|  75074 | javascript-ii            | 72.190.168.217  |           9.7144e-05  |
|  75075 | javascript-ii            | 72.190.184.30   |           9.7144e-05  |
|  75076 | javascript-ii            | 72.21.196.64    |           9.7144e-05  |
|  75077 | javascript-ii            | 72.48.255.177   |           9.7144e-05  |
|  75078 | javascript-ii            | 73.153.134.112  |           9.7144e-05  |
|  75079 | javascript-ii            | 73.153.77.234   |           9.7144e-05  |
|  75080 | javascript-ii            | 73.166.86.204   |           9.7144e-05  |
|  75081 | javascript-ii            | 73.186.43.188   |           9.7144e-05  |
|  75082 | javascript-ii            | 73.34.63.18     |           9.7144e-05  |
|  75083 | javascript-ii            | 73.77.171.11    |           9.7144e-05  |
|  75084 | javascript-ii            | 74.105.240.236  |           9.7144e-05  |
|  75085 | javascript-ii            | 75.87.2.162     |           9.7144e-05  |
|  75086 | javascript-ii            | 76.184.212.53   |           9.7144e-05  |
|  75087 | javascript-ii            | 76.184.52.198   |           9.7144e-05  |
|  75088 | javascript-ii            | 76.185.135.244  |           9.7144e-05  |
|  75089 | javascript-ii            | 76.185.135.35   |           9.7144e-05  |
|  75090 | javascript-ii            | 76.185.142.244  |           9.7144e-05  |
|  75091 | javascript-ii            | 76.185.149.233  |           9.7144e-05  |
|  75092 | javascript-ii            | 76.185.158.93   |           9.7144e-05  |
|  75093 | javascript-ii            | 76.185.209.80   |           9.7144e-05  |
|  75094 | javascript-ii            | 76.185.211.152  |           9.7144e-05  |
|  75095 | javascript-ii            | 76.185.216.146  |           9.7144e-05  |
|  75096 | javascript-ii            | 76.185.238.187  |           9.7144e-05  |
|  75097 | javascript-ii            | 76.186.189.183  |           9.7144e-05  |
|  75098 | javascript-ii            | 76.186.54.46    |           9.7144e-05  |
|  75099 | javascript-ii            | 76.187.7.114    |           9.7144e-05  |
|  75100 | javascript-ii            | 76.192.103.34   |           9.7144e-05  |
|  75101 | javascript-ii            | 76.198.200.203  |           9.7144e-05  |
|  75102 | javascript-ii            | 76.201.6.236    |           9.7144e-05  |
|  75103 | javascript-ii            | 76.237.205.238  |           9.7144e-05  |
|  75104 | javascript-ii            | 76.25.49.194    |           9.7144e-05  |
|  75105 | javascript-ii            | 76.250.145.252  |           9.7144e-05  |
|  75106 | javascript-ii            | 77.136.40.123   |           9.7144e-05  |
|  75107 | javascript-ii            | 87.158.135.133  |           9.7144e-05  |
|  75108 | javascript-ii            | 88.241.49.202   |           9.7144e-05  |
|  75109 | javascript-ii            | 89.187.164.131  |           9.7144e-05  |
|  75110 | javascript-ii            | 89.187.164.132  |           9.7144e-05  |
|  75111 | javascript-ii            | 89.187.164.133  |           9.7144e-05  |
|  75112 | javascript-ii            | 89.187.164.145  |           9.7144e-05  |
|  75113 | javascript-ii            | 89.187.164.146  |           9.7144e-05  |
|  75114 | javascript-ii            | 89.187.164.162  |           9.7144e-05  |
|  75115 | javascript-ii            | 89.187.164.168  |           9.7144e-05  |
|  75116 | javascript-ii            | 89.187.164.77   |           9.7144e-05  |
|  75117 | javascript-ii            | 96.40.48.51     |           9.7144e-05  |
|  75118 | javascript-ii            | 96.46.160.20    |           9.7144e-05  |
|  75119 | javascript-ii            | 96.8.128.88     |           9.7144e-05  |
|  75120 | javascript-ii            | 96.8.177.17     |           9.7144e-05  |
|  75121 | javascript-ii            | 96.8.177.189    |           9.7144e-05  |
|  75122 | javascript-ii            | 96.8.178.119    |           9.7144e-05  |
|  75123 | javascript-ii            | 96.8.178.32     |           9.7144e-05  |
|  75124 | javascript-ii            | 96.8.193.113    |           9.7144e-05  |
|  75125 | javascript-ii            | 96.8.248.232    |           9.7144e-05  |
|  75126 | javascript-ii            | 97.105.69.130   |           9.7144e-05  |
|  75127 | javascript-ii            | 97.123.183.37   |           9.7144e-05  |
|  75128 | javascript-ii            | 97.79.147.122   |           9.7144e-05  |
|  75129 | javascript-ii            | 97.98.85.151    |           9.7144e-05  |
|  75130 | javascript-ii            | 98.6.203.30     |           9.7144e-05  |
|  75131 | javascript-ii            | 99.100.172.39   |           9.7144e-05  |
|  75132 | javascript-ii            | 99.113.230.177  |           9.7144e-05  |
|  75133 | javascript-ii            | 99.132.130.148  |           9.7144e-05  |
|  75134 | javascript-ii            | 99.140.253.42   |           9.7144e-05  |
|  75135 | javascript-ii            | 99.158.247.126  |           9.7144e-05  |
|  75136 | javascript-ii            | 99.158.247.252  |           9.7144e-05  |
|  75137 | javascript-ii            | 99.203.154.20   |           9.7144e-05  |
|  75138 | javascript-ii            | 99.203.27.248   |           9.7144e-05  |
|  75139 | javascript-ii            | 99.27.138.170   |           9.7144e-05  |
|  75140 | javascript-ii            | 99.30.242.101   |           9.7144e-05  |
|  75141 | javascript-ii            | 99.47.142.88    |           9.7144e-05  |
|  75142 | javascript-ii            | 99.57.20.163    |           9.7144e-05  |
|  75143 | javascript-ii            | 99.6.251.67     |           9.7144e-05  |
|  75144 | javascript-ii            | 99.62.48.254    |           9.7144e-05  |
|  75145 | javascript-ii            | 99.76.234.122   |           9.7144e-05  |
|  78904 | jquery                   | 104.189.72.26   |           9.05715e-05 |
|  78905 | jquery                   | 104.219.136.100 |           9.05715e-05 |
|  78906 | jquery                   | 104.219.136.53  |           9.05715e-05 |
|  78907 | jquery                   | 104.237.80.107  |           9.05715e-05 |
|  78908 | jquery                   | 104.237.80.140  |           9.05715e-05 |
|  78909 | jquery                   | 104.237.80.15   |           9.05715e-05 |
|  78910 | jquery                   | 104.237.80.21   |           9.05715e-05 |
|  78911 | jquery                   | 104.237.80.25   |           9.05715e-05 |
|  78912 | jquery                   | 104.237.80.41   |           9.05715e-05 |
|  78913 | jquery                   | 104.237.80.58   |           9.05715e-05 |
|  78914 | jquery                   | 104.237.80.70   |           9.05715e-05 |
|  78915 | jquery                   | 104.37.31.16    |           9.05715e-05 |
|  78916 | jquery                   | 104.48.214.211  |           9.05715e-05 |
|  78917 | jquery                   | 104.5.159.147   |           9.05715e-05 |
|  78918 | jquery                   | 104.53.202.72   |           9.05715e-05 |
|  78919 | jquery                   | 104.53.206.121  |           9.05715e-05 |
|  78920 | jquery                   | 104.53.215.21   |           9.05715e-05 |
|  78921 | jquery                   | 107.135.88.8    |           9.05715e-05 |
|  78922 | jquery                   | 107.204.52.24   |           9.05715e-05 |
|  78923 | jquery                   | 107.242.125.50  |           9.05715e-05 |
|  78924 | jquery                   | 107.77.100.85   |           9.05715e-05 |
|  78925 | jquery                   | 107.77.105.28   |           9.05715e-05 |
|  78926 | jquery                   | 107.77.169.10   |           9.05715e-05 |
|  78927 | jquery                   | 107.77.169.9    |           9.05715e-05 |
|  78928 | jquery                   | 107.77.196.94   |           9.05715e-05 |
|  78929 | jquery                   | 107.77.197.189  |           9.05715e-05 |
|  78930 | jquery                   | 107.77.201.165  |           9.05715e-05 |
|  78931 | jquery                   | 107.77.217.114  |           9.05715e-05 |
|  78932 | jquery                   | 107.77.217.123  |           9.05715e-05 |
|  78933 | jquery                   | 107.77.217.58   |           9.05715e-05 |
|  78934 | jquery                   | 107.77.218.122  |           9.05715e-05 |
|  78935 | jquery                   | 107.77.218.91   |           9.05715e-05 |
|  78936 | jquery                   | 107.77.219.172  |           9.05715e-05 |
|  78937 | jquery                   | 107.77.219.175  |           9.05715e-05 |
|  78938 | jquery                   | 107.77.220.214  |           9.05715e-05 |
|  78939 | jquery                   | 107.77.221.115  |           9.05715e-05 |
|  78940 | jquery                   | 107.77.221.209  |           9.05715e-05 |
|  78941 | jquery                   | 107.77.221.227  |           9.05715e-05 |
|  78942 | jquery                   | 107.77.222.128  |           9.05715e-05 |
|  78943 | jquery                   | 107.77.222.224  |           9.05715e-05 |
|  78944 | jquery                   | 107.77.72.64    |           9.05715e-05 |
|  78945 | jquery                   | 108.174.85.238  |           9.05715e-05 |
|  78946 | jquery                   | 108.249.226.232 |           9.05715e-05 |
|  78947 | jquery                   | 108.7.195.50    |           9.05715e-05 |
|  78948 | jquery                   | 110.78.138.179  |           9.05715e-05 |
|  78949 | jquery                   | 12.247.3.42     |           9.05715e-05 |
|  78950 | jquery                   | 12.34.198.161   |           9.05715e-05 |
|  78951 | jquery                   | 12.5.63.210     |           9.05715e-05 |
|  78952 | jquery                   | 12.52.157.98    |           9.05715e-05 |
|  78953 | jquery                   | 12.71.181.245   |           9.05715e-05 |
|  78954 | jquery                   | 129.112.109.42  |           9.05715e-05 |
|  78955 | jquery                   | 129.115.195.52  |           9.05715e-05 |
|  78956 | jquery                   | 129.115.2.251   |           9.05715e-05 |
|  78957 | jquery                   | 130.45.53.218   |           9.05715e-05 |
|  78958 | jquery                   | 130.45.62.247   |           9.05715e-05 |
|  78959 | jquery                   | 136.49.100.118  |           9.05715e-05 |
|  78960 | jquery                   | 136.50.16.179   |           9.05715e-05 |
|  78961 | jquery                   | 136.50.24.253   |           9.05715e-05 |
|  78962 | jquery                   | 136.50.36.192   |           9.05715e-05 |
|  78963 | jquery                   | 136.50.5.46     |           9.05715e-05 |
|  78964 | jquery                   | 136.50.54.38    |           9.05715e-05 |
|  78965 | jquery                   | 136.50.56.65    |           9.05715e-05 |
|  78966 | jquery                   | 136.50.78.231   |           9.05715e-05 |
|  78967 | jquery                   | 136.50.82.165   |           9.05715e-05 |
|  78968 | jquery                   | 136.50.95.200   |           9.05715e-05 |
|  78969 | jquery                   | 136.62.157.140  |           9.05715e-05 |
|  78970 | jquery                   | 137.254.7.173   |           9.05715e-05 |
|  78971 | jquery                   | 148.66.37.91    |           9.05715e-05 |
|  78972 | jquery                   | 148.66.38.220   |           9.05715e-05 |
|  78973 | jquery                   | 155.94.250.100  |           9.05715e-05 |
|  78974 | jquery                   | 159.192.242.43  |           9.05715e-05 |
|  78975 | jquery                   | 162.194.247.205 |           9.05715e-05 |
|  78976 | jquery                   | 162.236.252.30  |           9.05715e-05 |
|  78977 | jquery                   | 162.245.202.3   |           9.05715e-05 |
|  78978 | jquery                   | 162.72.0.173    |           9.05715e-05 |
|  78979 | jquery                   | 165.225.48.86   |           9.05715e-05 |
|  78980 | jquery                   | 165.227.41.208  |           9.05715e-05 |
|  78981 | jquery                   | 165.231.210.20  |           9.05715e-05 |
|  78982 | jquery                   | 166.216.159.156 |           9.05715e-05 |
|  78983 | jquery                   | 166.251.109.223 |           9.05715e-05 |
|  78984 | jquery                   | 172.124.174.241 |           9.05715e-05 |
|  78985 | jquery                   | 172.127.187.185 |           9.05715e-05 |
|  78986 | jquery                   | 172.3.220.202   |           9.05715e-05 |
|  78987 | jquery                   | 172.3.29.85     |           9.05715e-05 |
|  78988 | jquery                   | 172.56.21.254   |           9.05715e-05 |
|  78989 | jquery                   | 172.56.6.247    |           9.05715e-05 |
|  78990 | jquery                   | 172.56.7.113    |           9.05715e-05 |
|  78991 | jquery                   | 172.56.7.119    |           9.05715e-05 |
|  78992 | jquery                   | 172.56.7.23     |           9.05715e-05 |
|  78993 | jquery                   | 172.58.102.138  |           9.05715e-05 |
|  78994 | jquery                   | 172.58.107.74   |           9.05715e-05 |
|  78995 | jquery                   | 172.58.110.152  |           9.05715e-05 |
|  78996 | jquery                   | 172.58.96.164   |           9.05715e-05 |
|  78997 | jquery                   | 172.58.97.32    |           9.05715e-05 |
|  78998 | jquery                   | 172.58.99.207   |           9.05715e-05 |
|  78999 | jquery                   | 172.9.1.31      |           9.05715e-05 |
|  79000 | jquery                   | 173.149.159.126 |           9.05715e-05 |
|  79001 | jquery                   | 173.149.192.150 |           9.05715e-05 |
|  79002 | jquery                   | 173.173.102.182 |           9.05715e-05 |
|  79003 | jquery                   | 173.173.121.126 |           9.05715e-05 |
|  79004 | jquery                   | 173.173.96.192  |           9.05715e-05 |
|  79005 | jquery                   | 173.174.153.61  |           9.05715e-05 |
|  79006 | jquery                   | 173.174.184.26  |           9.05715e-05 |
|  79007 | jquery                   | 173.174.190.61  |           9.05715e-05 |
|  79008 | jquery                   | 173.174.198.2   |           9.05715e-05 |
|  79009 | jquery                   | 173.174.211.231 |           9.05715e-05 |
|  79010 | jquery                   | 173.174.233.67  |           9.05715e-05 |
|  79011 | jquery                   | 173.174.244.161 |           9.05715e-05 |
|  79012 | jquery                   | 173.175.211.194 |           9.05715e-05 |
|  79013 | jquery                   | 173.227.210.1   |           9.05715e-05 |
|  79014 | jquery                   | 173.239.198.173 |           9.05715e-05 |
|  79015 | jquery                   | 173.239.232.120 |           9.05715e-05 |
|  79016 | jquery                   | 173.239.232.121 |           9.05715e-05 |
|  79017 | jquery                   | 173.239.232.126 |           9.05715e-05 |
|  79018 | jquery                   | 173.239.232.132 |           9.05715e-05 |
|  79019 | jquery                   | 173.239.232.135 |           9.05715e-05 |
|  79020 | jquery                   | 173.239.232.141 |           9.05715e-05 |
|  79021 | jquery                   | 173.239.232.164 |           9.05715e-05 |
|  79022 | jquery                   | 173.239.232.39  |           9.05715e-05 |
|  79023 | jquery                   | 173.239.232.48  |           9.05715e-05 |
|  79024 | jquery                   | 173.239.232.60  |           9.05715e-05 |
|  79025 | jquery                   | 173.239.232.62  |           9.05715e-05 |
|  79026 | jquery                   | 173.239.232.74  |           9.05715e-05 |
|  79027 | jquery                   | 173.239.232.97  |           9.05715e-05 |
|  79028 | jquery                   | 173.239.232.98  |           9.05715e-05 |
|  79029 | jquery                   | 174.128.177.247 |           9.05715e-05 |
|  79030 | jquery                   | 174.197.0.7     |           9.05715e-05 |
|  79031 | jquery                   | 174.207.2.105   |           9.05715e-05 |
|  79032 | jquery                   | 174.207.5.245   |           9.05715e-05 |
|  79033 | jquery                   | 174.246.194.48  |           9.05715e-05 |
|  79034 | jquery                   | 184.226.105.72  |           9.05715e-05 |
|  79035 | jquery                   | 184.226.14.57   |           9.05715e-05 |
|  79036 | jquery                   | 184.226.43.98   |           9.05715e-05 |
|  79037 | jquery                   | 185.247.70.196  |           9.05715e-05 |
|  79038 | jquery                   | 191.96.67.108   |           9.05715e-05 |
|  79039 | jquery                   | 191.96.67.226   |           9.05715e-05 |
|  79040 | jquery                   | 198.205.24.208  |           9.05715e-05 |
|  79041 | jquery                   | 198.55.125.205  |           9.05715e-05 |
|  79042 | jquery                   | 205.251.148.146 |           9.05715e-05 |
|  79043 | jquery                   | 205.251.148.82  |           9.05715e-05 |
|  79044 | jquery                   | 207.119.54.88   |           9.05715e-05 |
|  79045 | jquery                   | 207.119.56.213  |           9.05715e-05 |
|  79046 | jquery                   | 208.84.155.36   |           9.05715e-05 |
|  79047 | jquery                   | 209.182.242.18  |           9.05715e-05 |
|  79048 | jquery                   | 209.205.120.222 |           9.05715e-05 |
|  79049 | jquery                   | 209.205.221.162 |           9.05715e-05 |
|  79050 | jquery                   | 209.65.254.25   |           9.05715e-05 |
|  79051 | jquery                   | 209.92.162.135  |           9.05715e-05 |
|  79052 | jquery                   | 216.162.109.195 |           9.05715e-05 |
|  79053 | jquery                   | 216.177.181.227 |           9.05715e-05 |
|  79054 | jquery                   | 216.82.129.193  |           9.05715e-05 |
|  79055 | jquery                   | 24.153.137.59   |           9.05715e-05 |
|  79056 | jquery                   | 24.153.217.26   |           9.05715e-05 |
|  79057 | jquery                   | 24.153.246.211  |           9.05715e-05 |
|  79058 | jquery                   | 24.170.93.9     |           9.05715e-05 |
|  79059 | jquery                   | 24.175.229.211  |           9.05715e-05 |
|  79060 | jquery                   | 24.175.74.13    |           9.05715e-05 |
|  79061 | jquery                   | 24.242.143.6    |           9.05715e-05 |
|  79062 | jquery                   | 24.242.93.131   |           9.05715e-05 |
|  79063 | jquery                   | 24.243.10.238   |           9.05715e-05 |
|  79064 | jquery                   | 24.26.244.77    |           9.05715e-05 |
|  79065 | jquery                   | 24.26.248.56    |           9.05715e-05 |
|  79066 | jquery                   | 24.26.250.41    |           9.05715e-05 |
|  79067 | jquery                   | 24.26.254.147   |           9.05715e-05 |
|  79068 | jquery                   | 24.27.227.0     |           9.05715e-05 |
|  79069 | jquery                   | 24.28.41.37     |           9.05715e-05 |
|  79070 | jquery                   | 24.54.124.11    |           9.05715e-05 |
|  79071 | jquery                   | 24.73.246.228   |           9.05715e-05 |
|  79072 | jquery                   | 38.142.208.162  |           9.05715e-05 |
|  79073 | jquery                   | 4.34.182.238    |           9.05715e-05 |
|  79074 | jquery                   | 45.131.192.180  |           9.05715e-05 |
|  79075 | jquery                   | 45.131.192.231  |           9.05715e-05 |
|  79076 | jquery                   | 45.131.192.60   |           9.05715e-05 |
|  79077 | jquery                   | 45.131.192.92   |           9.05715e-05 |
|  79078 | jquery                   | 45.30.141.87    |           9.05715e-05 |
|  79079 | jquery                   | 45.56.151.79    |           9.05715e-05 |
|  79080 | jquery                   | 47.187.228.190  |           9.05715e-05 |
|  79081 | jquery                   | 47.187.238.163  |           9.05715e-05 |
|  79082 | jquery                   | 47.187.245.12   |           9.05715e-05 |
|  79083 | jquery                   | 47.189.62.198   |           9.05715e-05 |
|  79084 | jquery                   | 5.181.234.118   |           9.05715e-05 |
|  79085 | jquery                   | 50.226.114.86   |           9.05715e-05 |
|  79086 | jquery                   | 63.246.163.161  |           9.05715e-05 |
|  79087 | jquery                   | 64.132.225.203  |           9.05715e-05 |
|  79088 | jquery                   | 64.250.7.235    |           9.05715e-05 |
|  79089 | jquery                   | 65.131.228.148  |           9.05715e-05 |
|  79090 | jquery                   | 65.36.104.120   |           9.05715e-05 |
|  79091 | jquery                   | 65.43.113.131   |           9.05715e-05 |
|  79092 | jquery                   | 66.162.83.130   |           9.05715e-05 |
|  79093 | jquery                   | 66.25.163.2     |           9.05715e-05 |
|  79094 | jquery                   | 66.25.175.197   |           9.05715e-05 |
|  79095 | jquery                   | 66.25.69.211    |           9.05715e-05 |
|  79096 | jquery                   | 66.3.38.170     |           9.05715e-05 |
|  79097 | jquery                   | 66.69.24.31     |           9.05715e-05 |
|  79098 | jquery                   | 66.69.36.47     |           9.05715e-05 |
|  79099 | jquery                   | 66.69.57.38     |           9.05715e-05 |
|  79100 | jquery                   | 66.69.58.238    |           9.05715e-05 |
|  79101 | jquery                   | 67.10.128.187   |           9.05715e-05 |
|  79102 | jquery                   | 67.10.191.167   |           9.05715e-05 |
|  79103 | jquery                   | 67.10.213.134   |           9.05715e-05 |
|  79104 | jquery                   | 67.11.100.255   |           9.05715e-05 |
|  79105 | jquery                   | 67.11.100.82    |           9.05715e-05 |
|  79106 | jquery                   | 67.11.111.87    |           9.05715e-05 |
|  79107 | jquery                   | 67.11.145.202   |           9.05715e-05 |
|  79108 | jquery                   | 67.11.199.195   |           9.05715e-05 |
|  79109 | jquery                   | 67.11.223.207   |           9.05715e-05 |
|  79110 | jquery                   | 67.11.35.80     |           9.05715e-05 |
|  79111 | jquery                   | 67.11.41.202    |           9.05715e-05 |
|  79112 | jquery                   | 67.11.98.144    |           9.05715e-05 |
|  79113 | jquery                   | 67.48.59.123    |           9.05715e-05 |
|  79114 | jquery                   | 67.7.0.188      |           9.05715e-05 |
|  79115 | jquery                   | 67.80.48.111    |           9.05715e-05 |
|  79116 | jquery                   | 68.201.192.3    |           9.05715e-05 |
|  79117 | jquery                   | 68.203.203.127  |           9.05715e-05 |
|  79118 | jquery                   | 68.206.101.177  |           9.05715e-05 |
|  79119 | jquery                   | 68.206.246.215  |           9.05715e-05 |
|  79120 | jquery                   | 68.206.251.51   |           9.05715e-05 |
|  79121 | jquery                   | 68.249.0.44     |           9.05715e-05 |
|  79122 | jquery                   | 69.166.89.120   |           9.05715e-05 |
|  79123 | jquery                   | 69.212.157.148  |           9.05715e-05 |
|  79124 | jquery                   | 69.232.100.94   |           9.05715e-05 |
|  79125 | jquery                   | 69.239.143.192  |           9.05715e-05 |
|  79126 | jquery                   | 69.58.156.212   |           9.05715e-05 |
|  79127 | jquery                   | 70.114.29.134   |           9.05715e-05 |
|  79128 | jquery                   | 70.117.3.179    |           9.05715e-05 |
|  79129 | jquery                   | 70.118.67.138   |           9.05715e-05 |
|  79130 | jquery                   | 70.120.1.49     |           9.05715e-05 |
|  79131 | jquery                   | 70.120.213.195  |           9.05715e-05 |
|  79132 | jquery                   | 70.120.3.242    |           9.05715e-05 |
|  79133 | jquery                   | 70.120.39.192   |           9.05715e-05 |
|  79134 | jquery                   | 70.120.83.9     |           9.05715e-05 |
|  79135 | jquery                   | 70.120.93.125   |           9.05715e-05 |
|  79136 | jquery                   | 70.121.129.79   |           9.05715e-05 |
|  79137 | jquery                   | 70.121.163.5    |           9.05715e-05 |
|  79138 | jquery                   | 70.121.181.181  |           9.05715e-05 |
|  79139 | jquery                   | 70.121.205.219  |           9.05715e-05 |
|  79140 | jquery                   | 70.121.221.55   |           9.05715e-05 |
|  79141 | jquery                   | 70.123.137.196  |           9.05715e-05 |
|  79142 | jquery                   | 70.123.169.229  |           9.05715e-05 |
|  79143 | jquery                   | 70.123.200.22   |           9.05715e-05 |
|  79144 | jquery                   | 70.123.205.224  |           9.05715e-05 |
|  79145 | jquery                   | 70.123.224.242  |           9.05715e-05 |
|  79146 | jquery                   | 70.123.228.62   |           9.05715e-05 |
|  79147 | jquery                   | 70.123.239.48   |           9.05715e-05 |
|  79148 | jquery                   | 70.240.199.22   |           9.05715e-05 |
|  79149 | jquery                   | 71.132.254.162  |           9.05715e-05 |
|  79150 | jquery                   | 71.136.250.27   |           9.05715e-05 |
|  79151 | jquery                   | 71.136.252.51   |           9.05715e-05 |
|  79152 | jquery                   | 71.159.247.210  |           9.05715e-05 |
|  79153 | jquery                   | 71.221.194.87   |           9.05715e-05 |
|  79154 | jquery                   | 71.40.24.180    |           9.05715e-05 |
|  79155 | jquery                   | 71.78.167.131   |           9.05715e-05 |
|  79156 | jquery                   | 71.78.94.74     |           9.05715e-05 |
|  79157 | jquery                   | 72.12.112.229   |           9.05715e-05 |
|  79158 | jquery                   | 72.177.141.148  |           9.05715e-05 |
|  79159 | jquery                   | 72.177.206.117  |           9.05715e-05 |
|  79160 | jquery                   | 72.177.230.241  |           9.05715e-05 |
|  79161 | jquery                   | 72.179.108.103  |           9.05715e-05 |
|  79162 | jquery                   | 72.179.175.39   |           9.05715e-05 |
|  79163 | jquery                   | 72.180.171.13   |           9.05715e-05 |
|  79164 | jquery                   | 72.181.106.12   |           9.05715e-05 |
|  79165 | jquery                   | 72.181.113.238  |           9.05715e-05 |
|  79166 | jquery                   | 72.181.96.144   |           9.05715e-05 |
|  79167 | jquery                   | 72.181.97.136   |           9.05715e-05 |
|  79168 | jquery                   | 72.181.97.207   |           9.05715e-05 |
|  79169 | jquery                   | 72.183.4.143    |           9.05715e-05 |
|  79170 | jquery                   | 72.190.151.50   |           9.05715e-05 |
|  79171 | jquery                   | 72.190.161.136  |           9.05715e-05 |
|  79172 | jquery                   | 72.190.168.217  |           9.05715e-05 |
|  79173 | jquery                   | 72.190.170.159  |           9.05715e-05 |
|  79174 | jquery                   | 72.190.171.65   |           9.05715e-05 |
|  79175 | jquery                   | 72.190.175.190  |           9.05715e-05 |
|  79176 | jquery                   | 72.191.30.236   |           9.05715e-05 |
|  79177 | jquery                   | 72.191.32.140   |           9.05715e-05 |
|  79178 | jquery                   | 72.191.48.208   |           9.05715e-05 |
|  79179 | jquery                   | 72.21.196.64    |           9.05715e-05 |
|  79180 | jquery                   | 72.21.196.66    |           9.05715e-05 |
|  79181 | jquery                   | 73.166.32.184   |           9.05715e-05 |
|  79182 | jquery                   | 73.166.86.204   |           9.05715e-05 |
|  79183 | jquery                   | 74.76.254.34    |           9.05715e-05 |
|  79184 | jquery                   | 75.1.32.205     |           9.05715e-05 |
|  79185 | jquery                   | 75.1.44.127     |           9.05715e-05 |
|  79186 | jquery                   | 75.141.146.55   |           9.05715e-05 |
|  79187 | jquery                   | 75.87.28.58     |           9.05715e-05 |
|  79188 | jquery                   | 76.183.251.188  |           9.05715e-05 |
|  79189 | jquery                   | 76.184.52.198   |           9.05715e-05 |
|  79190 | jquery                   | 76.185.149.233  |           9.05715e-05 |
|  79191 | jquery                   | 76.185.155.178  |           9.05715e-05 |
|  79192 | jquery                   | 76.185.158.93   |           9.05715e-05 |
|  79193 | jquery                   | 76.185.172.182  |           9.05715e-05 |
|  79194 | jquery                   | 76.185.173.161  |           9.05715e-05 |
|  79195 | jquery                   | 76.185.182.104  |           9.05715e-05 |
|  79196 | jquery                   | 76.185.193.115  |           9.05715e-05 |
|  79197 | jquery                   | 76.185.211.152  |           9.05715e-05 |
|  79198 | jquery                   | 76.185.234.111  |           9.05715e-05 |
|  79199 | jquery                   | 76.186.2.234    |           9.05715e-05 |
|  79200 | jquery                   | 76.187.101.165  |           9.05715e-05 |
|  79201 | jquery                   | 76.198.200.203  |           9.05715e-05 |
|  79202 | jquery                   | 76.198.205.9    |           9.05715e-05 |
|  79203 | jquery                   | 76.201.6.236    |           9.05715e-05 |
|  79204 | jquery                   | 76.214.156.42   |           9.05715e-05 |
|  79205 | jquery                   | 79.220.63.16    |           9.05715e-05 |
|  79206 | jquery                   | 84.14.10.91     |           9.05715e-05 |
|  79207 | jquery                   | 87.158.135.133  |           9.05715e-05 |
|  79208 | jquery                   | 88.241.49.202   |           9.05715e-05 |
|  79209 | jquery                   | 89.159.14.216   |           9.05715e-05 |
|  79210 | jquery                   | 89.187.164.134  |           9.05715e-05 |
|  79211 | jquery                   | 89.187.164.147  |           9.05715e-05 |
|  79212 | jquery                   | 89.187.164.150  |           9.05715e-05 |
|  79213 | jquery                   | 89.187.164.161  |           9.05715e-05 |
|  79214 | jquery                   | 89.187.164.164  |           9.05715e-05 |
|  79215 | jquery                   | 89.187.164.165  |           9.05715e-05 |
|  79216 | jquery                   | 89.187.164.167  |           9.05715e-05 |
|  79217 | jquery                   | 89.187.164.168  |           9.05715e-05 |
|  79218 | jquery                   | 92.38.148.46    |           9.05715e-05 |
|  79219 | jquery                   | 92.38.148.51    |           9.05715e-05 |
|  79220 | jquery                   | 96.35.248.138   |           9.05715e-05 |
|  79221 | jquery                   | 96.40.48.51     |           9.05715e-05 |
|  79222 | jquery                   | 96.46.160.20    |           9.05715e-05 |
|  79223 | jquery                   | 96.66.54.118    |           9.05715e-05 |
|  79224 | jquery                   | 96.8.130.0      |           9.05715e-05 |
|  79225 | jquery                   | 96.8.178.32     |           9.05715e-05 |
|  79226 | jquery                   | 97.105.101.131  |           9.05715e-05 |
|  79227 | jquery                   | 97.105.90.179   |           9.05715e-05 |
|  79228 | jquery                   | 97.98.105.129   |           9.05715e-05 |
|  79229 | jquery                   | 98.197.56.20    |           9.05715e-05 |
|  79230 | jquery                   | 98.39.12.204    |           9.05715e-05 |
|  79231 | jquery                   | 98.6.95.27      |           9.05715e-05 |
|  79232 | jquery                   | 99.132.130.148  |           9.05715e-05 |
|  79233 | jquery                   | 99.137.88.24    |           9.05715e-05 |
|  79234 | jquery                   | 99.184.61.243   |           9.05715e-05 |
|  79235 | jquery                   | 99.203.154.58   |           9.05715e-05 |
|  79236 | jquery                   | 99.203.23.16    |           9.05715e-05 |
|  79237 | jquery                   | 99.203.26.158   |           9.05715e-05 |
|  79238 | jquery                   | 99.43.108.160   |           9.05715e-05 |
|  79239 | jquery                   | 99.47.142.88    |           9.05715e-05 |
|  79240 | jquery                   | 99.52.75.127    |           9.05715e-05 |
|  79241 | jquery                   | 99.52.78.67     |           9.05715e-05 |
|  79242 | jquery                   | 99.76.235.191   |           9.05715e-05 |
|  85914 | mysql                    | 104.131.47.203  |           9.42418e-05 |
|  85915 | mysql                    | 104.180.86.144  |           9.42418e-05 |
|  85916 | mysql                    | 104.190.250.60  |           9.42418e-05 |
|  85917 | mysql                    | 104.193.225.250 |           9.42418e-05 |
|  85918 | mysql                    | 104.219.136.100 |           9.42418e-05 |
|  85919 | mysql                    | 104.53.202.72   |           9.42418e-05 |
|  85920 | mysql                    | 104.53.215.21   |           9.42418e-05 |
|  85921 | mysql                    | 107.127.0.131   |           9.42418e-05 |
|  85922 | mysql                    | 107.136.68.158  |           9.42418e-05 |
|  85923 | mysql                    | 107.152.98.82   |           9.42418e-05 |
|  85924 | mysql                    | 107.77.100.15   |           9.42418e-05 |
|  85925 | mysql                    | 107.77.100.79   |           9.42418e-05 |
|  85926 | mysql                    | 107.77.105.112  |           9.42418e-05 |
|  85927 | mysql                    | 107.77.169.7    |           9.42418e-05 |
|  85928 | mysql                    | 107.77.198.116  |           9.42418e-05 |
|  85929 | mysql                    | 107.77.200.32   |           9.42418e-05 |
|  85930 | mysql                    | 107.77.200.83   |           9.42418e-05 |
|  85931 | mysql                    | 107.77.217.106  |           9.42418e-05 |
|  85932 | mysql                    | 107.77.217.218  |           9.42418e-05 |
|  85933 | mysql                    | 107.77.218.122  |           9.42418e-05 |
|  85934 | mysql                    | 107.77.218.40   |           9.42418e-05 |
|  85935 | mysql                    | 107.77.219.93   |           9.42418e-05 |
|  85936 | mysql                    | 107.77.220.194  |           9.42418e-05 |
|  85937 | mysql                    | 107.77.221.10   |           9.42418e-05 |
|  85938 | mysql                    | 107.77.221.110  |           9.42418e-05 |
|  85939 | mysql                    | 107.77.221.199  |           9.42418e-05 |
|  85940 | mysql                    | 107.77.221.209  |           9.42418e-05 |
|  85941 | mysql                    | 107.77.222.216  |           9.42418e-05 |
|  85942 | mysql                    | 107.77.64.52    |           9.42418e-05 |
|  85943 | mysql                    | 107.77.66.78    |           9.42418e-05 |
|  85944 | mysql                    | 107.77.80.128   |           9.42418e-05 |
|  85945 | mysql                    | 108.192.181.20  |           9.42418e-05 |
|  85946 | mysql                    | 108.199.117.96  |           9.42418e-05 |
|  85947 | mysql                    | 108.200.112.159 |           9.42418e-05 |
|  85948 | mysql                    | 108.220.101.224 |           9.42418e-05 |
|  85949 | mysql                    | 108.222.98.137  |           9.42418e-05 |
|  85950 | mysql                    | 108.93.10.214   |           9.42418e-05 |
|  85951 | mysql                    | 12.28.69.2      |           9.42418e-05 |
|  85952 | mysql                    | 12.31.71.58     |           9.42418e-05 |
|  85953 | mysql                    | 12.5.63.226     |           9.42418e-05 |
|  85954 | mysql                    | 12.71.181.245   |           9.42418e-05 |
|  85955 | mysql                    | 129.112.109.42  |           9.42418e-05 |
|  85956 | mysql                    | 129.115.195.34  |           9.42418e-05 |
|  85957 | mysql                    | 129.115.2.251   |           9.42418e-05 |
|  85958 | mysql                    | 130.45.43.174   |           9.42418e-05 |
|  85959 | mysql                    | 130.45.50.251   |           9.42418e-05 |
|  85960 | mysql                    | 130.45.53.218   |           9.42418e-05 |
|  85961 | mysql                    | 130.45.68.158   |           9.42418e-05 |
|  85962 | mysql                    | 131.194.172.132 |           9.42418e-05 |
|  85963 | mysql                    | 131.194.172.188 |           9.42418e-05 |
|  85964 | mysql                    | 136.50.0.240    |           9.42418e-05 |
|  85965 | mysql                    | 136.50.16.179   |           9.42418e-05 |
|  85966 | mysql                    | 136.50.20.17    |           9.42418e-05 |
|  85967 | mysql                    | 136.50.24.253   |           9.42418e-05 |
|  85968 | mysql                    | 136.50.54.38    |           9.42418e-05 |
|  85969 | mysql                    | 136.50.59.188   |           9.42418e-05 |
|  85970 | mysql                    | 136.50.63.72    |           9.42418e-05 |
|  85971 | mysql                    | 136.50.75.159   |           9.42418e-05 |
|  85972 | mysql                    | 136.50.77.182   |           9.42418e-05 |
|  85973 | mysql                    | 136.50.92.234   |           9.42418e-05 |
|  85974 | mysql                    | 136.50.95.200   |           9.42418e-05 |
|  85975 | mysql                    | 136.50.97.29    |           9.42418e-05 |
|  85976 | mysql                    | 136.50.98.51    |           9.42418e-05 |
|  85977 | mysql                    | 138.229.228.38  |           9.42418e-05 |
|  85978 | mysql                    | 148.66.37.91    |           9.42418e-05 |
|  85979 | mysql                    | 159.118.138.70  |           9.42418e-05 |
|  85980 | mysql                    | 162.200.119.143 |           9.42418e-05 |
|  85981 | mysql                    | 162.233.170.151 |           9.42418e-05 |
|  85982 | mysql                    | 162.245.202.3   |           9.42418e-05 |
|  85983 | mysql                    | 162.250.18.154  |           9.42418e-05 |
|  85984 | mysql                    | 166.247.169.116 |           9.42418e-05 |
|  85985 | mysql                    | 166.251.101.2   |           9.42418e-05 |
|  85986 | mysql                    | 166.251.106.194 |           9.42418e-05 |
|  85987 | mysql                    | 166.251.109.31  |           9.42418e-05 |
|  85988 | mysql                    | 172.110.80.227  |           9.42418e-05 |
|  85989 | mysql                    | 172.127.182.0   |           9.42418e-05 |
|  85990 | mysql                    | 172.5.103.185   |           9.42418e-05 |
|  85991 | mysql                    | 172.56.6.92     |           9.42418e-05 |
|  85992 | mysql                    | 172.56.7.244    |           9.42418e-05 |
|  85993 | mysql                    | 172.58.101.45   |           9.42418e-05 |
|  85994 | mysql                    | 172.58.102.228  |           9.42418e-05 |
|  85995 | mysql                    | 172.58.107.0    |           9.42418e-05 |
|  85996 | mysql                    | 172.58.107.102  |           9.42418e-05 |
|  85997 | mysql                    | 172.58.108.228  |           9.42418e-05 |
|  85998 | mysql                    | 172.58.109.54   |           9.42418e-05 |
|  85999 | mysql                    | 172.58.110.189  |           9.42418e-05 |
|  86000 | mysql                    | 172.58.111.5    |           9.42418e-05 |
|  86001 | mysql                    | 172.58.97.32    |           9.42418e-05 |
|  86002 | mysql                    | 172.58.99.240   |           9.42418e-05 |
|  86003 | mysql                    | 172.8.173.98    |           9.42418e-05 |
|  86004 | mysql                    | 172.87.148.22   |           9.42418e-05 |
|  86005 | mysql                    | 173.123.4.140   |           9.42418e-05 |
|  86006 | mysql                    | 173.126.107.20  |           9.42418e-05 |
|  86007 | mysql                    | 173.172.166.90  |           9.42418e-05 |
|  86008 | mysql                    | 173.173.101.109 |           9.42418e-05 |
|  86009 | mysql                    | 173.173.104.202 |           9.42418e-05 |
|  86010 | mysql                    | 173.173.121.126 |           9.42418e-05 |
|  86011 | mysql                    | 173.173.121.51  |           9.42418e-05 |
|  86012 | mysql                    | 173.173.127.101 |           9.42418e-05 |
|  86013 | mysql                    | 173.173.98.137  |           9.42418e-05 |
|  86014 | mysql                    | 173.174.155.172 |           9.42418e-05 |
|  86015 | mysql                    | 173.174.159.45  |           9.42418e-05 |
|  86016 | mysql                    | 173.174.196.212 |           9.42418e-05 |
|  86017 | mysql                    | 173.174.218.183 |           9.42418e-05 |
|  86018 | mysql                    | 173.175.19.30   |           9.42418e-05 |
|  86019 | mysql                    | 173.239.232.112 |           9.42418e-05 |
|  86020 | mysql                    | 173.239.232.123 |           9.42418e-05 |
|  86021 | mysql                    | 173.239.232.137 |           9.42418e-05 |
|  86022 | mysql                    | 173.239.232.155 |           9.42418e-05 |
|  86023 | mysql                    | 173.239.232.171 |           9.42418e-05 |
|  86024 | mysql                    | 173.239.232.20  |           9.42418e-05 |
|  86025 | mysql                    | 173.239.232.22  |           9.42418e-05 |
|  86026 | mysql                    | 173.239.232.42  |           9.42418e-05 |
|  86027 | mysql                    | 173.239.232.50  |           9.42418e-05 |
|  86028 | mysql                    | 173.239.232.61  |           9.42418e-05 |
|  86029 | mysql                    | 173.239.232.80  |           9.42418e-05 |
|  86030 | mysql                    | 173.239.232.99  |           9.42418e-05 |
|  86031 | mysql                    | 173.244.217.91  |           9.42418e-05 |
|  86032 | mysql                    | 174.128.180.149 |           9.42418e-05 |
|  86033 | mysql                    | 174.128.180.20  |           9.42418e-05 |
|  86034 | mysql                    | 174.128.180.21  |           9.42418e-05 |
|  86035 | mysql                    | 174.197.8.92    |           9.42418e-05 |
|  86036 | mysql                    | 174.207.1.185   |           9.42418e-05 |
|  86037 | mysql                    | 174.207.14.229  |           9.42418e-05 |
|  86038 | mysql                    | 174.207.2.137   |           9.42418e-05 |
|  86039 | mysql                    | 174.207.2.208   |           9.42418e-05 |
|  86040 | mysql                    | 174.246.193.155 |           9.42418e-05 |
|  86041 | mysql                    | 174.246.199.81  |           9.42418e-05 |
|  86042 | mysql                    | 174.51.14.27    |           9.42418e-05 |
|  86043 | mysql                    | 174.67.9.84     |           9.42418e-05 |
|  86044 | mysql                    | 174.69.206.136  |           9.42418e-05 |
|  86045 | mysql                    | 185.216.33.82   |           9.42418e-05 |
|  86046 | mysql                    | 185.247.70.196  |           9.42418e-05 |
|  86047 | mysql                    | 189.146.56.56   |           9.42418e-05 |
|  86048 | mysql                    | 192.107.1.225   |           9.42418e-05 |
|  86049 | mysql                    | 193.56.117.42   |           9.42418e-05 |
|  86050 | mysql                    | 195.206.104.106 |           9.42418e-05 |
|  86051 | mysql                    | 196.52.39.29    |           9.42418e-05 |
|  86052 | mysql                    | 196.52.39.32    |           9.42418e-05 |
|  86053 | mysql                    | 198.202.210.220 |           9.42418e-05 |
|  86054 | mysql                    | 201.103.134.102 |           9.42418e-05 |
|  86055 | mysql                    | 204.44.112.76   |           9.42418e-05 |
|  86056 | mysql                    | 205.251.148.130 |           9.42418e-05 |
|  86057 | mysql                    | 205.251.148.146 |           9.42418e-05 |
|  86058 | mysql                    | 205.251.148.82  |           9.42418e-05 |
|  86059 | mysql                    | 207.119.54.88   |           9.42418e-05 |
|  86060 | mysql                    | 207.119.56.213  |           9.42418e-05 |
|  86061 | mysql                    | 207.119.59.216  |           9.42418e-05 |
|  86062 | mysql                    | 207.189.160.150 |           9.42418e-05 |
|  86063 | mysql                    | 208.30.107.160  |           9.42418e-05 |
|  86064 | mysql                    | 208.84.155.237  |           9.42418e-05 |
|  86065 | mysql                    | 209.12.105.54   |           9.42418e-05 |
|  86066 | mysql                    | 209.133.66.84   |           9.42418e-05 |
|  86067 | mysql                    | 209.182.242.18  |           9.42418e-05 |
|  86068 | mysql                    | 209.205.221.178 |           9.42418e-05 |
|  86069 | mysql                    | 212.102.45.83   |           9.42418e-05 |
|  86070 | mysql                    | 216.54.250.134  |           9.42418e-05 |
|  86071 | mysql                    | 23.116.24.226   |           9.42418e-05 |
|  86072 | mysql                    | 23.227.134.242  |           9.42418e-05 |
|  86073 | mysql                    | 24.155.140.118  |           9.42418e-05 |
|  86074 | mysql                    | 24.160.137.86   |           9.42418e-05 |
|  86075 | mysql                    | 24.160.141.134  |           9.42418e-05 |
|  86076 | mysql                    | 24.173.152.222  |           9.42418e-05 |
|  86077 | mysql                    | 24.242.143.6    |           9.42418e-05 |
|  86078 | mysql                    | 24.242.150.231  |           9.42418e-05 |
|  86079 | mysql                    | 24.242.180.178  |           9.42418e-05 |
|  86080 | mysql                    | 24.243.10.238   |           9.42418e-05 |
|  86081 | mysql                    | 24.243.59.91    |           9.42418e-05 |
|  86082 | mysql                    | 24.243.7.150    |           9.42418e-05 |
|  86083 | mysql                    | 24.243.74.160   |           9.42418e-05 |
|  86084 | mysql                    | 24.26.241.169   |           9.42418e-05 |
|  86085 | mysql                    | 24.26.250.41    |           9.42418e-05 |
|  86086 | mysql                    | 24.27.227.1     |           9.42418e-05 |
|  86087 | mysql                    | 24.28.148.238   |           9.42418e-05 |
|  86088 | mysql                    | 24.28.66.30     |           9.42418e-05 |
|  86089 | mysql                    | 24.28.96.138    |           9.42418e-05 |
|  86090 | mysql                    | 24.9.177.142    |           9.42418e-05 |
|  86091 | mysql                    | 24.99.149.248   |           9.42418e-05 |
|  86092 | mysql                    | 34.203.225.141  |           9.42418e-05 |
|  86093 | mysql                    | 34.226.8.106    |           9.42418e-05 |
|  86094 | mysql                    | 38.142.208.162  |           9.42418e-05 |
|  86095 | mysql                    | 45.131.192.127  |           9.42418e-05 |
|  86096 | mysql                    | 45.131.192.58   |           9.42418e-05 |
|  86097 | mysql                    | 45.21.2.131     |           9.42418e-05 |
|  86098 | mysql                    | 45.21.39.47     |           9.42418e-05 |
|  86099 | mysql                    | 45.56.151.68    |           9.42418e-05 |
|  86100 | mysql                    | 45.56.151.8     |           9.42418e-05 |
|  86101 | mysql                    | 45.89.173.198   |           9.42418e-05 |
|  86102 | mysql                    | 47.183.143.149  |           9.42418e-05 |
|  86103 | mysql                    | 47.187.238.163  |           9.42418e-05 |
|  86104 | mysql                    | 47.187.245.12   |           9.42418e-05 |
|  86105 | mysql                    | 47.214.184.25   |           9.42418e-05 |
|  86106 | mysql                    | 47.221.223.242  |           9.42418e-05 |
|  86107 | mysql                    | 47.234.152.236  |           9.42418e-05 |
|  86108 | mysql                    | 50.202.129.130  |           9.42418e-05 |
|  86109 | mysql                    | 56.0.84.26      |           9.42418e-05 |
|  86110 | mysql                    | 64.125.109.186  |           9.42418e-05 |
|  86111 | mysql                    | 64.250.7.235    |           9.42418e-05 |
|  86112 | mysql                    | 64.44.80.196    |           9.42418e-05 |
|  86113 | mysql                    | 65.131.228.148  |           9.42418e-05 |
|  86114 | mysql                    | 65.36.104.120   |           9.42418e-05 |
|  86115 | mysql                    | 66.25.13.167    |           9.42418e-05 |
|  86116 | mysql                    | 66.25.163.226   |           9.42418e-05 |
|  86117 | mysql                    | 66.25.171.170   |           9.42418e-05 |
|  86118 | mysql                    | 66.25.171.4     |           9.42418e-05 |
|  86119 | mysql                    | 66.25.80.16     |           9.42418e-05 |
|  86120 | mysql                    | 66.3.38.202     |           9.42418e-05 |
|  86121 | mysql                    | 66.69.110.179   |           9.42418e-05 |
|  86122 | mysql                    | 66.69.60.160    |           9.42418e-05 |
|  86123 | mysql                    | 66.69.66.113    |           9.42418e-05 |
|  86124 | mysql                    | 66.69.67.230    |           9.42418e-05 |
|  86125 | mysql                    | 66.90.180.216   |           9.42418e-05 |
|  86126 | mysql                    | 67.10.132.216   |           9.42418e-05 |
|  86127 | mysql                    | 67.10.154.68    |           9.42418e-05 |
|  86128 | mysql                    | 67.10.161.67    |           9.42418e-05 |
|  86129 | mysql                    | 67.10.164.55    |           9.42418e-05 |
|  86130 | mysql                    | 67.11.100.255   |           9.42418e-05 |
|  86131 | mysql                    | 67.11.225.209   |           9.42418e-05 |
|  86132 | mysql                    | 67.11.236.189   |           9.42418e-05 |
|  86133 | mysql                    | 67.11.245.224   |           9.42418e-05 |
|  86134 | mysql                    | 67.11.35.80     |           9.42418e-05 |
|  86135 | mysql                    | 67.11.59.112    |           9.42418e-05 |
|  86136 | mysql                    | 68.206.101.177  |           9.42418e-05 |
|  86137 | mysql                    | 68.206.254.198  |           9.42418e-05 |
|  86138 | mysql                    | 68.252.68.142   |           9.42418e-05 |
|  86139 | mysql                    | 69.154.52.98    |           9.42418e-05 |
|  86140 | mysql                    | 69.166.71.6     |           9.42418e-05 |
|  86141 | mysql                    | 69.208.245.166  |           9.42418e-05 |
|  86142 | mysql                    | 69.232.103.179  |           9.42418e-05 |
|  86143 | mysql                    | 69.236.100.33   |           9.42418e-05 |
|  86144 | mysql                    | 70.112.179.142  |           9.42418e-05 |
|  86145 | mysql                    | 70.114.38.157   |           9.42418e-05 |
|  86146 | mysql                    | 70.114.7.210    |           9.42418e-05 |
|  86147 | mysql                    | 70.115.114.110  |           9.42418e-05 |
|  86148 | mysql                    | 70.117.13.151   |           9.42418e-05 |
|  86149 | mysql                    | 70.117.29.41    |           9.42418e-05 |
|  86150 | mysql                    | 70.117.5.148    |           9.42418e-05 |
|  86151 | mysql                    | 70.117.8.118    |           9.42418e-05 |
|  86152 | mysql                    | 70.118.48.163   |           9.42418e-05 |
|  86153 | mysql                    | 70.119.130.42   |           9.42418e-05 |
|  86154 | mysql                    | 70.120.57.140   |           9.42418e-05 |
|  86155 | mysql                    | 70.120.59.12    |           9.42418e-05 |
|  86156 | mysql                    | 70.120.93.125   |           9.42418e-05 |
|  86157 | mysql                    | 70.121.185.146  |           9.42418e-05 |
|  86158 | mysql                    | 70.121.187.122  |           9.42418e-05 |
|  86159 | mysql                    | 70.121.188.16   |           9.42418e-05 |
|  86160 | mysql                    | 70.121.200.203  |           9.42418e-05 |
|  86161 | mysql                    | 70.121.214.153  |           9.42418e-05 |
|  86162 | mysql                    | 70.122.206.28   |           9.42418e-05 |
|  86163 | mysql                    | 70.122.250.222  |           9.42418e-05 |
|  86164 | mysql                    | 70.123.110.205  |           9.42418e-05 |
|  86165 | mysql                    | 70.123.137.196  |           9.42418e-05 |
|  86166 | mysql                    | 70.123.201.199  |           9.42418e-05 |
|  86167 | mysql                    | 70.123.215.49   |           9.42418e-05 |
|  86168 | mysql                    | 70.123.215.59   |           9.42418e-05 |
|  86169 | mysql                    | 70.123.224.242  |           9.42418e-05 |
|  86170 | mysql                    | 70.123.239.48   |           9.42418e-05 |
|  86171 | mysql                    | 70.123.247.196  |           9.42418e-05 |
|  86172 | mysql                    | 70.125.148.96   |           9.42418e-05 |
|  86173 | mysql                    | 70.130.122.154  |           9.42418e-05 |
|  86174 | mysql                    | 70.94.132.9     |           9.42418e-05 |
|  86175 | mysql                    | 71.136.255.165  |           9.42418e-05 |
|  86176 | mysql                    | 71.183.39.187   |           9.42418e-05 |
|  86177 | mysql                    | 71.78.167.131   |           9.42418e-05 |
|  86178 | mysql                    | 72.12.112.229   |           9.42418e-05 |
|  86179 | mysql                    | 72.177.141.148  |           9.42418e-05 |
|  86180 | mysql                    | 72.177.165.210  |           9.42418e-05 |
|  86181 | mysql                    | 72.177.199.59   |           9.42418e-05 |
|  86182 | mysql                    | 72.177.209.244  |           9.42418e-05 |
|  86183 | mysql                    | 72.177.210.199  |           9.42418e-05 |
|  86184 | mysql                    | 72.177.229.125  |           9.42418e-05 |
|  86185 | mysql                    | 72.177.245.198  |           9.42418e-05 |
|  86186 | mysql                    | 72.180.51.14    |           9.42418e-05 |
|  86187 | mysql                    | 72.181.104.30   |           9.42418e-05 |
|  86188 | mysql                    | 72.181.106.12   |           9.42418e-05 |
|  86189 | mysql                    | 72.181.119.126  |           9.42418e-05 |
|  86190 | mysql                    | 72.181.125.198  |           9.42418e-05 |
|  86191 | mysql                    | 72.181.99.110   |           9.42418e-05 |
|  86192 | mysql                    | 72.190.128.167  |           9.42418e-05 |
|  86193 | mysql                    | 72.190.162.161  |           9.42418e-05 |
|  86194 | mysql                    | 72.190.165.220  |           9.42418e-05 |
|  86195 | mysql                    | 72.190.170.159  |           9.42418e-05 |
|  86196 | mysql                    | 72.190.172.10   |           9.42418e-05 |
|  86197 | mysql                    | 72.191.11.178   |           9.42418e-05 |
|  86198 | mysql                    | 72.191.31.35    |           9.42418e-05 |
|  86199 | mysql                    | 72.191.59.31    |           9.42418e-05 |
|  86200 | mysql                    | 72.201.107.178  |           9.42418e-05 |
|  86201 | mysql                    | 72.21.196.64    |           9.42418e-05 |
|  86202 | mysql                    | 73.39.175.210   |           9.42418e-05 |
|  86203 | mysql                    | 74.81.88.162    |           9.42418e-05 |
|  86204 | mysql                    | 75.35.140.23    |           9.42418e-05 |
|  86205 | mysql                    | 75.41.2.91      |           9.42418e-05 |
|  86206 | mysql                    | 75.54.33.182    |           9.42418e-05 |
|  86207 | mysql                    | 75.87.2.162     |           9.42418e-05 |
|  86208 | mysql                    | 76.115.148.248  |           9.42418e-05 |
|  86209 | mysql                    | 76.184.212.53   |           9.42418e-05 |
|  86210 | mysql                    | 76.185.135.244  |           9.42418e-05 |
|  86211 | mysql                    | 76.185.135.35   |           9.42418e-05 |
|  86212 | mysql                    | 76.185.145.231  |           9.42418e-05 |
|  86213 | mysql                    | 76.185.173.254  |           9.42418e-05 |
|  86214 | mysql                    | 76.185.234.25   |           9.42418e-05 |
|  86215 | mysql                    | 76.186.2.234    |           9.42418e-05 |
|  86216 | mysql                    | 76.186.80.227   |           9.42418e-05 |
|  86217 | mysql                    | 76.214.156.42   |           9.42418e-05 |
|  86218 | mysql                    | 76.25.25.4      |           9.42418e-05 |
|  86219 | mysql                    | 89.187.164.129  |           9.42418e-05 |
|  86220 | mysql                    | 89.187.164.132  |           9.42418e-05 |
|  86221 | mysql                    | 89.187.164.147  |           9.42418e-05 |
|  86222 | mysql                    | 89.187.164.162  |           9.42418e-05 |
|  86223 | mysql                    | 89.187.164.166  |           9.42418e-05 |
|  86224 | mysql                    | 89.187.164.75   |           9.42418e-05 |
|  86225 | mysql                    | 89.187.164.77   |           9.42418e-05 |
|  86226 | mysql                    | 92.38.148.10    |           9.42418e-05 |
|  86227 | mysql                    | 92.38.148.56    |           9.42418e-05 |
|  86228 | mysql                    | 92.38.148.65    |           9.42418e-05 |
|  86229 | mysql                    | 95.181.238.19   |           9.42418e-05 |
|  86230 | mysql                    | 96.44.144.194   |           9.42418e-05 |
|  86231 | mysql                    | 96.46.160.20    |           9.42418e-05 |
|  86232 | mysql                    | 96.8.128.193    |           9.42418e-05 |
|  86233 | mysql                    | 96.8.130.0      |           9.42418e-05 |
|  86234 | mysql                    | 96.8.130.240    |           9.42418e-05 |
|  86235 | mysql                    | 96.8.130.92     |           9.42418e-05 |
|  86236 | mysql                    | 96.8.131.77     |           9.42418e-05 |
|  86237 | mysql                    | 96.8.177.17     |           9.42418e-05 |
|  86238 | mysql                    | 96.8.177.90     |           9.42418e-05 |
|  86239 | mysql                    | 96.8.178.32     |           9.42418e-05 |
|  86240 | mysql                    | 96.8.193.113    |           9.42418e-05 |
|  86241 | mysql                    | 97.77.214.243   |           9.42418e-05 |
|  86242 | mysql                    | 97.79.141.50    |           9.42418e-05 |
|  86243 | mysql                    | 97.98.16.4      |           9.42418e-05 |
|  86244 | mysql                    | 98.39.12.204    |           9.42418e-05 |
|  86245 | mysql                    | 98.6.141.91     |           9.42418e-05 |
|  86246 | mysql                    | 98.6.222.42     |           9.42418e-05 |
|  86247 | mysql                    | 98.6.75.74      |           9.42418e-05 |
|  86248 | mysql                    | 98.6.90.2       |           9.42418e-05 |
|  86249 | mysql                    | 99.147.236.67   |           9.42418e-05 |
|  86250 | mysql                    | 99.147.239.234  |           9.42418e-05 |
|  86251 | mysql                    | 99.203.154.20   |           9.42418e-05 |
|  86252 | mysql                    | 99.203.155.161  |           9.42418e-05 |
|  86253 | mysql                    | 99.203.27.106   |           9.42418e-05 |
|  86254 | mysql                    | 99.203.27.13    |           9.42418e-05 |
|  86255 | mysql                    | 99.203.27.66    |           9.42418e-05 |
|  86256 | mysql                    | 99.203.27.70    |           9.42418e-05 |
|  86257 | mysql                    | 99.27.138.170   |           9.42418e-05 |
|  86258 | mysql                    | 99.27.139.172   |           9.42418e-05 |
|  86259 | mysql                    | 99.30.48.55     |           9.42418e-05 |
|  86260 | mysql                    | 99.43.108.160   |           9.42418e-05 |
|  86261 | mysql                    | 99.48.230.93    |           9.42418e-05 |
|  86262 | mysql                    | 99.5.223.96     |           9.42418e-05 |
|  86263 | mysql                    | 99.65.201.242   |           9.42418e-05 |
|  86264 | mysql                    | 99.76.131.43    |           9.42418e-05 |
|  86265 | mysql                    | 99.97.210.12    |           9.42418e-05 |
|  86266 | mysql                    | 99.98.36.168    |           9.42418e-05 |
|  98081 | search/search_index.json | 100.19.73.35    |           5.70321e-05 |
|  98082 | search/search_index.json | 104.191.11.147  |           5.70321e-05 |
|  98083 | search/search_index.json | 104.191.75.0    |           5.70321e-05 |
|  98084 | search/search_index.json | 104.237.80.57   |           5.70321e-05 |
|  98085 | search/search_index.json | 104.238.45.21   |           5.70321e-05 |
|  98086 | search/search_index.json | 104.238.45.6    |           5.70321e-05 |
|  98087 | search/search_index.json | 104.5.159.147   |           5.70321e-05 |
|  98088 | search/search_index.json | 104.5.221.205   |           5.70321e-05 |
|  98089 | search/search_index.json | 104.56.10.184   |           5.70321e-05 |
|  98090 | search/search_index.json | 107.117.224.90  |           5.70321e-05 |
|  98091 | search/search_index.json | 107.127.0.89    |           5.70321e-05 |
|  98092 | search/search_index.json | 107.128.3.66    |           5.70321e-05 |
|  98093 | search/search_index.json | 107.136.101.76  |           5.70321e-05 |
|  98094 | search/search_index.json | 107.152.102.198 |           5.70321e-05 |
|  98095 | search/search_index.json | 107.218.180.201 |           5.70321e-05 |
|  98096 | search/search_index.json | 107.77.169.12   |           5.70321e-05 |
|  98097 | search/search_index.json | 107.77.189.32   |           5.70321e-05 |
|  98098 | search/search_index.json | 107.77.189.63   |           5.70321e-05 |
|  98099 | search/search_index.json | 107.77.189.9    |           5.70321e-05 |
|  98100 | search/search_index.json | 107.77.196.93   |           5.70321e-05 |
|  98101 | search/search_index.json | 107.77.199.23   |           5.70321e-05 |
|  98102 | search/search_index.json | 107.77.201.138  |           5.70321e-05 |
|  98103 | search/search_index.json | 107.77.201.165  |           5.70321e-05 |
|  98104 | search/search_index.json | 107.77.217.99   |           5.70321e-05 |
|  98105 | search/search_index.json | 107.77.218.225  |           5.70321e-05 |
|  98106 | search/search_index.json | 107.77.218.37   |           5.70321e-05 |
|  98107 | search/search_index.json | 107.77.218.77   |           5.70321e-05 |
|  98108 | search/search_index.json | 107.77.218.8    |           5.70321e-05 |
|  98109 | search/search_index.json | 107.77.219.190  |           5.70321e-05 |
|  98110 | search/search_index.json | 107.77.220.190  |           5.70321e-05 |
|  98111 | search/search_index.json | 107.77.220.51   |           5.70321e-05 |
|  98112 | search/search_index.json | 107.77.220.73   |           5.70321e-05 |
|  98113 | search/search_index.json | 107.77.221.101  |           5.70321e-05 |
|  98114 | search/search_index.json | 107.77.221.108  |           5.70321e-05 |
|  98115 | search/search_index.json | 107.77.221.116  |           5.70321e-05 |
|  98116 | search/search_index.json | 107.77.221.96   |           5.70321e-05 |
|  98117 | search/search_index.json | 107.77.222.132  |           5.70321e-05 |
|  98118 | search/search_index.json | 107.77.222.174  |           5.70321e-05 |
|  98119 | search/search_index.json | 107.77.222.197  |           5.70321e-05 |
|  98120 | search/search_index.json | 108.105.62.187  |           5.70321e-05 |
|  98121 | search/search_index.json | 108.174.80.231  |           5.70321e-05 |
|  98122 | search/search_index.json | 108.18.249.150  |           5.70321e-05 |
|  98123 | search/search_index.json | 108.199.116.156 |           5.70321e-05 |
|  98124 | search/search_index.json | 108.199.117.106 |           5.70321e-05 |
|  98125 | search/search_index.json | 108.93.9.152    |           5.70321e-05 |
|  98126 | search/search_index.json | 12.154.218.253  |           5.70321e-05 |
|  98127 | search/search_index.json | 12.19.204.162   |           5.70321e-05 |
|  98128 | search/search_index.json | 12.31.120.82    |           5.70321e-05 |
|  98129 | search/search_index.json | 130.45.33.209   |           5.70321e-05 |
|  98130 | search/search_index.json | 130.45.50.251   |           5.70321e-05 |
|  98131 | search/search_index.json | 131.194.172.132 |           5.70321e-05 |
|  98132 | search/search_index.json | 131.194.172.226 |           5.70321e-05 |
|  98133 | search/search_index.json | 131.44.121.130  |           5.70321e-05 |
|  98134 | search/search_index.json | 134.6.226.3     |           5.70321e-05 |
|  98135 | search/search_index.json | 136.49.16.212   |           5.70321e-05 |
|  98136 | search/search_index.json | 136.50.11.9     |           5.70321e-05 |
|  98137 | search/search_index.json | 136.50.16.179   |           5.70321e-05 |
|  98138 | search/search_index.json | 136.50.17.203   |           5.70321e-05 |
|  98139 | search/search_index.json | 136.50.20.143   |           5.70321e-05 |
|  98140 | search/search_index.json | 136.50.54.220   |           5.70321e-05 |
|  98141 | search/search_index.json | 136.50.56.155   |           5.70321e-05 |
|  98142 | search/search_index.json | 136.50.58.119   |           5.70321e-05 |
|  98143 | search/search_index.json | 136.50.70.139   |           5.70321e-05 |
|  98144 | search/search_index.json | 136.50.73.42    |           5.70321e-05 |
|  98145 | search/search_index.json | 137.7.73.123    |           5.70321e-05 |
|  98146 | search/search_index.json | 148.66.37.82    |           5.70321e-05 |
|  98147 | search/search_index.json | 154.6.28.146    |           5.70321e-05 |
|  98148 | search/search_index.json | 156.146.38.132  |           5.70321e-05 |
|  98149 | search/search_index.json | 162.198.96.75   |           5.70321e-05 |
|  98150 | search/search_index.json | 162.233.170.149 |           5.70321e-05 |
|  98151 | search/search_index.json | 162.245.203.7   |           5.70321e-05 |
|  98152 | search/search_index.json | 165.227.41.208  |           5.70321e-05 |
|  98153 | search/search_index.json | 166.251.101.159 |           5.70321e-05 |
|  98154 | search/search_index.json | 166.251.106.113 |           5.70321e-05 |
|  98155 | search/search_index.json | 166.251.106.169 |           5.70321e-05 |
|  98156 | search/search_index.json | 166.251.109.109 |           5.70321e-05 |
|  98157 | search/search_index.json | 166.251.109.240 |           5.70321e-05 |
|  98158 | search/search_index.json | 166.251.109.31  |           5.70321e-05 |
|  98159 | search/search_index.json | 166.251.109.68  |           5.70321e-05 |
|  98160 | search/search_index.json | 169.57.90.139   |           5.70321e-05 |
|  98161 | search/search_index.json | 170.130.1.67    |           5.70321e-05 |
|  98162 | search/search_index.json | 172.124.64.127  |           5.70321e-05 |
|  98163 | search/search_index.json | 172.125.225.121 |           5.70321e-05 |
|  98164 | search/search_index.json | 172.127.182.225 |           5.70321e-05 |
|  98165 | search/search_index.json | 172.127.187.185 |           5.70321e-05 |
|  98166 | search/search_index.json | 172.56.6.54     |           5.70321e-05 |
|  98167 | search/search_index.json | 172.56.7.241    |           5.70321e-05 |
|  98168 | search/search_index.json | 172.56.7.51     |           5.70321e-05 |
|  98169 | search/search_index.json | 172.58.102.198  |           5.70321e-05 |
|  98170 | search/search_index.json | 172.58.102.244  |           5.70321e-05 |
|  98171 | search/search_index.json | 172.58.107.17   |           5.70321e-05 |
|  98172 | search/search_index.json | 172.58.107.67   |           5.70321e-05 |
|  98173 | search/search_index.json | 172.58.108.17   |           5.70321e-05 |
|  98174 | search/search_index.json | 172.58.109.121  |           5.70321e-05 |
|  98175 | search/search_index.json | 172.58.109.252  |           5.70321e-05 |
|  98176 | search/search_index.json | 172.58.110.151  |           5.70321e-05 |
|  98177 | search/search_index.json | 172.58.110.152  |           5.70321e-05 |
|  98178 | search/search_index.json | 172.58.111.26   |           5.70321e-05 |
|  98179 | search/search_index.json | 172.58.76.99    |           5.70321e-05 |
|  98180 | search/search_index.json | 172.58.97.217   |           5.70321e-05 |
|  98181 | search/search_index.json | 172.58.99.121   |           5.70321e-05 |
|  98182 | search/search_index.json | 172.58.99.163   |           5.70321e-05 |
|  98183 | search/search_index.json | 172.58.99.225   |           5.70321e-05 |
|  98184 | search/search_index.json | 172.58.99.43    |           5.70321e-05 |
|  98185 | search/search_index.json | 172.58.99.69    |           5.70321e-05 |
|  98186 | search/search_index.json | 172.8.173.98    |           5.70321e-05 |
|  98187 | search/search_index.json | 173.123.172.254 |           5.70321e-05 |
|  98188 | search/search_index.json | 173.123.186.102 |           5.70321e-05 |
|  98189 | search/search_index.json | 173.123.90.87   |           5.70321e-05 |
|  98190 | search/search_index.json | 173.149.157.87  |           5.70321e-05 |
|  98191 | search/search_index.json | 173.173.101.109 |           5.70321e-05 |
|  98192 | search/search_index.json | 173.173.107.110 |           5.70321e-05 |
|  98193 | search/search_index.json | 173.173.111.218 |           5.70321e-05 |
|  98194 | search/search_index.json | 173.173.173.208 |           5.70321e-05 |
|  98195 | search/search_index.json | 173.173.250.160 |           5.70321e-05 |
|  98196 | search/search_index.json | 173.173.96.82   |           5.70321e-05 |
|  98197 | search/search_index.json | 173.173.97.218  |           5.70321e-05 |
|  98198 | search/search_index.json | 173.174.134.234 |           5.70321e-05 |
|  98199 | search/search_index.json | 173.174.164.47  |           5.70321e-05 |
|  98200 | search/search_index.json | 173.174.211.244 |           5.70321e-05 |
|  98201 | search/search_index.json | 173.174.212.216 |           5.70321e-05 |
|  98202 | search/search_index.json | 173.174.220.17  |           5.70321e-05 |
|  98203 | search/search_index.json | 173.174.231.66  |           5.70321e-05 |
|  98204 | search/search_index.json | 173.175.100.201 |           5.70321e-05 |
|  98205 | search/search_index.json | 173.175.122.123 |           5.70321e-05 |
|  98206 | search/search_index.json | 173.175.211.194 |           5.70321e-05 |
|  98207 | search/search_index.json | 173.232.241.116 |           5.70321e-05 |
|  98208 | search/search_index.json | 173.239.232.107 |           5.70321e-05 |
|  98209 | search/search_index.json | 173.239.232.108 |           5.70321e-05 |
|  98210 | search/search_index.json | 173.239.232.130 |           5.70321e-05 |
|  98211 | search/search_index.json | 173.239.232.142 |           5.70321e-05 |
|  98212 | search/search_index.json | 173.239.232.154 |           5.70321e-05 |
|  98213 | search/search_index.json | 173.239.232.160 |           5.70321e-05 |
|  98214 | search/search_index.json | 173.239.232.161 |           5.70321e-05 |
|  98215 | search/search_index.json | 173.239.232.171 |           5.70321e-05 |
|  98216 | search/search_index.json | 173.239.232.173 |           5.70321e-05 |
|  98217 | search/search_index.json | 173.239.232.20  |           5.70321e-05 |
|  98218 | search/search_index.json | 173.239.232.27  |           5.70321e-05 |
|  98219 | search/search_index.json | 173.239.232.32  |           5.70321e-05 |
|  98220 | search/search_index.json | 173.239.232.37  |           5.70321e-05 |
|  98221 | search/search_index.json | 173.239.232.4   |           5.70321e-05 |
|  98222 | search/search_index.json | 173.239.232.44  |           5.70321e-05 |
|  98223 | search/search_index.json | 173.239.232.50  |           5.70321e-05 |
|  98224 | search/search_index.json | 173.239.232.62  |           5.70321e-05 |
|  98225 | search/search_index.json | 173.239.232.64  |           5.70321e-05 |
|  98226 | search/search_index.json | 173.239.232.66  |           5.70321e-05 |
|  98227 | search/search_index.json | 173.239.232.77  |           5.70321e-05 |
|  98228 | search/search_index.json | 173.239.232.79  |           5.70321e-05 |
|  98229 | search/search_index.json | 173.239.232.84  |           5.70321e-05 |
|  98230 | search/search_index.json | 173.239.232.88  |           5.70321e-05 |
|  98231 | search/search_index.json | 173.239.232.95  |           5.70321e-05 |
|  98232 | search/search_index.json | 173.30.84.132   |           5.70321e-05 |
|  98233 | search/search_index.json | 174.110.151.124 |           5.70321e-05 |
|  98234 | search/search_index.json | 174.197.9.102   |           5.70321e-05 |
|  98235 | search/search_index.json | 174.207.0.29    |           5.70321e-05 |
|  98236 | search/search_index.json | 174.207.0.84    |           5.70321e-05 |
|  98237 | search/search_index.json | 174.207.12.102  |           5.70321e-05 |
|  98238 | search/search_index.json | 174.207.14.75   |           5.70321e-05 |
|  98239 | search/search_index.json | 174.207.15.91   |           5.70321e-05 |
|  98240 | search/search_index.json | 174.207.17.95   |           5.70321e-05 |
|  98241 | search/search_index.json | 174.207.5.100   |           5.70321e-05 |
|  98242 | search/search_index.json | 174.207.5.31    |           5.70321e-05 |
|  98243 | search/search_index.json | 174.207.6.56    |           5.70321e-05 |
|  98244 | search/search_index.json | 174.207.8.218   |           5.70321e-05 |
|  98245 | search/search_index.json | 174.236.11.194  |           5.70321e-05 |
|  98246 | search/search_index.json | 174.236.11.30   |           5.70321e-05 |
|  98247 | search/search_index.json | 174.244.81.146  |           5.70321e-05 |
|  98248 | search/search_index.json | 174.246.196.4   |           5.70321e-05 |
|  98249 | search/search_index.json | 174.246.198.133 |           5.70321e-05 |
|  98250 | search/search_index.json | 174.246.199.81  |           5.70321e-05 |
|  98251 | search/search_index.json | 184.203.144.40  |           5.70321e-05 |
|  98252 | search/search_index.json | 184.226.46.205  |           5.70321e-05 |
|  98253 | search/search_index.json | 192.107.1.225   |           5.70321e-05 |
|  98254 | search/search_index.json | 193.56.117.3    |           5.70321e-05 |
|  98255 | search/search_index.json | 193.56.117.6    |           5.70321e-05 |
|  98256 | search/search_index.json | 193.56.117.75   |           5.70321e-05 |
|  98257 | search/search_index.json | 195.206.104.106 |           5.70321e-05 |
|  98258 | search/search_index.json | 195.206.104.107 |           5.70321e-05 |
|  98259 | search/search_index.json | 195.206.104.108 |           5.70321e-05 |
|  98260 | search/search_index.json | 198.98.87.16    |           5.70321e-05 |
|  98261 | search/search_index.json | 199.193.220.106 |           5.70321e-05 |
|  98262 | search/search_index.json | 205.251.148.122 |           5.70321e-05 |
|  98263 | search/search_index.json | 205.251.148.130 |           5.70321e-05 |
|  98264 | search/search_index.json | 205.251.148.138 |           5.70321e-05 |
|  98265 | search/search_index.json | 205.251.148.146 |           5.70321e-05 |
|  98266 | search/search_index.json | 205.251.148.74  |           5.70321e-05 |
|  98267 | search/search_index.json | 205.251.148.98  |           5.70321e-05 |
|  98268 | search/search_index.json | 205.251.150.218 |           5.70321e-05 |
|  98269 | search/search_index.json | 205.251.150.242 |           5.70321e-05 |
|  98270 | search/search_index.json | 206.109.23.139  |           5.70321e-05 |
|  98271 | search/search_index.json | 207.119.2.249   |           5.70321e-05 |
|  98272 | search/search_index.json | 207.119.48.226  |           5.70321e-05 |
|  98273 | search/search_index.json | 207.119.59.156  |           5.70321e-05 |
|  98274 | search/search_index.json | 208.84.155.36   |           5.70321e-05 |
|  98275 | search/search_index.json | 208.84.155.69   |           5.70321e-05 |
|  98276 | search/search_index.json | 209.184.113.43  |           5.70321e-05 |
|  98277 | search/search_index.json | 209.58.142.157  |           5.70321e-05 |
|  98278 | search/search_index.json | 216.54.250.134  |           5.70321e-05 |
|  98279 | search/search_index.json | 216.82.146.133  |           5.70321e-05 |
|  98280 | search/search_index.json | 217.167.133.44  |           5.70321e-05 |
|  98281 | search/search_index.json | 23.116.24.226   |           5.70321e-05 |
|  98282 | search/search_index.json | 23.117.144.200  |           5.70321e-05 |
|  98283 | search/search_index.json | 24.160.140.11   |           5.70321e-05 |
|  98284 | search/search_index.json | 24.160.158.182  |           5.70321e-05 |
|  98285 | search/search_index.json | 24.242.150.231  |           5.70321e-05 |
|  98286 | search/search_index.json | 24.242.180.178  |           5.70321e-05 |
|  98287 | search/search_index.json | 24.242.93.131   |           5.70321e-05 |
|  98288 | search/search_index.json | 24.243.10.53    |           5.70321e-05 |
|  98289 | search/search_index.json | 24.243.17.137   |           5.70321e-05 |
|  98290 | search/search_index.json | 24.243.3.85     |           5.70321e-05 |
|  98291 | search/search_index.json | 24.243.37.71    |           5.70321e-05 |
|  98292 | search/search_index.json | 24.243.49.105   |           5.70321e-05 |
|  98293 | search/search_index.json | 24.243.55.34    |           5.70321e-05 |
|  98294 | search/search_index.json | 24.243.6.109    |           5.70321e-05 |
|  98295 | search/search_index.json | 24.26.240.167   |           5.70321e-05 |
|  98296 | search/search_index.json | 24.26.255.20    |           5.70321e-05 |
|  98297 | search/search_index.json | 24.54.121.33    |           5.70321e-05 |
|  98298 | search/search_index.json | 3.101.35.99     |           5.70321e-05 |
|  98299 | search/search_index.json | 3.17.151.210    |           5.70321e-05 |
|  98300 | search/search_index.json | 45.131.192.185  |           5.70321e-05 |
|  98301 | search/search_index.json | 45.131.192.60   |           5.70321e-05 |
|  98302 | search/search_index.json | 45.131.192.63   |           5.70321e-05 |
|  98303 | search/search_index.json | 45.131.192.85   |           5.70321e-05 |
|  98304 | search/search_index.json | 45.132.226.7    |           5.70321e-05 |
|  98305 | search/search_index.json | 45.18.51.199    |           5.70321e-05 |
|  98306 | search/search_index.json | 45.20.117.105   |           5.70321e-05 |
|  98307 | search/search_index.json | 45.21.39.47     |           5.70321e-05 |
|  98308 | search/search_index.json | 45.23.251.14    |           5.70321e-05 |
|  98309 | search/search_index.json | 45.27.14.73     |           5.70321e-05 |
|  98310 | search/search_index.json | 45.31.212.88    |           5.70321e-05 |
|  98311 | search/search_index.json | 45.56.151.87    |           5.70321e-05 |
|  98312 | search/search_index.json | 45.83.89.58     |           5.70321e-05 |
|  98313 | search/search_index.json | 47.183.138.169  |           5.70321e-05 |
|  98314 | search/search_index.json | 47.187.247.193  |           5.70321e-05 |
|  98315 | search/search_index.json | 47.218.59.57    |           5.70321e-05 |
|  98316 | search/search_index.json | 47.234.188.230  |           5.70321e-05 |
|  98317 | search/search_index.json | 5.181.234.118   |           5.70321e-05 |
|  98318 | search/search_index.json | 50.27.36.194    |           5.70321e-05 |
|  98319 | search/search_index.json | 63.240.174.244  |           5.70321e-05 |
|  98320 | search/search_index.json | 64.44.80.196    |           5.70321e-05 |
|  98321 | search/search_index.json | 65.220.10.3     |           5.70321e-05 |
|  98322 | search/search_index.json | 66.176.160.135  |           5.70321e-05 |
|  98323 | search/search_index.json | 66.25.163.226   |           5.70321e-05 |
|  98324 | search/search_index.json | 66.25.164.171   |           5.70321e-05 |
|  98325 | search/search_index.json | 66.25.168.113   |           5.70321e-05 |
|  98326 | search/search_index.json | 66.69.1.31      |           5.70321e-05 |
|  98327 | search/search_index.json | 66.69.122.35    |           5.70321e-05 |
|  98328 | search/search_index.json | 66.69.219.165   |           5.70321e-05 |
|  98329 | search/search_index.json | 66.69.49.204    |           5.70321e-05 |
|  98330 | search/search_index.json | 66.69.60.109    |           5.70321e-05 |
|  98331 | search/search_index.json | 66.69.79.82     |           5.70321e-05 |
|  98332 | search/search_index.json | 66.76.100.195   |           5.70321e-05 |
|  98333 | search/search_index.json | 67.10.132.181   |           5.70321e-05 |
|  98334 | search/search_index.json | 67.10.156.23    |           5.70321e-05 |
|  98335 | search/search_index.json | 67.10.184.149   |           5.70321e-05 |
|  98336 | search/search_index.json | 67.11.120.157   |           5.70321e-05 |
|  98337 | search/search_index.json | 67.11.175.66    |           5.70321e-05 |
|  98338 | search/search_index.json | 67.11.208.216   |           5.70321e-05 |
|  98339 | search/search_index.json | 67.11.224.135   |           5.70321e-05 |
|  98340 | search/search_index.json | 67.11.62.128    |           5.70321e-05 |
|  98341 | search/search_index.json | 67.11.82.11     |           5.70321e-05 |
|  98342 | search/search_index.json | 67.11.97.204    |           5.70321e-05 |
|  98343 | search/search_index.json | 67.214.78.138   |           5.70321e-05 |
|  98344 | search/search_index.json | 67.7.11.231     |           5.70321e-05 |
|  98345 | search/search_index.json | 67.7.3.203      |           5.70321e-05 |
|  98346 | search/search_index.json | 67.79.232.243   |           5.70321e-05 |
|  98347 | search/search_index.json | 67.79.62.147    |           5.70321e-05 |
|  98348 | search/search_index.json | 67.80.48.111    |           5.70321e-05 |
|  98349 | search/search_index.json | 68.203.198.221  |           5.70321e-05 |
|  98350 | search/search_index.json | 68.203.222.3    |           5.70321e-05 |
|  98351 | search/search_index.json | 68.206.254.17   |           5.70321e-05 |
|  98352 | search/search_index.json | 68.206.254.198  |           5.70321e-05 |
|  98353 | search/search_index.json | 68.206.99.228   |           5.70321e-05 |
|  98354 | search/search_index.json | 69.232.49.24    |           5.70321e-05 |
|  98355 | search/search_index.json | 69.59.91.191    |           5.70321e-05 |
|  98356 | search/search_index.json | 69.9.44.152     |           5.70321e-05 |
|  98357 | search/search_index.json | 69.9.44.166     |           5.70321e-05 |
|  98358 | search/search_index.json | 69.9.44.167     |           5.70321e-05 |
|  98359 | search/search_index.json | 70.114.28.44    |           5.70321e-05 |
|  98360 | search/search_index.json | 70.116.81.203   |           5.70321e-05 |
|  98361 | search/search_index.json | 70.117.28.76    |           5.70321e-05 |
|  98362 | search/search_index.json | 70.117.71.59    |           5.70321e-05 |
|  98363 | search/search_index.json | 70.117.8.141    |           5.70321e-05 |
|  98364 | search/search_index.json | 70.120.3.91     |           5.70321e-05 |
|  98365 | search/search_index.json | 70.120.73.0     |           5.70321e-05 |
|  98366 | search/search_index.json | 70.120.73.115   |           5.70321e-05 |
|  98367 | search/search_index.json | 70.120.80.21    |           5.70321e-05 |
|  98368 | search/search_index.json | 70.121.163.5    |           5.70321e-05 |
|  98369 | search/search_index.json | 70.121.185.2    |           5.70321e-05 |
|  98370 | search/search_index.json | 70.121.187.122  |           5.70321e-05 |
|  98371 | search/search_index.json | 70.121.199.82   |           5.70321e-05 |
|  98372 | search/search_index.json | 70.121.205.5    |           5.70321e-05 |
|  98373 | search/search_index.json | 70.121.221.55   |           5.70321e-05 |
|  98374 | search/search_index.json | 70.123.110.205  |           5.70321e-05 |
|  98375 | search/search_index.json | 70.123.135.83   |           5.70321e-05 |
|  98376 | search/search_index.json | 70.123.169.229  |           5.70321e-05 |
|  98377 | search/search_index.json | 70.123.194.234  |           5.70321e-05 |
|  98378 | search/search_index.json | 70.123.228.86   |           5.70321e-05 |
|  98379 | search/search_index.json | 70.123.239.48   |           5.70321e-05 |
|  98380 | search/search_index.json | 70.123.252.212  |           5.70321e-05 |
|  98381 | search/search_index.json | 70.123.253.167  |           5.70321e-05 |
|  98382 | search/search_index.json | 70.125.137.193  |           5.70321e-05 |
|  98383 | search/search_index.json | 70.125.158.82   |           5.70321e-05 |
|  98384 | search/search_index.json | 70.130.124.250  |           5.70321e-05 |
|  98385 | search/search_index.json | 70.40.28.157    |           5.70321e-05 |
|  98386 | search/search_index.json | 70.94.132.38    |           5.70321e-05 |
|  98387 | search/search_index.json | 70.94.157.237   |           5.70321e-05 |
|  98388 | search/search_index.json | 70.94.167.166   |           5.70321e-05 |
|  98389 | search/search_index.json | 71.136.250.27   |           5.70321e-05 |
|  98390 | search/search_index.json | 71.136.254.125  |           5.70321e-05 |
|  98391 | search/search_index.json | 71.136.255.148  |           5.70321e-05 |
|  98392 | search/search_index.json | 71.136.255.165  |           5.70321e-05 |
|  98393 | search/search_index.json | 71.146.119.5    |           5.70321e-05 |
|  98394 | search/search_index.json | 71.158.155.73   |           5.70321e-05 |
|  98395 | search/search_index.json | 71.221.198.41   |           5.70321e-05 |
|  98396 | search/search_index.json | 71.221.46.112   |           5.70321e-05 |
|  98397 | search/search_index.json | 71.235.91.18    |           5.70321e-05 |
|  98398 | search/search_index.json | 71.41.192.26    |           5.70321e-05 |
|  98399 | search/search_index.json | 71.42.105.182   |           5.70321e-05 |
|  98400 | search/search_index.json | 71.42.198.186   |           5.70321e-05 |
|  98401 | search/search_index.json | 72.177.173.215  |           5.70321e-05 |
|  98402 | search/search_index.json | 72.177.174.24   |           5.70321e-05 |
|  98403 | search/search_index.json | 72.177.209.244  |           5.70321e-05 |
|  98404 | search/search_index.json | 72.177.229.125  |           5.70321e-05 |
|  98405 | search/search_index.json | 72.178.117.75   |           5.70321e-05 |
|  98406 | search/search_index.json | 72.178.188.133  |           5.70321e-05 |
|  98407 | search/search_index.json | 72.179.108.103  |           5.70321e-05 |
|  98408 | search/search_index.json | 72.181.105.69   |           5.70321e-05 |
|  98409 | search/search_index.json | 72.181.106.34   |           5.70321e-05 |
|  98410 | search/search_index.json | 72.181.109.235  |           5.70321e-05 |
|  98411 | search/search_index.json | 72.181.115.96   |           5.70321e-05 |
|  98412 | search/search_index.json | 72.181.118.217  |           5.70321e-05 |
|  98413 | search/search_index.json | 72.181.120.120  |           5.70321e-05 |
|  98414 | search/search_index.json | 72.181.145.92   |           5.70321e-05 |
|  98415 | search/search_index.json | 72.190.151.50   |           5.70321e-05 |
|  98416 | search/search_index.json | 72.190.175.190  |           5.70321e-05 |
|  98417 | search/search_index.json | 72.191.19.45    |           5.70321e-05 |
|  98418 | search/search_index.json | 72.191.4.136    |           5.70321e-05 |
|  98419 | search/search_index.json | 72.191.48.117   |           5.70321e-05 |
|  98420 | search/search_index.json | 72.191.50.176   |           5.70321e-05 |
|  98421 | search/search_index.json | 72.191.58.78    |           5.70321e-05 |
|  98422 | search/search_index.json | 72.193.254.235  |           5.70321e-05 |
|  98423 | search/search_index.json | 72.21.196.66    |           5.70321e-05 |
|  98424 | search/search_index.json | 72.48.255.177   |           5.70321e-05 |
|  98425 | search/search_index.json | 72.48.60.84     |           5.70321e-05 |
|  98426 | search/search_index.json | 73.136.220.123  |           5.70321e-05 |
|  98427 | search/search_index.json | 73.143.162.24   |           5.70321e-05 |
|  98428 | search/search_index.json | 73.166.86.204   |           5.70321e-05 |
|  98429 | search/search_index.json | 73.206.101.120  |           5.70321e-05 |
|  98430 | search/search_index.json | 74.81.88.74     |           5.70321e-05 |
|  98431 | search/search_index.json | 75.1.145.2      |           5.70321e-05 |
|  98432 | search/search_index.json | 75.15.240.105   |           5.70321e-05 |
|  98433 | search/search_index.json | 75.15.245.2     |           5.70321e-05 |
|  98434 | search/search_index.json | 76.115.148.248  |           5.70321e-05 |
|  98435 | search/search_index.json | 76.183.251.188  |           5.70321e-05 |
|  98436 | search/search_index.json | 76.185.135.244  |           5.70321e-05 |
|  98437 | search/search_index.json | 76.185.158.93   |           5.70321e-05 |
|  98438 | search/search_index.json | 76.185.193.115  |           5.70321e-05 |
|  98439 | search/search_index.json | 76.185.204.129  |           5.70321e-05 |
|  98440 | search/search_index.json | 76.185.206.252  |           5.70321e-05 |
|  98441 | search/search_index.json | 76.185.209.80   |           5.70321e-05 |
|  98442 | search/search_index.json | 76.185.216.146  |           5.70321e-05 |
|  98443 | search/search_index.json | 76.185.67.51    |           5.70321e-05 |
|  98444 | search/search_index.json | 76.186.189.183  |           5.70321e-05 |
|  98445 | search/search_index.json | 76.186.80.227   |           5.70321e-05 |
|  98446 | search/search_index.json | 76.201.6.236    |           5.70321e-05 |
|  98447 | search/search_index.json | 76.204.29.215   |           5.70321e-05 |
|  98448 | search/search_index.json | 76.214.115.41   |           5.70321e-05 |
|  98449 | search/search_index.json | 77.111.247.10   |           5.70321e-05 |
|  98450 | search/search_index.json | 77.136.40.123   |           5.70321e-05 |
|  98451 | search/search_index.json | 79.220.63.16    |           5.70321e-05 |
|  98452 | search/search_index.json | 89.187.164.130  |           5.70321e-05 |
|  98453 | search/search_index.json | 89.187.164.249  |           5.70321e-05 |
|  98454 | search/search_index.json | 89.187.164.76   |           5.70321e-05 |
|  98455 | search/search_index.json | 89.187.175.144  |           5.70321e-05 |
|  98456 | search/search_index.json | 89.187.177.72   |           5.70321e-05 |
|  98457 | search/search_index.json | 92.38.148.10    |           5.70321e-05 |
|  98458 | search/search_index.json | 96.46.160.21    |           5.70321e-05 |
|  98459 | search/search_index.json | 96.8.128.205    |           5.70321e-05 |
|  98460 | search/search_index.json | 96.8.128.69     |           5.70321e-05 |
|  98461 | search/search_index.json | 96.8.130.139    |           5.70321e-05 |
|  98462 | search/search_index.json | 96.8.130.193    |           5.70321e-05 |
|  98463 | search/search_index.json | 96.8.130.206    |           5.70321e-05 |
|  98464 | search/search_index.json | 96.8.130.212    |           5.70321e-05 |
|  98465 | search/search_index.json | 96.8.131.77     |           5.70321e-05 |
|  98466 | search/search_index.json | 96.8.151.49     |           5.70321e-05 |
|  98467 | search/search_index.json | 96.8.168.147    |           5.70321e-05 |
|  98468 | search/search_index.json | 96.8.178.42     |           5.70321e-05 |
|  98469 | search/search_index.json | 96.8.248.115    |           5.70321e-05 |
|  98470 | search/search_index.json | 96.8.248.20     |           5.70321e-05 |
|  98471 | search/search_index.json | 96.8.248.243    |           5.70321e-05 |
|  98472 | search/search_index.json | 97.105.81.147   |           5.70321e-05 |
|  98473 | search/search_index.json | 97.77.27.166    |           5.70321e-05 |
|  98474 | search/search_index.json | 97.99.239.216   |           5.70321e-05 |
|  98475 | search/search_index.json | 98.6.204.179    |           5.70321e-05 |
|  98476 | search/search_index.json | 98.6.52.59      |           5.70321e-05 |
|  98477 | search/search_index.json | 98.6.94.51      |           5.70321e-05 |
|  98478 | search/search_index.json | 99.126.113.140  |           5.70321e-05 |
|  98479 | search/search_index.json | 99.126.116.28   |           5.70321e-05 |
|  98480 | search/search_index.json | 99.132.130.148  |           5.70321e-05 |
|  98481 | search/search_index.json | 99.140.253.42   |           5.70321e-05 |
|  98482 | search/search_index.json | 99.158.251.110  |           5.70321e-05 |
|  98483 | search/search_index.json | 99.162.243.68   |           5.70321e-05 |
|  98484 | search/search_index.json | 99.203.154.172  |           5.70321e-05 |
|  98485 | search/search_index.json | 99.203.154.44   |           5.70321e-05 |
|  98486 | search/search_index.json | 99.203.154.84   |           5.70321e-05 |
|  98487 | search/search_index.json | 99.203.155.193  |           5.70321e-05 |
|  98488 | search/search_index.json | 99.203.26.188   |           5.70321e-05 |
|  98489 | search/search_index.json | 99.203.26.55    |           5.70321e-05 |
|  98490 | search/search_index.json | 99.203.26.63    |           5.70321e-05 |
|  98491 | search/search_index.json | 99.203.27.106   |           5.70321e-05 |
|  98492 | search/search_index.json | 99.203.27.182   |           5.70321e-05 |
|  98493 | search/search_index.json | 99.203.27.226   |           5.70321e-05 |
|  98494 | search/search_index.json | 99.203.27.248   |           5.70321e-05 |
|  98495 | search/search_index.json | 99.203.27.92    |           5.70321e-05 |
|  98496 | search/search_index.json | 99.203.91.132   |           5.70321e-05 |
|  98497 | search/search_index.json | 99.27.139.172   |           5.70321e-05 |
|  98498 | search/search_index.json | 99.43.108.160   |           5.70321e-05 |
|  98499 | search/search_index.json | 99.98.36.168    |           5.70321e-05 |
| 100360 | spring                   | 104.148.30.54   |           8.41538e-05 |
| 100361 | spring                   | 104.180.86.144  |           8.41538e-05 |
| 100362 | spring                   | 104.190.255.78  |           8.41538e-05 |
| 100363 | spring                   | 104.191.11.147  |           8.41538e-05 |
| 100364 | spring                   | 104.219.136.67  |           8.41538e-05 |
| 100365 | spring                   | 104.5.221.205   |           8.41538e-05 |
| 100366 | spring                   | 107.117.224.90  |           8.41538e-05 |
| 100367 | spring                   | 107.127.0.131   |           8.41538e-05 |
| 100368 | spring                   | 107.129.209.3   |           8.41538e-05 |
| 100369 | spring                   | 107.158.15.51   |           8.41538e-05 |
| 100370 | spring                   | 107.204.53.24   |           8.41538e-05 |
| 100371 | spring                   | 107.77.100.88   |           8.41538e-05 |
| 100372 | spring                   | 107.77.105.117  |           8.41538e-05 |
| 100373 | spring                   | 107.77.105.45   |           8.41538e-05 |
| 100374 | spring                   | 107.77.105.92   |           8.41538e-05 |
| 100375 | spring                   | 107.77.189.33   |           8.41538e-05 |
| 100376 | spring                   | 107.77.189.47   |           8.41538e-05 |
| 100377 | spring                   | 107.77.200.32   |           8.41538e-05 |
| 100378 | spring                   | 107.77.217.13   |           8.41538e-05 |
| 100379 | spring                   | 107.77.217.151  |           8.41538e-05 |
| 100380 | spring                   | 107.77.217.69   |           8.41538e-05 |
| 100381 | spring                   | 107.77.218.122  |           8.41538e-05 |
| 100382 | spring                   | 107.77.218.126  |           8.41538e-05 |
| 100383 | spring                   | 107.77.219.67   |           8.41538e-05 |
| 100384 | spring                   | 107.77.219.93   |           8.41538e-05 |
| 100385 | spring                   | 107.77.220.40   |           8.41538e-05 |
| 100386 | spring                   | 107.77.220.51   |           8.41538e-05 |
| 100387 | spring                   | 107.77.221.10   |           8.41538e-05 |
| 100388 | spring                   | 107.77.221.209  |           8.41538e-05 |
| 100389 | spring                   | 107.77.222.188  |           8.41538e-05 |
| 100390 | spring                   | 107.77.222.85   |           8.41538e-05 |
| 100391 | spring                   | 107.77.64.52    |           8.41538e-05 |
| 100392 | spring                   | 107.77.66.78    |           8.41538e-05 |
| 100393 | spring                   | 108.192.181.20  |           8.41538e-05 |
| 100394 | spring                   | 108.196.159.214 |           8.41538e-05 |
| 100395 | spring                   | 108.211.88.21   |           8.41538e-05 |
| 100396 | spring                   | 108.218.193.28  |           8.41538e-05 |
| 100397 | spring                   | 108.220.101.224 |           8.41538e-05 |
| 100398 | spring                   | 110.78.138.179  |           8.41538e-05 |
| 100399 | spring                   | 12.247.3.50     |           8.41538e-05 |
| 100400 | spring                   | 12.5.63.210     |           8.41538e-05 |
| 100401 | spring                   | 129.112.109.42  |           8.41538e-05 |
| 100402 | spring                   | 130.45.53.218   |           8.41538e-05 |
| 100403 | spring                   | 130.45.68.158   |           8.41538e-05 |
| 100404 | spring                   | 131.194.172.132 |           8.41538e-05 |
| 100405 | spring                   | 131.194.172.199 |           8.41538e-05 |
| 100406 | spring                   | 134.250.1.82    |           8.41538e-05 |
| 100407 | spring                   | 136.50.13.79    |           8.41538e-05 |
| 100408 | spring                   | 136.50.2.13     |           8.41538e-05 |
| 100409 | spring                   | 136.50.5.46     |           8.41538e-05 |
| 100410 | spring                   | 136.50.5.58     |           8.41538e-05 |
| 100411 | spring                   | 136.50.53.26    |           8.41538e-05 |
| 100412 | spring                   | 136.50.54.38    |           8.41538e-05 |
| 100413 | spring                   | 136.50.71.97    |           8.41538e-05 |
| 100414 | spring                   | 136.50.89.106   |           8.41538e-05 |
| 100415 | spring                   | 136.50.97.29    |           8.41538e-05 |
| 100416 | spring                   | 136.62.156.201  |           8.41538e-05 |
| 100417 | spring                   | 137.254.7.172   |           8.41538e-05 |
| 100418 | spring                   | 137.254.7.174   |           8.41538e-05 |
| 100419 | spring                   | 137.254.7.176   |           8.41538e-05 |
| 100420 | spring                   | 137.254.7.178   |           8.41538e-05 |
| 100421 | spring                   | 148.244.212.19  |           8.41538e-05 |
| 100422 | spring                   | 148.66.37.91    |           8.41538e-05 |
| 100423 | spring                   | 155.94.241.194  |           8.41538e-05 |
| 100424 | spring                   | 159.192.242.43  |           8.41538e-05 |
| 100425 | spring                   | 162.249.35.10   |           8.41538e-05 |
| 100426 | spring                   | 162.250.18.140  |           8.41538e-05 |
| 100427 | spring                   | 162.253.68.118  |           8.41538e-05 |
| 100428 | spring                   | 162.72.133.33   |           8.41538e-05 |
| 100429 | spring                   | 165.225.48.90   |           8.41538e-05 |
| 100430 | spring                   | 165.84.230.4    |           8.41538e-05 |
| 100431 | spring                   | 166.137.126.119 |           8.41538e-05 |
| 100432 | spring                   | 166.137.136.50  |           8.41538e-05 |
| 100433 | spring                   | 166.251.106.189 |           8.41538e-05 |
| 100434 | spring                   | 166.251.109.31  |           8.41538e-05 |
| 100435 | spring                   | 170.225.9.141   |           8.41538e-05 |
| 100436 | spring                   | 172.127.183.32  |           8.41538e-05 |
| 100437 | spring                   | 172.3.220.202   |           8.41538e-05 |
| 100438 | spring                   | 172.3.28.29     |           8.41538e-05 |
| 100439 | spring                   | 172.5.103.185   |           8.41538e-05 |
| 100440 | spring                   | 172.56.6.20     |           8.41538e-05 |
| 100441 | spring                   | 172.56.7.25     |           8.41538e-05 |
| 100442 | spring                   | 172.58.102.160  |           8.41538e-05 |
| 100443 | spring                   | 172.58.102.206  |           8.41538e-05 |
| 100444 | spring                   | 172.58.102.248  |           8.41538e-05 |
| 100445 | spring                   | 172.58.107.124  |           8.41538e-05 |
| 100446 | spring                   | 172.58.108.5    |           8.41538e-05 |
| 100447 | spring                   | 172.58.109.54   |           8.41538e-05 |
| 100448 | spring                   | 172.58.110.152  |           8.41538e-05 |
| 100449 | spring                   | 172.58.110.181  |           8.41538e-05 |
| 100450 | spring                   | 172.58.110.193  |           8.41538e-05 |
| 100451 | spring                   | 172.58.111.255  |           8.41538e-05 |
| 100452 | spring                   | 172.58.67.18    |           8.41538e-05 |
| 100453 | spring                   | 172.58.99.40    |           8.41538e-05 |
| 100454 | spring                   | 172.87.148.22   |           8.41538e-05 |
| 100455 | spring                   | 173.123.102.174 |           8.41538e-05 |
| 100456 | spring                   | 173.123.21.58   |           8.41538e-05 |
| 100457 | spring                   | 173.127.115.88  |           8.41538e-05 |
| 100458 | spring                   | 173.127.49.144  |           8.41538e-05 |
| 100459 | spring                   | 173.149.177.187 |           8.41538e-05 |
| 100460 | spring                   | 173.149.180.43  |           8.41538e-05 |
| 100461 | spring                   | 173.172.166.90  |           8.41538e-05 |
| 100462 | spring                   | 173.173.102.198 |           8.41538e-05 |
| 100463 | spring                   | 173.173.104.14  |           8.41538e-05 |
| 100464 | spring                   | 173.173.104.57  |           8.41538e-05 |
| 100465 | spring                   | 173.173.121.126 |           8.41538e-05 |
| 100466 | spring                   | 173.173.250.160 |           8.41538e-05 |
| 100467 | spring                   | 173.174.164.88  |           8.41538e-05 |
| 100468 | spring                   | 173.174.199.207 |           8.41538e-05 |
| 100469 | spring                   | 173.174.210.2   |           8.41538e-05 |
| 100470 | spring                   | 173.175.117.44  |           8.41538e-05 |
| 100471 | spring                   | 173.175.122.123 |           8.41538e-05 |
| 100472 | spring                   | 173.175.19.30   |           8.41538e-05 |
| 100473 | spring                   | 173.175.211.194 |           8.41538e-05 |
| 100474 | spring                   | 173.227.210.1   |           8.41538e-05 |
| 100475 | spring                   | 173.239.218.8   |           8.41538e-05 |
| 100476 | spring                   | 173.239.232.20  |           8.41538e-05 |
| 100477 | spring                   | 173.239.232.5   |           8.41538e-05 |
| 100478 | spring                   | 173.239.232.60  |           8.41538e-05 |
| 100479 | spring                   | 173.239.232.8   |           8.41538e-05 |
| 100480 | spring                   | 173.239.232.99  |           8.41538e-05 |
| 100481 | spring                   | 173.244.217.91  |           8.41538e-05 |
| 100482 | spring                   | 173.30.84.132   |           8.41538e-05 |
| 100483 | spring                   | 174.207.1.185   |           8.41538e-05 |
| 100484 | spring                   | 174.207.2.137   |           8.41538e-05 |
| 100485 | spring                   | 174.207.21.3    |           8.41538e-05 |
| 100486 | spring                   | 174.207.5.31    |           8.41538e-05 |
| 100487 | spring                   | 174.207.7.252   |           8.41538e-05 |
| 100488 | spring                   | 174.236.11.30   |           8.41538e-05 |
| 100489 | spring                   | 174.236.6.113   |           8.41538e-05 |
| 100490 | spring                   | 174.240.134.5   |           8.41538e-05 |
| 100491 | spring                   | 174.246.199.81  |           8.41538e-05 |
| 100492 | spring                   | 174.249.44.163  |           8.41538e-05 |
| 100493 | spring                   | 174.255.131.179 |           8.41538e-05 |
| 100494 | spring                   | 184.203.123.248 |           8.41538e-05 |
| 100495 | spring                   | 184.203.242.102 |           8.41538e-05 |
| 100496 | spring                   | 184.226.100.247 |           8.41538e-05 |
| 100497 | spring                   | 184.226.30.104  |           8.41538e-05 |
| 100498 | spring                   | 184.226.6.201   |           8.41538e-05 |
| 100499 | spring                   | 184.62.220.53   |           8.41538e-05 |
| 100500 | spring                   | 185.151.12.218  |           8.41538e-05 |
| 100501 | spring                   | 185.247.70.196  |           8.41538e-05 |
| 100502 | spring                   | 185.247.70.228  |           8.41538e-05 |
| 100503 | spring                   | 192.138.47.30   |           8.41538e-05 |
| 100504 | spring                   | 192.67.54.12    |           8.41538e-05 |
| 100505 | spring                   | 196.52.39.29    |           8.41538e-05 |
| 100506 | spring                   | 199.59.33.126   |           8.41538e-05 |
| 100507 | spring                   | 204.44.112.76   |           8.41538e-05 |
| 100508 | spring                   | 205.251.150.218 |           8.41538e-05 |
| 100509 | spring                   | 205.251.151.42  |           8.41538e-05 |
| 100510 | spring                   | 206.81.232.6    |           8.41538e-05 |
| 100511 | spring                   | 207.189.160.150 |           8.41538e-05 |
| 100512 | spring                   | 207.239.160.254 |           8.41538e-05 |
| 100513 | spring                   | 208.180.188.245 |           8.41538e-05 |
| 100514 | spring                   | 209.12.105.53   |           8.41538e-05 |
| 100515 | spring                   | 209.182.242.18  |           8.41538e-05 |
| 100516 | spring                   | 209.184.113.43  |           8.41538e-05 |
| 100517 | spring                   | 209.205.221.162 |           8.41538e-05 |
| 100518 | spring                   | 212.102.45.115  |           8.41538e-05 |
| 100519 | spring                   | 216.177.187.98  |           8.41538e-05 |
| 100520 | spring                   | 23.116.24.226   |           8.41538e-05 |
| 100521 | spring                   | 23.227.134.242  |           8.41538e-05 |
| 100522 | spring                   | 24.160.139.79   |           8.41538e-05 |
| 100523 | spring                   | 24.173.152.222  |           8.41538e-05 |
| 100524 | spring                   | 24.242.143.6    |           8.41538e-05 |
| 100525 | spring                   | 24.242.183.202  |           8.41538e-05 |
| 100526 | spring                   | 24.243.16.96    |           8.41538e-05 |
| 100527 | spring                   | 24.243.31.59    |           8.41538e-05 |
| 100528 | spring                   | 24.243.39.1     |           8.41538e-05 |
| 100529 | spring                   | 24.243.59.138   |           8.41538e-05 |
| 100530 | spring                   | 24.26.242.224   |           8.41538e-05 |
| 100531 | spring                   | 24.26.246.52    |           8.41538e-05 |
| 100532 | spring                   | 24.26.251.39    |           8.41538e-05 |
| 100533 | spring                   | 24.28.109.234   |           8.41538e-05 |
| 100534 | spring                   | 24.28.148.245   |           8.41538e-05 |
| 100535 | spring                   | 24.28.159.140   |           8.41538e-05 |
| 100536 | spring                   | 24.9.177.142    |           8.41538e-05 |
| 100537 | spring                   | 24.99.149.248   |           8.41538e-05 |
| 100538 | spring                   | 31.14.72.18     |           8.41538e-05 |
| 100539 | spring                   | 34.226.8.106    |           8.41538e-05 |
| 100540 | spring                   | 45.131.192.166  |           8.41538e-05 |
| 100541 | spring                   | 45.21.32.233    |           8.41538e-05 |
| 100542 | spring                   | 45.22.97.134    |           8.41538e-05 |
| 100543 | spring                   | 45.31.213.223   |           8.41538e-05 |
| 100544 | spring                   | 45.56.151.122   |           8.41538e-05 |
| 100545 | spring                   | 47.187.247.193  |           8.41538e-05 |
| 100546 | spring                   | 47.221.223.242  |           8.41538e-05 |
| 100547 | spring                   | 47.234.152.236  |           8.41538e-05 |
| 100548 | spring                   | 5.181.234.114   |           8.41538e-05 |
| 100549 | spring                   | 50.84.78.82     |           8.41538e-05 |
| 100550 | spring                   | 56.0.143.23     |           8.41538e-05 |
| 100551 | spring                   | 64.233.172.169  |           8.41538e-05 |
| 100552 | spring                   | 65.36.104.120   |           8.41538e-05 |
| 100553 | spring                   | 65.36.105.55    |           8.41538e-05 |
| 100554 | spring                   | 66.102.6.173    |           8.41538e-05 |
| 100555 | spring                   | 66.193.245.94   |           8.41538e-05 |
| 100556 | spring                   | 66.25.162.204   |           8.41538e-05 |
| 100557 | spring                   | 66.25.163.226   |           8.41538e-05 |
| 100558 | spring                   | 66.25.166.124   |           8.41538e-05 |
| 100559 | spring                   | 66.25.171.170   |           8.41538e-05 |
| 100560 | spring                   | 66.25.172.244   |           8.41538e-05 |
| 100561 | spring                   | 66.3.38.170     |           8.41538e-05 |
| 100562 | spring                   | 66.64.5.233     |           8.41538e-05 |
| 100563 | spring                   | 66.69.110.179   |           8.41538e-05 |
| 100564 | spring                   | 66.69.219.165   |           8.41538e-05 |
| 100565 | spring                   | 66.69.48.5      |           8.41538e-05 |
| 100566 | spring                   | 66.69.58.238    |           8.41538e-05 |
| 100567 | spring                   | 66.69.60.160    |           8.41538e-05 |
| 100568 | spring                   | 66.90.156.73    |           8.41538e-05 |
| 100569 | spring                   | 66.90.180.216   |           8.41538e-05 |
| 100570 | spring                   | 67.10.128.179   |           8.41538e-05 |
| 100571 | spring                   | 67.10.128.187   |           8.41538e-05 |
| 100572 | spring                   | 67.10.140.180   |           8.41538e-05 |
| 100573 | spring                   | 67.10.154.68    |           8.41538e-05 |
| 100574 | spring                   | 67.10.183.5     |           8.41538e-05 |
| 100575 | spring                   | 67.10.184.254   |           8.41538e-05 |
| 100576 | spring                   | 67.11.100.255   |           8.41538e-05 |
| 100577 | spring                   | 67.11.100.82    |           8.41538e-05 |
| 100578 | spring                   | 67.11.104.47    |           8.41538e-05 |
| 100579 | spring                   | 67.11.111.87    |           8.41538e-05 |
| 100580 | spring                   | 67.11.145.202   |           8.41538e-05 |
| 100581 | spring                   | 67.11.167.40    |           8.41538e-05 |
| 100582 | spring                   | 67.11.207.37    |           8.41538e-05 |
| 100583 | spring                   | 67.11.229.0     |           8.41538e-05 |
| 100584 | spring                   | 67.11.24.12     |           8.41538e-05 |
| 100585 | spring                   | 67.11.245.224   |           8.41538e-05 |
| 100586 | spring                   | 67.11.39.16     |           8.41538e-05 |
| 100587 | spring                   | 67.11.41.202    |           8.41538e-05 |
| 100588 | spring                   | 67.11.98.144    |           8.41538e-05 |
| 100589 | spring                   | 67.142.235.252  |           8.41538e-05 |
| 100590 | spring                   | 67.7.7.139      |           8.41538e-05 |
| 100591 | spring                   | 68.201.204.191  |           8.41538e-05 |
| 100592 | spring                   | 68.203.198.221  |           8.41538e-05 |
| 100593 | spring                   | 68.203.215.197  |           8.41538e-05 |
| 100594 | spring                   | 68.206.198.66   |           8.41538e-05 |
| 100595 | spring                   | 68.206.246.215  |           8.41538e-05 |
| 100596 | spring                   | 68.206.99.228   |           8.41538e-05 |
| 100597 | spring                   | 68.227.97.247   |           8.41538e-05 |
| 100598 | spring                   | 68.249.0.44     |           8.41538e-05 |
| 100599 | spring                   | 69.132.169.243  |           8.41538e-05 |
| 100600 | spring                   | 69.153.19.127   |           8.41538e-05 |
| 100601 | spring                   | 69.154.52.98    |           8.41538e-05 |
| 100602 | spring                   | 69.160.252.231  |           8.41538e-05 |
| 100603 | spring                   | 69.166.71.6     |           8.41538e-05 |
| 100604 | spring                   | 69.231.76.212   |           8.41538e-05 |
| 100605 | spring                   | 69.239.143.192  |           8.41538e-05 |
| 100606 | spring                   | 69.9.44.166     |           8.41538e-05 |
| 100607 | spring                   | 69.9.44.186     |           8.41538e-05 |
| 100608 | spring                   | 70.113.107.71   |           8.41538e-05 |
| 100609 | spring                   | 70.114.139.150  |           8.41538e-05 |
| 100610 | spring                   | 70.115.114.110  |           8.41538e-05 |
| 100611 | spring                   | 70.117.31.37    |           8.41538e-05 |
| 100612 | spring                   | 70.118.168.239  |           8.41538e-05 |
| 100613 | spring                   | 70.120.1.49     |           8.41538e-05 |
| 100614 | spring                   | 70.120.213.195  |           8.41538e-05 |
| 100615 | spring                   | 70.120.3.242    |           8.41538e-05 |
| 100616 | spring                   | 70.120.59.12    |           8.41538e-05 |
| 100617 | spring                   | 70.120.93.125   |           8.41538e-05 |
| 100618 | spring                   | 70.121.171.184  |           8.41538e-05 |
| 100619 | spring                   | 70.121.185.146  |           8.41538e-05 |
| 100620 | spring                   | 70.121.188.16   |           8.41538e-05 |
| 100621 | spring                   | 70.121.191.85   |           8.41538e-05 |
| 100622 | spring                   | 70.121.200.203  |           8.41538e-05 |
| 100623 | spring                   | 70.121.204.76   |           8.41538e-05 |
| 100624 | spring                   | 70.121.214.153  |           8.41538e-05 |
| 100625 | spring                   | 70.122.206.28   |           8.41538e-05 |
| 100626 | spring                   | 70.122.250.222  |           8.41538e-05 |
| 100627 | spring                   | 70.123.135.83   |           8.41538e-05 |
| 100628 | spring                   | 70.123.137.196  |           8.41538e-05 |
| 100629 | spring                   | 70.123.200.145  |           8.41538e-05 |
| 100630 | spring                   | 70.125.135.252  |           8.41538e-05 |
| 100631 | spring                   | 70.125.137.193  |           8.41538e-05 |
| 100632 | spring                   | 70.125.138.191  |           8.41538e-05 |
| 100633 | spring                   | 70.130.122.154  |           8.41538e-05 |
| 100634 | spring                   | 70.130.124.250  |           8.41538e-05 |
| 100635 | spring                   | 70.239.184.149  |           8.41538e-05 |
| 100636 | spring                   | 70.94.129.79    |           8.41538e-05 |
| 100637 | spring                   | 70.94.132.38    |           8.41538e-05 |
| 100638 | spring                   | 70.94.160.147   |           8.41538e-05 |
| 100639 | spring                   | 71.136.254.125  |           8.41538e-05 |
| 100640 | spring                   | 71.40.81.37     |           8.41538e-05 |
| 100641 | spring                   | 71.78.144.202   |           8.41538e-05 |
| 100642 | spring                   | 71.78.167.131   |           8.41538e-05 |
| 100643 | spring                   | 72.177.15.120   |           8.41538e-05 |
| 100644 | spring                   | 72.177.199.59   |           8.41538e-05 |
| 100645 | spring                   | 72.177.210.199  |           8.41538e-05 |
| 100646 | spring                   | 72.177.241.50   |           8.41538e-05 |
| 100647 | spring                   | 72.178.117.75   |           8.41538e-05 |
| 100648 | spring                   | 72.179.161.39   |           8.41538e-05 |
| 100649 | spring                   | 72.179.174.165  |           8.41538e-05 |
| 100650 | spring                   | 72.179.185.12   |           8.41538e-05 |
| 100651 | spring                   | 72.180.103.93   |           8.41538e-05 |
| 100652 | spring                   | 72.180.149.37   |           8.41538e-05 |
| 100653 | spring                   | 72.180.51.14    |           8.41538e-05 |
| 100654 | spring                   | 72.181.110.80   |           8.41538e-05 |
| 100655 | spring                   | 72.181.113.238  |           8.41538e-05 |
| 100656 | spring                   | 72.181.120.73   |           8.41538e-05 |
| 100657 | spring                   | 72.181.97.207   |           8.41538e-05 |
| 100658 | spring                   | 72.190.118.203  |           8.41538e-05 |
| 100659 | spring                   | 72.190.170.159  |           8.41538e-05 |
| 100660 | spring                   | 72.190.175.190  |           8.41538e-05 |
| 100661 | spring                   | 72.191.28.232   |           8.41538e-05 |
| 100662 | spring                   | 72.191.59.31    |           8.41538e-05 |
| 100663 | spring                   | 72.45.179.227   |           8.41538e-05 |
| 100664 | spring                   | 73.153.134.112  |           8.41538e-05 |
| 100665 | spring                   | 73.186.43.188   |           8.41538e-05 |
| 100666 | spring                   | 73.219.222.176  |           8.41538e-05 |
| 100667 | spring                   | 73.34.63.18     |           8.41538e-05 |
| 100668 | spring                   | 73.39.175.210   |           8.41538e-05 |
| 100669 | spring                   | 74.81.88.114    |           8.41538e-05 |
| 100670 | spring                   | 74.81.88.42     |           8.41538e-05 |
| 100671 | spring                   | 75.1.166.163    |           8.41538e-05 |
| 100672 | spring                   | 75.1.166.50     |           8.41538e-05 |
| 100673 | spring                   | 75.1.195.169    |           8.41538e-05 |
| 100674 | spring                   | 75.15.242.25    |           8.41538e-05 |
| 100675 | spring                   | 75.33.163.123   |           8.41538e-05 |
| 100676 | spring                   | 75.54.33.161    |           8.41538e-05 |
| 100677 | spring                   | 76.184.48.62    |           8.41538e-05 |
| 100678 | spring                   | 76.184.52.198   |           8.41538e-05 |
| 100679 | spring                   | 76.185.121.198  |           8.41538e-05 |
| 100680 | spring                   | 76.185.135.244  |           8.41538e-05 |
| 100681 | spring                   | 76.185.135.35   |           8.41538e-05 |
| 100682 | spring                   | 76.185.152.173  |           8.41538e-05 |
| 100683 | spring                   | 76.185.155.178  |           8.41538e-05 |
| 100684 | spring                   | 76.185.182.104  |           8.41538e-05 |
| 100685 | spring                   | 76.185.234.111  |           8.41538e-05 |
| 100686 | spring                   | 76.186.80.227   |           8.41538e-05 |
| 100687 | spring                   | 76.187.53.87    |           8.41538e-05 |
| 100688 | spring                   | 76.198.205.9    |           8.41538e-05 |
| 100689 | spring                   | 76.25.49.194    |           8.41538e-05 |
| 100690 | spring                   | 87.158.135.133  |           8.41538e-05 |
| 100691 | spring                   | 88.241.49.202   |           8.41538e-05 |
| 100692 | spring                   | 89.187.164.131  |           8.41538e-05 |
| 100693 | spring                   | 89.187.164.161  |           8.41538e-05 |
| 100694 | spring                   | 89.187.164.166  |           8.41538e-05 |
| 100695 | spring                   | 89.187.164.168  |           8.41538e-05 |
| 100696 | spring                   | 96.40.234.116   |           8.41538e-05 |
| 100697 | spring                   | 96.44.144.194   |           8.41538e-05 |
| 100698 | spring                   | 96.46.160.20    |           8.41538e-05 |
| 100699 | spring                   | 96.8.128.193    |           8.41538e-05 |
| 100700 | spring                   | 96.8.128.205    |           8.41538e-05 |
| 100701 | spring                   | 96.8.130.92     |           8.41538e-05 |
| 100702 | spring                   | 96.8.172.51     |           8.41538e-05 |
| 100703 | spring                   | 96.8.177.78     |           8.41538e-05 |
| 100704 | spring                   | 96.8.193.113    |           8.41538e-05 |
| 100705 | spring                   | 96.8.248.232    |           8.41538e-05 |
| 100706 | spring                   | 96.89.11.37     |           8.41538e-05 |
| 100707 | spring                   | 97.105.101.131  |           8.41538e-05 |
| 100708 | spring                   | 97.105.164.91   |           8.41538e-05 |
| 100709 | spring                   | 97.118.255.115  |           8.41538e-05 |
| 100710 | spring                   | 97.77.126.60    |           8.41538e-05 |
| 100711 | spring                   | 97.77.161.196   |           8.41538e-05 |
| 100712 | spring                   | 97.79.140.198   |           8.41538e-05 |
| 100713 | spring                   | 97.79.141.50    |           8.41538e-05 |
| 100714 | spring                   | 97.98.85.151    |           8.41538e-05 |
| 100715 | spring                   | 98.6.1.177      |           8.41538e-05 |
| 100716 | spring                   | 98.6.151.46     |           8.41538e-05 |
| 100717 | spring                   | 99.126.116.28   |           8.41538e-05 |
| 100718 | spring                   | 99.152.157.19   |           8.41538e-05 |
| 100719 | spring                   | 99.162.240.204  |           8.41538e-05 |
| 100720 | spring                   | 99.203.154.197  |           8.41538e-05 |
| 100721 | spring                   | 99.203.154.20   |           8.41538e-05 |
| 100722 | spring                   | 99.203.26.253   |           8.41538e-05 |
| 100723 | spring                   | 99.203.26.72    |           8.41538e-05 |
| 100724 | spring                   | 99.203.26.81    |           8.41538e-05 |
| 100725 | spring                   | 99.203.27.241   |           8.41538e-05 |
| 100726 | spring                   | 99.203.27.83    |           8.41538e-05 |
| 100727 | spring                   | 99.203.27.96    |           8.41538e-05 |
| 100728 | spring                   | 99.30.48.55     |           8.41538e-05 |
| 100729 | spring                   | 99.48.229.126   |           8.41538e-05 |
| 100730 | spring                   | 99.48.230.93    |           8.41538e-05 |
| 100731 | spring                   | 99.52.75.127    |           8.41538e-05 |
| 100732 | spring                   | 99.65.201.242   |           8.41538e-05 |
| 100733 | spring                   | 99.98.36.168    |           8.41538e-05 |
| 111083 | toc                      | 104.176.91.161  |           5.68473e-05 |
| 111084 | toc                      | 104.180.86.144  |           5.68473e-05 |
| 111085 | toc                      | 104.182.14.234  |           5.68473e-05 |
| 111086 | toc                      | 104.223.91.53   |           5.68473e-05 |
| 111087 | toc                      | 104.237.80.109  |           5.68473e-05 |
| 111088 | toc                      | 104.237.80.12   |           5.68473e-05 |
| 111089 | toc                      | 104.237.80.140  |           5.68473e-05 |
| 111090 | toc                      | 104.237.80.141  |           5.68473e-05 |
| 111091 | toc                      | 104.237.80.151  |           5.68473e-05 |
| 111092 | toc                      | 104.237.80.157  |           5.68473e-05 |
| 111093 | toc                      | 104.237.80.51   |           5.68473e-05 |
| 111094 | toc                      | 104.237.80.53   |           5.68473e-05 |
| 111095 | toc                      | 104.237.80.59   |           5.68473e-05 |
| 111096 | toc                      | 104.237.80.68   |           5.68473e-05 |
| 111097 | toc                      | 104.237.80.83   |           5.68473e-05 |
| 111098 | toc                      | 104.5.159.147   |           5.68473e-05 |
| 111099 | toc                      | 104.53.215.7    |           5.68473e-05 |
| 111100 | toc                      | 104.58.135.163  |           5.68473e-05 |
| 111101 | toc                      | 107.117.168.103 |           5.68473e-05 |
| 111102 | toc                      | 107.128.3.66    |           5.68473e-05 |
| 111103 | toc                      | 107.135.88.8    |           5.68473e-05 |
| 111104 | toc                      | 107.204.53.24   |           5.68473e-05 |
| 111105 | toc                      | 107.242.125.38  |           5.68473e-05 |
| 111106 | toc                      | 107.242.125.50  |           5.68473e-05 |
| 111107 | toc                      | 107.242.125.8   |           5.68473e-05 |
| 111108 | toc                      | 107.77.100.128  |           5.68473e-05 |
| 111109 | toc                      | 107.77.169.1    |           5.68473e-05 |
| 111110 | toc                      | 107.77.169.13   |           5.68473e-05 |
| 111111 | toc                      | 107.77.169.19   |           5.68473e-05 |
| 111112 | toc                      | 107.77.169.57   |           5.68473e-05 |
| 111113 | toc                      | 107.77.169.9    |           5.68473e-05 |
| 111114 | toc                      | 107.77.173.2    |           5.68473e-05 |
| 111115 | toc                      | 107.77.173.4    |           5.68473e-05 |
| 111116 | toc                      | 107.77.189.16   |           5.68473e-05 |
| 111117 | toc                      | 107.77.189.2    |           5.68473e-05 |
| 111118 | toc                      | 107.77.189.21   |           5.68473e-05 |
| 111119 | toc                      | 107.77.189.23   |           5.68473e-05 |
| 111120 | toc                      | 107.77.189.26   |           5.68473e-05 |
| 111121 | toc                      | 107.77.189.27   |           5.68473e-05 |
| 111122 | toc                      | 107.77.189.30   |           5.68473e-05 |
| 111123 | toc                      | 107.77.189.37   |           5.68473e-05 |
| 111124 | toc                      | 107.77.189.49   |           5.68473e-05 |
| 111125 | toc                      | 107.77.189.50   |           5.68473e-05 |
| 111126 | toc                      | 107.77.189.55   |           5.68473e-05 |
| 111127 | toc                      | 107.77.189.62   |           5.68473e-05 |
| 111128 | toc                      | 107.77.189.64   |           5.68473e-05 |
| 111129 | toc                      | 107.77.200.140  |           5.68473e-05 |
| 111130 | toc                      | 107.77.208.117  |           5.68473e-05 |
| 111131 | toc                      | 107.77.208.209  |           5.68473e-05 |
| 111132 | toc                      | 107.77.217.101  |           5.68473e-05 |
| 111133 | toc                      | 107.77.217.127  |           5.68473e-05 |
| 111134 | toc                      | 107.77.217.13   |           5.68473e-05 |
| 111135 | toc                      | 107.77.217.152  |           5.68473e-05 |
| 111136 | toc                      | 107.77.217.168  |           5.68473e-05 |
| 111137 | toc                      | 107.77.217.196  |           5.68473e-05 |
| 111138 | toc                      | 107.77.217.200  |           5.68473e-05 |
| 111139 | toc                      | 107.77.217.21   |           5.68473e-05 |
| 111140 | toc                      | 107.77.217.218  |           5.68473e-05 |
| 111141 | toc                      | 107.77.217.32   |           5.68473e-05 |
| 111142 | toc                      | 107.77.217.4    |           5.68473e-05 |
| 111143 | toc                      | 107.77.217.65   |           5.68473e-05 |
| 111144 | toc                      | 107.77.218.139  |           5.68473e-05 |
| 111145 | toc                      | 107.77.218.199  |           5.68473e-05 |
| 111146 | toc                      | 107.77.218.226  |           5.68473e-05 |
| 111147 | toc                      | 107.77.218.26   |           5.68473e-05 |
| 111148 | toc                      | 107.77.218.58   |           5.68473e-05 |
| 111149 | toc                      | 107.77.218.81   |           5.68473e-05 |
| 111150 | toc                      | 107.77.219.106  |           5.68473e-05 |
| 111151 | toc                      | 107.77.219.13   |           5.68473e-05 |
| 111152 | toc                      | 107.77.219.187  |           5.68473e-05 |
| 111153 | toc                      | 107.77.219.189  |           5.68473e-05 |
| 111154 | toc                      | 107.77.219.217  |           5.68473e-05 |
| 111155 | toc                      | 107.77.219.219  |           5.68473e-05 |
| 111156 | toc                      | 107.77.219.47   |           5.68473e-05 |
| 111157 | toc                      | 107.77.219.64   |           5.68473e-05 |
| 111158 | toc                      | 107.77.219.66   |           5.68473e-05 |
| 111159 | toc                      | 107.77.219.69   |           5.68473e-05 |
| 111160 | toc                      | 107.77.219.96   |           5.68473e-05 |
| 111161 | toc                      | 107.77.220.110  |           5.68473e-05 |
| 111162 | toc                      | 107.77.220.125  |           5.68473e-05 |
| 111163 | toc                      | 107.77.220.149  |           5.68473e-05 |
| 111164 | toc                      | 107.77.220.154  |           5.68473e-05 |
| 111165 | toc                      | 107.77.220.166  |           5.68473e-05 |
| 111166 | toc                      | 107.77.220.176  |           5.68473e-05 |
| 111167 | toc                      | 107.77.220.195  |           5.68473e-05 |
| 111168 | toc                      | 107.77.220.202  |           5.68473e-05 |
| 111169 | toc                      | 107.77.220.231  |           5.68473e-05 |
| 111170 | toc                      | 107.77.220.64   |           5.68473e-05 |
| 111171 | toc                      | 107.77.220.86   |           5.68473e-05 |
| 111172 | toc                      | 107.77.220.90   |           5.68473e-05 |
| 111173 | toc                      | 107.77.221.136  |           5.68473e-05 |
| 111174 | toc                      | 107.77.221.145  |           5.68473e-05 |
| 111175 | toc                      | 107.77.221.152  |           5.68473e-05 |
| 111176 | toc                      | 107.77.221.153  |           5.68473e-05 |
| 111177 | toc                      | 107.77.221.157  |           5.68473e-05 |
| 111178 | toc                      | 107.77.221.173  |           5.68473e-05 |
| 111179 | toc                      | 107.77.221.194  |           5.68473e-05 |
| 111180 | toc                      | 107.77.221.49   |           5.68473e-05 |
| 111181 | toc                      | 107.77.221.58   |           5.68473e-05 |
| 111182 | toc                      | 107.77.221.7    |           5.68473e-05 |
| 111183 | toc                      | 107.77.221.72   |           5.68473e-05 |
| 111184 | toc                      | 107.77.221.76   |           5.68473e-05 |
| 111185 | toc                      | 107.77.221.96   |           5.68473e-05 |
| 111186 | toc                      | 107.77.222.142  |           5.68473e-05 |
| 111187 | toc                      | 107.77.222.17   |           5.68473e-05 |
| 111188 | toc                      | 107.77.222.18   |           5.68473e-05 |
| 111189 | toc                      | 107.77.222.211  |           5.68473e-05 |
| 111190 | toc                      | 107.77.222.37   |           5.68473e-05 |
| 111191 | toc                      | 107.77.222.63   |           5.68473e-05 |
| 111192 | toc                      | 107.77.222.92   |           5.68473e-05 |
| 111193 | toc                      | 107.77.245.7    |           5.68473e-05 |
| 111194 | toc                      | 107.77.66.121   |           5.68473e-05 |
| 111195 | toc                      | 107.77.72.49    |           5.68473e-05 |
| 111196 | toc                      | 107.77.80.29    |           5.68473e-05 |
| 111197 | toc                      | 107.77.80.76    |           5.68473e-05 |
| 111198 | toc                      | 107.77.80.91    |           5.68473e-05 |
| 111199 | toc                      | 108.192.181.20  |           5.68473e-05 |
| 111200 | toc                      | 108.220.101.224 |           5.68473e-05 |
| 111201 | toc                      | 108.222.98.137  |           5.68473e-05 |
| 111202 | toc                      | 12.151.185.131  |           5.68473e-05 |
| 111203 | toc                      | 12.162.134.81   |           5.68473e-05 |
| 111204 | toc                      | 12.197.196.242  |           5.68473e-05 |
| 111205 | toc                      | 12.247.3.50     |           5.68473e-05 |
| 111206 | toc                      | 12.68.243.204   |           5.68473e-05 |
| 111207 | toc                      | 130.45.33.209   |           5.68473e-05 |
| 111208 | toc                      | 130.45.51.161   |           5.68473e-05 |
| 111209 | toc                      | 130.45.59.45    |           5.68473e-05 |
| 111210 | toc                      | 131.194.184.74  |           5.68473e-05 |
| 111211 | toc                      | 131.44.121.130  |           5.68473e-05 |
| 111212 | toc                      | 136.50.0.240    |           5.68473e-05 |
| 111213 | toc                      | 136.50.13.79    |           5.68473e-05 |
| 111214 | toc                      | 136.50.2.13     |           5.68473e-05 |
| 111215 | toc                      | 136.50.29.193   |           5.68473e-05 |
| 111216 | toc                      | 136.50.71.97    |           5.68473e-05 |
| 111217 | toc                      | 136.50.72.144   |           5.68473e-05 |
| 111218 | toc                      | 136.50.79.102   |           5.68473e-05 |
| 111219 | toc                      | 136.50.99.17    |           5.68473e-05 |
| 111220 | toc                      | 137.254.7.172   |           5.68473e-05 |
| 111221 | toc                      | 137.254.7.173   |           5.68473e-05 |
| 111222 | toc                      | 137.254.7.174   |           5.68473e-05 |
| 111223 | toc                      | 137.254.7.178   |           5.68473e-05 |
| 111224 | toc                      | 137.254.7.184   |           5.68473e-05 |
| 111225 | toc                      | 148.66.37.91    |           5.68473e-05 |
| 111226 | toc                      | 154.3.251.115   |           5.68473e-05 |
| 111227 | toc                      | 154.6.28.146    |           5.68473e-05 |
| 111228 | toc                      | 155.94.241.194  |           5.68473e-05 |
| 111229 | toc                      | 162.200.114.251 |           5.68473e-05 |
| 111230 | toc                      | 162.239.219.93  |           5.68473e-05 |
| 111231 | toc                      | 162.245.203.7   |           5.68473e-05 |
| 111232 | toc                      | 162.72.2.198    |           5.68473e-05 |
| 111233 | toc                      | 165.225.48.84   |           5.68473e-05 |
| 111234 | toc                      | 165.225.48.93   |           5.68473e-05 |
| 111235 | toc                      | 165.91.13.35    |           5.68473e-05 |
| 111236 | toc                      | 166.205.190.126 |           5.68473e-05 |
| 111237 | toc                      | 166.205.97.126  |           5.68473e-05 |
| 111238 | toc                      | 166.251.106.84  |           5.68473e-05 |
| 111239 | toc                      | 167.24.24.150   |           5.68473e-05 |
| 111240 | toc                      | 172.108.140.130 |           5.68473e-05 |
| 111241 | toc                      | 172.127.187.185 |           5.68473e-05 |
| 111242 | toc                      | 172.56.14.207   |           5.68473e-05 |
| 111243 | toc                      | 172.56.14.78    |           5.68473e-05 |
| 111244 | toc                      | 172.56.15.73    |           5.68473e-05 |
| 111245 | toc                      | 172.56.6.112    |           5.68473e-05 |
| 111246 | toc                      | 172.56.6.130    |           5.68473e-05 |
| 111247 | toc                      | 172.56.6.20     |           5.68473e-05 |
| 111248 | toc                      | 172.56.6.31     |           5.68473e-05 |
| 111249 | toc                      | 172.56.6.62     |           5.68473e-05 |
| 111250 | toc                      | 172.56.6.79     |           5.68473e-05 |
| 111251 | toc                      | 172.56.7.129    |           5.68473e-05 |
| 111252 | toc                      | 172.56.7.141    |           5.68473e-05 |
| 111253 | toc                      | 172.56.7.151    |           5.68473e-05 |
| 111254 | toc                      | 172.56.7.180    |           5.68473e-05 |
| 111255 | toc                      | 172.56.7.21     |           5.68473e-05 |
| 111256 | toc                      | 172.56.7.232    |           5.68473e-05 |
| 111257 | toc                      | 172.56.7.238    |           5.68473e-05 |
| 111258 | toc                      | 172.56.7.241    |           5.68473e-05 |
| 111259 | toc                      | 172.56.7.6      |           5.68473e-05 |
| 111260 | toc                      | 172.58.101.148  |           5.68473e-05 |
| 111261 | toc                      | 172.58.101.208  |           5.68473e-05 |
| 111262 | toc                      | 172.58.101.37   |           5.68473e-05 |
| 111263 | toc                      | 172.58.101.45   |           5.68473e-05 |
| 111264 | toc                      | 172.58.102.135  |           5.68473e-05 |
| 111265 | toc                      | 172.58.102.136  |           5.68473e-05 |
| 111266 | toc                      | 172.58.102.152  |           5.68473e-05 |
| 111267 | toc                      | 172.58.102.153  |           5.68473e-05 |
| 111268 | toc                      | 172.58.102.155  |           5.68473e-05 |
| 111269 | toc                      | 172.58.102.174  |           5.68473e-05 |
| 111270 | toc                      | 172.58.102.176  |           5.68473e-05 |
| 111271 | toc                      | 172.58.102.183  |           5.68473e-05 |
| 111272 | toc                      | 172.58.102.206  |           5.68473e-05 |
| 111273 | toc                      | 172.58.102.215  |           5.68473e-05 |
| 111274 | toc                      | 172.58.102.218  |           5.68473e-05 |
| 111275 | toc                      | 172.58.102.230  |           5.68473e-05 |
| 111276 | toc                      | 172.58.102.240  |           5.68473e-05 |
| 111277 | toc                      | 172.58.103.143  |           5.68473e-05 |
| 111278 | toc                      | 172.58.103.188  |           5.68473e-05 |
| 111279 | toc                      | 172.58.103.55   |           5.68473e-05 |
| 111280 | toc                      | 172.58.104.239  |           5.68473e-05 |
| 111281 | toc                      | 172.58.106.58   |           5.68473e-05 |
| 111282 | toc                      | 172.58.107.0    |           5.68473e-05 |
| 111283 | toc                      | 172.58.107.194  |           5.68473e-05 |
| 111284 | toc                      | 172.58.107.207  |           5.68473e-05 |
| 111285 | toc                      | 172.58.107.216  |           5.68473e-05 |
| 111286 | toc                      | 172.58.107.223  |           5.68473e-05 |
| 111287 | toc                      | 172.58.107.51   |           5.68473e-05 |
| 111288 | toc                      | 172.58.108.134  |           5.68473e-05 |
| 111289 | toc                      | 172.58.108.214  |           5.68473e-05 |
| 111290 | toc                      | 172.58.108.232  |           5.68473e-05 |
| 111291 | toc                      | 172.58.109.121  |           5.68473e-05 |
| 111292 | toc                      | 172.58.109.15   |           5.68473e-05 |
| 111293 | toc                      | 172.58.109.173  |           5.68473e-05 |
| 111294 | toc                      | 172.58.109.187  |           5.68473e-05 |
| 111295 | toc                      | 172.58.109.222  |           5.68473e-05 |
| 111296 | toc                      | 172.58.109.225  |           5.68473e-05 |
| 111297 | toc                      | 172.58.109.241  |           5.68473e-05 |
| 111298 | toc                      | 172.58.109.245  |           5.68473e-05 |
| 111299 | toc                      | 172.58.109.33   |           5.68473e-05 |
| 111300 | toc                      | 172.58.109.44   |           5.68473e-05 |
| 111301 | toc                      | 172.58.109.48   |           5.68473e-05 |
| 111302 | toc                      | 172.58.109.54   |           5.68473e-05 |
| 111303 | toc                      | 172.58.109.59   |           5.68473e-05 |
| 111304 | toc                      | 172.58.109.64   |           5.68473e-05 |
| 111305 | toc                      | 172.58.109.80   |           5.68473e-05 |
| 111306 | toc                      | 172.58.109.83   |           5.68473e-05 |
| 111307 | toc                      | 172.58.110.181  |           5.68473e-05 |
| 111308 | toc                      | 172.58.110.183  |           5.68473e-05 |
| 111309 | toc                      | 172.58.110.217  |           5.68473e-05 |
| 111310 | toc                      | 172.58.111.153  |           5.68473e-05 |
| 111311 | toc                      | 172.58.111.175  |           5.68473e-05 |
| 111312 | toc                      | 172.58.111.180  |           5.68473e-05 |
| 111313 | toc                      | 172.58.111.192  |           5.68473e-05 |
| 111314 | toc                      | 172.58.111.200  |           5.68473e-05 |
| 111315 | toc                      | 172.58.111.203  |           5.68473e-05 |
| 111316 | toc                      | 172.58.111.204  |           5.68473e-05 |
| 111317 | toc                      | 172.58.111.247  |           5.68473e-05 |
| 111318 | toc                      | 172.58.111.255  |           5.68473e-05 |
| 111319 | toc                      | 172.58.111.26   |           5.68473e-05 |
| 111320 | toc                      | 172.58.204.130  |           5.68473e-05 |
| 111321 | toc                      | 172.58.3.231    |           5.68473e-05 |
| 111322 | toc                      | 172.58.67.77    |           5.68473e-05 |
| 111323 | toc                      | 172.58.68.245   |           5.68473e-05 |
| 111324 | toc                      | 172.58.70.191   |           5.68473e-05 |
| 111325 | toc                      | 172.58.96.187   |           5.68473e-05 |
| 111326 | toc                      | 172.58.96.230   |           5.68473e-05 |
| 111327 | toc                      | 172.58.96.54    |           5.68473e-05 |
| 111328 | toc                      | 172.58.97.120   |           5.68473e-05 |
| 111329 | toc                      | 172.58.97.183   |           5.68473e-05 |
| 111330 | toc                      | 172.58.97.217   |           5.68473e-05 |
| 111331 | toc                      | 172.58.99.11    |           5.68473e-05 |
| 111332 | toc                      | 172.58.99.111   |           5.68473e-05 |
| 111333 | toc                      | 172.58.99.114   |           5.68473e-05 |
| 111334 | toc                      | 172.58.99.124   |           5.68473e-05 |
| 111335 | toc                      | 172.58.99.125   |           5.68473e-05 |
| 111336 | toc                      | 172.58.99.165   |           5.68473e-05 |
| 111337 | toc                      | 172.58.99.191   |           5.68473e-05 |
| 111338 | toc                      | 172.58.99.192   |           5.68473e-05 |
| 111339 | toc                      | 172.58.99.225   |           5.68473e-05 |
| 111340 | toc                      | 172.58.99.53    |           5.68473e-05 |
| 111341 | toc                      | 172.58.99.71    |           5.68473e-05 |
| 111342 | toc                      | 172.85.200.202  |           5.68473e-05 |
| 111343 | toc                      | 173.123.102.174 |           5.68473e-05 |
| 111344 | toc                      | 173.123.21.58   |           5.68473e-05 |
| 111345 | toc                      | 173.123.5.242   |           5.68473e-05 |
| 111346 | toc                      | 173.127.114.36  |           5.68473e-05 |
| 111347 | toc                      | 173.127.47.45   |           5.68473e-05 |
| 111348 | toc                      | 173.149.155.61  |           5.68473e-05 |
| 111349 | toc                      | 173.149.160.96  |           5.68473e-05 |
| 111350 | toc                      | 173.149.234.119 |           5.68473e-05 |
| 111351 | toc                      | 173.149.249.154 |           5.68473e-05 |
| 111352 | toc                      | 173.172.166.90  |           5.68473e-05 |
| 111353 | toc                      | 173.173.98.137  |           5.68473e-05 |
| 111354 | toc                      | 173.174.164.47  |           5.68473e-05 |
| 111355 | toc                      | 173.175.211.194 |           5.68473e-05 |
| 111356 | toc                      | 173.232.241.116 |           5.68473e-05 |
| 111357 | toc                      | 173.239.198.230 |           5.68473e-05 |
| 111358 | toc                      | 173.239.232.103 |           5.68473e-05 |
| 111359 | toc                      | 173.239.232.104 |           5.68473e-05 |
| 111360 | toc                      | 173.239.232.107 |           5.68473e-05 |
| 111361 | toc                      | 173.239.232.108 |           5.68473e-05 |
| 111362 | toc                      | 173.239.232.113 |           5.68473e-05 |
| 111363 | toc                      | 173.239.232.119 |           5.68473e-05 |
| 111364 | toc                      | 173.239.232.12  |           5.68473e-05 |
| 111365 | toc                      | 173.239.232.125 |           5.68473e-05 |
| 111366 | toc                      | 173.239.232.141 |           5.68473e-05 |
| 111367 | toc                      | 173.239.232.142 |           5.68473e-05 |
| 111368 | toc                      | 173.239.232.147 |           5.68473e-05 |
| 111369 | toc                      | 173.239.232.156 |           5.68473e-05 |
| 111370 | toc                      | 173.239.232.20  |           5.68473e-05 |
| 111371 | toc                      | 173.239.232.24  |           5.68473e-05 |
| 111372 | toc                      | 173.239.232.26  |           5.68473e-05 |
| 111373 | toc                      | 173.239.232.29  |           5.68473e-05 |
| 111374 | toc                      | 173.239.232.35  |           5.68473e-05 |
| 111375 | toc                      | 173.239.232.37  |           5.68473e-05 |
| 111376 | toc                      | 173.239.232.4   |           5.68473e-05 |
| 111377 | toc                      | 173.239.232.42  |           5.68473e-05 |
| 111378 | toc                      | 173.239.232.48  |           5.68473e-05 |
| 111379 | toc                      | 173.239.232.5   |           5.68473e-05 |
| 111380 | toc                      | 173.239.232.52  |           5.68473e-05 |
| 111381 | toc                      | 173.239.232.58  |           5.68473e-05 |
| 111382 | toc                      | 173.239.232.60  |           5.68473e-05 |
| 111383 | toc                      | 173.239.232.69  |           5.68473e-05 |
| 111384 | toc                      | 173.239.232.76  |           5.68473e-05 |
| 111385 | toc                      | 173.239.232.88  |           5.68473e-05 |
| 111386 | toc                      | 173.239.232.89  |           5.68473e-05 |
| 111387 | toc                      | 173.239.232.90  |           5.68473e-05 |
| 111388 | toc                      | 174.193.4.63    |           5.68473e-05 |
| 111389 | toc                      | 174.194.1.50    |           5.68473e-05 |
| 111390 | toc                      | 174.196.196.21  |           5.68473e-05 |
| 111391 | toc                      | 174.197.1.205   |           5.68473e-05 |
| 111392 | toc                      | 174.197.1.23    |           5.68473e-05 |
| 111393 | toc                      | 174.197.19.54   |           5.68473e-05 |
| 111394 | toc                      | 174.197.2.198   |           5.68473e-05 |
| 111395 | toc                      | 174.206.18.197  |           5.68473e-05 |
| 111396 | toc                      | 174.206.2.148   |           5.68473e-05 |
| 111397 | toc                      | 174.207.0.103   |           5.68473e-05 |
| 111398 | toc                      | 174.207.10.43   |           5.68473e-05 |
| 111399 | toc                      | 174.207.11.139  |           5.68473e-05 |
| 111400 | toc                      | 174.207.14.251  |           5.68473e-05 |
| 111401 | toc                      | 174.207.15.123  |           5.68473e-05 |
| 111402 | toc                      | 174.207.15.137  |           5.68473e-05 |
| 111403 | toc                      | 174.207.15.176  |           5.68473e-05 |
| 111404 | toc                      | 174.207.15.228  |           5.68473e-05 |
| 111405 | toc                      | 174.207.16.1    |           5.68473e-05 |
| 111406 | toc                      | 174.207.16.65   |           5.68473e-05 |
| 111407 | toc                      | 174.207.17.141  |           5.68473e-05 |
| 111408 | toc                      | 174.207.17.170  |           5.68473e-05 |
| 111409 | toc                      | 174.207.17.181  |           5.68473e-05 |
| 111410 | toc                      | 174.207.18.173  |           5.68473e-05 |
| 111411 | toc                      | 174.207.2.122   |           5.68473e-05 |
| 111412 | toc                      | 174.207.22.2    |           5.68473e-05 |
| 111413 | toc                      | 174.207.23.109  |           5.68473e-05 |
| 111414 | toc                      | 174.207.23.167  |           5.68473e-05 |
| 111415 | toc                      | 174.207.23.199  |           5.68473e-05 |
| 111416 | toc                      | 174.207.23.41   |           5.68473e-05 |
| 111417 | toc                      | 174.207.24.231  |           5.68473e-05 |
| 111418 | toc                      | 174.207.24.245  |           5.68473e-05 |
| 111419 | toc                      | 174.207.24.42   |           5.68473e-05 |
| 111420 | toc                      | 174.207.25.219  |           5.68473e-05 |
| 111421 | toc                      | 174.207.25.229  |           5.68473e-05 |
| 111422 | toc                      | 174.207.26.100  |           5.68473e-05 |
| 111423 | toc                      | 174.207.26.73   |           5.68473e-05 |
| 111424 | toc                      | 174.207.3.54    |           5.68473e-05 |
| 111425 | toc                      | 174.207.34.2    |           5.68473e-05 |
| 111426 | toc                      | 174.207.5.143   |           5.68473e-05 |
| 111427 | toc                      | 174.207.6.56    |           5.68473e-05 |
| 111428 | toc                      | 174.207.8.123   |           5.68473e-05 |
| 111429 | toc                      | 174.207.8.153   |           5.68473e-05 |
| 111430 | toc                      | 174.246.129.39  |           5.68473e-05 |
| 111431 | toc                      | 174.246.193.161 |           5.68473e-05 |
| 111432 | toc                      | 174.246.197.129 |           5.68473e-05 |
| 111433 | toc                      | 174.246.198.28  |           5.68473e-05 |
| 111434 | toc                      | 174.249.45.199  |           5.68473e-05 |
| 111435 | toc                      | 174.255.132.169 |           5.68473e-05 |
| 111436 | toc                      | 174.255.134.225 |           5.68473e-05 |
| 111437 | toc                      | 174.255.146.154 |           5.68473e-05 |
| 111438 | toc                      | 174.48.18.249   |           5.68473e-05 |
| 111439 | toc                      | 174.67.9.84     |           5.68473e-05 |
| 111440 | toc                      | 181.214.227.23  |           5.68473e-05 |
| 111441 | toc                      | 181.214.227.47  |           5.68473e-05 |
| 111442 | toc                      | 184.203.144.40  |           5.68473e-05 |
| 111443 | toc                      | 184.203.172.25  |           5.68473e-05 |
| 111444 | toc                      | 184.203.189.129 |           5.68473e-05 |
| 111445 | toc                      | 184.203.6.32    |           5.68473e-05 |
| 111446 | toc                      | 184.226.22.205  |           5.68473e-05 |
| 111447 | toc                      | 184.226.93.17   |           5.68473e-05 |
| 111448 | toc                      | 187.237.25.33   |           5.68473e-05 |
| 111449 | toc                      | 191.96.67.5     |           5.68473e-05 |
| 111450 | toc                      | 193.148.18.75   |           5.68473e-05 |
| 111451 | toc                      | 193.56.117.80   |           5.68473e-05 |
| 111452 | toc                      | 199.193.220.210 |           5.68473e-05 |
| 111453 | toc                      | 200.68.128.26   |           5.68473e-05 |
| 111454 | toc                      | 201.174.135.51  |           5.68473e-05 |
| 111455 | toc                      | 205.251.148.74  |           5.68473e-05 |
| 111456 | toc                      | 205.251.150.218 |           5.68473e-05 |
| 111457 | toc                      | 205.251.151.18  |           5.68473e-05 |
| 111458 | toc                      | 206.214.78.35   |           5.68473e-05 |
| 111459 | toc                      | 207.119.3.149   |           5.68473e-05 |
| 111460 | toc                      | 207.239.160.254 |           5.68473e-05 |
| 111461 | toc                      | 208.54.83.163   |           5.68473e-05 |
| 111462 | toc                      | 208.54.83.194   |           5.68473e-05 |
| 111463 | toc                      | 208.54.83.222   |           5.68473e-05 |
| 111464 | toc                      | 208.54.86.183   |           5.68473e-05 |
| 111465 | toc                      | 208.54.86.247   |           5.68473e-05 |
| 111466 | toc                      | 208.73.111.171  |           5.68473e-05 |
| 111467 | toc                      | 209.107.189.26  |           5.68473e-05 |
| 111468 | toc                      | 209.133.66.84   |           5.68473e-05 |
| 111469 | toc                      | 209.135.132.136 |           5.68473e-05 |
| 111470 | toc                      | 216.162.109.195 |           5.68473e-05 |
| 111471 | toc                      | 216.177.187.98  |           5.68473e-05 |
| 111472 | toc                      | 216.54.250.134  |           5.68473e-05 |
| 111473 | toc                      | 217.167.133.44  |           5.68473e-05 |
| 111474 | toc                      | 23.117.144.200  |           5.68473e-05 |
| 111475 | toc                      | 23.30.51.53     |           5.68473e-05 |
| 111476 | toc                      | 24.153.217.26   |           5.68473e-05 |
| 111477 | toc                      | 24.155.24.134   |           5.68473e-05 |
| 111478 | toc                      | 24.160.137.86   |           5.68473e-05 |
| 111479 | toc                      | 24.160.145.169  |           5.68473e-05 |
| 111480 | toc                      | 24.160.4.6      |           5.68473e-05 |
| 111481 | toc                      | 24.167.9.61     |           5.68473e-05 |
| 111482 | toc                      | 24.175.229.211  |           5.68473e-05 |
| 111483 | toc                      | 24.242.93.131   |           5.68473e-05 |
| 111484 | toc                      | 24.243.19.100   |           5.68473e-05 |
| 111485 | toc                      | 24.243.38.118   |           5.68473e-05 |
| 111486 | toc                      | 24.243.70.180   |           5.68473e-05 |
| 111487 | toc                      | 24.26.232.177   |           5.68473e-05 |
| 111488 | toc                      | 24.26.251.39    |           5.68473e-05 |
| 111489 | toc                      | 24.28.152.103   |           5.68473e-05 |
| 111490 | toc                      | 24.28.70.46     |           5.68473e-05 |
| 111491 | toc                      | 3.209.254.178   |           5.68473e-05 |
| 111492 | toc                      | 4.15.231.83     |           5.68473e-05 |
| 111493 | toc                      | 45.131.192.121  |           5.68473e-05 |
| 111494 | toc                      | 45.131.192.151  |           5.68473e-05 |
| 111495 | toc                      | 45.131.192.211  |           5.68473e-05 |
| 111496 | toc                      | 45.131.192.85   |           5.68473e-05 |
| 111497 | toc                      | 45.248.77.171   |           5.68473e-05 |
| 111498 | toc                      | 45.31.212.143   |           5.68473e-05 |
| 111499 | toc                      | 45.56.151.8     |           5.68473e-05 |
| 111500 | toc                      | 45.56.151.93    |           5.68473e-05 |
| 111501 | toc                      | 45.83.89.58     |           5.68473e-05 |
| 111502 | toc                      | 46.218.181.194  |           5.68473e-05 |
| 111503 | toc                      | 47.183.134.235  |           5.68473e-05 |
| 111504 | toc                      | 47.183.138.169  |           5.68473e-05 |
| 111505 | toc                      | 47.187.240.67   |           5.68473e-05 |
| 111506 | toc                      | 50.237.25.227   |           5.68473e-05 |
| 111507 | toc                      | 50.84.244.130   |           5.68473e-05 |
| 111508 | toc                      | 51.9.81.63      |           5.68473e-05 |
| 111509 | toc                      | 52.144.111.133  |           5.68473e-05 |
| 111510 | toc                      | 56.0.143.23     |           5.68473e-05 |
| 111511 | toc                      | 56.0.143.25     |           5.68473e-05 |
| 111512 | toc                      | 63.240.174.244  |           5.68473e-05 |
| 111513 | toc                      | 63.246.163.161  |           5.68473e-05 |
| 111514 | toc                      | 64.134.148.108  |           5.68473e-05 |
| 111515 | toc                      | 64.183.232.242  |           5.68473e-05 |
| 111516 | toc                      | 64.233.172.173  |           5.68473e-05 |
| 111517 | toc                      | 64.250.9.218    |           5.68473e-05 |
| 111518 | toc                      | 64.9.239.13     |           5.68473e-05 |
| 111519 | toc                      | 64.9.239.21     |           5.68473e-05 |
| 111520 | toc                      | 64.9.239.30     |           5.68473e-05 |
| 111521 | toc                      | 66.194.160.106  |           5.68473e-05 |
| 111522 | toc                      | 66.25.130.242   |           5.68473e-05 |
| 111523 | toc                      | 66.25.163.122   |           5.68473e-05 |
| 111524 | toc                      | 66.25.169.158   |           5.68473e-05 |
| 111525 | toc                      | 66.25.173.125   |           5.68473e-05 |
| 111526 | toc                      | 66.68.53.12     |           5.68473e-05 |
| 111527 | toc                      | 66.69.48.5      |           5.68473e-05 |
| 111528 | toc                      | 66.69.65.3      |           5.68473e-05 |
| 111529 | toc                      | 67.10.171.227   |           5.68473e-05 |
| 111530 | toc                      | 67.10.183.5     |           5.68473e-05 |
| 111531 | toc                      | 67.10.191.167   |           5.68473e-05 |
| 111532 | toc                      | 67.10.213.134   |           5.68473e-05 |
| 111533 | toc                      | 67.11.120.84    |           5.68473e-05 |
| 111534 | toc                      | 67.11.212.182   |           5.68473e-05 |
| 111535 | toc                      | 67.11.33.106    |           5.68473e-05 |
| 111536 | toc                      | 67.11.39.16     |           5.68473e-05 |
| 111537 | toc                      | 67.11.56.110    |           5.68473e-05 |
| 111538 | toc                      | 67.11.59.112    |           5.68473e-05 |
| 111539 | toc                      | 67.11.97.204    |           5.68473e-05 |
| 111540 | toc                      | 67.11.99.52     |           5.68473e-05 |
| 111541 | toc                      | 67.54.151.131   |           5.68473e-05 |
| 111542 | toc                      | 67.79.20.162    |           5.68473e-05 |
| 111543 | toc                      | 67.79.62.147    |           5.68473e-05 |
| 111544 | toc                      | 67.79.98.186    |           5.68473e-05 |
| 111545 | toc                      | 68.201.78.226   |           5.68473e-05 |
| 111546 | toc                      | 68.203.215.174  |           5.68473e-05 |
| 111547 | toc                      | 68.206.198.66   |           5.68473e-05 |
| 111548 | toc                      | 69.148.0.213    |           5.68473e-05 |
| 111549 | toc                      | 69.213.15.192   |           5.68473e-05 |
| 111550 | toc                      | 69.232.100.94   |           5.68473e-05 |
| 111551 | toc                      | 69.232.49.24    |           5.68473e-05 |
| 111552 | toc                      | 70.112.164.11   |           5.68473e-05 |
| 111553 | toc                      | 70.112.179.142  |           5.68473e-05 |
| 111554 | toc                      | 70.112.212.15   |           5.68473e-05 |
| 111555 | toc                      | 70.114.56.233   |           5.68473e-05 |
| 111556 | toc                      | 70.117.13.151   |           5.68473e-05 |
| 111557 | toc                      | 70.117.148.31   |           5.68473e-05 |
| 111558 | toc                      | 70.117.19.13    |           5.68473e-05 |
| 111559 | toc                      | 70.117.5.148    |           5.68473e-05 |
| 111560 | toc                      | 70.117.71.59    |           5.68473e-05 |
| 111561 | toc                      | 70.120.24.167   |           5.68473e-05 |
| 111562 | toc                      | 70.120.28.81    |           5.68473e-05 |
| 111563 | toc                      | 70.120.43.13    |           5.68473e-05 |
| 111564 | toc                      | 70.120.93.125   |           5.68473e-05 |
| 111565 | toc                      | 70.121.187.122  |           5.68473e-05 |
| 111566 | toc                      | 70.121.200.203  |           5.68473e-05 |
| 111567 | toc                      | 70.121.215.53   |           5.68473e-05 |
| 111568 | toc                      | 70.123.164.223  |           5.68473e-05 |
| 111569 | toc                      | 70.123.169.229  |           5.68473e-05 |
| 111570 | toc                      | 70.130.124.250  |           5.68473e-05 |
| 111571 | toc                      | 70.133.251.114  |           5.68473e-05 |
| 111572 | toc                      | 70.94.150.75    |           5.68473e-05 |
| 111573 | toc                      | 70.94.161.227   |           5.68473e-05 |
| 111574 | toc                      | 70.94.165.250   |           5.68473e-05 |
| 111575 | toc                      | 70.94.181.28    |           5.68473e-05 |
| 111576 | toc                      | 71.135.71.73    |           5.68473e-05 |
| 111577 | toc                      | 71.137.144.185  |           5.68473e-05 |
| 111578 | toc                      | 71.150.218.181  |           5.68473e-05 |
| 111579 | toc                      | 71.158.155.118  |           5.68473e-05 |
| 111580 | toc                      | 71.159.247.210  |           5.68473e-05 |
| 111581 | toc                      | 71.221.194.87   |           5.68473e-05 |
| 111582 | toc                      | 71.221.202.248  |           5.68473e-05 |
| 111583 | toc                      | 71.221.210.29   |           5.68473e-05 |
| 111584 | toc                      | 71.221.235.149  |           5.68473e-05 |
| 111585 | toc                      | 71.221.237.129  |           5.68473e-05 |
| 111586 | toc                      | 71.42.105.147   |           5.68473e-05 |
| 111587 | toc                      | 71.78.167.131   |           5.68473e-05 |
| 111588 | toc                      | 71.81.176.46    |           5.68473e-05 |
| 111589 | toc                      | 72.169.80.108   |           5.68473e-05 |
| 111590 | toc                      | 72.177.15.120   |           5.68473e-05 |
| 111591 | toc                      | 72.177.185.29   |           5.68473e-05 |
| 111592 | toc                      | 72.177.210.199  |           5.68473e-05 |
| 111593 | toc                      | 72.177.226.58   |           5.68473e-05 |
| 111594 | toc                      | 72.177.241.50   |           5.68473e-05 |
| 111595 | toc                      | 72.178.188.133  |           5.68473e-05 |
| 111596 | toc                      | 72.179.167.36   |           5.68473e-05 |
| 111597 | toc                      | 72.179.183.68   |           5.68473e-05 |
| 111598 | toc                      | 72.179.189.42   |           5.68473e-05 |
| 111599 | toc                      | 72.180.46.191   |           5.68473e-05 |
| 111600 | toc                      | 72.181.104.64   |           5.68473e-05 |
| 111601 | toc                      | 72.181.105.69   |           5.68473e-05 |
| 111602 | toc                      | 72.181.106.34   |           5.68473e-05 |
| 111603 | toc                      | 72.181.120.147  |           5.68473e-05 |
| 111604 | toc                      | 72.181.125.198  |           5.68473e-05 |
| 111605 | toc                      | 72.181.145.92   |           5.68473e-05 |
| 111606 | toc                      | 72.181.99.110   |           5.68473e-05 |
| 111607 | toc                      | 72.181.99.132   |           5.68473e-05 |
| 111608 | toc                      | 72.181.99.88    |           5.68473e-05 |
| 111609 | toc                      | 72.190.151.50   |           5.68473e-05 |
| 111610 | toc                      | 72.190.171.65   |           5.68473e-05 |
| 111611 | toc                      | 72.190.172.10   |           5.68473e-05 |
| 111612 | toc                      | 72.190.175.190  |           5.68473e-05 |
| 111613 | toc                      | 72.190.178.21   |           5.68473e-05 |
| 111614 | toc                      | 72.190.237.180  |           5.68473e-05 |
| 111615 | toc                      | 72.191.140.150  |           5.68473e-05 |
| 111616 | toc                      | 72.191.26.144   |           5.68473e-05 |
| 111617 | toc                      | 72.191.28.232   |           5.68473e-05 |
| 111618 | toc                      | 72.191.50.176   |           5.68473e-05 |
| 111619 | toc                      | 72.193.254.235  |           5.68473e-05 |
| 111620 | toc                      | 72.205.214.27   |           5.68473e-05 |
| 111621 | toc                      | 72.21.196.64    |           5.68473e-05 |
| 111622 | toc                      | 72.21.196.67    |           5.68473e-05 |
| 111623 | toc                      | 72.26.31.59     |           5.68473e-05 |
| 111624 | toc                      | 73.206.101.120  |           5.68473e-05 |
| 111625 | toc                      | 73.76.119.133   |           5.68473e-05 |
| 111626 | toc                      | 74.81.88.26     |           5.68473e-05 |
| 111627 | toc                      | 74.81.88.74     |           5.68473e-05 |
| 111628 | toc                      | 75.15.242.245   |           5.68473e-05 |
| 111629 | toc                      | 75.177.248.24   |           5.68473e-05 |
| 111630 | toc                      | 75.35.140.23    |           5.68473e-05 |
| 111631 | toc                      | 75.57.25.192    |           5.68473e-05 |
| 111632 | toc                      | 75.87.28.58     |           5.68473e-05 |
| 111633 | toc                      | 76.182.226.251  |           5.68473e-05 |
| 111634 | toc                      | 76.185.121.198  |           5.68473e-05 |
| 111635 | toc                      | 76.185.135.244  |           5.68473e-05 |
| 111636 | toc                      | 76.185.136.75   |           5.68473e-05 |
| 111637 | toc                      | 76.185.145.231  |           5.68473e-05 |
| 111638 | toc                      | 76.185.149.233  |           5.68473e-05 |
| 111639 | toc                      | 76.185.173.254  |           5.68473e-05 |
| 111640 | toc                      | 76.185.182.104  |           5.68473e-05 |
| 111641 | toc                      | 76.185.227.82   |           5.68473e-05 |
| 111642 | toc                      | 76.185.234.111  |           5.68473e-05 |
| 111643 | toc                      | 76.186.2.234    |           5.68473e-05 |
| 111644 | toc                      | 76.186.26.141   |           5.68473e-05 |
| 111645 | toc                      | 76.187.101.165  |           5.68473e-05 |
| 111646 | toc                      | 76.187.107.183  |           5.68473e-05 |
| 111647 | toc                      | 76.187.164.109  |           5.68473e-05 |
| 111648 | toc                      | 76.218.3.43     |           5.68473e-05 |
| 111649 | toc                      | 76.237.206.217  |           5.68473e-05 |
| 111650 | toc                      | 89.187.164.130  |           5.68473e-05 |
| 111651 | toc                      | 89.187.164.131  |           5.68473e-05 |
| 111652 | toc                      | 89.187.164.133  |           5.68473e-05 |
| 111653 | toc                      | 89.187.164.146  |           5.68473e-05 |
| 111654 | toc                      | 89.187.164.150  |           5.68473e-05 |
| 111655 | toc                      | 89.187.164.76   |           5.68473e-05 |
| 111656 | toc                      | 89.187.164.77   |           5.68473e-05 |
| 111657 | toc                      | 89.187.164.78   |           5.68473e-05 |
| 111658 | toc                      | 89.187.164.79   |           5.68473e-05 |
| 111659 | toc                      | 89.187.175.174  |           5.68473e-05 |
| 111660 | toc                      | 89.81.166.143   |           5.68473e-05 |
| 111661 | toc                      | 92.38.148.56    |           5.68473e-05 |
| 111662 | toc                      | 96.46.160.14    |           5.68473e-05 |
| 111663 | toc                      | 96.8.172.51     |           5.68473e-05 |
| 111664 | toc                      | 96.8.177.78     |           5.68473e-05 |
| 111665 | toc                      | 96.8.177.90     |           5.68473e-05 |
| 111666 | toc                      | 96.8.178.150    |           5.68473e-05 |
| 111667 | toc                      | 96.8.178.180    |           5.68473e-05 |
| 111668 | toc                      | 96.8.178.42     |           5.68473e-05 |
| 111669 | toc                      | 97.100.13.134   |           5.68473e-05 |
| 111670 | toc                      | 97.103.225.152  |           5.68473e-05 |
| 111671 | toc                      | 97.105.19.60    |           5.68473e-05 |
| 111672 | toc                      | 97.105.54.251   |           5.68473e-05 |
| 111673 | toc                      | 97.105.81.147   |           5.68473e-05 |
| 111674 | toc                      | 97.117.115.55   |           5.68473e-05 |
| 111675 | toc                      | 97.123.183.37   |           5.68473e-05 |
| 111676 | toc                      | 97.77.214.243   |           5.68473e-05 |
| 111677 | toc                      | 98.6.151.46     |           5.68473e-05 |
| 111678 | toc                      | 98.6.204.195    |           5.68473e-05 |
| 111679 | toc                      | 98.6.44.42      |           5.68473e-05 |
| 111680 | toc                      | 99.132.128.74   |           5.68473e-05 |
| 111681 | toc                      | 99.132.129.169  |           5.68473e-05 |
| 111682 | toc                      | 99.137.94.0     |           5.68473e-05 |
| 111683 | toc                      | 99.151.201.25   |           5.68473e-05 |
| 111684 | toc                      | 99.154.3.0      |           5.68473e-05 |
| 111685 | toc                      | 99.158.247.126  |           5.68473e-05 |
| 111686 | toc                      | 99.158.247.252  |           5.68473e-05 |
| 111687 | toc                      | 99.158.248.205  |           5.68473e-05 |
| 111688 | toc                      | 99.184.61.243   |           5.68473e-05 |
| 111689 | toc                      | 99.184.63.30    |           5.68473e-05 |
| 111690 | toc                      | 99.203.154.149  |           5.68473e-05 |
| 111691 | toc                      | 99.203.154.172  |           5.68473e-05 |
| 111692 | toc                      | 99.203.154.182  |           5.68473e-05 |
| 111693 | toc                      | 99.203.154.89   |           5.68473e-05 |
| 111694 | toc                      | 99.203.155.0    |           5.68473e-05 |
| 111695 | toc                      | 99.203.155.134  |           5.68473e-05 |
| 111696 | toc                      | 99.203.155.161  |           5.68473e-05 |
| 111697 | toc                      | 99.203.155.185  |           5.68473e-05 |
| 111698 | toc                      | 99.203.212.171  |           5.68473e-05 |
| 111699 | toc                      | 99.203.212.200  |           5.68473e-05 |
| 111700 | toc                      | 99.203.213.3    |           5.68473e-05 |
| 111701 | toc                      | 99.203.22.172   |           5.68473e-05 |
| 111702 | toc                      | 99.203.22.39    |           5.68473e-05 |
| 111703 | toc                      | 99.203.23.196   |           5.68473e-05 |
| 111704 | toc                      | 99.203.26.115   |           5.68473e-05 |
| 111705 | toc                      | 99.203.26.138   |           5.68473e-05 |
| 111706 | toc                      | 99.203.26.146   |           5.68473e-05 |
| 111707 | toc                      | 99.203.26.158   |           5.68473e-05 |
| 111708 | toc                      | 99.203.26.188   |           5.68473e-05 |
| 111709 | toc                      | 99.203.26.190   |           5.68473e-05 |
| 111710 | toc                      | 99.203.26.208   |           5.68473e-05 |
| 111711 | toc                      | 99.203.26.48    |           5.68473e-05 |
| 111712 | toc                      | 99.203.26.5     |           5.68473e-05 |
| 111713 | toc                      | 99.203.26.55    |           5.68473e-05 |
| 111714 | toc                      | 99.203.26.98    |           5.68473e-05 |
| 111715 | toc                      | 99.203.27.1     |           5.68473e-05 |
| 111716 | toc                      | 99.203.27.131   |           5.68473e-05 |
| 111717 | toc                      | 99.203.27.140   |           5.68473e-05 |
| 111718 | toc                      | 99.203.27.182   |           5.68473e-05 |
| 111719 | toc                      | 99.203.27.218   |           5.68473e-05 |
| 111720 | toc                      | 99.203.27.226   |           5.68473e-05 |
| 111721 | toc                      | 99.203.27.45    |           5.68473e-05 |
| 111722 | toc                      | 99.203.90.115   |           5.68473e-05 |
| 111723 | toc                      | 99.203.90.129   |           5.68473e-05 |
| 111724 | toc                      | 99.203.90.253   |           5.68473e-05 |
| 111725 | toc                      | 99.203.90.71    |           5.68473e-05 |
| 111726 | toc                      | 99.203.90.84    |           5.68473e-05 |
| 111727 | toc                      | 99.203.91.132   |           5.68473e-05 |
| 111728 | toc                      | 99.203.91.76    |           5.68473e-05 |
| 111729 | toc                      | 99.203.91.97    |           5.68473e-05 |
| 111730 | toc                      | 99.76.235.191   |           5.68473e-05 |
| 111731 | toc                      | 99.97.208.154   |           5.68473e-05 |

#### User_Id given End_Date

| user_id   | end_date   | proba_user_id_given_end_date   |
|-----------|------------|--------------------------------|

#### Cohort_Id given Name

| cohort_id   | name   | proba_cohort_id_given_name   |
|-------------|--------|------------------------------|

#### Time given Slack

| time   | slack   | proba_time_given_slack   |
|--------|---------|--------------------------|

#### Cohort_Id given Slack

| cohort_id   | slack   | proba_cohort_id_given_slack   |
|-------------|---------|-------------------------------|

#### Time given End_Date

| time   | end_date   | proba_time_given_end_date   |
|--------|------------|-----------------------------|

#### Cohort_Id given End_Date

| cohort_id   | end_date   | proba_cohort_id_given_end_date   |
|-------------|------------|----------------------------------|

#### Ip given Program_Id

| ip   | program_id   | proba_ip_given_program_id   |
|------|--------------|-----------------------------|

#### Cohort_Id given Ip

|      |   cohort_id | ip              |   proba_cohort_id_given_ip |
|-----:|------------:|:----------------|---------------------------:|
|  753 |          22 | 107.77.217.101  |                9.70058e-05 |
|  754 |          22 | 107.77.80.125   |                9.70058e-05 |
|  755 |          22 | 12.27.152.137   |                9.70058e-05 |
|  756 |          22 | 137.254.7.176   |                9.70058e-05 |
|  757 |          22 | 166.251.109.241 |                9.70058e-05 |
|  758 |          22 | 167.24.104.150  |                9.70058e-05 |
|  759 |          22 | 172.58.100.107  |                9.70058e-05 |
|  760 |          22 | 172.58.70.146   |                9.70058e-05 |
|  761 |          22 | 174.207.7.167   |                9.70058e-05 |
|  762 |          22 | 196.52.2.30     |                9.70058e-05 |
|  763 |          22 | 72.190.180.197  |                9.70058e-05 |
|  764 |          22 | 8.31.0.128      |                9.70058e-05 |
|  765 |          22 | 89.81.166.143   |                9.70058e-05 |
|  766 |          22 | 97.77.126.60    |                9.70058e-05 |
|  767 |          22 | 104.129.196.183 |                6.46705e-05 |
|  768 |          22 | 107.77.100.45   |                6.46705e-05 |
|  769 |          22 | 107.77.218.40   |                6.46705e-05 |
|  770 |          22 | 107.77.66.96    |                6.46705e-05 |
|  771 |          22 | 107.77.72.112   |                6.46705e-05 |
|  772 |          22 | 107.77.80.57    |                6.46705e-05 |
|  773 |          22 | 107.77.80.82    |                6.46705e-05 |
|  774 |          22 | 166.251.112.88  |                6.46705e-05 |
|  775 |          22 | 174.207.0.15    |                6.46705e-05 |
|  776 |          22 | 174.207.13.44   |                6.46705e-05 |
|  777 |          22 | 174.207.3.185   |                6.46705e-05 |
|  778 |          22 | 192.171.117.234 |                6.46705e-05 |
|  779 |          22 | 209.205.122.182 |                6.46705e-05 |
|  780 |          22 | 70.125.151.73   |                6.46705e-05 |
|  781 |          22 | 72.178.96.177   |                6.46705e-05 |
|  782 |          22 | 72.181.121.192  |                6.46705e-05 |
|  783 |          22 | 90.85.119.75    |                6.46705e-05 |
|  784 |          22 | 98.6.222.42     |                6.46705e-05 |
|  785 |          22 | 99.203.155.147  |                6.46705e-05 |
|  786 |          22 | 99.203.26.185   |                6.46705e-05 |
|  787 |          22 | 99.203.26.200   |                6.46705e-05 |
|  788 |          22 | 104.184.220.38  |                3.23353e-05 |
|  789 |          22 | 107.181.165.202 |                3.23353e-05 |
|  790 |          22 | 107.77.100.113  |                3.23353e-05 |
|  791 |          22 | 107.77.100.84   |                3.23353e-05 |
|  792 |          22 | 107.77.100.91   |                3.23353e-05 |
|  793 |          22 | 107.77.64.27    |                3.23353e-05 |
|  794 |          22 | 107.77.64.50    |                3.23353e-05 |
|  795 |          22 | 107.77.64.51    |                3.23353e-05 |
|  796 |          22 | 107.77.64.80    |                3.23353e-05 |
|  797 |          22 | 107.77.66.47    |                3.23353e-05 |
|  798 |          22 | 107.77.72.97    |                3.23353e-05 |
|  799 |          22 | 107.77.80.20    |                3.23353e-05 |
|  800 |          22 | 137.254.7.169   |                3.23353e-05 |
|  801 |          22 | 172.58.111.139  |                3.23353e-05 |
|  802 |          22 | 172.58.111.172  |                3.23353e-05 |
|  803 |          22 | 172.58.67.186   |                3.23353e-05 |
|  804 |          22 | 172.58.96.172   |                3.23353e-05 |
|  805 |          22 | 173.173.100.232 |                3.23353e-05 |
|  806 |          22 | 174.207.0.177   |                3.23353e-05 |
|  807 |          22 | 174.207.12.44   |                3.23353e-05 |
|  808 |          22 | 174.207.5.177   |                3.23353e-05 |
|  809 |          22 | 189.146.46.149  |                3.23353e-05 |
|  810 |          22 | 196.52.2.49     |                3.23353e-05 |
|  811 |          22 | 196.52.2.70     |                3.23353e-05 |
|  812 |          22 | 209.205.122.186 |                3.23353e-05 |
|  813 |          22 | 24.243.34.38    |                3.23353e-05 |
|  814 |          22 | 24.27.227.0     |                3.23353e-05 |
|  815 |          22 | 24.27.227.1     |                3.23353e-05 |
|  816 |          22 | 37.60.189.18    |                3.23353e-05 |
|  817 |          22 | 64.132.225.203  |                3.23353e-05 |
|  818 |          22 | 66.69.68.137    |                3.23353e-05 |
|  819 |          22 | 70.123.229.111  |                3.23353e-05 |
|  820 |          22 | 71.42.245.250   |                3.23353e-05 |
|  821 |          22 | 71.78.185.26    |                3.23353e-05 |
|  822 |          22 | 76.214.156.188  |                3.23353e-05 |
|  823 |          22 | 96.27.161.213   |                3.23353e-05 |
|  824 |          22 | 97.77.127.220   |                3.23353e-05 |
|  825 |          22 | 99.203.154.83   |                3.23353e-05 |
|  826 |          22 | 99.203.154.93   |                3.23353e-05 |
|  827 |          22 | 99.203.26.93    |                3.23353e-05 |
| 1009 |          23 | 108.178.64.170  |                7.00918e-05 |
| 1010 |          23 | 12.247.3.42     |                7.00918e-05 |
| 1011 |          23 | 166.137.143.91  |                7.00918e-05 |
| 1012 |          23 | 172.58.101.148  |                7.00918e-05 |
| 1013 |          23 | 172.58.108.182  |                7.00918e-05 |
| 1014 |          23 | 172.58.109.183  |                7.00918e-05 |
| 1015 |          23 | 172.58.109.6    |                7.00918e-05 |
| 1016 |          23 | 172.58.110.34   |                7.00918e-05 |
| 1017 |          23 | 172.58.111.204  |                7.00918e-05 |
| 1018 |          23 | 172.58.67.181   |                7.00918e-05 |
| 1019 |          23 | 172.58.99.59    |                7.00918e-05 |
| 1020 |          23 | 174.128.180.20  |                7.00918e-05 |
| 1021 |          23 | 174.128.181.119 |                7.00918e-05 |
| 1022 |          23 | 174.255.132.169 |                7.00918e-05 |
| 1023 |          23 | 174.255.137.106 |                7.00918e-05 |
| 1024 |          23 | 64.233.172.173  |                7.00918e-05 |
| 1025 |          23 | 66.102.6.173    |                7.00918e-05 |
| 1026 |          23 | 67.11.199.195   |                7.00918e-05 |
| 1027 |          23 | 68.206.249.115  |                7.00918e-05 |
| 1028 |          23 | 72.190.185.235  |                7.00918e-05 |
| 1029 |          23 | 75.54.32.23     |                7.00918e-05 |
| 1030 |          23 | 99.203.26.80    |                7.00918e-05 |
| 1031 |          23 | 99.48.229.126   |                7.00918e-05 |
| 1032 |          23 | 104.156.210.199 |                3.50459e-05 |
| 1033 |          23 | 107.77.105.20   |                3.50459e-05 |
| 1034 |          23 | 107.77.105.36   |                3.50459e-05 |
| 1035 |          23 | 107.77.219.202  |                3.50459e-05 |
| 1036 |          23 | 107.77.66.54    |                3.50459e-05 |
| 1037 |          23 | 107.77.72.63    |                3.50459e-05 |
| 1038 |          23 | 107.77.80.32    |                3.50459e-05 |
| 1039 |          23 | 130.45.19.88    |                3.50459e-05 |
| 1040 |          23 | 167.24.104.150  |                3.50459e-05 |
| 1041 |          23 | 172.56.6.109    |                3.50459e-05 |
| 1042 |          23 | 172.56.6.37     |                3.50459e-05 |
| 1043 |          23 | 172.58.111.248  |                3.50459e-05 |
| 1044 |          23 | 172.58.67.35    |                3.50459e-05 |
| 1045 |          23 | 172.58.70.192   |                3.50459e-05 |
| 1046 |          23 | 172.58.96.229   |                3.50459e-05 |
| 1047 |          23 | 173.127.108.144 |                3.50459e-05 |
| 1048 |          23 | 174.128.180.149 |                3.50459e-05 |
| 1049 |          23 | 174.128.180.196 |                3.50459e-05 |
| 1050 |          23 | 174.128.180.67  |                3.50459e-05 |
| 1051 |          23 | 174.255.135.219 |                3.50459e-05 |
| 1052 |          23 | 174.255.136.80  |                3.50459e-05 |
| 1053 |          23 | 174.255.138.88  |                3.50459e-05 |
| 1054 |          23 | 174.255.148.28  |                3.50459e-05 |
| 1055 |          23 | 174.255.186.213 |                3.50459e-05 |
| 1056 |          23 | 205.157.120.4   |                3.50459e-05 |
| 1057 |          23 | 24.214.254.244  |                3.50459e-05 |
| 1058 |          23 | 45.30.140.223   |                3.50459e-05 |
| 1059 |          23 | 50.84.194.42    |                3.50459e-05 |
| 1060 |          23 | 64.233.172.169  |                3.50459e-05 |
| 1061 |          23 | 64.250.227.83   |                3.50459e-05 |
| 1062 |          23 | 66.102.6.169    |                3.50459e-05 |
| 1063 |          23 | 66.102.6.171    |                3.50459e-05 |
| 1064 |          23 | 66.193.245.94   |                3.50459e-05 |
| 1065 |          23 | 66.3.37.210     |                3.50459e-05 |
| 1066 |          23 | 66.69.37.36     |                3.50459e-05 |
| 1067 |          23 | 69.169.52.37    |                3.50459e-05 |
| 1068 |          23 | 69.9.44.135     |                3.50459e-05 |
| 1069 |          23 | 69.9.44.136     |                3.50459e-05 |
| 1070 |          23 | 69.9.44.150     |                3.50459e-05 |
| 1071 |          23 | 69.9.44.197     |                3.50459e-05 |
| 1072 |          23 | 70.120.75.219   |                3.50459e-05 |
| 1073 |          23 | 71.78.46.154    |                3.50459e-05 |
| 1074 |          23 | 72.181.98.126   |                3.50459e-05 |
| 1075 |          23 | 75.54.32.45     |                3.50459e-05 |
| 1076 |          23 | 97.105.53.195   |                3.50459e-05 |
| 1077 |          23 | 98.6.99.131     |                3.50459e-05 |
| 1078 |          23 | 99.203.27.254   |                3.50459e-05 |
| 1188 |          24 | 107.77.100.48   |                8.41845e-05 |
| 1189 |          24 | 107.77.200.140  |                8.41845e-05 |
| 1190 |          24 | 107.77.217.9    |                8.41845e-05 |
| 1191 |          24 | 107.77.218.110  |                8.41845e-05 |
| 1192 |          24 | 107.77.221.157  |                8.41845e-05 |
| 1193 |          24 | 129.115.2.245   |                8.41845e-05 |
| 1194 |          24 | 172.127.185.99  |                8.41845e-05 |
| 1195 |          24 | 172.58.109.251  |                8.41845e-05 |
| 1196 |          24 | 172.58.67.38    |                8.41845e-05 |
| 1197 |          24 | 50.84.244.130   |                8.41845e-05 |
| 1198 |          24 | 71.42.218.131   |                8.41845e-05 |
| 1199 |          24 | 74.197.69.72    |                8.41845e-05 |
| 1200 |          24 | 99.203.154.58   |                8.41845e-05 |
| 1201 |          24 | 104.63.15.53    |                5.6123e-05  |
| 1202 |          24 | 107.77.105.117  |                5.6123e-05  |
| 1203 |          24 | 107.77.105.78   |                5.6123e-05  |
| 1204 |          24 | 107.77.66.26    |                5.6123e-05  |
| 1205 |          24 | 107.77.80.83    |                5.6123e-05  |
| 1206 |          24 | 108.220.101.224 |                5.6123e-05  |
| 1207 |          24 | 166.251.109.229 |                5.6123e-05  |
| 1208 |          24 | 172.58.109.48   |                5.6123e-05  |
| 1209 |          24 | 172.58.109.83   |                5.6123e-05  |
| 1210 |          24 | 172.58.70.163   |                5.6123e-05  |
| 1211 |          24 | 173.173.127.101 |                5.6123e-05  |
| 1212 |          24 | 24.160.4.6      |                5.6123e-05  |
| 1213 |          24 | 71.78.144.202   |                5.6123e-05  |
| 1214 |          24 | 72.181.33.132   |                5.6123e-05  |
| 1215 |          24 | 98.6.151.46     |                5.6123e-05  |
| 1216 |          24 | 99.203.27.241   |                5.6123e-05  |
| 1217 |          24 | 107.77.100.57   |                2.80615e-05 |
| 1218 |          24 | 107.77.197.78   |                2.80615e-05 |
| 1219 |          24 | 107.77.200.229  |                2.80615e-05 |
| 1220 |          24 | 107.77.220.108  |                2.80615e-05 |
| 1221 |          24 | 107.77.94.33    |                2.80615e-05 |
| 1222 |          24 | 166.137.136.55  |                2.80615e-05 |
| 1223 |          24 | 172.223.38.19   |                2.80615e-05 |
| 1224 |          24 | 172.58.67.105   |                2.80615e-05 |
| 1225 |          24 | 172.58.99.179   |                2.80615e-05 |
| 1226 |          24 | 172.58.99.22    |                2.80615e-05 |
| 1227 |          24 | 45.31.212.160   |                2.80615e-05 |
| 1228 |          24 | 65.36.106.75    |                2.80615e-05 |
| 1229 |          24 | 68.201.240.17   |                2.80615e-05 |
| 1230 |          24 | 72.181.100.92   |                2.80615e-05 |
| 1231 |          24 | 72.181.37.22    |                2.80615e-05 |
| 1232 |          24 | 72.190.182.108  |                2.80615e-05 |
| 1233 |          24 | 76.250.155.202  |                2.80615e-05 |
| 1234 |          24 | 97.79.234.116   |                2.80615e-05 |
| 1235 |          24 | 99.203.26.118   |                2.80615e-05 |
| 1236 |          24 | 99.203.27.213   |                2.80615e-05 |
| 1542 |          25 | 104.143.92.30   |                7.81677e-05 |
| 1543 |          25 | 104.237.80.10   |                7.81677e-05 |
| 1544 |          25 | 104.237.80.111  |                7.81677e-05 |
| 1545 |          25 | 104.237.80.147  |                7.81677e-05 |
| 1546 |          25 | 104.237.80.16   |                7.81677e-05 |
| 1547 |          25 | 104.237.80.36   |                7.81677e-05 |
| 1548 |          25 | 104.237.80.40   |                7.81677e-05 |
| 1549 |          25 | 104.237.80.48   |                7.81677e-05 |
| 1550 |          25 | 104.237.80.88   |                7.81677e-05 |
| 1551 |          25 | 107.77.169.12   |                7.81677e-05 |
| 1552 |          25 | 107.77.173.4    |                7.81677e-05 |
| 1553 |          25 | 107.77.217.21   |                7.81677e-05 |
| 1554 |          25 | 107.77.221.115  |                7.81677e-05 |
| 1555 |          25 | 107.77.222.174  |                7.81677e-05 |
| 1556 |          25 | 107.77.222.216  |                7.81677e-05 |
| 1557 |          25 | 107.77.222.63   |                7.81677e-05 |
| 1558 |          25 | 107.77.80.76    |                7.81677e-05 |
| 1559 |          25 | 108.11.10.198   |                7.81677e-05 |
| 1560 |          25 | 155.94.250.100  |                7.81677e-05 |
| 1561 |          25 | 172.56.6.71     |                7.81677e-05 |
| 1562 |          25 | 172.58.102.174  |                7.81677e-05 |
| 1563 |          25 | 172.58.67.18    |                7.81677e-05 |
| 1564 |          25 | 172.58.67.65    |                7.81677e-05 |
| 1565 |          25 | 172.58.99.67    |                7.81677e-05 |
| 1566 |          25 | 172.58.99.99    |                7.81677e-05 |
| 1567 |          25 | 173.239.232.100 |                7.81677e-05 |
| 1568 |          25 | 173.239.232.102 |                7.81677e-05 |
| 1569 |          25 | 173.239.232.103 |                7.81677e-05 |
| 1570 |          25 | 173.239.232.107 |                7.81677e-05 |
| 1571 |          25 | 173.239.232.113 |                7.81677e-05 |
| 1572 |          25 | 173.239.232.132 |                7.81677e-05 |
| 1573 |          25 | 173.239.232.139 |                7.81677e-05 |
| 1574 |          25 | 173.239.232.16  |                7.81677e-05 |
| 1575 |          25 | 173.239.232.166 |                7.81677e-05 |
| 1576 |          25 | 173.239.232.29  |                7.81677e-05 |
| 1577 |          25 | 173.239.232.41  |                7.81677e-05 |
| 1578 |          25 | 173.239.232.47  |                7.81677e-05 |
| 1579 |          25 | 173.239.232.51  |                7.81677e-05 |
| 1580 |          25 | 173.239.232.70  |                7.81677e-05 |
| 1581 |          25 | 173.239.232.94  |                7.81677e-05 |
| 1582 |          25 | 196.52.2.71     |                7.81677e-05 |
| 1583 |          25 | 205.251.150.154 |                7.81677e-05 |
| 1584 |          25 | 208.54.86.156   |                7.81677e-05 |
| 1585 |          25 | 52.176.52.196   |                7.81677e-05 |
| 1586 |          25 | 104.237.80.106  |                3.90839e-05 |
| 1587 |          25 | 104.237.80.117  |                3.90839e-05 |
| 1588 |          25 | 104.237.80.120  |                3.90839e-05 |
| 1589 |          25 | 104.237.80.144  |                3.90839e-05 |
| 1590 |          25 | 104.237.80.24   |                3.90839e-05 |
| 1591 |          25 | 104.237.80.34   |                3.90839e-05 |
| 1592 |          25 | 104.237.80.39   |                3.90839e-05 |
| 1593 |          25 | 104.237.80.44   |                3.90839e-05 |
| 1594 |          25 | 104.237.80.57   |                3.90839e-05 |
| 1595 |          25 | 104.237.80.75   |                3.90839e-05 |
| 1596 |          25 | 104.237.80.93   |                3.90839e-05 |
| 1597 |          25 | 107.77.173.9    |                3.90839e-05 |
| 1598 |          25 | 107.77.217.216  |                3.90839e-05 |
| 1599 |          25 | 107.77.217.54   |                3.90839e-05 |
| 1600 |          25 | 107.77.219.27   |                3.90839e-05 |
| 1601 |          25 | 107.77.220.227  |                3.90839e-05 |
| 1602 |          25 | 107.77.220.47   |                3.90839e-05 |
| 1603 |          25 | 107.77.221.72   |                3.90839e-05 |
| 1604 |          25 | 107.77.221.80   |                3.90839e-05 |
| 1605 |          25 | 107.77.222.55   |                3.90839e-05 |
| 1606 |          25 | 108.11.6.201    |                3.90839e-05 |
| 1607 |          25 | 165.91.12.128   |                3.90839e-05 |
| 1608 |          25 | 172.56.6.250    |                3.90839e-05 |
| 1609 |          25 | 172.56.7.199    |                3.90839e-05 |
| 1610 |          25 | 172.58.109.223  |                3.90839e-05 |
| 1611 |          25 | 172.58.67.210   |                3.90839e-05 |
| 1612 |          25 | 172.58.99.178   |                3.90839e-05 |
| 1613 |          25 | 172.58.99.225   |                3.90839e-05 |
| 1614 |          25 | 173.239.198.246 |                3.90839e-05 |
| 1615 |          25 | 173.239.232.104 |                3.90839e-05 |
| 1616 |          25 | 173.239.232.111 |                3.90839e-05 |
| 1617 |          25 | 173.239.232.115 |                3.90839e-05 |
| 1618 |          25 | 173.239.232.122 |                3.90839e-05 |
| 1619 |          25 | 173.239.232.127 |                3.90839e-05 |
| 1620 |          25 | 173.239.232.146 |                3.90839e-05 |
| 1621 |          25 | 173.239.232.157 |                3.90839e-05 |
| 1622 |          25 | 173.239.232.163 |                3.90839e-05 |
| 1623 |          25 | 173.239.232.24  |                3.90839e-05 |
| 1624 |          25 | 173.239.232.28  |                3.90839e-05 |
| 1625 |          25 | 173.239.232.32  |                3.90839e-05 |
| 1626 |          25 | 173.239.232.37  |                3.90839e-05 |
| 1627 |          25 | 173.239.232.52  |                3.90839e-05 |
| 1628 |          25 | 173.239.232.61  |                3.90839e-05 |
| 1629 |          25 | 173.239.232.7   |                3.90839e-05 |
| 1630 |          25 | 173.239.232.85  |                3.90839e-05 |
| 1631 |          25 | 173.239.232.87  |                3.90839e-05 |
| 1632 |          25 | 173.239.240.116 |                3.90839e-05 |
| 1633 |          25 | 173.239.240.49  |                3.90839e-05 |
| 1634 |          25 | 196.52.2.59     |                3.90839e-05 |
| 1635 |          25 | 199.116.115.143 |                3.90839e-05 |
| 1636 |          25 | 199.116.115.147 |                3.90839e-05 |
| 1637 |          25 | 199.116.118.166 |                3.90839e-05 |
| 1638 |          25 | 205.251.148.178 |                3.90839e-05 |
| 1639 |          25 | 205.251.150.234 |                3.90839e-05 |
| 1640 |          25 | 45.56.151.121   |                3.90839e-05 |
| 1641 |          25 | 66.69.111.115   |                3.90839e-05 |
| 1642 |          25 | 67.48.40.83     |                3.90839e-05 |
| 1643 |          25 | 70.94.183.166   |                3.90839e-05 |
| 1644 |          25 | 71.146.33.118   |                3.90839e-05 |
| 1645 |          25 | 97.77.126.60    |                3.90839e-05 |
| 1646 |          25 | 99.203.23.170   |                3.90839e-05 |
| 1759 |          26 | 104.53.215.3    |                7.20747e-05 |
| 1760 |          26 | 107.77.169.3    |                7.20747e-05 |
| 1761 |          26 | 107.77.221.57   |                7.20747e-05 |
| 1762 |          26 | 108.200.112.159 |                7.20747e-05 |
| 1763 |          26 | 108.220.102.149 |                7.20747e-05 |
| 1764 |          26 | 170.248.173.246 |                7.20747e-05 |
| 1765 |          26 | 207.119.59.156  |                7.20747e-05 |
| 1766 |          26 | 208.54.86.247   |                7.20747e-05 |
| 1767 |          26 | 209.107.189.26  |                7.20747e-05 |
| 1768 |          26 | 71.221.9.148    |                7.20747e-05 |
| 1769 |          26 | 72.177.237.78   |                7.20747e-05 |
| 1770 |          26 | 104.176.91.161  |                3.60373e-05 |
| 1771 |          26 | 104.184.27.54   |                3.60373e-05 |
| 1772 |          26 | 107.77.217.137  |                3.60373e-05 |
| 1773 |          26 | 107.77.217.20   |                3.60373e-05 |
| 1774 |          26 | 107.77.218.198  |                3.60373e-05 |
| 1775 |          26 | 107.77.219.19   |                3.60373e-05 |
| 1776 |          26 | 107.77.220.123  |                3.60373e-05 |
| 1777 |          26 | 107.77.220.42   |                3.60373e-05 |
| 1778 |          26 | 107.77.220.81   |                3.60373e-05 |
| 1779 |          26 | 107.77.221.63   |                3.60373e-05 |
| 1780 |          26 | 107.77.222.181  |                3.60373e-05 |
| 1781 |          26 | 107.77.222.90   |                3.60373e-05 |
| 1782 |          26 | 107.77.222.92   |                3.60373e-05 |
| 1783 |          26 | 170.248.173.247 |                3.60373e-05 |
| 1784 |          26 | 173.174.136.11  |                3.60373e-05 |
| 1785 |          26 | 173.174.201.87  |                3.60373e-05 |
| 1786 |          26 | 174.207.25.208  |                3.60373e-05 |
| 1787 |          26 | 24.27.56.43     |                3.60373e-05 |
| 1788 |          26 | 56.0.143.25     |                3.60373e-05 |
| 1789 |          26 | 71.135.71.73    |                3.60373e-05 |
| 1790 |          26 | 71.40.247.91    |                3.60373e-05 |
| 1933 |          27 | 107.77.217.125  |                9.64181e-05 |
| 1934 |          27 | 107.77.219.174  |                9.64181e-05 |
| 1935 |          27 | 107.77.220.166  |                9.64181e-05 |
| 1936 |          27 | 166.251.101.159 |                9.64181e-05 |
| 1937 |          27 | 172.56.6.240    |                9.64181e-05 |
| 1938 |          27 | 172.58.106.143  |                9.64181e-05 |
| 1939 |          27 | 172.58.3.231    |                9.64181e-05 |
| 1940 |          27 | 172.58.67.180   |                9.64181e-05 |
| 1941 |          27 | 174.207.0.29    |                9.64181e-05 |
| 1942 |          27 | 174.207.23.199  |                9.64181e-05 |
| 1943 |          27 | 216.53.156.133  |                9.64181e-05 |
| 1944 |          27 | 66.25.165.226   |                9.64181e-05 |
| 1945 |          27 | 66.25.167.65    |                9.64181e-05 |
| 1946 |          27 | 72.181.99.132   |                9.64181e-05 |
| 1947 |          27 | 72.183.4.143    |                9.64181e-05 |
| 1948 |          27 | 99.203.26.215   |                9.64181e-05 |
| 1949 |          27 | 99.203.26.98    |                9.64181e-05 |
| 1950 |          27 | 107.77.198.37   |                4.8209e-05  |
| 1951 |          27 | 107.77.222.200  |                4.8209e-05  |
| 1952 |          27 | 107.77.222.41   |                4.8209e-05  |
| 1953 |          27 | 12.184.218.52   |                4.8209e-05  |
| 1954 |          27 | 165.225.48.85   |                4.8209e-05  |
| 1955 |          27 | 172.56.6.73     |                4.8209e-05  |
| 1956 |          27 | 174.207.23.109  |                4.8209e-05  |
| 1957 |          27 | 24.27.227.129   |                4.8209e-05  |
| 1958 |          27 | 24.28.137.129   |                4.8209e-05  |
| 1959 |          27 | 66.25.168.77    |                4.8209e-05  |
| 1960 |          27 | 70.94.161.227   |                4.8209e-05  |
| 1961 |          27 | 71.40.74.50     |                4.8209e-05  |
| 1962 |          27 | 72.128.131.66   |                4.8209e-05  |
| 1963 |          27 | 73.166.220.146  |                4.8209e-05  |
| 1964 |          27 | 96.88.69.9      |                4.8209e-05  |
| 1965 |          27 | 97.77.127.220   |                4.8209e-05  |
| 1966 |          27 | 99.203.154.130  |                4.8209e-05  |
| 1967 |          27 | 99.203.26.14    |                4.8209e-05  |
| 1968 |          27 | 99.203.27.77    |                4.8209e-05  |
| 2113 |          28 | 67.22.173.176   |                9.5203e-05  |
| 2114 |          28 | 72.181.125.198  |                9.5203e-05  |
| 2115 |          28 | 97.98.14.0      |                9.5203e-05  |
| 2116 |          28 | 12.247.3.50     |                8.33026e-05 |
| 2117 |          28 | 173.173.113.96  |                8.33026e-05 |
| 2118 |          28 | 174.128.181.89  |                8.33026e-05 |
| 2119 |          28 | 191.96.67.226   |                8.33026e-05 |
| 2120 |          28 | 71.221.235.149  |                8.33026e-05 |
| 2121 |          28 | 89.187.175.174  |                8.33026e-05 |
| 2122 |          28 | 99.203.26.250   |                8.33026e-05 |
| 2123 |          28 | 107.77.169.6    |                7.14022e-05 |
| 2124 |          28 | 127.0.0.1       |                7.14022e-05 |
| 2125 |          28 | 154.3.250.165   |                7.14022e-05 |
| 2126 |          28 | 174.128.180.71  |                7.14022e-05 |
| 2127 |          28 | 207.239.160.254 |                7.14022e-05 |
| 2128 |          28 | 72.173.98.235   |                7.14022e-05 |
| 2129 |          28 | 73.119.63.36    |                7.14022e-05 |
| 2130 |          28 | 73.219.222.176  |                7.14022e-05 |
| 2131 |          28 | 97.105.54.251   |                7.14022e-05 |
| 2132 |          28 | 98.6.1.177      |                7.14022e-05 |
| 2133 |          28 | 162.250.18.82   |                5.95019e-05 |
| 2134 |          28 | 166.205.97.137  |                5.95019e-05 |
| 2135 |          28 | 172.58.99.178   |                5.95019e-05 |
| 2136 |          28 | 174.128.180.150 |                5.95019e-05 |
| 2137 |          28 | 174.128.180.9   |                5.95019e-05 |
| 2138 |          28 | 24.28.70.46     |                5.95019e-05 |
| 2139 |          28 | 4.16.216.161    |                5.95019e-05 |
| 2140 |          28 | 50.84.218.3     |                5.95019e-05 |
| 2141 |          28 | 70.124.99.231   |                5.95019e-05 |
| 2142 |          28 | 72.181.120.120  |                5.95019e-05 |
| 2143 |          28 | 75.144.206.81   |                5.95019e-05 |
| 2144 |          28 | 76.205.165.180  |                5.95019e-05 |
| 2145 |          28 | 107.77.169.8    |                4.76015e-05 |
| 2146 |          28 | 107.77.189.50   |                4.76015e-05 |
| 2147 |          28 | 165.227.41.208  |                4.76015e-05 |
| 2148 |          28 | 170.248.173.245 |                4.76015e-05 |
| 2149 |          28 | 173.175.194.1   |                4.76015e-05 |
| 2150 |          28 | 174.128.180.152 |                4.76015e-05 |
| 2151 |          28 | 174.128.181.232 |                4.76015e-05 |
| 2152 |          28 | 174.128.181.71  |                4.76015e-05 |
| 2153 |          28 | 174.207.15.123  |                4.76015e-05 |
| 2154 |          28 | 192.241.233.20  |                4.76015e-05 |
| 2155 |          28 | 208.54.83.145   |                4.76015e-05 |
| 2156 |          28 | 24.28.41.114    |                4.76015e-05 |
| 2157 |          28 | 46.218.181.194  |                4.76015e-05 |
| 2158 |          28 | 67.10.185.18    |                4.76015e-05 |
| 2159 |          28 | 72.181.120.63   |                4.76015e-05 |
| 2160 |          28 | 99.203.31.78    |                4.76015e-05 |
| 2161 |          28 | 107.77.169.10   |                3.57011e-05 |
| 2162 |          28 | 107.77.169.11   |                3.57011e-05 |
| 2163 |          28 | 107.77.189.16   |                3.57011e-05 |
| 2164 |          28 | 107.77.189.23   |                3.57011e-05 |
| 2165 |          28 | 107.77.189.37   |                3.57011e-05 |
| 2166 |          28 | 108.213.191.10  |                3.57011e-05 |
| 2167 |          28 | 129.115.2.245   |                3.57011e-05 |
| 2168 |          28 | 130.45.47.106   |                3.57011e-05 |
| 2169 |          28 | 162.197.43.200  |                3.57011e-05 |
| 2170 |          28 | 166.205.97.126  |                3.57011e-05 |
| 2171 |          28 | 167.71.126.76   |                3.57011e-05 |
| 2172 |          28 | 172.58.107.207  |                3.57011e-05 |
| 2173 |          28 | 172.58.107.99   |                3.57011e-05 |
| 2174 |          28 | 172.58.108.134  |                3.57011e-05 |
| 2175 |          28 | 172.58.108.214  |                3.57011e-05 |
| 2176 |          28 | 172.58.109.59   |                3.57011e-05 |
| 2177 |          28 | 172.58.97.151   |                3.57011e-05 |
| 2178 |          28 | 172.58.97.217   |                3.57011e-05 |
| 2179 |          28 | 174.128.180.102 |                3.57011e-05 |
| 2180 |          28 | 174.128.180.115 |                3.57011e-05 |
| 2181 |          28 | 174.128.180.163 |                3.57011e-05 |
| 2182 |          28 | 174.128.180.19  |                3.57011e-05 |
| 2183 |          28 | 174.128.181.152 |                3.57011e-05 |
| 2184 |          28 | 174.128.181.69  |                3.57011e-05 |
| 2185 |          28 | 174.196.196.21  |                3.57011e-05 |
| 2186 |          28 | 201.174.135.51  |                3.57011e-05 |
| 2187 |          28 | 3.209.254.178   |                3.57011e-05 |
| 2188 |          28 | 67.7.3.203      |                3.57011e-05 |
| 2189 |          28 | 69.148.0.213    |                3.57011e-05 |
| 2190 |          28 | 69.172.183.120  |                3.57011e-05 |
| 2191 |          28 | 70.121.216.135  |                3.57011e-05 |
| 2192 |          28 | 72.191.30.213   |                3.57011e-05 |
| 2193 |          28 | 73.115.118.19   |                3.57011e-05 |
| 2194 |          28 | 75.54.33.182    |                3.57011e-05 |
| 2195 |          28 | 75.54.33.190    |                3.57011e-05 |
| 2196 |          28 | 96.8.193.160    |                3.57011e-05 |
| 2197 |          28 | 98.6.52.59      |                3.57011e-05 |
| 2198 |          28 | 99.203.155.126  |                3.57011e-05 |
| 2199 |          28 | 99.203.90.208   |                3.57011e-05 |
| 2200 |          28 | 104.238.45.20   |                2.38007e-05 |
| 2201 |          28 | 107.77.169.1    |                2.38007e-05 |
| 2202 |          28 | 107.77.218.40   |                2.38007e-05 |
| 2203 |          28 | 107.77.222.125  |                2.38007e-05 |
| 2204 |          28 | 129.115.2.251   |                2.38007e-05 |
| 2205 |          28 | 144.178.1.46    |                2.38007e-05 |
| 2206 |          28 | 166.205.190.113 |                2.38007e-05 |
| 2207 |          28 | 166.205.97.85   |                2.38007e-05 |
| 2208 |          28 | 172.56.6.129    |                2.38007e-05 |
| 2209 |          28 | 172.58.102.250  |                2.38007e-05 |
| 2210 |          28 | 172.58.111.200  |                2.38007e-05 |
| 2211 |          28 | 172.58.96.187   |                2.38007e-05 |
| 2212 |          28 | 172.58.96.85    |                2.38007e-05 |
| 2213 |          28 | 172.58.97.152   |                2.38007e-05 |
| 2214 |          28 | 172.58.99.132   |                2.38007e-05 |
| 2215 |          28 | 172.58.99.25    |                2.38007e-05 |
| 2216 |          28 | 173.123.204.117 |                2.38007e-05 |
| 2217 |          28 | 174.128.180.3   |                2.38007e-05 |
| 2218 |          28 | 174.128.181.166 |                2.38007e-05 |
| 2219 |          28 | 174.128.181.167 |                2.38007e-05 |
| 2220 |          28 | 174.197.1.123   |                2.38007e-05 |
| 2221 |          28 | 174.197.10.65   |                2.38007e-05 |
| 2222 |          28 | 187.237.25.33   |                2.38007e-05 |
| 2223 |          28 | 192.241.235.228 |                2.38007e-05 |
| 2224 |          28 | 200.68.128.26   |                2.38007e-05 |
| 2225 |          28 | 207.87.181.170  |                2.38007e-05 |
| 2226 |          28 | 208.54.83.163   |                2.38007e-05 |
| 2227 |          28 | 24.27.227.129   |                2.38007e-05 |
| 2228 |          28 | 50.84.215.165   |                2.38007e-05 |
| 2229 |          28 | 64.124.33.178   |                2.38007e-05 |
| 2230 |          28 | 67.7.0.188      |                2.38007e-05 |
| 2231 |          28 | 70.116.81.203   |                2.38007e-05 |
| 2232 |          28 | 70.123.219.46   |                2.38007e-05 |
| 2233 |          28 | 71.221.192.10   |                2.38007e-05 |
| 2234 |          28 | 71.42.125.202   |                2.38007e-05 |
| 2235 |          28 | 72.177.229.125  |                2.38007e-05 |
| 2236 |          28 | 72.177.94.59    |                2.38007e-05 |
| 2237 |          28 | 72.181.107.144  |                2.38007e-05 |
| 2238 |          28 | 75.54.33.188    |                2.38007e-05 |
| 2239 |          28 | 76.185.247.236  |                2.38007e-05 |
| 2240 |          28 | 99.203.155.145  |                2.38007e-05 |
| 2241 |          28 | 99.203.22.172   |                2.38007e-05 |
| 2242 |          28 | 99.203.27.106   |                2.38007e-05 |
| 2243 |          28 | 99.98.245.245   |                2.38007e-05 |
| 2244 |          28 | 107.77.169.9    |                1.19004e-05 |
| 2245 |          28 | 107.77.200.202  |                1.19004e-05 |
| 2246 |          28 | 107.77.201.52   |                1.19004e-05 |
| 2247 |          28 | 107.77.219.187  |                1.19004e-05 |
| 2248 |          28 | 136.50.61.52    |                1.19004e-05 |
| 2249 |          28 | 154.3.250.53    |                1.19004e-05 |
| 2250 |          28 | 172.56.15.51    |                1.19004e-05 |
| 2251 |          28 | 172.58.102.205  |                1.19004e-05 |
| 2252 |          28 | 172.58.108.93   |                1.19004e-05 |
| 2253 |          28 | 172.58.109.163  |                1.19004e-05 |
| 2254 |          28 | 172.58.110.211  |                1.19004e-05 |
| 2255 |          28 | 172.58.96.54    |                1.19004e-05 |
| 2256 |          28 | 172.58.99.137   |                1.19004e-05 |
| 2257 |          28 | 174.128.180.24  |                1.19004e-05 |
| 2258 |          28 | 174.128.180.55  |                1.19004e-05 |
| 2259 |          28 | 174.128.181.105 |                1.19004e-05 |
| 2260 |          28 | 174.197.7.127   |                1.19004e-05 |
| 2261 |          28 | 187.237.231.72  |                1.19004e-05 |
| 2262 |          28 | 187.237.231.85  |                1.19004e-05 |
| 2263 |          28 | 204.156.181.62  |                1.19004e-05 |
| 2264 |          28 | 208.116.188.250 |                1.19004e-05 |
| 2265 |          28 | 208.54.83.234   |                1.19004e-05 |
| 2266 |          28 | 24.173.208.242  |                1.19004e-05 |
| 2267 |          28 | 24.243.20.39    |                1.19004e-05 |
| 2268 |          28 | 64.9.239.9      |                1.19004e-05 |
| 2269 |          28 | 65.36.96.192    |                1.19004e-05 |
| 2270 |          28 | 66.68.220.218   |                1.19004e-05 |
| 2271 |          28 | 70.114.21.86    |                1.19004e-05 |
| 2272 |          28 | 70.120.15.165   |                1.19004e-05 |
| 2273 |          28 | 71.221.223.195  |                1.19004e-05 |
| 2274 |          28 | 72.179.166.181  |                1.19004e-05 |
| 2275 |          28 | 72.181.112.227  |                1.19004e-05 |
| 2276 |          28 | 99.203.143.2    |                1.19004e-05 |
| 2277 |          28 | 99.203.154.95   |                1.19004e-05 |
| 2278 |          28 | 99.203.155.182  |                1.19004e-05 |
| 2279 |          28 | 99.203.155.78   |                1.19004e-05 |
| 2280 |          28 | 99.203.212.173  |                1.19004e-05 |
| 2281 |          28 | 99.203.212.205  |                1.19004e-05 |
| 2282 |          28 | 99.203.213.105  |                1.19004e-05 |
| 2283 |          28 | 99.203.213.59   |                1.19004e-05 |
| 2284 |          28 | 99.203.26.9     |                1.19004e-05 |
| 2285 |          28 | 99.203.27.231   |                1.19004e-05 |
| 2286 |          28 | 99.203.27.9     |                1.19004e-05 |
| 2441 |          29 | 107.77.218.199  |                7.87484e-05 |
| 2442 |          29 | 107.77.218.58   |                7.87484e-05 |
| 2443 |          29 | 107.77.245.7    |                7.87484e-05 |
| 2444 |          29 | 166.251.106.182 |                7.87484e-05 |
| 2445 |          29 | 172.56.7.151    |                7.87484e-05 |
| 2446 |          29 | 172.58.103.188  |                7.87484e-05 |
| 2447 |          29 | 172.58.106.58   |                7.87484e-05 |
| 2448 |          29 | 172.58.109.245  |                7.87484e-05 |
| 2449 |          29 | 172.58.109.64   |                7.87484e-05 |
| 2450 |          29 | 172.58.111.140  |                7.87484e-05 |
| 2451 |          29 | 174.207.21.3    |                7.87484e-05 |
| 2452 |          29 | 64.9.239.13     |                7.87484e-05 |
| 2453 |          29 | 64.9.239.21     |                7.87484e-05 |
| 2454 |          29 | 98.6.52.59      |                7.87484e-05 |
| 2455 |          29 | 99.203.154.190  |                7.87484e-05 |
| 2456 |          29 | 99.203.154.198  |                7.87484e-05 |
| 2457 |          29 | 99.203.154.208  |                7.87484e-05 |
| 2458 |          29 | 99.203.155.190  |                7.87484e-05 |
| 2459 |          29 | 99.203.155.215  |                7.87484e-05 |
| 2460 |          29 | 99.203.155.225  |                7.87484e-05 |
| 2461 |          29 | 107.77.169.1    |                5.2499e-05  |
| 2462 |          29 | 107.77.217.32   |                5.2499e-05  |
| 2463 |          29 | 107.77.218.42   |                5.2499e-05  |
| 2464 |          29 | 131.194.172.128 |                5.2499e-05  |
| 2465 |          29 | 131.194.172.217 |                5.2499e-05  |
| 2466 |          29 | 131.194.172.75  |                5.2499e-05  |
| 2467 |          29 | 136.49.90.35    |                5.2499e-05  |
| 2468 |          29 | 159.118.138.70  |                5.2499e-05  |
| 2469 |          29 | 174.207.26.73   |                5.2499e-05  |
| 2470 |          29 | 196.52.60.56    |                5.2499e-05  |
| 2471 |          29 | 64.9.239.12     |                5.2499e-05  |
| 2472 |          29 | 64.9.239.34     |                5.2499e-05  |
| 2473 |          29 | 70.120.43.13    |                5.2499e-05  |
| 2474 |          29 | 96.8.128.69     |                5.2499e-05  |
| 2475 |          29 | 96.8.219.155    |                5.2499e-05  |
| 2476 |          29 | 99.203.154.102  |                5.2499e-05  |
| 2477 |          29 | 99.203.154.246  |                5.2499e-05  |
| 2478 |          29 | 99.203.154.30   |                5.2499e-05  |
| 2479 |          29 | 99.203.155.185  |                5.2499e-05  |
| 2480 |          29 | 99.203.155.241  |                5.2499e-05  |
| 2481 |          29 | 99.203.26.2     |                5.2499e-05  |
| 2482 |          29 | 99.203.26.20    |                5.2499e-05  |
| 2483 |          29 | 99.203.27.109   |                5.2499e-05  |
| 2484 |          29 | 99.203.27.120   |                5.2499e-05  |
| 2485 |          29 | 99.203.27.34    |                5.2499e-05  |
| 2486 |          29 | 99.203.90.42    |                5.2499e-05  |
| 2487 |          29 | 99.203.91.127   |                5.2499e-05  |
| 2488 |          29 | 107.77.198.150  |                2.62495e-05 |
| 2489 |          29 | 107.77.220.149  |                2.62495e-05 |
| 2490 |          29 | 107.77.220.26   |                2.62495e-05 |
| 2491 |          29 | 131.194.172.30  |                2.62495e-05 |
| 2492 |          29 | 136.49.24.186   |                2.62495e-05 |
| 2493 |          29 | 165.225.48.93   |                2.62495e-05 |
| 2494 |          29 | 172.56.6.31     |                2.62495e-05 |
| 2495 |          29 | 172.58.109.108  |                2.62495e-05 |
| 2496 |          29 | 172.58.109.220  |                2.62495e-05 |
| 2497 |          29 | 172.58.109.30   |                2.62495e-05 |
| 2498 |          29 | 173.123.45.184  |                2.62495e-05 |
| 2499 |          29 | 173.127.56.209  |                2.62495e-05 |
| 2500 |          29 | 174.207.15.199  |                2.62495e-05 |
| 2501 |          29 | 174.207.2.32    |                2.62495e-05 |
| 2502 |          29 | 174.207.3.247   |                2.62495e-05 |
| 2503 |          29 | 193.37.252.68   |                2.62495e-05 |
| 2504 |          29 | 65.121.50.194   |                2.62495e-05 |
| 2505 |          29 | 96.8.128.98     |                2.62495e-05 |
| 2506 |          29 | 99.203.154.118  |                2.62495e-05 |
| 2507 |          29 | 99.203.154.55   |                2.62495e-05 |
| 2508 |          29 | 99.203.155.123  |                2.62495e-05 |
| 2509 |          29 | 99.203.155.142  |                2.62495e-05 |
| 2510 |          29 | 99.203.155.17   |                2.62495e-05 |
| 2511 |          29 | 99.203.155.176  |                2.62495e-05 |
| 2512 |          29 | 99.203.26.125   |                2.62495e-05 |
| 2513 |          29 | 99.203.26.251   |                2.62495e-05 |
| 2514 |          29 | 99.203.26.81    |                2.62495e-05 |
| 2515 |          29 | 99.203.26.97    |                2.62495e-05 |
| 2516 |          29 | 99.203.27.219   |                2.62495e-05 |
| 2517 |          29 | 99.203.27.48    |                2.62495e-05 |
| 2518 |          29 | 99.203.27.9     |                2.62495e-05 |
| 2519 |          29 | 99.203.91.141   |                2.62495e-05 |
| 2520 |          29 | 99.203.91.196   |                2.62495e-05 |
| 2521 |          29 | 99.203.91.36    |                2.62495e-05 |
| 2522 |          29 | 99.61.71.158    |                2.62495e-05 |
| 2632 |          31 | 165.225.48.90   |                7.88675e-05 |
| 2633 |          31 | 172.56.6.211    |                7.88675e-05 |
| 2634 |          31 | 172.56.6.247    |                7.88675e-05 |
| 2635 |          31 | 172.58.67.60    |                7.88675e-05 |
| 2636 |          31 | 173.239.232.62  |                7.88675e-05 |
| 2637 |          31 | 174.207.12.247  |                7.88675e-05 |
| 2638 |          31 | 174.207.15.228  |                7.88675e-05 |
| 2639 |          31 | 199.10.64.84    |                7.88675e-05 |
| 2640 |          31 | 64.148.251.171  |                7.88675e-05 |
| 2641 |          31 | 67.142.203.140  |                7.88675e-05 |
| 2642 |          31 | 68.81.93.120    |                7.88675e-05 |
| 2643 |          31 | 70.123.160.242  |                7.88675e-05 |
| 2644 |          31 | 71.42.105.141   |                7.88675e-05 |
| 2645 |          31 | 72.177.203.50   |                7.88675e-05 |
| 2646 |          31 | 99.203.154.149  |                7.88675e-05 |
| 2647 |          31 | 99.203.27.92    |                7.88675e-05 |
| 2648 |          31 | 107.77.219.231  |                3.94337e-05 |
| 2649 |          31 | 107.77.221.19   |                3.94337e-05 |
| 2650 |          31 | 166.251.101.233 |                3.94337e-05 |
| 2651 |          31 | 166.251.105.253 |                3.94337e-05 |
| 2652 |          31 | 166.251.112.234 |                3.94337e-05 |
| 2653 |          31 | 172.56.14.207   |                3.94337e-05 |
| 2654 |          31 | 172.56.7.179    |                3.94337e-05 |
| 2655 |          31 | 174.225.136.248 |                3.94337e-05 |
| 2656 |          31 | 185.247.70.4    |                3.94337e-05 |
| 2657 |          31 | 185.247.70.76   |                3.94337e-05 |
| 2658 |          31 | 208.54.83.222   |                3.94337e-05 |
| 2659 |          31 | 24.26.235.149   |                3.94337e-05 |
| 2660 |          31 | 38.140.212.203  |                3.94337e-05 |
| 2661 |          31 | 64.134.155.129  |                3.94337e-05 |
| 2662 |          31 | 67.79.222.107   |                3.94337e-05 |
| 2663 |          31 | 71.42.105.147   |                3.94337e-05 |
| 2664 |          31 | 71.42.105.183   |                3.94337e-05 |
| 2665 |          31 | 71.78.144.162   |                3.94337e-05 |
| 2666 |          31 | 99.203.154.221  |                3.94337e-05 |
| 2667 |          31 | 99.203.155.5    |                3.94337e-05 |
| 2668 |          31 | 99.203.155.92   |                3.94337e-05 |
| 2669 |          31 | 99.203.155.98   |                3.94337e-05 |
| 2670 |          31 | 99.203.26.219   |                3.94337e-05 |
| 2671 |          31 | 99.203.27.110   |                3.94337e-05 |
| 2672 |          31 | 99.203.27.189   |                3.94337e-05 |
| 2673 |          31 | 99.203.27.196   |                3.94337e-05 |
| 2674 |          31 | 99.203.27.24    |                3.94337e-05 |
| 2774 |          32 | 107.77.217.214  |                6.81292e-05 |
| 2775 |          32 | 107.77.217.37   |                6.81292e-05 |
| 2776 |          32 | 107.77.217.47   |                6.81292e-05 |
| 2777 |          32 | 107.77.218.35   |                6.81292e-05 |
| 2778 |          32 | 107.77.219.47   |                6.81292e-05 |
| 2779 |          32 | 107.77.220.18   |                6.81292e-05 |
| 2780 |          32 | 107.77.220.195  |                6.81292e-05 |
| 2781 |          32 | 172.56.7.186    |                6.81292e-05 |
| 2782 |          32 | 172.56.7.6      |                6.81292e-05 |
| 2783 |          32 | 172.58.107.66   |                6.81292e-05 |
| 2784 |          32 | 172.58.111.5    |                6.81292e-05 |
| 2785 |          32 | 40.141.115.69   |                6.81292e-05 |
| 2786 |          32 | 71.42.105.167   |                6.81292e-05 |
| 2787 |          32 | 71.42.105.172   |                6.81292e-05 |
| 2788 |          32 | 76.186.54.220   |                6.81292e-05 |
| 2789 |          32 | 76.250.156.19   |                6.81292e-05 |
| 2790 |          32 | 107.77.217.10   |                3.40646e-05 |
| 2791 |          32 | 107.77.217.49   |                3.40646e-05 |
| 2792 |          32 | 107.77.218.210  |                3.40646e-05 |
| 2793 |          32 | 107.77.221.36   |                3.40646e-05 |
| 2794 |          32 | 107.77.221.42   |                3.40646e-05 |
| 2795 |          32 | 107.77.221.76   |                3.40646e-05 |
| 2796 |          32 | 107.77.222.231  |                3.40646e-05 |
| 2797 |          32 | 172.58.109.140  |                3.40646e-05 |
| 2798 |          32 | 174.207.16.237  |                3.40646e-05 |
| 2799 |          32 | 174.207.24.226  |                3.40646e-05 |
| 2800 |          32 | 64.250.4.125    |                3.40646e-05 |
| 2801 |          32 | 66.69.76.167    |                3.40646e-05 |
| 2802 |          32 | 71.78.61.162    |                3.40646e-05 |
| 2803 |          32 | 97.105.165.178  |                3.40646e-05 |
| 2942 |          33 | 107.242.125.8   |                9.82077e-05 |
| 2943 |          33 | 107.77.217.126  |                9.82077e-05 |
| 2944 |          33 | 107.77.217.69   |                9.82077e-05 |
| 2945 |          33 | 107.77.219.173  |                9.82077e-05 |
| 2946 |          33 | 12.247.3.42     |                9.82077e-05 |
| 2947 |          33 | 162.248.14.168  |                9.82077e-05 |
| 2948 |          33 | 166.251.106.102 |                9.82077e-05 |
| 2949 |          33 | 172.58.102.240  |                9.82077e-05 |
| 2950 |          33 | 173.123.163.49  |                9.82077e-05 |
| 2951 |          33 | 173.173.109.125 |                9.82077e-05 |
| 2952 |          33 | 192.107.1.225   |                9.82077e-05 |
| 2953 |          33 | 24.160.141.134  |                9.82077e-05 |
| 2954 |          33 | 31.14.72.26     |                9.82077e-05 |
| 2955 |          33 | 67.10.161.67    |                9.82077e-05 |
| 2956 |          33 | 70.117.19.13    |                9.82077e-05 |
| 2957 |          33 | 73.14.59.241    |                9.82077e-05 |
| 2958 |          33 | 76.255.12.121   |                9.82077e-05 |
| 2959 |          33 | 97.105.53.195   |                9.82077e-05 |
| 2960 |          33 | 107.77.189.21   |                7.36558e-05 |
| 2961 |          33 | 107.77.189.63   |                7.36558e-05 |
| 2962 |          33 | 107.77.196.38   |                7.36558e-05 |
| 2963 |          33 | 107.77.217.168  |                7.36558e-05 |
| 2964 |          33 | 107.77.218.48   |                7.36558e-05 |
| 2965 |          33 | 107.77.220.130  |                7.36558e-05 |
| 2966 |          33 | 107.77.220.191  |                7.36558e-05 |
| 2967 |          33 | 107.77.220.47   |                7.36558e-05 |
| 2968 |          33 | 162.197.40.243  |                7.36558e-05 |
| 2969 |          33 | 162.71.236.108  |                7.36558e-05 |
| 2970 |          33 | 166.251.106.216 |                7.36558e-05 |
| 2971 |          33 | 172.56.7.208    |                7.36558e-05 |
| 2972 |          33 | 172.58.102.228  |                7.36558e-05 |
| 2973 |          33 | 172.58.99.18    |                7.36558e-05 |
| 2974 |          33 | 174.206.2.148   |                7.36558e-05 |
| 2975 |          33 | 174.207.0.103   |                7.36558e-05 |
| 2976 |          33 | 174.207.15.137  |                7.36558e-05 |
| 2977 |          33 | 174.207.16.65   |                7.36558e-05 |
| 2978 |          33 | 174.207.17.181  |                7.36558e-05 |
| 2979 |          33 | 174.207.22.2    |                7.36558e-05 |
| 2980 |          33 | 174.207.23.41   |                7.36558e-05 |
| 2981 |          33 | 174.207.24.231  |                7.36558e-05 |
| 2982 |          33 | 174.207.24.245  |                7.36558e-05 |
| 2983 |          33 | 174.207.25.219  |                7.36558e-05 |
| 2984 |          33 | 174.207.34.2    |                7.36558e-05 |
| 2985 |          33 | 174.207.5.143   |                7.36558e-05 |
| 2986 |          33 | 174.207.8.123   |                7.36558e-05 |
| 2987 |          33 | 212.102.40.134  |                7.36558e-05 |
| 2988 |          33 | 31.14.72.17     |                7.36558e-05 |
| 2989 |          33 | 31.14.72.25     |                7.36558e-05 |
| 2990 |          33 | 50.84.152.78    |                7.36558e-05 |
| 2991 |          33 | 67.10.164.55    |                7.36558e-05 |
| 2992 |          33 | 72.128.151.210  |                7.36558e-05 |
| 2993 |          33 | 75.131.8.233    |                7.36558e-05 |
| 2994 |          33 | 89.187.164.81   |                7.36558e-05 |
| 2995 |          33 | 99.154.3.0      |                7.36558e-05 |
| 2996 |          33 | 99.203.154.89   |                7.36558e-05 |
| 2997 |          33 | 99.203.90.211   |                7.36558e-05 |
| 2998 |          33 | 99.66.104.96    |                7.36558e-05 |
| 2999 |          33 | 104.223.95.94   |                4.91039e-05 |
| 3000 |          33 | 107.128.5.96    |                4.91039e-05 |
| 3001 |          33 | 107.77.169.9    |                4.91039e-05 |
| 3002 |          33 | 107.77.189.49   |                4.91039e-05 |
| 3003 |          33 | 107.77.201.226  |                4.91039e-05 |
| 3004 |          33 | 107.77.221.136  |                4.91039e-05 |
| 3005 |          33 | 107.77.222.131  |                4.91039e-05 |
| 3006 |          33 | 107.77.222.8    |                4.91039e-05 |
| 3007 |          33 | 130.45.24.15    |                4.91039e-05 |
| 3008 |          33 | 162.194.246.150 |                4.91039e-05 |
| 3009 |          33 | 172.58.110.156  |                4.91039e-05 |
| 3010 |          33 | 172.58.99.142   |                4.91039e-05 |
| 3011 |          33 | 172.85.165.138  |                4.91039e-05 |
| 3012 |          33 | 173.174.212.216 |                4.91039e-05 |
| 3013 |          33 | 174.207.14.251  |                4.91039e-05 |
| 3014 |          33 | 174.246.197.129 |                4.91039e-05 |
| 3015 |          33 | 206.121.37.170  |                4.91039e-05 |
| 3016 |          33 | 208.54.86.244   |                4.91039e-05 |
| 3017 |          33 | 31.14.72.10     |                4.91039e-05 |
| 3018 |          33 | 31.14.72.18     |                4.91039e-05 |
| 3019 |          33 | 31.14.72.31     |                4.91039e-05 |
| 3020 |          33 | 31.14.72.32     |                4.91039e-05 |
| 3021 |          33 | 66.194.160.106  |                4.91039e-05 |
| 3022 |          33 | 67.79.223.235   |                4.91039e-05 |
| 3023 |          33 | 99.203.90.147   |                4.91039e-05 |
| 3024 |          33 | 99.203.91.58    |                4.91039e-05 |
| 3025 |          33 | 104.223.91.55   |                2.45519e-05 |
| 3026 |          33 | 104.223.95.92   |                2.45519e-05 |
| 3027 |          33 | 107.77.169.55   |                2.45519e-05 |
| 3028 |          33 | 107.77.221.160  |                2.45519e-05 |
| 3029 |          33 | 107.77.221.40   |                2.45519e-05 |
| 3030 |          33 | 107.77.222.171  |                2.45519e-05 |
| 3031 |          33 | 161.129.70.3    |                2.45519e-05 |
| 3032 |          33 | 166.251.101.230 |                2.45519e-05 |
| 3033 |          33 | 170.248.173.246 |                2.45519e-05 |
| 3034 |          33 | 174.246.161.224 |                2.45519e-05 |
| 3035 |          33 | 174.246.198.233 |                2.45519e-05 |
| 3036 |          33 | 201.163.149.250 |                2.45519e-05 |
| 3037 |          33 | 31.14.72.19     |                2.45519e-05 |
| 3038 |          33 | 66.154.110.205  |                2.45519e-05 |
| 3039 |          33 | 70.117.59.30    |                2.45519e-05 |
| 3040 |          33 | 72.191.30.236   |                2.45519e-05 |
| 3041 |          33 | 74.3.157.69     |                2.45519e-05 |
| 3042 |          33 | 97.77.227.131   |                2.45519e-05 |
| 3198 |          34 | 107.77.217.37   |                7.53636e-05 |
| 3199 |          34 | 107.77.218.153  |                7.53636e-05 |
| 3200 |          34 | 107.77.218.203  |                7.53636e-05 |
| 3201 |          34 | 107.77.221.19   |                7.53636e-05 |
| 3202 |          34 | 12.97.187.15    |                7.53636e-05 |
| 3203 |          34 | 129.115.195.14  |                7.53636e-05 |
| 3204 |          34 | 129.115.195.42  |                7.53636e-05 |
| 3205 |          34 | 167.24.24.150   |                7.53636e-05 |
| 3206 |          34 | 172.58.107.104  |                7.53636e-05 |
| 3207 |          34 | 173.127.77.151  |                7.53636e-05 |
| 3208 |          34 | 174.207.4.87    |                7.53636e-05 |
| 3209 |          34 | 174.244.80.215  |                7.53636e-05 |
| 3210 |          34 | 184.203.109.10  |                7.53636e-05 |
| 3211 |          34 | 184.203.21.120  |                7.53636e-05 |
| 3212 |          34 | 184.226.92.76   |                7.53636e-05 |
| 3213 |          34 | 209.6.64.234    |                7.53636e-05 |
| 3214 |          34 | 67.11.141.218   |                7.53636e-05 |
| 3215 |          34 | 70.120.28.245   |                7.53636e-05 |
| 3216 |          34 | 73.34.177.179   |                7.53636e-05 |
| 3217 |          34 | 107.77.217.54   |                3.76818e-05 |
| 3218 |          34 | 107.77.220.85   |                3.76818e-05 |
| 3219 |          34 | 107.77.221.14   |                3.76818e-05 |
| 3220 |          34 | 107.77.221.166  |                3.76818e-05 |
| 3221 |          34 | 107.77.221.66   |                3.76818e-05 |
| 3222 |          34 | 107.77.222.197  |                3.76818e-05 |
| 3223 |          34 | 12.131.11.26    |                3.76818e-05 |
| 3224 |          34 | 12.5.63.210     |                3.76818e-05 |
| 3225 |          34 | 12.97.187.16    |                3.76818e-05 |
| 3226 |          34 | 129.115.195.10  |                3.76818e-05 |
| 3227 |          34 | 172.58.107.107  |                3.76818e-05 |
| 3228 |          34 | 172.58.107.139  |                3.76818e-05 |
| 3229 |          34 | 172.58.110.148  |                3.76818e-05 |
| 3230 |          34 | 172.58.110.227  |                3.76818e-05 |
| 3231 |          34 | 172.58.70.158   |                3.76818e-05 |
| 3232 |          34 | 172.58.96.115   |                3.76818e-05 |
| 3233 |          34 | 173.127.40.25   |                3.76818e-05 |
| 3234 |          34 | 173.127.7.219   |                3.76818e-05 |
| 3235 |          34 | 174.197.4.51    |                3.76818e-05 |
| 3236 |          34 | 174.197.6.43    |                3.76818e-05 |
| 3237 |          34 | 174.197.9.102   |                3.76818e-05 |
| 3238 |          34 | 174.207.12.122  |                3.76818e-05 |
| 3239 |          34 | 174.207.4.226   |                3.76818e-05 |
| 3240 |          34 | 174.244.81.91   |                3.76818e-05 |
| 3241 |          34 | 174.246.161.148 |                3.76818e-05 |
| 3242 |          34 | 184.203.5.0     |                3.76818e-05 |
| 3243 |          34 | 184.226.46.205  |                3.76818e-05 |
| 3244 |          34 | 187.201.77.208  |                3.76818e-05 |
| 3245 |          34 | 189.166.150.196 |                3.76818e-05 |
| 3246 |          34 | 24.153.137.59   |                3.76818e-05 |
| 3247 |          34 | 24.227.144.238  |                3.76818e-05 |
| 3248 |          34 | 24.27.227.1     |                3.76818e-05 |
| 3249 |          34 | 50.207.115.213  |                3.76818e-05 |
| 3250 |          34 | 67.79.36.55     |                3.76818e-05 |
| 3251 |          34 | 71.41.242.68    |                3.76818e-05 |
| 3252 |          34 | 71.42.230.162   |                3.76818e-05 |
| 3253 |          34 | 71.78.189.122   |                3.76818e-05 |
| 3254 |          34 | 72.181.117.73   |                3.76818e-05 |
| 3255 |          34 | 73.163.186.32   |                3.76818e-05 |
| 3256 |          34 | 76.193.64.119   |                3.76818e-05 |
| 3257 |          34 | 89.187.175.48   |                3.76818e-05 |
| 3258 |          34 | 96.8.130.209    |                3.76818e-05 |
| 3372 |          51 | 107.77.217.144  |                9.12187e-05 |
| 3373 |          51 | 107.77.217.6    |                9.12187e-05 |
| 3374 |          51 | 107.77.218.22   |                9.12187e-05 |
| 3375 |          51 | 107.77.219.20   |                9.12187e-05 |
| 3376 |          51 | 107.77.219.64   |                9.12187e-05 |
| 3377 |          51 | 107.77.221.227  |                9.12187e-05 |
| 3378 |          51 | 172.58.107.237  |                9.12187e-05 |
| 3379 |          51 | 174.207.15.244  |                9.12187e-05 |
| 3380 |          51 | 174.207.17.170  |                9.12187e-05 |
| 3381 |          51 | 174.207.6.56    |                9.12187e-05 |
| 3382 |          51 | 71.81.176.46    |                9.12187e-05 |
| 3383 |          51 | 72.180.171.13   |                9.12187e-05 |
| 3384 |          51 | 72.181.117.226  |                9.12187e-05 |
| 3385 |          51 | 97.77.146.130   |                9.12187e-05 |
| 3386 |          51 | 107.77.189.64   |                6.08125e-05 |
| 3387 |          51 | 107.77.217.140  |                6.08125e-05 |
| 3388 |          51 | 107.77.217.156  |                6.08125e-05 |
| 3389 |          51 | 107.77.217.39   |                6.08125e-05 |
| 3390 |          51 | 107.77.217.63   |                6.08125e-05 |
| 3391 |          51 | 173.174.169.114 |                6.08125e-05 |
| 3392 |          51 | 174.206.24.69   |                6.08125e-05 |
| 3393 |          51 | 174.207.12.102  |                6.08125e-05 |
| 3394 |          51 | 174.207.2.208   |                6.08125e-05 |
| 3395 |          51 | 174.246.196.23  |                6.08125e-05 |
| 3396 |          51 | 72.178.121.51   |                6.08125e-05 |
| 3397 |          51 | 76.185.247.236  |                6.08125e-05 |
| 3398 |          51 | 107.77.189.15   |                3.04062e-05 |
| 3399 |          51 | 107.77.217.129  |                3.04062e-05 |
| 3400 |          51 | 107.77.217.211  |                3.04062e-05 |
| 3401 |          51 | 107.77.217.67   |                3.04062e-05 |
| 3402 |          51 | 107.77.218.109  |                3.04062e-05 |
| 3403 |          51 | 107.77.218.158  |                3.04062e-05 |
| 3404 |          51 | 107.77.218.42   |                3.04062e-05 |
| 3405 |          51 | 107.77.218.64   |                3.04062e-05 |
| 3406 |          51 | 107.77.218.99   |                3.04062e-05 |
| 3407 |          51 | 107.77.219.210  |                3.04062e-05 |
| 3408 |          51 | 107.77.219.80   |                3.04062e-05 |
| 3409 |          51 | 107.77.220.54   |                3.04062e-05 |
| 3410 |          51 | 107.77.221.32   |                3.04062e-05 |
| 3411 |          51 | 107.77.221.97   |                3.04062e-05 |
| 3412 |          51 | 107.77.222.175  |                3.04062e-05 |
| 3413 |          51 | 107.77.222.97   |                3.04062e-05 |
| 3414 |          51 | 108.199.117.188 |                3.04062e-05 |
| 3415 |          51 | 12.252.48.110   |                3.04062e-05 |
| 3416 |          51 | 172.58.107.163  |                3.04062e-05 |
| 3417 |          51 | 172.58.107.176  |                3.04062e-05 |
| 3418 |          51 | 172.58.110.234  |                3.04062e-05 |
| 3419 |          51 | 173.127.44.65   |                3.04062e-05 |
| 3420 |          51 | 174.207.1.195   |                3.04062e-05 |
| 3421 |          51 | 174.207.2.195   |                3.04062e-05 |
| 3422 |          51 | 174.207.5.154   |                3.04062e-05 |
| 3423 |          51 | 174.207.8.218   |                3.04062e-05 |
| 3424 |          51 | 70.123.183.233  |                3.04062e-05 |
| 3425 |          51 | 71.40.4.250     |                3.04062e-05 |
| 3426 |          51 | 96.8.178.202    |                3.04062e-05 |
| 3642 |          52 | 107.77.217.110  |                7.13445e-05 |
| 3643 |          52 | 107.77.219.83   |                7.13445e-05 |
| 3644 |          52 | 107.77.220.169  |                7.13445e-05 |
| 3645 |          52 | 107.77.221.91   |                7.13445e-05 |
| 3646 |          52 | 173.123.11.246  |                7.13445e-05 |
| 3647 |          52 | 173.123.113.69  |                7.13445e-05 |
| 3648 |          52 | 173.123.129.83  |                7.13445e-05 |
| 3649 |          52 | 173.127.53.126  |                7.13445e-05 |
| 3650 |          52 | 173.239.232.120 |                7.13445e-05 |
| 3651 |          52 | 173.239.232.167 |                7.13445e-05 |
| 3652 |          52 | 173.239.232.175 |                7.13445e-05 |
| 3653 |          52 | 173.239.232.60  |                7.13445e-05 |
| 3654 |          52 | 173.239.232.7   |                7.13445e-05 |
| 3655 |          52 | 174.207.15.240  |                7.13445e-05 |
| 3656 |          52 | 174.207.17.75   |                7.13445e-05 |
| 3657 |          52 | 184.203.110.171 |                7.13445e-05 |
| 3658 |          52 | 184.203.153.83  |                7.13445e-05 |
| 3659 |          52 | 184.203.35.148  |                7.13445e-05 |
| 3660 |          52 | 184.226.38.142  |                7.13445e-05 |
| 3661 |          52 | 184.226.43.98   |                7.13445e-05 |
| 3662 |          52 | 184.226.5.254   |                7.13445e-05 |
| 3663 |          52 | 184.226.91.216  |                7.13445e-05 |
| 3664 |          52 | 192.171.117.214 |                7.13445e-05 |
| 3665 |          52 | 205.251.148.130 |                7.13445e-05 |
| 3666 |          52 | 205.251.150.202 |                7.13445e-05 |
| 3667 |          52 | 45.56.151.68    |                7.13445e-05 |
| 3668 |          52 | 72.169.80.108   |                7.13445e-05 |
| 3669 |          52 | 72.169.81.20    |                7.13445e-05 |
| 3670 |          52 | 72.169.81.64    |                7.13445e-05 |
| 3671 |          52 | 72.169.81.79    |                7.13445e-05 |
| 3672 |          52 | 74.81.88.130    |                7.13445e-05 |
| 3673 |          52 | 74.81.88.90     |                7.13445e-05 |
| 3674 |          52 | 77.111.247.10   |                7.13445e-05 |
| 3675 |          52 | 107.77.208.122  |                3.56722e-05 |
| 3676 |          52 | 107.77.217.125  |                3.56722e-05 |
| 3677 |          52 | 107.77.217.81   |                3.56722e-05 |
| 3678 |          52 | 107.77.218.137  |                3.56722e-05 |
| 3679 |          52 | 107.77.220.67   |                3.56722e-05 |
| 3680 |          52 | 107.77.220.71   |                3.56722e-05 |
| 3681 |          52 | 107.77.220.81   |                3.56722e-05 |
| 3682 |          52 | 107.77.221.13   |                3.56722e-05 |
| 3683 |          52 | 107.77.221.202  |                3.56722e-05 |
| 3684 |          52 | 107.77.222.168  |                3.56722e-05 |
| 3685 |          52 | 107.77.222.200  |                3.56722e-05 |
| 3686 |          52 | 136.50.24.156   |                3.56722e-05 |
| 3687 |          52 | 172.58.107.121  |                3.56722e-05 |
| 3688 |          52 | 172.58.107.4    |                3.56722e-05 |
| 3689 |          52 | 172.58.110.189  |                3.56722e-05 |
| 3690 |          52 | 173.123.135.208 |                3.56722e-05 |
| 3691 |          52 | 173.123.79.79   |                3.56722e-05 |
| 3692 |          52 | 173.127.107.19  |                3.56722e-05 |
| 3693 |          52 | 173.127.33.133  |                3.56722e-05 |
| 3694 |          52 | 173.127.36.228  |                3.56722e-05 |
| 3695 |          52 | 173.149.136.60  |                3.56722e-05 |
| 3696 |          52 | 173.149.160.96  |                3.56722e-05 |
| 3697 |          52 | 173.149.241.156 |                3.56722e-05 |
| 3698 |          52 | 173.149.246.55  |                3.56722e-05 |
| 3699 |          52 | 173.239.232.101 |                3.56722e-05 |
| 3700 |          52 | 173.239.232.136 |                3.56722e-05 |
| 3701 |          52 | 173.239.232.141 |                3.56722e-05 |
| 3702 |          52 | 173.239.232.142 |                3.56722e-05 |
| 3703 |          52 | 173.239.232.149 |                3.56722e-05 |
| 3704 |          52 | 173.239.232.154 |                3.56722e-05 |
| 3705 |          52 | 173.239.232.163 |                3.56722e-05 |
| 3706 |          52 | 173.239.232.170 |                3.56722e-05 |
| 3707 |          52 | 173.239.232.173 |                3.56722e-05 |
| 3708 |          52 | 173.239.232.95  |                3.56722e-05 |
| 3709 |          52 | 174.246.198.133 |                3.56722e-05 |
| 3710 |          52 | 184.203.143.190 |                3.56722e-05 |
| 3711 |          52 | 184.203.149.107 |                3.56722e-05 |
| 3712 |          52 | 184.203.189.129 |                3.56722e-05 |
| 3713 |          52 | 184.203.198.216 |                3.56722e-05 |
| 3714 |          52 | 184.203.98.134  |                3.56722e-05 |
| 3715 |          52 | 184.226.32.25   |                3.56722e-05 |
| 3716 |          52 | 184.226.79.141  |                3.56722e-05 |
| 3717 |          52 | 38.70.11.18     |                3.56722e-05 |
| 3718 |          52 | 70.121.228.51   |                3.56722e-05 |
| 3719 |          52 | 70.123.210.135  |                3.56722e-05 |
| 3720 |          52 | 71.41.242.76    |                3.56722e-05 |
| 3721 |          52 | 72.169.80.141   |                3.56722e-05 |
| 3722 |          52 | 72.177.198.47   |                3.56722e-05 |
| 3723 |          52 | 72.181.57.49    |                3.56722e-05 |
| 3724 |          52 | 74.81.88.18     |                3.56722e-05 |
| 3725 |          52 | 98.6.141.91     |                3.56722e-05 |
| 3726 |          52 | 99.203.212.90   |                3.56722e-05 |
| 3727 |          52 | 99.203.213.15   |                3.56722e-05 |
| 3728 |          52 | 99.203.213.192  |                3.56722e-05 |
| 3729 |          52 | 99.203.213.199  |                3.56722e-05 |
| 3730 |          52 | 99.203.213.25   |                3.56722e-05 |
| 3731 |          52 | 99.203.90.176   |                3.56722e-05 |
| 3732 |          52 | 99.203.90.177   |                3.56722e-05 |
| 3733 |          52 | 99.203.90.180   |                3.56722e-05 |
| 3734 |          52 | 99.203.90.207   |                3.56722e-05 |
| 3735 |          52 | 99.203.90.36    |                3.56722e-05 |
| 3736 |          52 | 99.203.91.87    |                3.56722e-05 |
| 3737 |          52 | 99.203.91.92    |                3.56722e-05 |
| 3860 |          53 | 136.50.75.159   |                8.12964e-05 |
| 3861 |          53 | 166.205.190.141 |                8.12964e-05 |
| 3862 |          53 | 172.56.6.145    |                8.12964e-05 |
| 3863 |          53 | 172.56.6.79     |                8.12964e-05 |
| 3864 |          53 | 172.58.101.238  |                8.12964e-05 |
| 3865 |          53 | 172.58.107.0    |                8.12964e-05 |
| 3866 |          53 | 172.58.109.225  |                8.12964e-05 |
| 3867 |          53 | 172.58.99.225   |                8.12964e-05 |
| 3868 |          53 | 45.131.192.121  |                8.12964e-05 |
| 3869 |          53 | 68.203.203.127  |                8.12964e-05 |
| 3870 |          53 | 71.40.42.51     |                8.12964e-05 |
| 3871 |          53 | 76.255.12.121   |                8.12964e-05 |
| 3872 |          53 | 107.77.189.26   |                5.41976e-05 |
| 3873 |          53 | 107.77.222.188  |                5.41976e-05 |
| 3874 |          53 | 12.246.113.186  |                5.41976e-05 |
| 3875 |          53 | 172.58.101.72   |                5.41976e-05 |
| 3876 |          53 | 172.58.102.135  |                5.41976e-05 |
| 3877 |          53 | 172.58.103.175  |                5.41976e-05 |
| 3878 |          53 | 172.58.109.128  |                5.41976e-05 |
| 3879 |          53 | 172.58.109.15   |                5.41976e-05 |
| 3880 |          53 | 172.58.67.117   |                5.41976e-05 |
| 3881 |          53 | 193.56.117.100  |                5.41976e-05 |
| 3882 |          53 | 24.243.58.134   |                5.41976e-05 |
| 3883 |          53 | 70.122.52.140   |                5.41976e-05 |
| 3884 |          53 | 71.78.10.227    |                5.41976e-05 |
| 3885 |          53 | 107.77.189.2    |                2.70988e-05 |
| 3886 |          53 | 107.77.189.30   |                2.70988e-05 |
| 3887 |          53 | 107.77.217.18   |                2.70988e-05 |
| 3888 |          53 | 107.77.217.70   |                2.70988e-05 |
| 3889 |          53 | 107.77.218.11   |                2.70988e-05 |
| 3890 |          53 | 107.77.218.168  |                2.70988e-05 |
| 3891 |          53 | 107.77.219.194  |                2.70988e-05 |
| 3892 |          53 | 107.77.219.83   |                2.70988e-05 |
| 3893 |          53 | 107.77.221.210  |                2.70988e-05 |
| 3894 |          53 | 107.77.222.31   |                2.70988e-05 |
| 3895 |          53 | 172.125.4.125   |                2.70988e-05 |
| 3896 |          53 | 172.56.6.147    |                2.70988e-05 |
| 3897 |          53 | 172.56.7.58     |                2.70988e-05 |
| 3898 |          53 | 172.58.101.155  |                2.70988e-05 |
| 3899 |          53 | 172.58.101.215  |                2.70988e-05 |
| 3900 |          53 | 172.58.102.183  |                2.70988e-05 |
| 3901 |          53 | 172.58.103.199  |                2.70988e-05 |
| 3902 |          53 | 172.58.107.116  |                2.70988e-05 |
| 3903 |          53 | 172.58.109.21   |                2.70988e-05 |
| 3904 |          53 | 172.58.109.69   |                2.70988e-05 |
| 3905 |          53 | 172.58.109.80   |                2.70988e-05 |
| 3906 |          53 | 172.58.109.83   |                2.70988e-05 |
| 3907 |          53 | 172.58.111.216  |                2.70988e-05 |
| 3908 |          53 | 172.58.111.248  |                2.70988e-05 |
| 3909 |          53 | 172.58.99.75    |                2.70988e-05 |
| 3910 |          53 | 174.207.10.127  |                2.70988e-05 |
| 3911 |          53 | 174.207.15.2    |                2.70988e-05 |
| 3912 |          53 | 45.132.115.22   |                2.70988e-05 |
| 3913 |          53 | 96.8.132.129    |                2.70988e-05 |
| 3914 |          53 | 96.8.177.40     |                2.70988e-05 |
| 3915 |          53 | 99.203.212.119  |                2.70988e-05 |
| 3916 |          53 | 99.43.140.149   |                2.70988e-05 |
| 4088 |          55 | 107.77.217.210  |                9.26698e-05 |
| 4089 |          55 | 107.77.217.229  |                9.26698e-05 |
| 4090 |          55 | 107.77.220.42   |                9.26698e-05 |
| 4091 |          55 | 107.77.221.127  |                9.26698e-05 |
| 4092 |          55 | 107.77.221.165  |                9.26698e-05 |
| 4093 |          55 | 107.77.221.200  |                9.26698e-05 |
| 4094 |          55 | 107.77.221.211  |                9.26698e-05 |
| 4095 |          55 | 108.160.237.17  |                9.26698e-05 |
| 4096 |          55 | 162.89.0.47     |                9.26698e-05 |
| 4097 |          55 | 166.181.85.9    |                9.26698e-05 |
| 4098 |          55 | 166.247.59.40   |                9.26698e-05 |
| 4099 |          55 | 166.251.101.149 |                9.26698e-05 |
| 4100 |          55 | 166.251.101.225 |                9.26698e-05 |
| 4101 |          55 | 166.251.106.121 |                9.26698e-05 |
| 4102 |          55 | 166.251.112.197 |                9.26698e-05 |
| 4103 |          55 | 172.58.107.245  |                9.26698e-05 |
| 4104 |          55 | 172.58.110.141  |                9.26698e-05 |
| 4105 |          55 | 172.58.110.144  |                9.26698e-05 |
| 4106 |          55 | 172.58.110.170  |                9.26698e-05 |
| 4107 |          55 | 172.58.110.199  |                9.26698e-05 |
| 4108 |          55 | 172.58.110.210  |                9.26698e-05 |
| 4109 |          55 | 172.58.110.242  |                9.26698e-05 |
| 4110 |          55 | 172.58.67.135   |                9.26698e-05 |
| 4111 |          55 | 173.123.125.46  |                9.26698e-05 |
| 4112 |          55 | 173.123.148.85  |                9.26698e-05 |
| 4113 |          55 | 173.123.211.50  |                9.26698e-05 |
| 4114 |          55 | 173.127.20.174  |                9.26698e-05 |
| 4115 |          55 | 173.149.192.220 |                9.26698e-05 |
| 4116 |          55 | 173.149.224.202 |                9.26698e-05 |
| 4117 |          55 | 174.246.193.205 |                9.26698e-05 |
| 4118 |          55 | 174.246.198.172 |                9.26698e-05 |
| 4119 |          55 | 209.36.87.18    |                9.26698e-05 |
| 4120 |          55 | 221.229.204.176 |                9.26698e-05 |
| 4121 |          55 | 24.153.137.59   |                9.26698e-05 |
| 4122 |          55 | 70.40.29.112    |                9.26698e-05 |
| 4123 |          55 | 76.185.252.88   |                9.26698e-05 |
| 4124 |          55 | 98.6.99.123     |                9.26698e-05 |
| 4125 |          55 | 99.203.90.140   |                9.26698e-05 |
| 4126 |          55 | 107.77.198.136  |                4.63349e-05 |
| 4127 |          55 | 107.77.198.177  |                4.63349e-05 |
| 4128 |          55 | 107.77.217.131  |                4.63349e-05 |
| 4129 |          55 | 107.77.217.85   |                4.63349e-05 |
| 4130 |          55 | 107.77.218.136  |                4.63349e-05 |
| 4131 |          55 | 107.77.218.154  |                4.63349e-05 |
| 4132 |          55 | 107.77.219.100  |                4.63349e-05 |
| 4133 |          55 | 107.77.219.175  |                4.63349e-05 |
| 4134 |          55 | 107.77.221.157  |                4.63349e-05 |
| 4135 |          55 | 107.77.221.160  |                4.63349e-05 |
| 4136 |          55 | 107.77.221.82   |                4.63349e-05 |
| 4137 |          55 | 107.77.221.92   |                4.63349e-05 |
| 4138 |          55 | 129.111.179.104 |                4.63349e-05 |
| 4139 |          55 | 166.247.59.41   |                4.63349e-05 |
| 4140 |          55 | 166.251.101.21  |                4.63349e-05 |
| 4141 |          55 | 166.251.101.210 |                4.63349e-05 |
| 4142 |          55 | 166.251.109.47  |                4.63349e-05 |
| 4143 |          55 | 166.251.112.195 |                4.63349e-05 |
| 4144 |          55 | 172.58.102.173  |                4.63349e-05 |
| 4145 |          55 | 172.58.102.206  |                4.63349e-05 |
| 4146 |          55 | 172.58.102.252  |                4.63349e-05 |
| 4147 |          55 | 172.58.107.101  |                4.63349e-05 |
| 4148 |          55 | 172.58.107.123  |                4.63349e-05 |
| 4149 |          55 | 172.58.107.137  |                4.63349e-05 |
| 4150 |          55 | 172.58.107.146  |                4.63349e-05 |
| 4151 |          55 | 172.58.107.192  |                4.63349e-05 |
| 4152 |          55 | 172.58.107.33   |                4.63349e-05 |
| 4153 |          55 | 172.58.107.35   |                4.63349e-05 |
| 4154 |          55 | 172.58.107.63   |                4.63349e-05 |
| 4155 |          55 | 172.58.107.89   |                4.63349e-05 |
| 4156 |          55 | 172.58.110.135  |                4.63349e-05 |
| 4157 |          55 | 172.58.110.154  |                4.63349e-05 |
| 4158 |          55 | 172.58.110.171  |                4.63349e-05 |
| 4159 |          55 | 172.58.110.184  |                4.63349e-05 |
| 4160 |          55 | 172.58.67.152   |                4.63349e-05 |
| 4161 |          55 | 172.58.99.204   |                4.63349e-05 |
| 4162 |          55 | 172.58.99.6     |                4.63349e-05 |
| 4163 |          55 | 173.123.196.135 |                4.63349e-05 |
| 4164 |          55 | 173.127.59.159  |                4.63349e-05 |
| 4165 |          55 | 173.149.140.36  |                4.63349e-05 |
| 4166 |          55 | 173.149.239.160 |                4.63349e-05 |
| 4167 |          55 | 173.149.245.229 |                4.63349e-05 |
| 4168 |          55 | 174.197.7.219   |                4.63349e-05 |
| 4169 |          55 | 174.207.0.183   |                4.63349e-05 |
| 4170 |          55 | 174.207.14.170  |                4.63349e-05 |
| 4171 |          55 | 174.207.16.31   |                4.63349e-05 |
| 4172 |          55 | 174.207.7.100   |                4.63349e-05 |
| 4173 |          55 | 174.244.81.249  |                4.63349e-05 |
| 4174 |          55 | 184.203.15.95   |                4.63349e-05 |
| 4175 |          55 | 184.203.207.171 |                4.63349e-05 |
| 4176 |          55 | 185.245.87.158  |                4.63349e-05 |
| 4177 |          55 | 63.97.8.65      |                4.63349e-05 |
| 4178 |          55 | 70.117.30.148   |                4.63349e-05 |
| 4179 |          55 | 71.40.24.180    |                4.63349e-05 |
| 4180 |          55 | 71.42.140.139   |                4.63349e-05 |
| 4181 |          55 | 72.178.102.21   |                4.63349e-05 |
| 4182 |          55 | 72.181.126.124  |                4.63349e-05 |
| 4277 |          56 | 107.242.113.36  |                8.93708e-05 |
| 4278 |          56 | 172.58.107.216  |                8.93708e-05 |
| 4279 |          56 | 172.58.107.223  |                8.93708e-05 |
| 4280 |          56 | 172.58.110.174  |                8.93708e-05 |
| 4281 |          56 | 97.105.164.91   |                8.93708e-05 |
| 4282 |          56 | 99.30.242.101   |                8.93708e-05 |
| 4283 |          56 | 104.14.116.184  |                5.95806e-05 |
| 4284 |          56 | 172.58.107.203  |                5.95806e-05 |
| 4285 |          56 | 172.58.110.183  |                5.95806e-05 |
| 4286 |          56 | 172.58.110.220  |                5.95806e-05 |
| 4287 |          56 | 172.58.110.239  |                5.95806e-05 |
| 4288 |          56 | 172.58.111.247  |                5.95806e-05 |
| 4289 |          56 | 70.5.31.154     |                5.95806e-05 |
| 4290 |          56 | 71.146.116.173  |                5.95806e-05 |
| 4291 |          56 | 72.181.99.88    |                5.95806e-05 |
| 4292 |          56 | 107.77.222.113  |                2.97903e-05 |
| 4293 |          56 | 172.58.107.141  |                2.97903e-05 |
| 4294 |          56 | 172.58.107.145  |                2.97903e-05 |
| 4295 |          56 | 172.58.107.177  |                2.97903e-05 |
| 4296 |          56 | 172.58.107.190  |                2.97903e-05 |
| 4297 |          56 | 172.58.107.23   |                2.97903e-05 |
| 4298 |          56 | 172.58.107.230  |                2.97903e-05 |
| 4299 |          56 | 172.58.107.239  |                2.97903e-05 |
| 4300 |          56 | 172.58.107.3    |                2.97903e-05 |
| 4301 |          56 | 172.58.107.34   |                2.97903e-05 |
| 4302 |          56 | 172.58.107.62   |                2.97903e-05 |
| 4303 |          56 | 172.58.108.142  |                2.97903e-05 |
| 4304 |          56 | 172.58.108.157  |                2.97903e-05 |
| 4305 |          56 | 172.58.108.17   |                2.97903e-05 |
| 4306 |          56 | 172.58.109.124  |                2.97903e-05 |
| 4307 |          56 | 172.58.110.140  |                2.97903e-05 |
| 4308 |          56 | 172.58.111.140  |                2.97903e-05 |
| 4309 |          56 | 174.244.48.221  |                2.97903e-05 |
| 4310 |          56 | 174.246.132.173 |                2.97903e-05 |
| 4311 |          56 | 174.246.135.44  |                2.97903e-05 |
| 4312 |          56 | 47.186.8.231    |                2.97903e-05 |
| 4313 |          56 | 47.6.87.22      |                2.97903e-05 |
| 4314 |          56 | 71.78.157.46    |                2.97903e-05 |
| 4315 |          56 | 71.78.37.150    |                2.97903e-05 |
| 4316 |          56 | 72.181.156.185  |                2.97903e-05 |
| 4317 |          56 | 76.182.166.199  |                2.97903e-05 |
| 4414 |          57 | 107.77.222.141  |                8.8642e-05  |
| 4415 |          57 | 107.77.222.165  |                8.8642e-05  |
| 4416 |          57 | 172.56.6.62     |                8.8642e-05  |
| 4417 |          57 | 172.58.103.143  |                8.8642e-05  |
| 4418 |          57 | 173.127.105.106 |                8.8642e-05  |
| 4419 |          57 | 176.113.73.30   |                8.8642e-05  |
| 4420 |          57 | 185.244.215.42  |                8.8642e-05  |
| 4421 |          57 | 23.227.143.114  |                8.8642e-05  |
| 4422 |          57 | 24.28.146.220   |                8.8642e-05  |
| 4423 |          57 | 5.181.234.38    |                8.8642e-05  |
| 4424 |          57 | 72.177.15.120   |                8.8642e-05  |
| 4425 |          57 | 72.179.161.188  |                8.8642e-05  |
| 4426 |          57 | 75.1.220.159    |                8.8642e-05  |
| 4427 |          57 | 76.185.204.129  |                8.8642e-05  |
| 4428 |          57 | 107.152.102.194 |                5.90947e-05 |
| 4429 |          57 | 107.77.217.187  |                5.90947e-05 |
| 4430 |          57 | 107.77.217.4    |                5.90947e-05 |
| 4431 |          57 | 107.77.218.175  |                5.90947e-05 |
| 4432 |          57 | 107.77.219.106  |                5.90947e-05 |
| 4433 |          57 | 192.119.10.210  |                5.90947e-05 |
| 4434 |          57 | 212.103.49.182  |                5.90947e-05 |
| 4435 |          57 | 23.227.143.66   |                5.90947e-05 |
| 4436 |          57 | 70.123.197.193  |                5.90947e-05 |
| 4437 |          57 | 92.38.148.46    |                5.90947e-05 |
| 4438 |          57 | 107.77.217.17   |                2.95473e-05 |
| 4439 |          57 | 107.77.218.167  |                2.95473e-05 |
| 4440 |          57 | 107.77.218.49   |                2.95473e-05 |
| 4441 |          57 | 107.77.219.22   |                2.95473e-05 |
| 4442 |          57 | 107.77.221.107  |                2.95473e-05 |
| 4443 |          57 | 107.77.221.18   |                2.95473e-05 |
| 4444 |          57 | 107.77.221.69   |                2.95473e-05 |
| 4445 |          57 | 107.77.222.144  |                2.95473e-05 |
| 4446 |          57 | 107.77.222.175  |                2.95473e-05 |
| 4447 |          57 | 107.77.222.57   |                2.95473e-05 |
| 4448 |          57 | 107.77.222.95   |                2.95473e-05 |
| 4449 |          57 | 108.89.71.245   |                2.95473e-05 |
| 4450 |          57 | 172.58.109.175  |                2.95473e-05 |
| 4451 |          57 | 172.58.30.200   |                2.95473e-05 |
| 4452 |          57 | 173.174.147.198 |                2.95473e-05 |
| 4453 |          57 | 212.103.49.178  |                2.95473e-05 |
| 4454 |          57 | 67.220.183.10   |                2.95473e-05 |
| 4455 |          57 | 70.125.229.227  |                2.95473e-05 |
| 4456 |          57 | 76.185.216.144  |                2.95473e-05 |
| 4457 |          57 | 92.38.148.50    |                2.95473e-05 |
| 4458 |          57 | 99.203.213.177  |                2.95473e-05 |
| 4558 |          58 | 107.77.198.228  |                6.69905e-05 |
| 4559 |          58 | 135.26.38.208   |                6.69905e-05 |
| 4560 |          58 | 172.58.109.196  |                6.69905e-05 |
| 4561 |          58 | 172.58.111.213  |                6.69905e-05 |
| 4562 |          58 | 50.30.92.29     |                6.69905e-05 |
| 4563 |          58 | 73.3.242.85     |                6.69905e-05 |
| 4564 |          58 | 89.187.175.23   |                6.69905e-05 |
| 4565 |          58 | 99.203.212.32   |                6.69905e-05 |
| 4566 |          58 | 99.48.230.77    |                6.69905e-05 |
| 4567 |          58 | 107.158.15.59   |                3.34952e-05 |
| 4568 |          58 | 107.77.217.19   |                3.34952e-05 |
| 4569 |          58 | 172.58.107.3    |                3.34952e-05 |
| 4570 |          58 | 172.58.110.149  |                3.34952e-05 |
| 4571 |          58 | 172.58.96.31    |                3.34952e-05 |
| 4572 |          58 | 172.58.96.4     |                3.34952e-05 |
| 4573 |          58 | 173.232.243.163 |                3.34952e-05 |
| 4574 |          58 | 38.133.200.21   |                3.34952e-05 |
| 4575 |          58 | 50.30.92.26     |                3.34952e-05 |
| 4576 |          58 | 69.59.91.191    |                3.34952e-05 |
| 4577 |          58 | 70.238.240.119  |                3.34952e-05 |
| 4578 |          58 | 72.179.189.42   |                3.34952e-05 |
| 4579 |          58 | 72.182.113.134  |                3.34952e-05 |
| 4580 |          58 | 99.203.213.118  |                3.34952e-05 |
| 4722 |          59 | 107.127.0.122   |                9.37061e-05 |
| 4723 |          59 | 107.77.220.21   |                9.37061e-05 |
| 4724 |          59 | 107.77.222.115  |                9.37061e-05 |
| 4725 |          59 | 107.77.222.175  |                9.37061e-05 |
| 4726 |          59 | 107.77.222.7    |                9.37061e-05 |
| 4727 |          59 | 172.58.102.131  |                9.37061e-05 |
| 4728 |          59 | 172.58.102.144  |                9.37061e-05 |
| 4729 |          59 | 172.58.110.195  |                9.37061e-05 |
| 4730 |          59 | 172.58.110.210  |                9.37061e-05 |
| 4731 |          59 | 172.58.110.236  |                9.37061e-05 |
| 4732 |          59 | 172.58.67.130   |                9.37061e-05 |
| 4733 |          59 | 172.58.68.25    |                9.37061e-05 |
| 4734 |          59 | 172.58.99.136   |                9.37061e-05 |
| 4735 |          59 | 172.58.99.251   |                9.37061e-05 |
| 4736 |          59 | 173.123.150.183 |                9.37061e-05 |
| 4737 |          59 | 173.123.172.254 |                9.37061e-05 |
| 4738 |          59 | 184.203.177.106 |                9.37061e-05 |
| 4739 |          59 | 184.226.101.48  |                9.37061e-05 |
| 4740 |          59 | 67.11.185.145   |                9.37061e-05 |
| 4741 |          59 | 99.203.154.98   |                9.37061e-05 |
| 4742 |          59 | 99.203.155.102  |                9.37061e-05 |
| 4743 |          59 | 99.203.26.103   |                9.37061e-05 |
| 4744 |          59 | 99.203.26.85    |                9.37061e-05 |
| 4745 |          59 | 99.203.27.141   |                9.37061e-05 |
| 4746 |          59 | 99.203.27.90    |                9.37061e-05 |
| 4747 |          59 | 99.203.40.221   |                9.37061e-05 |
| 4748 |          59 | 107.127.0.90    |                6.24707e-05 |
| 4749 |          59 | 107.77.218.55   |                6.24707e-05 |
| 4750 |          59 | 107.77.218.83   |                6.24707e-05 |
| 4751 |          59 | 107.77.220.12   |                6.24707e-05 |
| 4752 |          59 | 107.77.222.108  |                6.24707e-05 |
| 4753 |          59 | 107.77.222.18   |                6.24707e-05 |
| 4754 |          59 | 107.77.222.195  |                6.24707e-05 |
| 4755 |          59 | 107.77.222.231  |                6.24707e-05 |
| 4756 |          59 | 155.95.98.43    |                6.24707e-05 |
| 4757 |          59 | 155.95.98.61    |                6.24707e-05 |
| 4758 |          59 | 172.58.102.136  |                6.24707e-05 |
| 4759 |          59 | 172.58.102.201  |                6.24707e-05 |
| 4760 |          59 | 172.58.102.207  |                6.24707e-05 |
| 4761 |          59 | 172.58.107.212  |                6.24707e-05 |
| 4762 |          59 | 172.58.107.219  |                6.24707e-05 |
| 4763 |          59 | 172.58.107.3    |                6.24707e-05 |
| 4764 |          59 | 172.58.110.222  |                6.24707e-05 |
| 4765 |          59 | 172.58.70.161   |                6.24707e-05 |
| 4766 |          59 | 172.58.71.157   |                6.24707e-05 |
| 4767 |          59 | 172.58.99.108   |                6.24707e-05 |
| 4768 |          59 | 172.58.99.51    |                6.24707e-05 |
| 4769 |          59 | 173.149.157.75  |                6.24707e-05 |
| 4770 |          59 | 173.149.192.116 |                6.24707e-05 |
| 4771 |          59 | 173.173.244.231 |                6.24707e-05 |
| 4772 |          59 | 174.197.11.85   |                6.24707e-05 |
| 4773 |          59 | 174.244.67.71   |                6.24707e-05 |
| 4774 |          59 | 184.203.160.10  |                6.24707e-05 |
| 4775 |          59 | 184.223.233.47  |                6.24707e-05 |
| 4776 |          59 | 184.226.64.13   |                6.24707e-05 |
| 4777 |          59 | 70.114.115.136  |                6.24707e-05 |
| 4778 |          59 | 96.8.130.236    |                6.24707e-05 |
| 4779 |          59 | 96.8.178.201    |                6.24707e-05 |
| 4780 |          59 | 99.203.154.100  |                6.24707e-05 |
| 4781 |          59 | 99.203.154.222  |                6.24707e-05 |
| 4782 |          59 | 99.203.154.244  |                6.24707e-05 |
| 4783 |          59 | 99.203.155.22   |                6.24707e-05 |
| 4784 |          59 | 99.203.212.61   |                6.24707e-05 |
| 4785 |          59 | 99.203.26.185   |                6.24707e-05 |
| 4786 |          59 | 99.203.26.235   |                6.24707e-05 |
| 4787 |          59 | 99.203.27.127   |                6.24707e-05 |
| 4788 |          59 | 99.203.27.151   |                6.24707e-05 |
| 4789 |          59 | 104.190.247.101 |                3.12354e-05 |
| 4790 |          59 | 107.127.0.123   |                3.12354e-05 |
| 4791 |          59 | 107.77.220.52   |                3.12354e-05 |
| 4792 |          59 | 107.77.221.226  |                3.12354e-05 |
| 4793 |          59 | 107.77.222.216  |                3.12354e-05 |
| 4794 |          59 | 12.132.167.253  |                3.12354e-05 |
| 4795 |          59 | 12.87.39.218    |                3.12354e-05 |
| 4796 |          59 | 162.71.236.112  |                3.12354e-05 |
| 4797 |          59 | 172.56.7.48     |                3.12354e-05 |
| 4798 |          59 | 172.58.107.61   |                3.12354e-05 |
| 4799 |          59 | 172.58.107.84   |                3.12354e-05 |
| 4800 |          59 | 172.58.110.147  |                3.12354e-05 |
| 4801 |          59 | 172.58.110.164  |                3.12354e-05 |
| 4802 |          59 | 172.58.110.207  |                3.12354e-05 |
| 4803 |          59 | 172.58.67.160   |                3.12354e-05 |
| 4804 |          59 | 172.58.67.200   |                3.12354e-05 |
| 4805 |          59 | 172.58.70.194   |                3.12354e-05 |
| 4806 |          59 | 172.58.70.232   |                3.12354e-05 |
| 4807 |          59 | 172.58.99.4     |                3.12354e-05 |
| 4808 |          59 | 172.58.99.71    |                3.12354e-05 |
| 4809 |          59 | 184.203.197.206 |                3.12354e-05 |
| 4810 |          59 | 184.226.114.60  |                3.12354e-05 |
| 4811 |          59 | 72.20.140.35    |                3.12354e-05 |
| 4812 |          59 | 96.8.178.242    |                3.12354e-05 |
| 4813 |          59 | 99.203.154.146  |                3.12354e-05 |
| 4814 |          59 | 99.203.154.42   |                3.12354e-05 |
| 4815 |          59 | 99.203.155.226  |                3.12354e-05 |
| 4816 |          59 | 99.203.155.70   |                3.12354e-05 |
| 4817 |          59 | 99.203.26.44    |                3.12354e-05 |
| 4818 |          59 | 99.203.26.6     |                3.12354e-05 |
| 4819 |          59 | 99.203.26.61    |                3.12354e-05 |
| 4820 |          59 | 99.203.26.71    |                3.12354e-05 |
| 4821 |          59 | 99.203.26.91    |                3.12354e-05 |
| 4822 |          59 | 99.203.27.111   |                3.12354e-05 |
| 4823 |          59 | 99.203.27.212   |                3.12354e-05 |
| 4824 |          59 | 99.203.27.32    |                3.12354e-05 |
| 4825 |          59 | 99.203.27.94    |                3.12354e-05 |
| 4903 |          61 | 107.242.125.44  |                5.64557e-05 |
| 4904 |          61 | 107.242.125.9   |                5.64557e-05 |
| 4905 |          61 | 107.77.200.143  |                5.64557e-05 |
| 4906 |          61 | 107.77.201.138  |                5.64557e-05 |
| 4907 |          61 | 107.77.201.153  |                5.64557e-05 |
| 4908 |          61 | 107.77.201.227  |                5.64557e-05 |
| 4909 |          61 | 107.77.222.142  |                5.64557e-05 |
| 4910 |          61 | 172.56.6.5      |                5.64557e-05 |
| 4911 |          61 | 172.58.111.183  |                5.64557e-05 |
| 4912 |          61 | 172.58.97.156   |                5.64557e-05 |
| 4913 |          61 | 45.131.192.121  |                5.64557e-05 |
| 4914 |          61 | 45.131.192.135  |                5.64557e-05 |
| 4915 |          61 | 45.131.192.190  |                5.64557e-05 |
| 4916 |          61 | 47.183.194.27   |                5.64557e-05 |
| 4917 |          61 | 47.184.217.245  |                5.64557e-05 |
| 5020 |          62 | 107.77.219.219  |                8.08429e-05 |
| 5021 |          62 | 107.77.219.94   |                8.08429e-05 |
| 5022 |          62 | 107.77.222.190  |                8.08429e-05 |
| 5023 |          62 | 172.58.102.136  |                8.08429e-05 |
| 5024 |          62 | 172.58.102.152  |                8.08429e-05 |
| 5025 |          62 | 172.58.102.155  |                8.08429e-05 |
| 5026 |          62 | 172.58.96.164   |                8.08429e-05 |
| 5027 |          62 | 172.58.96.63    |                8.08429e-05 |
| 5028 |          62 | 172.58.99.124   |                8.08429e-05 |
| 5029 |          62 | 172.58.99.125   |                8.08429e-05 |
| 5030 |          62 | 174.197.1.205   |                8.08429e-05 |
| 5031 |          62 | 24.243.50.45    |                8.08429e-05 |
| 5032 |          62 | 47.184.42.157   |                8.08429e-05 |
| 5033 |          62 | 50.237.25.227   |                8.08429e-05 |
| 5034 |          62 | 72.128.140.202  |                8.08429e-05 |
| 5035 |          62 | 73.206.101.120  |                8.08429e-05 |
| 5036 |          62 | 96.8.177.36     |                8.08429e-05 |
| 5037 |          62 | 99.158.248.118  |                8.08429e-05 |
| 5038 |          62 | 107.117.168.103 |                5.38953e-05 |
| 5039 |          62 | 107.77.217.194  |                5.38953e-05 |
| 5040 |          62 | 107.77.217.69   |                5.38953e-05 |
| 5041 |          62 | 107.77.218.9    |                5.38953e-05 |
| 5042 |          62 | 107.77.219.11   |                5.38953e-05 |
| 5043 |          62 | 107.77.219.139  |                5.38953e-05 |
| 5044 |          62 | 107.77.221.150  |                5.38953e-05 |
| 5045 |          62 | 107.77.222.18   |                5.38953e-05 |
| 5046 |          62 | 172.56.6.145    |                5.38953e-05 |
| 5047 |          62 | 172.58.96.188   |                5.38953e-05 |
| 5048 |          62 | 172.58.99.114   |                5.38953e-05 |
| 5049 |          62 | 174.246.133.136 |                5.38953e-05 |
| 5050 |          62 | 24.27.42.164    |                5.38953e-05 |
| 5051 |          62 | 47.183.143.86   |                5.38953e-05 |
| 5052 |          62 | 50.84.10.102    |                5.38953e-05 |
| 5053 |          62 | 97.99.68.133    |                5.38953e-05 |
| 5054 |          62 | 98.199.170.30   |                5.38953e-05 |
| 5055 |          62 | 99.151.200.207  |                5.38953e-05 |
| 5056 |          62 | 99.76.233.227   |                5.38953e-05 |
| 5057 |          62 | 107.117.224.88  |                2.69476e-05 |
| 5058 |          62 | 107.127.0.108   |                2.69476e-05 |
| 5059 |          62 | 107.127.0.137   |                2.69476e-05 |
| 5060 |          62 | 107.127.0.22    |                2.69476e-05 |
| 5061 |          62 | 107.127.0.57    |                2.69476e-05 |
| 5062 |          62 | 107.150.31.108  |                2.69476e-05 |
| 5063 |          62 | 107.77.217.123  |                2.69476e-05 |
| 5064 |          62 | 107.77.217.144  |                2.69476e-05 |
| 5065 |          62 | 107.77.218.230  |                2.69476e-05 |
| 5066 |          62 | 107.77.218.65   |                2.69476e-05 |
| 5067 |          62 | 107.77.219.190  |                2.69476e-05 |
| 5068 |          62 | 107.77.219.69   |                2.69476e-05 |
| 5069 |          62 | 107.77.220.227  |                2.69476e-05 |
| 5070 |          62 | 107.77.221.80   |                2.69476e-05 |
| 5071 |          62 | 107.77.222.153  |                2.69476e-05 |
| 5072 |          62 | 136.34.163.5    |                2.69476e-05 |
| 5073 |          62 | 165.166.121.80  |                2.69476e-05 |
| 5074 |          62 | 172.56.6.146    |                2.69476e-05 |
| 5075 |          62 | 172.58.102.252  |                2.69476e-05 |
| 5076 |          62 | 172.58.99.231   |                2.69476e-05 |
| 5077 |          62 | 174.246.192.204 |                2.69476e-05 |
| 5078 |          62 | 192.24.18.31    |                2.69476e-05 |
| 5079 |          62 | 67.166.11.44    |                2.69476e-05 |
| 5184 |         132 | 107.117.225.105 |                8.44202e-05 |
| 5185 |         132 | 107.77.220.9    |                8.44202e-05 |
| 5186 |         132 | 107.77.221.58   |                8.44202e-05 |
| 5187 |         132 | 107.77.222.127  |                8.44202e-05 |
| 5188 |         132 | 107.77.222.226  |                8.44202e-05 |
| 5189 |         132 | 107.77.235.213  |                8.44202e-05 |
| 5190 |         132 | 107.77.236.53   |                8.44202e-05 |
| 5191 |         132 | 12.34.198.161   |                8.44202e-05 |
| 5192 |         132 | 172.56.15.73    |                8.44202e-05 |
| 5193 |         132 | 172.56.7.188    |                8.44202e-05 |
| 5194 |         132 | 172.58.103.253  |                8.44202e-05 |
| 5195 |         132 | 172.58.108.228  |                8.44202e-05 |
| 5196 |         132 | 172.58.71.93    |                8.44202e-05 |
| 5197 |         132 | 174.197.10.16   |                8.44202e-05 |
| 5198 |         132 | 174.197.3.235   |                8.44202e-05 |
| 5199 |         132 | 174.246.139.18  |                8.44202e-05 |
| 5200 |         132 | 96.8.193.73     |                8.44202e-05 |
| 5201 |         132 | 107.77.201.101  |                4.22101e-05 |
| 5202 |         132 | 154.21.28.158   |                4.22101e-05 |
| 5203 |         132 | 172.14.106.110  |                4.22101e-05 |
| 5204 |         132 | 172.56.21.254   |                4.22101e-05 |
| 5205 |         132 | 172.56.7.212    |                4.22101e-05 |
| 5206 |         132 | 172.56.7.59     |                4.22101e-05 |
| 5207 |         132 | 172.58.102.172  |                4.22101e-05 |
| 5208 |         132 | 172.58.102.174  |                4.22101e-05 |
| 5209 |         132 | 172.58.102.248  |                4.22101e-05 |
| 5210 |         132 | 172.58.96.108   |                4.22101e-05 |
| 5211 |         132 | 172.58.99.157   |                4.22101e-05 |
| 5212 |         132 | 172.58.99.205   |                4.22101e-05 |
| 5213 |         132 | 172.58.99.230   |                4.22101e-05 |
| 5214 |         132 | 172.58.99.44    |                4.22101e-05 |
| 5215 |         132 | 174.197.10.230  |                4.22101e-05 |
| 5216 |         132 | 174.242.79.208  |                4.22101e-05 |
| 5217 |         132 | 174.246.205.229 |                4.22101e-05 |
| 5218 |         132 | 174.255.129.190 |                4.22101e-05 |
| 5219 |         132 | 181.214.227.23  |                4.22101e-05 |
| 5220 |         132 | 50.24.136.80    |                4.22101e-05 |
| 5221 |         132 | 70.158.101.52   |                4.22101e-05 |
| 5222 |         132 | 96.43.16.139    |                4.22101e-05 |
| 5369 |         133 | 172.58.102.136  |                6.79579e-05 |
| 5370 |         133 | 172.58.102.139  |                6.79579e-05 |
| 5371 |         133 | 172.58.102.162  |                6.79579e-05 |
| 5372 |         133 | 172.58.102.178  |                6.79579e-05 |
| 5373 |         133 | 172.58.102.191  |                6.79579e-05 |
| 5374 |         133 | 172.58.102.200  |                6.79579e-05 |
| 5375 |         133 | 172.58.102.219  |                6.79579e-05 |
| 5376 |         133 | 172.58.102.250  |                6.79579e-05 |
| 5377 |         133 | 172.58.107.129  |                6.79579e-05 |
| 5378 |         133 | 172.58.107.176  |                6.79579e-05 |
| 5379 |         133 | 172.58.107.178  |                6.79579e-05 |
| 5380 |         133 | 172.58.107.22   |                6.79579e-05 |
| 5381 |         133 | 172.58.107.235  |                6.79579e-05 |
| 5382 |         133 | 172.58.107.241  |                6.79579e-05 |
| 5383 |         133 | 172.58.107.27   |                6.79579e-05 |
| 5384 |         133 | 172.58.108.187  |                6.79579e-05 |
| 5385 |         133 | 172.58.108.216  |                6.79579e-05 |
| 5386 |         133 | 172.58.108.41   |                6.79579e-05 |
| 5387 |         133 | 172.58.110.158  |                6.79579e-05 |
| 5388 |         133 | 172.58.110.164  |                6.79579e-05 |
| 5389 |         133 | 172.58.110.172  |                6.79579e-05 |
| 5390 |         133 | 172.58.111.115  |                6.79579e-05 |
| 5391 |         133 | 172.58.111.64   |                6.79579e-05 |
| 5392 |         133 | 172.58.111.66   |                6.79579e-05 |
| 5393 |         133 | 172.58.111.76   |                6.79579e-05 |
| 5394 |         133 | 172.58.111.8    |                6.79579e-05 |
| 5395 |         133 | 172.58.67.118   |                6.79579e-05 |
| 5396 |         133 | 172.58.67.159   |                6.79579e-05 |
| 5397 |         133 | 172.58.67.243   |                6.79579e-05 |
| 5398 |         133 | 172.58.67.246   |                6.79579e-05 |
| 5399 |         133 | 172.58.68.191   |                6.79579e-05 |
| 5400 |         133 | 172.58.68.21    |                6.79579e-05 |
| 5401 |         133 | 172.58.68.33    |                6.79579e-05 |
| 5402 |         133 | 172.58.70.167   |                6.79579e-05 |
| 5403 |         133 | 172.58.70.176   |                6.79579e-05 |
| 5404 |         133 | 172.58.70.237   |                6.79579e-05 |
| 5405 |         133 | 172.58.71.71    |                6.79579e-05 |
| 5406 |         133 | 172.58.76.161   |                6.79579e-05 |
| 5407 |         133 | 172.58.76.170   |                6.79579e-05 |
| 5408 |         133 | 172.58.76.180   |                6.79579e-05 |
| 5409 |         133 | 172.58.76.49    |                6.79579e-05 |
| 5410 |         133 | 172.58.79.29    |                6.79579e-05 |
| 5411 |         133 | 172.58.79.69    |                6.79579e-05 |
| 5412 |         133 | 172.58.99.113   |                6.79579e-05 |
| 5413 |         133 | 172.58.99.146   |                6.79579e-05 |
| 5414 |         133 | 172.58.99.16    |                6.79579e-05 |
| 5415 |         133 | 172.58.99.179   |                6.79579e-05 |
| 5416 |         133 | 172.58.99.20    |                6.79579e-05 |
| 5417 |         133 | 172.58.99.23    |                6.79579e-05 |
| 5418 |         133 | 172.58.99.238   |                6.79579e-05 |
| 5419 |         133 | 172.58.99.57    |                6.79579e-05 |
| 5420 |         133 | 172.58.99.6     |                6.79579e-05 |
| 5421 |         133 | 184.169.45.4    |                6.79579e-05 |
| 5422 |         133 | 212.102.41.19   |                6.79579e-05 |
| 5423 |         133 | 212.102.41.46   |                6.79579e-05 |
| 5424 |         133 | 67.11.239.97    |                6.79579e-05 |
| 5539 |         134 | 107.77.200.151  |                6.01576e-05 |
| 5540 |         134 | 154.29.131.133  |                6.01576e-05 |
| 5541 |         134 | 154.29.131.203  |                6.01576e-05 |
| 5542 |         134 | 154.3.250.87    |                6.01576e-05 |
| 5543 |         134 | 154.3.251.184   |                6.01576e-05 |
| 5544 |         134 | 162.200.113.53  |                6.01576e-05 |
| 5545 |         134 | 172.56.7.46     |                6.01576e-05 |
| 5546 |         134 | 172.58.158.230  |                6.01576e-05 |
| 5547 |         134 | 172.58.97.185   |                6.01576e-05 |
| 5548 |         134 | 193.56.117.80   |                6.01576e-05 |
| 5549 |         134 | 212.102.40.36   |                6.01576e-05 |
| 5550 |         134 | 217.138.206.86  |                6.01576e-05 |
| 5551 |         134 | 62.149.20.55    |                6.01576e-05 |
| 5552 |         134 | 66.69.47.20     |                6.01576e-05 |
| 5553 |         134 | 95.181.238.17   |                6.01576e-05 |
| 5554 |         134 | 95.181.238.25   |                6.01576e-05 |
| 5555 |         134 | 95.181.238.34   |                6.01576e-05 |
| 5637 |         135 | 136.50.11.9     |                6.09868e-05 |
| 5638 |         135 | 172.56.6.157    |                6.09868e-05 |
| 5639 |         135 | 172.56.7.141    |                6.09868e-05 |
| 5640 |         135 | 172.58.101.150  |                6.09868e-05 |
| 5641 |         135 | 172.58.107.239  |                6.09868e-05 |
| 5642 |         135 | 172.58.110.231  |                6.09868e-05 |
| 5643 |         135 | 172.58.77.193   |                6.09868e-05 |
| 5644 |         135 | 172.58.79.188   |                6.09868e-05 |
| 5645 |         135 | 173.173.97.218  |                6.09868e-05 |
| 5646 |         135 | 174.197.1.23    |                6.09868e-05 |
| 5647 |         135 | 174.197.2.151   |                6.09868e-05 |
| 5648 |         135 | 24.28.41.137    |                6.09868e-05 |
| 5649 |         135 | 24.28.41.36     |                6.09868e-05 |
| 5650 |         135 | 67.11.159.44    |                6.09868e-05 |

#### User_Id given Ip

|     |   user_id | ip             |   proba_user_id_given_ip |
|----:|----------:|:---------------|-------------------------:|
| 175 |        11 | 172.56.15.51   |              5.58254e-05 |
| 176 |        11 | 208.54.83.234  |              5.58254e-05 |
| 177 |        11 | 72.205.179.123 |              5.58254e-05 |
| 178 |        11 | 99.203.143.2   |              5.58254e-05 |
| 179 |        11 | 99.203.155.182 |              5.58254e-05 |
| 180 |        11 | 99.203.155.78  |              5.58254e-05 |
| 181 |        11 | 99.203.26.9    |              5.58254e-05 |
| 182 |        11 | 99.203.27.231  |              5.58254e-05 |
| 542 |        53 | 75.54.33.178   |              8.11096e-05 |
| 593 |        64 | 187.237.231.72 |              6.1267e-05  |
| 594 |        64 | 187.237.231.85 |              6.1267e-05  |
| 595 |        64 | 70.114.21.86   |              6.1267e-05  |

#### Time given Ip

| time   | ip   | proba_time_given_ip   |
|--------|------|-----------------------|

#### Date given Program_Id

| date   | program_id   | proba_date_given_program_id   |
|--------|--------------|-------------------------------|

#### Path given Cohort_Id

|       | path                     |   cohort_id |   proba_path_given_cohort_id |
|------:|:-------------------------|------------:|-----------------------------:|
|    50 | /                        |           6 |                  8.72334e-05 |
|    51 | /                        |           4 |                  2.18083e-05 |
|    52 | /                        |           5 |                  2.18083e-05 |
|    53 | /                        |           9 |                  2.18083e-05 |
|  7288 | html-css                 |           6 |                  7.61789e-05 |
|  8011 | java-i                   |           2 |                  9.55384e-05 |
|  8012 | java-i                   |          12 |                  9.55384e-05 |
|  8260 | java-ii                  |          12 |                  8.2122e-05  |
|  8613 | java-iii                 |          12 |                  7.59532e-05 |
|  8614 | java-iii                 |         139 |                  7.59532e-05 |
|  8991 | javascript-i             |           2 |                  5.4936e-05  |
|  8992 | javascript-i             |          12 |                  5.4936e-05  |
| 10081 | jquery                   |          19 |                  9.05715e-05 |
| 11908 | search/search_index.json |          11 |                  5.70321e-05 |
| 11909 | search/search_index.json |          15 |                  5.70321e-05 |
| 12479 | spring                   |          55 |                  8.41538e-05 |
| 13356 | toc                      |          15 |                  5.68473e-05 |

#### Path given Start_Date

|       | path                     | start_date   |   proba_path_given_start_date |
|------:|:-------------------------|:-------------|------------------------------:|
|    47 | /                        | 2015-02-03   |                   8.72334e-05 |
|    48 | /                        | 2014-10-20   |                   2.18083e-05 |
|    49 | /                        | 2014-11-18   |                   2.18083e-05 |
|    50 | /                        | 2015-03-30   |                   2.18083e-05 |
|  7156 | html-css                 | 2015-02-03   |                   7.61789e-05 |
|  7863 | java-i                   | 2014-06-04   |                   9.55384e-05 |
|  7864 | java-i                   | 2016-03-08   |                   9.55384e-05 |
|  8105 | java-ii                  | 2016-03-08   |                   8.2122e-05  |
|  8449 | java-iii                 | 2016-03-08   |                   7.59532e-05 |
|  8450 | java-iii                 | 2021-04-12   |                   7.59532e-05 |
|  8817 | javascript-i             | 2014-06-04   |                   5.4936e-05  |
|  8818 | javascript-i             | 2016-03-08   |                   5.4936e-05  |
|  9878 | jquery                   | 2017-06-05   |                   9.05715e-05 |
| 11656 | search/search_index.json | 2016-01-20   |                   5.70321e-05 |
| 11657 | search/search_index.json | 2016-09-26   |                   5.70321e-05 |
| 12203 | spring                   | 2020-02-03   |                   8.41538e-05 |
| 13065 | toc                      | 2016-09-26   |                   5.68473e-05 |

#### User_Id given Cohort_Id

| user_id   | cohort_id   | proba_user_id_given_cohort_id   |
|-----------|-------------|---------------------------------|

#### Id given Slack

| id   | slack   | proba_id_given_slack   |
|------|---------|------------------------|

#### User_Id given Start_Date

| user_id   | start_date   | proba_user_id_given_start_date   |
|-----------|--------------|----------------------------------|

#### Id given Name

| id   | name   | proba_id_given_name   |
|------|--------|-----------------------|

#### Slack given End_Date

| slack   | end_date   | proba_slack_given_end_date   |
|---------|------------|------------------------------|

#### Time given Cohort_Id

| time   | cohort_id   | proba_time_given_cohort_id   |
|--------|-------------|------------------------------|

#### Ip given Id

|      | ip           |   id |   proba_ip_given_id |
|-----:|:-------------|-----:|--------------------:|
| 5413 | 97.105.19.58 |   18 |         6.32798e-05 |
| 5414 | 97.105.19.58 |   57 |         4.83905e-05 |
| 5415 | 97.105.19.58 |   22 |         3.35011e-05 |

#### Start_Date given End_Date

| start_date   | end_date   | proba_start_date_given_end_date   |
|--------------|------------|-----------------------------------|

#### Time given Start_Date

| time   | start_date   | proba_time_given_start_date   |
|--------|--------------|-------------------------------|

#### Cohort_Id given Start_Date

| cohort_id   | start_date   | proba_cohort_id_given_start_date   |
|-------------|--------------|------------------------------------|

#### Id given End_Date

| id   | end_date   | proba_id_given_end_date   |
|------|------------|---------------------------|

#### Path given User_Id

|        | path                     |   user_id |   proba_path_given_user_id |
|-------:|:-------------------------|----------:|---------------------------:|
|    755 | /                        |        16 |                8.72334e-05 |
|    756 | /                        |        70 |                8.72334e-05 |
|    757 | /                        |        92 |                8.72334e-05 |
|    758 | /                        |       101 |                8.72334e-05 |
|    759 | /                        |       173 |                8.72334e-05 |
|    760 | /                        |       210 |                8.72334e-05 |
|    761 | /                        |       278 |                8.72334e-05 |
|    762 | /                        |       287 |                8.72334e-05 |
|    763 | /                        |       298 |                8.72334e-05 |
|    764 | /                        |       322 |                8.72334e-05 |
|    765 | /                        |       387 |                8.72334e-05 |
|    766 | /                        |       517 |                8.72334e-05 |
|    767 | /                        |       537 |                8.72334e-05 |
|    768 | /                        |       608 |                8.72334e-05 |
|    769 | /                        |       611 |                8.72334e-05 |
|    770 | /                        |       745 |                8.72334e-05 |
|    771 | /                        |       748 |                8.72334e-05 |
|    772 | /                        |       856 |                8.72334e-05 |
|    773 | /                        |       858 |                8.72334e-05 |
|    774 | /                        |       863 |                8.72334e-05 |
|    775 | /                        |       881 |                8.72334e-05 |
|    776 | /                        |       890 |                8.72334e-05 |
|    777 | /                        |       913 |                8.72334e-05 |
|    778 | /                        |        45 |                6.5425e-05  |
|    779 | /                        |        46 |                6.5425e-05  |
|    780 | /                        |        49 |                6.5425e-05  |
|    781 | /                        |        84 |                6.5425e-05  |
|    782 | /                        |        98 |                6.5425e-05  |
|    783 | /                        |       112 |                6.5425e-05  |
|    784 | /                        |       144 |                6.5425e-05  |
|    785 | /                        |       154 |                6.5425e-05  |
|    786 | /                        |       164 |                6.5425e-05  |
|    787 | /                        |       169 |                6.5425e-05  |
|    788 | /                        |       283 |                6.5425e-05  |
|    789 | /                        |       311 |                6.5425e-05  |
|    790 | /                        |       370 |                6.5425e-05  |
|    791 | /                        |       371 |                6.5425e-05  |
|    792 | /                        |       443 |                6.5425e-05  |
|    793 | /                        |       461 |                6.5425e-05  |
|    794 | /                        |       486 |                6.5425e-05  |
|    795 | /                        |       559 |                6.5425e-05  |
|    796 | /                        |       610 |                6.5425e-05  |
|    797 | /                        |       755 |                6.5425e-05  |
|    798 | /                        |       817 |                6.5425e-05  |
|    799 | /                        |       931 |                6.5425e-05  |
|    800 | /                        |       949 |                6.5425e-05  |
|    801 | /                        |       962 |                6.5425e-05  |
|    802 | /                        |       969 |                6.5425e-05  |
|    803 | /                        |       975 |                6.5425e-05  |
|    804 | /                        |       980 |                6.5425e-05  |
|    805 | /                        |        13 |                4.36167e-05 |
|    806 | /                        |        24 |                4.36167e-05 |
|    807 | /                        |        47 |                4.36167e-05 |
|    808 | /                        |        68 |                4.36167e-05 |
|    809 | /                        |        69 |                4.36167e-05 |
|    810 | /                        |        95 |                4.36167e-05 |
|    811 | /                        |       109 |                4.36167e-05 |
|    812 | /                        |       113 |                4.36167e-05 |
|    813 | /                        |       141 |                4.36167e-05 |
|    814 | /                        |       149 |                4.36167e-05 |
|    815 | /                        |       181 |                4.36167e-05 |
|    816 | /                        |       214 |                4.36167e-05 |
|    817 | /                        |       255 |                4.36167e-05 |
|    818 | /                        |       396 |                4.36167e-05 |
|    819 | /                        |       399 |                4.36167e-05 |
|    820 | /                        |       401 |                4.36167e-05 |
|    821 | /                        |       402 |                4.36167e-05 |
|    822 | /                        |       411 |                4.36167e-05 |
|    823 | /                        |       427 |                4.36167e-05 |
|    824 | /                        |       436 |                4.36167e-05 |
|    825 | /                        |       529 |                4.36167e-05 |
|    826 | /                        |       543 |                4.36167e-05 |
|    827 | /                        |       573 |                4.36167e-05 |
|    828 | /                        |       648 |                4.36167e-05 |
|    829 | /                        |       779 |                4.36167e-05 |
|    830 | /                        |       824 |                4.36167e-05 |
|    831 | /                        |       852 |                4.36167e-05 |
|    832 | /                        |       855 |                4.36167e-05 |
|    833 | /                        |       889 |                4.36167e-05 |
|    834 | /                        |       941 |                4.36167e-05 |
|    835 | /                        |       956 |                4.36167e-05 |
|    836 | /                        |       959 |                4.36167e-05 |
|    837 | /                        |       961 |                4.36167e-05 |
|    838 | /                        |       963 |                4.36167e-05 |
|    839 | /                        |       966 |                4.36167e-05 |
|    840 | /                        |       968 |                4.36167e-05 |
|    841 | /                        |       972 |                4.36167e-05 |
|    842 | /                        |       973 |                4.36167e-05 |
|    843 | /                        |       976 |                4.36167e-05 |
|    844 | /                        |       977 |                4.36167e-05 |
|    845 | /                        |       981 |                4.36167e-05 |
|    846 | /                        |        17 |                2.18083e-05 |
|    847 | /                        |        66 |                2.18083e-05 |
|    848 | /                        |        71 |                2.18083e-05 |
|    849 | /                        |        77 |                2.18083e-05 |
|    850 | /                        |        81 |                2.18083e-05 |
|    851 | /                        |        85 |                2.18083e-05 |
|    852 | /                        |        87 |                2.18083e-05 |
|    853 | /                        |       102 |                2.18083e-05 |
|    854 | /                        |       104 |                2.18083e-05 |
|    855 | /                        |       107 |                2.18083e-05 |
|    856 | /                        |       114 |                2.18083e-05 |
|    857 | /                        |       115 |                2.18083e-05 |
|    858 | /                        |       125 |                2.18083e-05 |
|    859 | /                        |       150 |                2.18083e-05 |
|    860 | /                        |       152 |                2.18083e-05 |
|    861 | /                        |       163 |                2.18083e-05 |
|    862 | /                        |       174 |                2.18083e-05 |
|    863 | /                        |       177 |                2.18083e-05 |
|    864 | /                        |       178 |                2.18083e-05 |
|    865 | /                        |       209 |                2.18083e-05 |
|    866 | /                        |       216 |                2.18083e-05 |
|    867 | /                        |       244 |                2.18083e-05 |
|    868 | /                        |       250 |                2.18083e-05 |
|    869 | /                        |       251 |                2.18083e-05 |
|    870 | /                        |       279 |                2.18083e-05 |
|    871 | /                        |       286 |                2.18083e-05 |
|    872 | /                        |       316 |                2.18083e-05 |
|    873 | /                        |       340 |                2.18083e-05 |
|    874 | /                        |       348 |                2.18083e-05 |
|    875 | /                        |       388 |                2.18083e-05 |
|    876 | /                        |       398 |                2.18083e-05 |
|    877 | /                        |       405 |                2.18083e-05 |
|    878 | /                        |       432 |                2.18083e-05 |
|    879 | /                        |       462 |                2.18083e-05 |
|    880 | /                        |       463 |                2.18083e-05 |
|    881 | /                        |       487 |                2.18083e-05 |
|    882 | /                        |       519 |                2.18083e-05 |
|    883 | /                        |       574 |                2.18083e-05 |
|    884 | /                        |       592 |                2.18083e-05 |
|    885 | /                        |       597 |                2.18083e-05 |
|    886 | /                        |       619 |                2.18083e-05 |
|    887 | /                        |       649 |                2.18083e-05 |
|    888 | /                        |       679 |                2.18083e-05 |
|    889 | /                        |       697 |                2.18083e-05 |
|    890 | /                        |       734 |                2.18083e-05 |
|    891 | /                        |       740 |                2.18083e-05 |
|    892 | /                        |       741 |                2.18083e-05 |
|    893 | /                        |       772 |                2.18083e-05 |
|    894 | /                        |       787 |                2.18083e-05 |
|    895 | /                        |       832 |                2.18083e-05 |
|    896 | /                        |       857 |                2.18083e-05 |
|    897 | /                        |       879 |                2.18083e-05 |
|    898 | /                        |       918 |                2.18083e-05 |
|    899 | /                        |       940 |                2.18083e-05 |
|    900 | /                        |       952 |                2.18083e-05 |
|  36538 | html-css                 |         8 |                7.61789e-05 |
|  36539 | html-css                 |         9 |                7.61789e-05 |
|  36540 | html-css                 |        12 |                7.61789e-05 |
|  36541 | html-css                 |        28 |                7.61789e-05 |
|  36542 | html-css                 |        32 |                7.61789e-05 |
|  36543 | html-css                 |        57 |                7.61789e-05 |
|  36544 | html-css                 |        72 |                7.61789e-05 |
|  36545 | html-css                 |        81 |                7.61789e-05 |
|  36546 | html-css                 |        84 |                7.61789e-05 |
|  36547 | html-css                 |        94 |                7.61789e-05 |
|  36548 | html-css                 |        95 |                7.61789e-05 |
|  36549 | html-css                 |       104 |                7.61789e-05 |
|  36550 | html-css                 |       141 |                7.61789e-05 |
|  36551 | html-css                 |       147 |                7.61789e-05 |
|  36552 | html-css                 |       148 |                7.61789e-05 |
|  36553 | html-css                 |       150 |                7.61789e-05 |
|  36554 | html-css                 |       216 |                7.61789e-05 |
|  36555 | html-css                 |       242 |                7.61789e-05 |
|  36556 | html-css                 |       255 |                7.61789e-05 |
|  36557 | html-css                 |       278 |                7.61789e-05 |
|  36558 | html-css                 |       280 |                7.61789e-05 |
|  36559 | html-css                 |       283 |                7.61789e-05 |
|  36560 | html-css                 |       286 |                7.61789e-05 |
|  36561 | html-css                 |       305 |                7.61789e-05 |
|  36562 | html-css                 |       370 |                7.61789e-05 |
|  36563 | html-css                 |       388 |                7.61789e-05 |
|  36564 | html-css                 |       396 |                7.61789e-05 |
|  36565 | html-css                 |       400 |                7.61789e-05 |
|  36566 | html-css                 |       401 |                7.61789e-05 |
|  36567 | html-css                 |       426 |                7.61789e-05 |
|  36568 | html-css                 |       459 |                7.61789e-05 |
|  36569 | html-css                 |       472 |                7.61789e-05 |
|  36570 | html-css                 |       611 |                7.61789e-05 |
|  36571 | html-css                 |       629 |                7.61789e-05 |
|  36572 | html-css                 |       702 |                7.61789e-05 |
|  36573 | html-css                 |       704 |                7.61789e-05 |
|  36574 | html-css                 |       708 |                7.61789e-05 |
|  36575 | html-css                 |       734 |                7.61789e-05 |
|  36576 | html-css                 |       777 |                7.61789e-05 |
|  36577 | html-css                 |       779 |                7.61789e-05 |
|  36578 | html-css                 |       784 |                7.61789e-05 |
|  36579 | html-css                 |       798 |                7.61789e-05 |
|  36580 | html-css                 |       812 |                7.61789e-05 |
|  36581 | html-css                 |       828 |                7.61789e-05 |
|  36582 | html-css                 |       832 |                7.61789e-05 |
|  36583 | html-css                 |       857 |                7.61789e-05 |
|  36584 | html-css                 |       894 |                7.61789e-05 |
|  36585 | html-css                 |       935 |                7.61789e-05 |
|  36586 | html-css                 |       941 |                7.61789e-05 |
|  36587 | html-css                 |       942 |                7.61789e-05 |
|  36588 | html-css                 |       951 |                7.61789e-05 |
|  36589 | html-css                 |       954 |                7.61789e-05 |
|  36590 | html-css                 |       958 |                7.61789e-05 |
|  36591 | html-css                 |       960 |                7.61789e-05 |
|  36592 | html-css                 |       965 |                7.61789e-05 |
|  45650 | java-i                   |        13 |                9.55384e-05 |
|  45651 | java-i                   |        27 |                9.55384e-05 |
|  45652 | java-i                   |        28 |                9.55384e-05 |
|  45653 | java-i                   |        52 |                9.55384e-05 |
|  45654 | java-i                   |        65 |                9.55384e-05 |
|  45655 | java-i                   |       112 |                9.55384e-05 |
|  45656 | java-i                   |       115 |                9.55384e-05 |
|  45657 | java-i                   |       126 |                9.55384e-05 |
|  45658 | java-i                   |       161 |                9.55384e-05 |
|  45659 | java-i                   |       164 |                9.55384e-05 |
|  45660 | java-i                   |       170 |                9.55384e-05 |
|  45661 | java-i                   |       180 |                9.55384e-05 |
|  45662 | java-i                   |       201 |                9.55384e-05 |
|  45663 | java-i                   |       232 |                9.55384e-05 |
|  45664 | java-i                   |       255 |                9.55384e-05 |
|  45665 | java-i                   |       278 |                9.55384e-05 |
|  45666 | java-i                   |       280 |                9.55384e-05 |
|  45667 | java-i                   |       281 |                9.55384e-05 |
|  45668 | java-i                   |       285 |                9.55384e-05 |
|  45669 | java-i                   |       310 |                9.55384e-05 |
|  45670 | java-i                   |       311 |                9.55384e-05 |
|  45671 | java-i                   |       316 |                9.55384e-05 |
|  45672 | java-i                   |       370 |                9.55384e-05 |
|  45673 | java-i                   |       371 |                9.55384e-05 |
|  45674 | java-i                   |       398 |                9.55384e-05 |
|  45675 | java-i                   |       402 |                9.55384e-05 |
|  45676 | java-i                   |       411 |                9.55384e-05 |
|  45677 | java-i                   |       427 |                9.55384e-05 |
|  45678 | java-i                   |       442 |                9.55384e-05 |
|  45679 | java-i                   |       459 |                9.55384e-05 |
|  45680 | java-i                   |       467 |                9.55384e-05 |
|  45681 | java-i                   |       539 |                9.55384e-05 |
|  45682 | java-i                   |       601 |                9.55384e-05 |
|  45683 | java-i                   |       603 |                9.55384e-05 |
|  45684 | java-i                   |       618 |                9.55384e-05 |
|  45685 | java-i                   |       645 |                9.55384e-05 |
|  45686 | java-i                   |       665 |                9.55384e-05 |
|  45687 | java-i                   |       735 |                9.55384e-05 |
|  45688 | java-i                   |       738 |                9.55384e-05 |
|  45689 | java-i                   |       741 |                9.55384e-05 |
|  45690 | java-i                   |       758 |                9.55384e-05 |
|  45691 | java-i                   |       766 |                9.55384e-05 |
|  45692 | java-i                   |       797 |                9.55384e-05 |
|  45693 | java-i                   |       800 |                9.55384e-05 |
|  45694 | java-i                   |       812 |                9.55384e-05 |
|  45695 | java-i                   |       813 |                9.55384e-05 |
|  45696 | java-i                   |       833 |                9.55384e-05 |
|  45697 | java-i                   |       861 |                9.55384e-05 |
|  45698 | java-i                   |       871 |                9.55384e-05 |
|  45699 | java-i                   |       874 |                9.55384e-05 |
|  45700 | java-i                   |       876 |                9.55384e-05 |
|  45701 | java-i                   |       878 |                9.55384e-05 |
|  45702 | java-i                   |       881 |                9.55384e-05 |
|  45703 | java-i                   |       883 |                9.55384e-05 |
|  45704 | java-i                   |       891 |                9.55384e-05 |
|  45705 | java-i                   |       917 |                9.55384e-05 |
|  45706 | java-i                   |       921 |                9.55384e-05 |
|  45707 | java-i                   |       924 |                9.55384e-05 |
|  45708 | java-i                   |       925 |                9.55384e-05 |
|  45709 | java-i                   |       932 |                9.55384e-05 |
|  45710 | java-i                   |       936 |                9.55384e-05 |
|  45711 | java-i                   |       937 |                9.55384e-05 |
|  45712 | java-i                   |       938 |                9.55384e-05 |
|  45713 | java-i                   |       954 |                9.55384e-05 |
|  45714 | java-i                   |       970 |                9.55384e-05 |
|  49233 | java-ii                  |        15 |                8.2122e-05  |
|  49234 | java-ii                  |        21 |                8.2122e-05  |
|  49235 | java-ii                  |        28 |                8.2122e-05  |
|  49236 | java-ii                  |        52 |                8.2122e-05  |
|  49237 | java-ii                  |        56 |                8.2122e-05  |
|  49238 | java-ii                  |        57 |                8.2122e-05  |
|  49239 | java-ii                  |        65 |                8.2122e-05  |
|  49240 | java-ii                  |       108 |                8.2122e-05  |
|  49241 | java-ii                  |       149 |                8.2122e-05  |
|  49242 | java-ii                  |       161 |                8.2122e-05  |
|  49243 | java-ii                  |       164 |                8.2122e-05  |
|  49244 | java-ii                  |       167 |                8.2122e-05  |
|  49245 | java-ii                  |       168 |                8.2122e-05  |
|  49246 | java-ii                  |       218 |                8.2122e-05  |
|  49247 | java-ii                  |       253 |                8.2122e-05  |
|  49248 | java-ii                  |       254 |                8.2122e-05  |
|  49249 | java-ii                  |       255 |                8.2122e-05  |
|  49250 | java-ii                  |       278 |                8.2122e-05  |
|  49251 | java-ii                  |       281 |                8.2122e-05  |
|  49252 | java-ii                  |       284 |                8.2122e-05  |
|  49253 | java-ii                  |       297 |                8.2122e-05  |
|  49254 | java-ii                  |       310 |                8.2122e-05  |
|  49255 | java-ii                  |       316 |                8.2122e-05  |
|  49256 | java-ii                  |       411 |                8.2122e-05  |
|  49257 | java-ii                  |       422 |                8.2122e-05  |
|  49258 | java-ii                  |       442 |                8.2122e-05  |
|  49259 | java-ii                  |       497 |                8.2122e-05  |
|  49260 | java-ii                  |       515 |                8.2122e-05  |
|  49261 | java-ii                  |       519 |                8.2122e-05  |
|  49262 | java-ii                  |       572 |                8.2122e-05  |
|  49263 | java-ii                  |       594 |                8.2122e-05  |
|  49264 | java-ii                  |       603 |                8.2122e-05  |
|  49265 | java-ii                  |       611 |                8.2122e-05  |
|  49266 | java-ii                  |       618 |                8.2122e-05  |
|  49267 | java-ii                  |       738 |                8.2122e-05  |
|  49268 | java-ii                  |       752 |                8.2122e-05  |
|  49269 | java-ii                  |       805 |                8.2122e-05  |
|  49270 | java-ii                  |       809 |                8.2122e-05  |
|  49271 | java-ii                  |       813 |                8.2122e-05  |
|  49272 | java-ii                  |       824 |                8.2122e-05  |
|  49273 | java-ii                  |       833 |                8.2122e-05  |
|  49274 | java-ii                  |       862 |                8.2122e-05  |
|  49275 | java-ii                  |       870 |                8.2122e-05  |
|  49276 | java-ii                  |       871 |                8.2122e-05  |
|  49277 | java-ii                  |       874 |                8.2122e-05  |
|  49278 | java-ii                  |       876 |                8.2122e-05  |
|  49279 | java-ii                  |       884 |                8.2122e-05  |
|  49280 | java-ii                  |       886 |                8.2122e-05  |
|  49281 | java-ii                  |       887 |                8.2122e-05  |
|  49282 | java-ii                  |       921 |                8.2122e-05  |
|  49283 | java-ii                  |       924 |                8.2122e-05  |
|  49284 | java-ii                  |       930 |                8.2122e-05  |
|  49285 | java-ii                  |       938 |                8.2122e-05  |
|  49286 | java-ii                  |       939 |                8.2122e-05  |
|  54102 | java-iii                 |        13 |                7.59532e-05 |
|  54103 | java-iii                 |        15 |                7.59532e-05 |
|  54104 | java-iii                 |        24 |                7.59532e-05 |
|  54105 | java-iii                 |        28 |                7.59532e-05 |
|  54106 | java-iii                 |        47 |                7.59532e-05 |
|  54107 | java-iii                 |        57 |                7.59532e-05 |
|  54108 | java-iii                 |        67 |                7.59532e-05 |
|  54109 | java-iii                 |        75 |                7.59532e-05 |
|  54110 | java-iii                 |        88 |                7.59532e-05 |
|  54111 | java-iii                 |        96 |                7.59532e-05 |
|  54112 | java-iii                 |       105 |                7.59532e-05 |
|  54113 | java-iii                 |       147 |                7.59532e-05 |
|  54114 | java-iii                 |       162 |                7.59532e-05 |
|  54115 | java-iii                 |       164 |                7.59532e-05 |
|  54116 | java-iii                 |       202 |                7.59532e-05 |
|  54117 | java-iii                 |       218 |                7.59532e-05 |
|  54118 | java-iii                 |       222 |                7.59532e-05 |
|  54119 | java-iii                 |       237 |                7.59532e-05 |
|  54120 | java-iii                 |       244 |                7.59532e-05 |
|  54121 | java-iii                 |       284 |                7.59532e-05 |
|  54122 | java-iii                 |       310 |                7.59532e-05 |
|  54123 | java-iii                 |       312 |                7.59532e-05 |
|  54124 | java-iii                 |       346 |                7.59532e-05 |
|  54125 | java-iii                 |       371 |                7.59532e-05 |
|  54126 | java-iii                 |       422 |                7.59532e-05 |
|  54127 | java-iii                 |       467 |                7.59532e-05 |
|  54128 | java-iii                 |       572 |                7.59532e-05 |
|  54129 | java-iii                 |       594 |                7.59532e-05 |
|  54130 | java-iii                 |       601 |                7.59532e-05 |
|  54131 | java-iii                 |       611 |                7.59532e-05 |
|  54132 | java-iii                 |       738 |                7.59532e-05 |
|  54133 | java-iii                 |       752 |                7.59532e-05 |
|  54134 | java-iii                 |       777 |                7.59532e-05 |
|  54135 | java-iii                 |       800 |                7.59532e-05 |
|  54136 | java-iii                 |       805 |                7.59532e-05 |
|  54137 | java-iii                 |       829 |                7.59532e-05 |
|  54138 | java-iii                 |       854 |                7.59532e-05 |
|  54139 | java-iii                 |       863 |                7.59532e-05 |
|  54140 | java-iii                 |       871 |                7.59532e-05 |
|  54141 | java-iii                 |       884 |                7.59532e-05 |
|  54142 | java-iii                 |       886 |                7.59532e-05 |
|  54143 | java-iii                 |       887 |                7.59532e-05 |
|  54144 | java-iii                 |       936 |                7.59532e-05 |
|  54145 | java-iii                 |       941 |                7.59532e-05 |
|  54146 | java-iii                 |       942 |                7.59532e-05 |
|  54147 | java-iii                 |       954 |                7.59532e-05 |
|  58921 | javascript-i             |        17 |                5.4936e-05  |
|  58922 | javascript-i             |        24 |                5.4936e-05  |
|  58923 | javascript-i             |        33 |                5.4936e-05  |
|  58924 | javascript-i             |        52 |                5.4936e-05  |
|  58925 | javascript-i             |        55 |                5.4936e-05  |
|  58926 | javascript-i             |        67 |                5.4936e-05  |
|  58927 | javascript-i             |        90 |                5.4936e-05  |
|  58928 | javascript-i             |        94 |                5.4936e-05  |
|  58929 | javascript-i             |        97 |                5.4936e-05  |
|  58930 | javascript-i             |       104 |                5.4936e-05  |
|  58931 | javascript-i             |       105 |                5.4936e-05  |
|  58932 | javascript-i             |       147 |                5.4936e-05  |
|  58933 | javascript-i             |       149 |                5.4936e-05  |
|  58934 | javascript-i             |       150 |                5.4936e-05  |
|  58935 | javascript-i             |       151 |                5.4936e-05  |
|  58936 | javascript-i             |       155 |                5.4936e-05  |
|  58937 | javascript-i             |       156 |                5.4936e-05  |
|  58938 | javascript-i             |       178 |                5.4936e-05  |
|  58939 | javascript-i             |       216 |                5.4936e-05  |
|  58940 | javascript-i             |       243 |                5.4936e-05  |
|  58941 | javascript-i             |       254 |                5.4936e-05  |
|  58942 | javascript-i             |       278 |                5.4936e-05  |
|  58943 | javascript-i             |       280 |                5.4936e-05  |
|  58944 | javascript-i             |       281 |                5.4936e-05  |
|  58945 | javascript-i             |       320 |                5.4936e-05  |
|  58946 | javascript-i             |       340 |                5.4936e-05  |
|  58947 | javascript-i             |       432 |                5.4936e-05  |
|  58948 | javascript-i             |       505 |                5.4936e-05  |
|  58949 | javascript-i             |       529 |                5.4936e-05  |
|  58950 | javascript-i             |       572 |                5.4936e-05  |
|  58951 | javascript-i             |       581 |                5.4936e-05  |
|  58952 | javascript-i             |       618 |                5.4936e-05  |
|  58953 | javascript-i             |       654 |                5.4936e-05  |
|  58954 | javascript-i             |       661 |                5.4936e-05  |
|  58955 | javascript-i             |       704 |                5.4936e-05  |
|  58956 | javascript-i             |       745 |                5.4936e-05  |
|  58957 | javascript-i             |       777 |                5.4936e-05  |
|  58958 | javascript-i             |       832 |                5.4936e-05  |
|  58959 | javascript-i             |       860 |                5.4936e-05  |
|  58960 | javascript-i             |       889 |                5.4936e-05  |
|  58961 | javascript-i             |       892 |                5.4936e-05  |
|  58962 | javascript-i             |       894 |                5.4936e-05  |
|  58963 | javascript-i             |       958 |                5.4936e-05  |
|  58964 | javascript-i             |       961 |                5.4936e-05  |
|  58965 | javascript-i             |       972 |                5.4936e-05  |
|  71288 | javascript-ii            |        16 |                9.7144e-05  |
|  71289 | javascript-ii            |        32 |                9.7144e-05  |
|  71290 | javascript-ii            |        33 |                9.7144e-05  |
|  71291 | javascript-ii            |        35 |                9.7144e-05  |
|  71292 | javascript-ii            |        42 |                9.7144e-05  |
|  71293 | javascript-ii            |        52 |                9.7144e-05  |
|  71294 | javascript-ii            |        65 |                9.7144e-05  |
|  71295 | javascript-ii            |        83 |                9.7144e-05  |
|  71296 | javascript-ii            |        94 |                9.7144e-05  |
|  71297 | javascript-ii            |        98 |                9.7144e-05  |
|  71298 | javascript-ii            |       105 |                9.7144e-05  |
|  71299 | javascript-ii            |       126 |                9.7144e-05  |
|  71300 | javascript-ii            |       129 |                9.7144e-05  |
|  71301 | javascript-ii            |       150 |                9.7144e-05  |
|  71302 | javascript-ii            |       151 |                9.7144e-05  |
|  71303 | javascript-ii            |       156 |                9.7144e-05  |
|  71304 | javascript-ii            |       175 |                9.7144e-05  |
|  71305 | javascript-ii            |       180 |                9.7144e-05  |
|  71306 | javascript-ii            |       248 |                9.7144e-05  |
|  71307 | javascript-ii            |       255 |                9.7144e-05  |
|  71308 | javascript-ii            |       284 |                9.7144e-05  |
|  71309 | javascript-ii            |       285 |                9.7144e-05  |
|  71310 | javascript-ii            |       392 |                9.7144e-05  |
|  71311 | javascript-ii            |       432 |                9.7144e-05  |
|  71312 | javascript-ii            |       442 |                9.7144e-05  |
|  71313 | javascript-ii            |       513 |                9.7144e-05  |
|  71314 | javascript-ii            |       517 |                9.7144e-05  |
|  71315 | javascript-ii            |       539 |                9.7144e-05  |
|  71316 | javascript-ii            |       572 |                9.7144e-05  |
|  71317 | javascript-ii            |       594 |                9.7144e-05  |
|  71318 | javascript-ii            |       628 |                9.7144e-05  |
|  71319 | javascript-ii            |       651 |                9.7144e-05  |
|  71320 | javascript-ii            |       711 |                9.7144e-05  |
|  71321 | javascript-ii            |       775 |                9.7144e-05  |
|  71322 | javascript-ii            |       811 |                9.7144e-05  |
|  71323 | javascript-ii            |       817 |                9.7144e-05  |
|  71324 | javascript-ii            |       826 |                9.7144e-05  |
|  71325 | javascript-ii            |       828 |                9.7144e-05  |
|  71326 | javascript-ii            |       853 |                9.7144e-05  |
|  71327 | javascript-ii            |       854 |                9.7144e-05  |
|  71328 | javascript-ii            |       861 |                9.7144e-05  |
|  71329 | javascript-ii            |       886 |                9.7144e-05  |
|  71330 | javascript-ii            |       893 |                9.7144e-05  |
|  71331 | javascript-ii            |       920 |                9.7144e-05  |
|  71332 | javascript-ii            |       921 |                9.7144e-05  |
|  71333 | javascript-ii            |       936 |                9.7144e-05  |
|  71334 | javascript-ii            |       941 |                9.7144e-05  |
|  71335 | javascript-ii            |       946 |                9.7144e-05  |
|  71336 | javascript-ii            |       950 |                9.7144e-05  |
|  71337 | javascript-ii            |       951 |                9.7144e-05  |
|  71338 | javascript-ii            |       969 |                9.7144e-05  |
|  71339 | javascript-ii            |       970 |                9.7144e-05  |
|  71340 | javascript-ii            |       979 |                9.7144e-05  |
|  71341 | javascript-ii            |       981 |                9.7144e-05  |
|  74938 | jquery                   |        13 |                9.05715e-05 |
|  74939 | jquery                   |        27 |                9.05715e-05 |
|  74940 | jquery                   |        33 |                9.05715e-05 |
|  74941 | jquery                   |        47 |                9.05715e-05 |
|  74942 | jquery                   |        50 |                9.05715e-05 |
|  74943 | jquery                   |        52 |                9.05715e-05 |
|  74944 | jquery                   |       104 |                9.05715e-05 |
|  74945 | jquery                   |       105 |                9.05715e-05 |
|  74946 | jquery                   |       150 |                9.05715e-05 |
|  74947 | jquery                   |       151 |                9.05715e-05 |
|  74948 | jquery                   |       156 |                9.05715e-05 |
|  74949 | jquery                   |       164 |                9.05715e-05 |
|  74950 | jquery                   |       179 |                9.05715e-05 |
|  74951 | jquery                   |       218 |                9.05715e-05 |
|  74952 | jquery                   |       251 |                9.05715e-05 |
|  74953 | jquery                   |       253 |                9.05715e-05 |
|  74954 | jquery                   |       254 |                9.05715e-05 |
|  74955 | jquery                   |       255 |                9.05715e-05 |
|  74956 | jquery                   |       313 |                9.05715e-05 |
|  74957 | jquery                   |       370 |                9.05715e-05 |
|  74958 | jquery                   |       432 |                9.05715e-05 |
|  74959 | jquery                   |       464 |                9.05715e-05 |
|  74960 | jquery                   |       503 |                9.05715e-05 |
|  74961 | jquery                   |       505 |                9.05715e-05 |
|  74962 | jquery                   |       572 |                9.05715e-05 |
|  74963 | jquery                   |       594 |                9.05715e-05 |
|  74964 | jquery                   |       629 |                9.05715e-05 |
|  74965 | jquery                   |       676 |                9.05715e-05 |
|  74966 | jquery                   |       712 |                9.05715e-05 |
|  74967 | jquery                   |       734 |                9.05715e-05 |
|  74968 | jquery                   |       765 |                9.05715e-05 |
|  74969 | jquery                   |       784 |                9.05715e-05 |
|  74970 | jquery                   |       806 |                9.05715e-05 |
|  74971 | jquery                   |       818 |                9.05715e-05 |
|  74972 | jquery                   |       854 |                9.05715e-05 |
|  74973 | jquery                   |       864 |                9.05715e-05 |
|  74974 | jquery                   |       894 |                9.05715e-05 |
|  74975 | jquery                   |       915 |                9.05715e-05 |
|  74976 | jquery                   |       916 |                9.05715e-05 |
|  74977 | jquery                   |       921 |                9.05715e-05 |
|  74978 | jquery                   |       925 |                9.05715e-05 |
|  74979 | jquery                   |       938 |                9.05715e-05 |
|  74980 | jquery                   |       939 |                9.05715e-05 |
|  74981 | jquery                   |       941 |                9.05715e-05 |
|  74982 | jquery                   |       943 |                9.05715e-05 |
|  74983 | jquery                   |       946 |                9.05715e-05 |
|  74984 | jquery                   |       947 |                9.05715e-05 |
|  74985 | jquery                   |       948 |                9.05715e-05 |
|  74986 | jquery                   |       958 |                9.05715e-05 |
|  74987 | jquery                   |       970 |                9.05715e-05 |
|  82296 | mysql                    |        13 |                9.42418e-05 |
|  82297 | mysql                    |        16 |                9.42418e-05 |
|  82298 | mysql                    |        17 |                9.42418e-05 |
|  82299 | mysql                    |        22 |                9.42418e-05 |
|  82300 | mysql                    |        45 |                9.42418e-05 |
|  82301 | mysql                    |        57 |                9.42418e-05 |
|  82302 | mysql                    |        65 |                9.42418e-05 |
|  82303 | mysql                    |        94 |                9.42418e-05 |
|  82304 | mysql                    |       150 |                9.42418e-05 |
|  82305 | mysql                    |       164 |                9.42418e-05 |
|  82306 | mysql                    |       168 |                9.42418e-05 |
|  82307 | mysql                    |       170 |                9.42418e-05 |
|  82308 | mysql                    |       216 |                9.42418e-05 |
|  82309 | mysql                    |       237 |                9.42418e-05 |
|  82310 | mysql                    |       243 |                9.42418e-05 |
|  82311 | mysql                    |       276 |                9.42418e-05 |
|  82312 | mysql                    |       284 |                9.42418e-05 |
|  82313 | mysql                    |       310 |                9.42418e-05 |
|  82314 | mysql                    |       327 |                9.42418e-05 |
|  82315 | mysql                    |       370 |                9.42418e-05 |
|  82316 | mysql                    |       371 |                9.42418e-05 |
|  82317 | mysql                    |       376 |                9.42418e-05 |
|  82318 | mysql                    |       401 |                9.42418e-05 |
|  82319 | mysql                    |       443 |                9.42418e-05 |
|  82320 | mysql                    |       476 |                9.42418e-05 |
|  82321 | mysql                    |       572 |                9.42418e-05 |
|  82322 | mysql                    |       581 |                9.42418e-05 |
|  82323 | mysql                    |       594 |                9.42418e-05 |
|  82324 | mysql                    |       611 |                9.42418e-05 |
|  82325 | mysql                    |       691 |                9.42418e-05 |
|  82326 | mysql                    |       694 |                9.42418e-05 |
|  82327 | mysql                    |       698 |                9.42418e-05 |
|  82328 | mysql                    |       704 |                9.42418e-05 |
|  82329 | mysql                    |       711 |                9.42418e-05 |
|  82330 | mysql                    |       738 |                9.42418e-05 |
|  82331 | mysql                    |       744 |                9.42418e-05 |
|  82332 | mysql                    |       752 |                9.42418e-05 |
|  82333 | mysql                    |       777 |                9.42418e-05 |
|  82334 | mysql                    |       779 |                9.42418e-05 |
|  82335 | mysql                    |       800 |                9.42418e-05 |
|  82336 | mysql                    |       802 |                9.42418e-05 |
|  82337 | mysql                    |       805 |                9.42418e-05 |
|  82338 | mysql                    |       833 |                9.42418e-05 |
|  82339 | mysql                    |       854 |                9.42418e-05 |
|  82340 | mysql                    |       867 |                9.42418e-05 |
|  82341 | mysql                    |       869 |                9.42418e-05 |
|  82342 | mysql                    |       870 |                9.42418e-05 |
|  82343 | mysql                    |       884 |                9.42418e-05 |
|  82344 | mysql                    |       915 |                9.42418e-05 |
|  82345 | mysql                    |       932 |                9.42418e-05 |
|  82346 | mysql                    |       941 |                9.42418e-05 |
|  82347 | mysql                    |       950 |                9.42418e-05 |
|  94423 | search/search_index.json |        25 |                5.70321e-05 |
|  94424 | search/search_index.json |        57 |                5.70321e-05 |
|  94425 | search/search_index.json |       105 |                5.70321e-05 |
|  94426 | search/search_index.json |       119 |                5.70321e-05 |
|  94427 | search/search_index.json |       126 |                5.70321e-05 |
|  94428 | search/search_index.json |       134 |                5.70321e-05 |
|  94429 | search/search_index.json |       145 |                5.70321e-05 |
|  94430 | search/search_index.json |       157 |                5.70321e-05 |
|  94431 | search/search_index.json |       164 |                5.70321e-05 |
|  94432 | search/search_index.json |       181 |                5.70321e-05 |
|  94433 | search/search_index.json |       187 |                5.70321e-05 |
|  94434 | search/search_index.json |       188 |                5.70321e-05 |
|  94435 | search/search_index.json |       192 |                5.70321e-05 |
|  94436 | search/search_index.json |       196 |                5.70321e-05 |
|  94437 | search/search_index.json |       242 |                5.70321e-05 |
|  94438 | search/search_index.json |       245 |                5.70321e-05 |
|  94439 | search/search_index.json |       255 |                5.70321e-05 |
|  94440 | search/search_index.json |       257 |                5.70321e-05 |
|  94441 | search/search_index.json |       278 |                5.70321e-05 |
|  94442 | search/search_index.json |       304 |                5.70321e-05 |
|  94443 | search/search_index.json |       310 |                5.70321e-05 |
|  94444 | search/search_index.json |       321 |                5.70321e-05 |
|  94445 | search/search_index.json |       444 |                5.70321e-05 |
|  94446 | search/search_index.json |       470 |                5.70321e-05 |
|  94447 | search/search_index.json |       508 |                5.70321e-05 |
|  94448 | search/search_index.json |       519 |                5.70321e-05 |
|  94449 | search/search_index.json |       572 |                5.70321e-05 |
|  94450 | search/search_index.json |       588 |                5.70321e-05 |
|  94451 | search/search_index.json |       607 |                5.70321e-05 |
|  94452 | search/search_index.json |       621 |                5.70321e-05 |
|  94453 | search/search_index.json |       654 |                5.70321e-05 |
|  94454 | search/search_index.json |       661 |                5.70321e-05 |
|  94455 | search/search_index.json |       680 |                5.70321e-05 |
|  94456 | search/search_index.json |       745 |                5.70321e-05 |
|  94457 | search/search_index.json |       749 |                5.70321e-05 |
|  94458 | search/search_index.json |       766 |                5.70321e-05 |
|  94459 | search/search_index.json |       804 |                5.70321e-05 |
|  94460 | search/search_index.json |       807 |                5.70321e-05 |
|  94461 | search/search_index.json |       811 |                5.70321e-05 |
|  94462 | search/search_index.json |       837 |                5.70321e-05 |
|  94463 | search/search_index.json |       865 |                5.70321e-05 |
|  94464 | search/search_index.json |       875 |                5.70321e-05 |
|  94465 | search/search_index.json |       883 |                5.70321e-05 |
|  94466 | search/search_index.json |       897 |                5.70321e-05 |
|  94467 | search/search_index.json |       903 |                5.70321e-05 |
|  94468 | search/search_index.json |       923 |                5.70321e-05 |
|  94469 | search/search_index.json |       924 |                5.70321e-05 |
|  94470 | search/search_index.json |       925 |                5.70321e-05 |
|  94471 | search/search_index.json |       926 |                5.70321e-05 |
|  94472 | search/search_index.json |       927 |                5.70321e-05 |
|  94473 | search/search_index.json |       934 |                5.70321e-05 |
|  94474 | search/search_index.json |       944 |                5.70321e-05 |
|  94475 | search/search_index.json |       945 |                5.70321e-05 |
|  94476 | search/search_index.json |       953 |                5.70321e-05 |
|  94477 | search/search_index.json |       963 |                5.70321e-05 |
|  94478 | search/search_index.json |       964 |                5.70321e-05 |
|  94479 | search/search_index.json |       974 |                5.70321e-05 |
|  97383 | spring                   |        13 |                8.41538e-05 |
|  97384 | spring                   |        23 |                8.41538e-05 |
|  97385 | spring                   |        37 |                8.41538e-05 |
|  97386 | spring                   |        51 |                8.41538e-05 |
|  97387 | spring                   |        75 |                8.41538e-05 |
|  97388 | spring                   |        80 |                8.41538e-05 |
|  97389 | spring                   |        96 |                8.41538e-05 |
|  97390 | spring                   |        98 |                8.41538e-05 |
|  97391 | spring                   |       149 |                8.41538e-05 |
|  97392 | spring                   |       164 |                8.41538e-05 |
|  97393 | spring                   |       170 |                8.41538e-05 |
|  97394 | spring                   |       218 |                8.41538e-05 |
|  97395 | spring                   |       222 |                8.41538e-05 |
|  97396 | spring                   |       237 |                8.41538e-05 |
|  97397 | spring                   |       301 |                8.41538e-05 |
|  97398 | spring                   |       310 |                8.41538e-05 |
|  97399 | spring                   |       346 |                8.41538e-05 |
|  97400 | spring                   |       427 |                8.41538e-05 |
|  97401 | spring                   |       431 |                8.41538e-05 |
|  97402 | spring                   |       442 |                8.41538e-05 |
|  97403 | spring                   |       444 |                8.41538e-05 |
|  97404 | spring                   |       459 |                8.41538e-05 |
|  97405 | spring                   |       467 |                8.41538e-05 |
|  97406 | spring                   |       472 |                8.41538e-05 |
|  97407 | spring                   |       480 |                8.41538e-05 |
|  97408 | spring                   |       481 |                8.41538e-05 |
|  97409 | spring                   |       492 |                8.41538e-05 |
|  97410 | spring                   |       539 |                8.41538e-05 |
|  97411 | spring                   |       543 |                8.41538e-05 |
|  97412 | spring                   |       552 |                8.41538e-05 |
|  97413 | spring                   |       572 |                8.41538e-05 |
|  97414 | spring                   |       594 |                8.41538e-05 |
|  97415 | spring                   |       601 |                8.41538e-05 |
|  97416 | spring                   |       607 |                8.41538e-05 |
|  97417 | spring                   |       651 |                8.41538e-05 |
|  97418 | spring                   |       659 |                8.41538e-05 |
|  97419 | spring                   |       700 |                8.41538e-05 |
|  97420 | spring                   |       704 |                8.41538e-05 |
|  97421 | spring                   |       752 |                8.41538e-05 |
|  97422 | spring                   |       757 |                8.41538e-05 |
|  97423 | spring                   |       779 |                8.41538e-05 |
|  97424 | spring                   |       786 |                8.41538e-05 |
|  97425 | spring                   |       797 |                8.41538e-05 |
|  97426 | spring                   |       800 |                8.41538e-05 |
|  97427 | spring                   |       807 |                8.41538e-05 |
|  97428 | spring                   |       813 |                8.41538e-05 |
|  97429 | spring                   |       820 |                8.41538e-05 |
|  97430 | spring                   |       822 |                8.41538e-05 |
|  97431 | spring                   |       829 |                8.41538e-05 |
|  97432 | spring                   |       830 |                8.41538e-05 |
|  97433 | spring                   |       833 |                8.41538e-05 |
|  97434 | spring                   |       854 |                8.41538e-05 |
|  97435 | spring                   |       856 |                8.41538e-05 |
|  97436 | spring                   |       861 |                8.41538e-05 |
|  97437 | spring                   |       862 |                8.41538e-05 |
|  97438 | spring                   |       884 |                8.41538e-05 |
|  97439 | spring                   |       886 |                8.41538e-05 |
|  97440 | spring                   |       890 |                8.41538e-05 |
|  97441 | spring                   |       917 |                8.41538e-05 |
|  97442 | spring                   |       936 |                8.41538e-05 |
|  97443 | spring                   |       939 |                8.41538e-05 |
|  97444 | spring                   |       942 |                8.41538e-05 |
|  97445 | spring                   |       946 |                8.41538e-05 |
|  97446 | spring                   |       950 |                8.41538e-05 |
|  97447 | spring                   |       958 |                8.41538e-05 |
|  97448 | spring                   |       970 |                8.41538e-05 |
| 106670 | toc                      |        13 |                5.68473e-05 |
| 106671 | toc                      |        15 |                5.68473e-05 |
| 106672 | toc                      |        37 |                5.68473e-05 |
| 106673 | toc                      |        38 |                5.68473e-05 |
| 106674 | toc                      |        41 |                5.68473e-05 |
| 106675 | toc                      |        76 |                5.68473e-05 |
| 106676 | toc                      |        80 |                5.68473e-05 |
| 106677 | toc                      |       118 |                5.68473e-05 |
| 106678 | toc                      |       126 |                5.68473e-05 |
| 106679 | toc                      |       135 |                5.68473e-05 |
| 106680 | toc                      |       138 |                5.68473e-05 |
| 106681 | toc                      |       156 |                5.68473e-05 |
| 106682 | toc                      |       162 |                5.68473e-05 |
| 106683 | toc                      |       197 |                5.68473e-05 |
| 106684 | toc                      |       244 |                5.68473e-05 |
| 106685 | toc                      |       252 |                5.68473e-05 |
| 106686 | toc                      |       254 |                5.68473e-05 |
| 106687 | toc                      |       280 |                5.68473e-05 |
| 106688 | toc                      |       287 |                5.68473e-05 |
| 106689 | toc                      |       295 |                5.68473e-05 |
| 106690 | toc                      |       300 |                5.68473e-05 |
| 106691 | toc                      |       348 |                5.68473e-05 |
| 106692 | toc                      |       389 |                5.68473e-05 |
| 106693 | toc                      |       396 |                5.68473e-05 |
| 106694 | toc                      |       400 |                5.68473e-05 |
| 106695 | toc                      |       404 |                5.68473e-05 |
| 106696 | toc                      |       409 |                5.68473e-05 |
| 106697 | toc                      |       411 |                5.68473e-05 |
| 106698 | toc                      |       413 |                5.68473e-05 |
| 106699 | toc                      |       420 |                5.68473e-05 |
| 106700 | toc                      |       427 |                5.68473e-05 |
| 106701 | toc                      |       443 |                5.68473e-05 |
| 106702 | toc                      |       457 |                5.68473e-05 |
| 106703 | toc                      |       459 |                5.68473e-05 |
| 106704 | toc                      |       464 |                5.68473e-05 |
| 106705 | toc                      |       493 |                5.68473e-05 |
| 106706 | toc                      |       519 |                5.68473e-05 |
| 106707 | toc                      |       574 |                5.68473e-05 |
| 106708 | toc                      |       581 |                5.68473e-05 |
| 106709 | toc                      |       594 |                5.68473e-05 |
| 106710 | toc                      |       598 |                5.68473e-05 |
| 106711 | toc                      |       623 |                5.68473e-05 |
| 106712 | toc                      |       644 |                5.68473e-05 |
| 106713 | toc                      |       653 |                5.68473e-05 |
| 106714 | toc                      |       669 |                5.68473e-05 |
| 106715 | toc                      |       712 |                5.68473e-05 |
| 106716 | toc                      |       734 |                5.68473e-05 |
| 106717 | toc                      |       744 |                5.68473e-05 |
| 106718 | toc                      |       745 |                5.68473e-05 |
| 106719 | toc                      |       748 |                5.68473e-05 |
| 106720 | toc                      |       772 |                5.68473e-05 |
| 106721 | toc                      |       777 |                5.68473e-05 |
| 106722 | toc                      |       779 |                5.68473e-05 |
| 106723 | toc                      |       786 |                5.68473e-05 |
| 106724 | toc                      |       793 |                5.68473e-05 |
| 106725 | toc                      |       821 |                5.68473e-05 |
| 106726 | toc                      |       855 |                5.68473e-05 |
| 106727 | toc                      |       877 |                5.68473e-05 |
| 106728 | toc                      |       936 |                5.68473e-05 |
| 106729 | toc                      |       941 |                5.68473e-05 |
| 106730 | toc                      |       956 |                5.68473e-05 |
| 106731 | toc                      |       966 |                5.68473e-05 |
| 106732 | toc                      |       967 |                5.68473e-05 |
| 106733 | toc                      |       968 |                5.68473e-05 |
| 106734 | toc                      |       972 |                5.68473e-05 |
| 106735 | toc                      |       974 |                5.68473e-05 |
| 106736 | toc                      |       975 |                5.68473e-05 |
| 106737 | toc                      |       976 |                5.68473e-05 |

#### Date given Id

| date   | id   | proba_date_given_id   |
|--------|------|-----------------------|

#### Ip given Name

|      | ip           | name      |   proba_ip_given_name |
|-----:|:-------------|:----------|----------------------:|
| 5413 | 97.105.19.58 | Pinnacles |           6.32798e-05 |
| 5414 | 97.105.19.58 | Ganymede  |           4.83905e-05 |
| 5415 | 97.105.19.58 | Teddy     |           3.35011e-05 |

#### Time given User_Id

| time   | user_id   | proba_time_given_user_id   |
|--------|-----------|----------------------------|

#### Name given Slack

| name   | slack   | proba_name_given_slack   |
|--------|---------|--------------------------|

#### Name given End_Date

| name   | end_date   | proba_name_given_end_date   |
|--------|------------|-----------------------------|

#### Ip given Slack

|      | ip           | slack      |   proba_ip_given_slack |
|-----:|:-------------|:-----------|-----------------------:|
| 5405 | 97.105.19.58 | #pinnacles |            6.32798e-05 |
| 5406 | 97.105.19.58 | #ganymede  |            4.83905e-05 |
| 5407 | 97.105.19.58 | #teddy     |            3.35011e-05 |

#### Date given Path

| date   | path   | proba_date_given_path   |
|--------|--------|-------------------------|

#### Ip given End_Date

|      | ip           | end_date   |   proba_ip_given_end_date |
|-----:|:-------------|:-----------|--------------------------:|
| 5412 | 97.105.19.58 | 2017-07-20 |               6.32798e-05 |
| 5413 | 97.105.19.58 | 2020-08-20 |               4.83905e-05 |
| 5414 | 97.105.19.58 | 2018-05-17 |               3.35011e-05 |

#### Slack given Start_Date

| slack   | start_date   | proba_slack_given_start_date   |
|---------|--------------|--------------------------------|

#### End_Date given Program_Id

| end_date   | program_id   | proba_end_date_given_program_id   |
|------------|--------------|-----------------------------------|

#### Id given Start_Date

| id   | start_date   | proba_id_given_start_date   |
|------|--------------|-----------------------------|

#### Date given Name

| date   | name   | proba_date_given_name   |
|--------|--------|-------------------------|

#### Path given Program_Id

|    | path   | program_id                |   proba_path_given_program_id |
|---:|:-------|:--------------------------|------------------------------:|
| 10 | /      | Front End Web Development |                   2.18083e-05 |

#### Date given Slack

| date   | slack   | proba_date_given_slack   |
|--------|---------|--------------------------|

#### Name given Program_Id

| name   | program_id   | proba_name_given_program_id   |
|--------|--------------|-------------------------------|

#### Time given Program_Id

| time   | program_id   | proba_time_given_program_id   |
|--------|--------------|-------------------------------|

#### User_Id given Program_Id

| user_id   | program_id   | proba_user_id_given_program_id   |
|-----------|--------------|----------------------------------|

#### Date given End_Date

| date   | end_date   | proba_date_given_end_date   |
|--------|------------|-----------------------------|

#### Cohort_Id given Program_Id

| cohort_id   | program_id   | proba_cohort_id_given_program_id   |
|-------------|--------------|------------------------------------|

#### Date given Ip

| date   | ip   | proba_date_given_ip   |
|--------|------|-----------------------|

#### Name given Start_Date

| name   | start_date   | proba_name_given_start_date   |
|--------|--------------|-------------------------------|

#### Ip given Start_Date

|      | ip           | start_date   |   proba_ip_given_start_date |
|-----:|:-------------|:-------------|----------------------------:|
| 5402 | 97.105.19.58 | 2017-03-27   |                 6.32798e-05 |
| 5403 | 97.105.19.58 | 2020-03-23   |                 4.83905e-05 |
| 5404 | 97.105.19.58 | 2018-01-08   |                 3.35011e-05 |

